# Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import os
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime, timedelta

# Cargar bases de datos 

In [2]:
# Configuración de pandas para mostrar más dígitos en la salida
pd.set_option('display.float_format', '{:.22f}'.format)

In [3]:
# Imprime el directorio actual
print("Directorio inicial:", os.getcwd())

# Cambiar al directorio 'FORMATO_290'
os.chdir('..')

# Confirma el cambio de directorio
print("Directorio después de moverse hacia arriba:", os.getcwd())

Directorio inicial: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290\Codigo
Directorio después de moverse hacia arriba: D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290


In [4]:
ruta = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COVI.xlsx"

In [5]:
# Leer hoja específica y normalizar columnas
AÑO_ACTUAL = pd.read_excel(ruta, sheet_name="AÑO ACTUAL", dtype=str)
AÑO_ACTUAL.columns = [x.upper() for x in AÑO_ACTUAL.columns]

In [6]:
# Cargar hoja "RAMOS_COVI" como texto y estandarizar nombres de columnas
RAMOS_COVI = pd.read_excel(ruta, sheet_name="RAMOS_COVI", dtype=str)
RAMOS_COVI.columns = [x.upper() for x in RAMOS_COVI.columns]
RAMOS_COVI = RAMOS_COVI[['SOCIEDAD', 'LINE OF BUSS', 'COD_RAMO', 'RAMO']].drop_duplicates()

In [7]:
# Leer hoja específica y normalizar columnas
CUENTAS = pd.read_excel(ruta, sheet_name="CUENTAS", dtype=str)
CUENTAS.columns = [x.upper() for x in CUENTAS.columns]
CUENTAS = CUENTAS[['SOCIEDAD', 'CUENTA', 'CONCEPTO', 'ALTERNATIVA 6']].drop_duplicates()

In [8]:
UNIDAD_DE_CAPTURA = pd.read_excel(ruta, sheet_name="UNIDAD_DE_CAPTURA", dtype=str)
UNIDAD_DE_CAPTURA.columns = [x.upper() for x in UNIDAD_DE_CAPTURA.columns]

In [9]:
# Leer la hoja POLIZAS desde el archivo principal
POLIZAS = pd.read_excel(ruta, sheet_name="POLIZAS", dtype=str)
POLIZAS.columns = [x.upper() for x in POLIZAS.columns]

# Convertir PERIODO solo a fecha con formato yyyy-mm-dd
POLIZAS['PERIODO'] = pd.to_datetime(POLIZAS['PERIODO']).dt.strftime("%Y-%m-%d")

# Separar en AÑO, MES y DÍA como texto
POLIZAS[['AÑO', 'MES', 'DÍA']] = POLIZAS['PERIODO'].str.split("-", expand=True)

In [10]:
BALANCE_WP = pd.read_excel(ruta,sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)

# Renombrar columnas para que coincidan con la estructura anterior
BALANCE_WP.columns = ["ALTERNATIVA 6", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]

In [11]:
# Leer hoja BD_Local2 desde el archivo principal
Base = pd.read_excel(ruta, sheet_name="BD_Local2", dtype=str)
Base.columns = [x.upper() for x in Base.columns]
Base = Base.apply(lambda col: col.str.strip().str.replace(r'\s+', ' ', regex=True) if col.dtype == 'object' else col)

In [12]:
# Combinar datos
bd = AÑO_ACTUAL.merge(CUENTAS, on=['CUENTA', 'SOCIEDAD'], how='left')
bd = bd.merge(RAMOS_COVI, on=['LINE OF BUSS', 'SOCIEDAD'], how='left')
bd = bd.merge(UNIDAD_DE_CAPTURA, on='CONCEPTO', how='left').drop_duplicates()

variables = [
    "SALDO PERIODO 01", "SALDO PERIODO 02", "SALDO PERIODO 03", "SALDO PERIODO 04", 
    "SALDO PERIODO 05", "SALDO PERIODO 06", "SALDO PERIODO 07", "SALDO PERIODO 08", 
    "SALDO PERIODO 09", "SALDO PERIODO 10", "SALDO PERIODO 11", "SALDO PERIODO 12", 
    "SALDO PERIODO 13"
]

for variable in variables:
    if variable not in bd.columns:
        bd[variable] = np.nan

columnas_a_convertir = [
    "TOTAL CUENTA", "ARRASTRE SALDOS", "SALDO PERIODO 01", "SALDO PERIODO 02", "SALDO PERIODO 03",
    "SALDO PERIODO 04", "SALDO PERIODO 05", "SALDO PERIODO 06", "SALDO PERIODO 07",
    "SALDO PERIODO 08", "SALDO PERIODO 09", "SALDO PERIODO 10", "SALDO PERIODO 11",
    "SALDO PERIODO 12", "SALDO PERIODO 13","COD_RAMO"
]

for columna in columnas_a_convertir:
    bd[columna] = pd.to_numeric(bd[columna], errors='coerce')

    
bd['TOTAL'] = bd[['ARRASTRE SALDOS'] + variables].sum(axis=1, skipna=True)

In [14]:
ruta = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\FORMATO_290\Bases\Plantilla_290_COVI.xlsx"
wb = openpyxl.load_workbook(ruta)
ws = wb['Hoja1']

# UNIDAD DE CAPTURA 1


## CONSULTA DE TOTALES PRIMAS RETENIDAS UNIDAD 1    

In [15]:
unidad1_total = bd[bd['UNIDAD'] == 1].groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad1_total['TOTAL'] = unidad1_total['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Cargamos el workbook y seleccionamos la hoja
wb = openpyxl.load_workbook('Bases/Plantilla_290_COVI.xlsx')
ws = wb['Hoja1']

def write_to_excel(data_frame, concept, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=3).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad1_total, "Primas Emitidas Directas", 4)
write_to_excel(unidad1_total, "Primas aceptadas en coaseguro", 5)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6)
write_to_excel(unidad1_total, "Aceptadas Reaseguro interior", 7)
write_to_excel(unidad1_total, "Aceptadas Reaseguro exterior", 8)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9)
write_to_excel(unidad1_total, "Cedidas Reaseguro interior (resta)", 10)
write_to_excel(unidad1_total, "Cedidas Reaseguro exterior (resta)", 11)
write_to_excel(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 12)
write_to_excel(unidad1_total, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)

def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
c4_005 = get_value(unidad1_total, "Primas Emitidas Directas")
c5_010 = get_value(unidad1_total, "Primas aceptadas en coaseguro")
c6_015 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
c7_020 = get_value(unidad1_total, "Aceptadas Reaseguro interior")
c8_025 = get_value(unidad1_total, "Aceptadas Reaseguro exterior")
c9_030 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
c10_035 = get_value(unidad1_total, "Cedidas Reaseguro interior (resta)")
c11_040 = get_value(unidad1_total, "Cedidas Reaseguro exterior (resta)")
c12_045 = get_value(unidad1_total, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
c13_050 = get_value(unidad1_total, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Calcular el total general según la fórmula dada
c14_999 = (c4_005 + c5_010 - c6_015 + c7_020 + c8_025 - c9_030 - c10_035 - c11_040 + c12_045 + c13_050)

# Escribir el total general en Excel
ws = wb['Hoja1']
ws.cell(row=14, column=3).value = c14_999


##  SUBTOTAL - RAMOS UNIDAD 1      

In [16]:
unidad1_SUBTOTAL = bd[bd['UNIDAD'] == 1].groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad1_SUBTOTAL['TOTAL'] = unidad1_SUBTOTAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Llamadas a la función para escribir en Excel
concepts = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

for concept, row_num in concepts:
    write_to_excel(unidad1_SUBTOTAL, concept, 4, row_num)


def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
d4_005 = get_value(unidad1_SUBTOTAL, "Primas Emitidas Directas")
d5_010 = get_value(unidad1_SUBTOTAL, "Primas aceptadas en coaseguro")
d6_015 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
d7_020 = get_value(unidad1_SUBTOTAL, "Aceptadas Reaseguro interior")
d8_025 = get_value(unidad1_SUBTOTAL, "Aceptadas Reaseguro exterior")
d9_030 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
d10_035 = get_value(unidad1_SUBTOTAL, "Cedidas Reaseguro interior (resta)")
d11_040 = get_value(unidad1_SUBTOTAL, "Cedidas Reaseguro exterior (resta)")
d12_045 = get_value(unidad1_SUBTOTAL, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
d13_050 = get_value(unidad1_SUBTOTAL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular d14_999
d14_999 = (d4_005 + d5_010 - d6_015 + d7_020 + d8_025 - d9_030 - d10_035 - d11_040 + d12_045 + d13_050)

# Escribir el total general en Excel
ws = wb['Hoja1']
if d14_999 == 0:  # Verificar si el resultado es 0, lo que implica que no hubo datos para calcular
    d14_999 = {'Total': 0}  # Crear un diccionario para simular un DataFrame con un solo valor 'Total' = 0
    ws.cell(row=14, column=4).value = d14_999['Total']
else:
    ws.cell(row=14, column=4).value = d14_999


## 26 EXEQUIAS  UNIDAD 1   

In [17]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 26
unidad1_EXEQUIAS = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 26)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_EXEQUIAS['TOTAL'] = unidad1_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 26, 4),
    ("Primas aceptadas en coaseguro", 26, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 26, 6),
    ("Aceptadas Reaseguro interior", 26, 7),
    ("Aceptadas Reaseguro exterior", 26, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 26, 9),
    ("Cedidas Reaseguro interior (resta)", 26, 10),
    ("Cedidas Reaseguro exterior (resta)", 26, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 26, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 26, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_EXEQUIAS, concept, ramo, 28, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab4_005 = get_value(unidad1_EXEQUIAS, "Primas Emitidas Directas", 26)
ab5_010 = get_value(unidad1_EXEQUIAS, "Primas aceptadas en coaseguro", 26)
ab6_015 = get_value(unidad1_EXEQUIAS, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 26)
ab7_020 = get_value(unidad1_EXEQUIAS, "Aceptadas Reaseguro interior", 26)
ab8_025 = get_value(unidad1_EXEQUIAS, "Aceptadas Reaseguro exterior", 26)
ab9_030 = get_value(unidad1_EXEQUIAS, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 26)
ab10_035 = get_value(unidad1_EXEQUIAS, "Cedidas Reaseguro interior (resta)", 26)
ab11_040 = get_value(unidad1_EXEQUIAS, "Cedidas Reaseguro exterior (resta)", 26)
ab12_045 = get_value(unidad1_EXEQUIAS, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 26)
ab13_050 = get_value(unidad1_EXEQUIAS, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 26)

# Aplicar la fórmula específica para calcular ab14_999
ab14_999 = (ab4_005 + ab5_010 - ab6_015 + ab7_020 + ab8_025 - ab9_030 - ab10_035 - ab11_040 + ab12_045 + ab13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=28, value=ab14_999)


<Cell 'Hoja1'.AB14>

## 27 ACCIDENTES PERSONALES UNIDAD 1 

In [18]:

# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 27
unidad1_ACCPERSONALES = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 27)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_ACCPERSONALES['TOTAL'] = unidad1_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_ACCPERSONALES, concept, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac4_005 = get_value(unidad1_ACCPERSONALES, "Primas Emitidas Directas")
ac5_010 = get_value(unidad1_ACCPERSONALES, "Primas aceptadas en coaseguro")
ac6_015 = get_value(unidad1_ACCPERSONALES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ac7_020 = get_value(unidad1_ACCPERSONALES, "Aceptadas Reaseguro interior")
ac8_025 = get_value(unidad1_ACCPERSONALES, "Aceptadas Reaseguro exterior")
ac9_030 = get_value(unidad1_ACCPERSONALES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ac10_035 = get_value(unidad1_ACCPERSONALES, "Cedidas Reaseguro interior (resta)")
ac11_040 = get_value(unidad1_ACCPERSONALES, "Cedidas Reaseguro exterior (resta)")
ac12_045 = get_value(unidad1_ACCPERSONALES, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ac13_050 = get_value(unidad1_ACCPERSONALES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular aC14_999
ac14_999 = (ac4_005 + ac5_010 - ac6_015 + ac7_020 + ac8_025 - ac9_030 - ac10_035 - ac11_040 + ac12_045 + ac13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=29, value=ac14_999)

<Cell 'Hoja1'.AC14>

##   28 COLECTIVO VIDA UNIDAD 1  

In [19]:

# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_COLECVI = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_COLECVI['TOTAL'] = unidad1_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 28, 4),
    ("Primas aceptadas en coaseguro", 28, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 28, 6),
    ("Aceptadas Reaseguro interior", 28, 7),
    ("Aceptadas Reaseguro exterior", 28, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 28, 9),
    ("Cedidas Reaseguro interior (resta)", 28, 10),
    ("Cedidas Reaseguro exterior (resta)", 28, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 28, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 28, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_COLECVI, concept, ramo, 30, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad4_005 = get_value(unidad1_COLECVI, "Primas Emitidas Directas", 28)
ad5_010 = get_value(unidad1_COLECVI, "Primas aceptadas en coaseguro", 28)
ad6_015 = get_value(unidad1_COLECVI, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 28)
ad7_020 = get_value(unidad1_COLECVI, "Aceptadas Reaseguro interior", 28)
ad8_025 = get_value(unidad1_COLECVI, "Aceptadas Reaseguro exterior", 28)
ad9_030 = get_value(unidad1_COLECVI, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 28)
ad10_035 = get_value(unidad1_COLECVI, "Cedidas Reaseguro interior (resta)", 28)
ad11_040 = get_value(unidad1_COLECVI, "Cedidas Reaseguro exterior (resta)", 28)
ad12_045 = get_value(unidad1_COLECVI, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 28)
ad13_050 = get_value(unidad1_COLECVI, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 28)

# Aplicar la fórmula específica para calcular ad14_999
ad14_999 = (ad4_005 + ad5_010 - ad6_015 + ad7_020 + ad8_025 - ad9_030 - ad10_035 - ad11_040 + ad12_045 + ad13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=30, value=ad14_999)



<Cell 'Hoja1'.AD14>

## 29 EDUCATIVO  UNIDAD 1 

In [20]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_EDUCATIVO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_EDUCATIVO['TOTAL'] = unidad1_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 29, 4),
    ("Primas aceptadas en coaseguro", 29, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 29, 6),
    ("Aceptadas Reaseguro interior", 29, 7),
    ("Aceptadas Reaseguro exterior", 29, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 29, 9),
    ("Cedidas Reaseguro interior (resta)", 29, 10),
    ("Cedidas Reaseguro exterior (resta)", 29, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 29, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 29, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_EDUCATIVO, concept, ramo, 31, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae4_005 = get_value(unidad1_EDUCATIVO, "Primas Emitidas Directas", 29)
ae5_010 = get_value(unidad1_EDUCATIVO, "Primas aceptadas en coaseguro", 29)
ae6_015 = get_value(unidad1_EDUCATIVO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 29)
ae7_020 = get_value(unidad1_EDUCATIVO, "Aceptadas Reaseguro interior", 29)
ae8_025 = get_value(unidad1_EDUCATIVO, "Aceptadas Reaseguro exterior", 29)
ae9_030 = get_value(unidad1_EDUCATIVO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 29)
ae10_035 = get_value(unidad1_EDUCATIVO, "Cedidas Reaseguro interior (resta)", 29)
ae11_040 = get_value(unidad1_EDUCATIVO, "Cedidas Reaseguro exterior (resta)", 29)
ae12_045 = get_value(unidad1_EDUCATIVO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 29)
ae13_050 = get_value(unidad1_EDUCATIVO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 29)

# Aplicar la fórmula específica para calcular ae14_999
ae14_999 = (ae4_005 + ae5_010 - ae6_015 + ae7_020 + ae8_025 - ae9_030 - ae10_035 - ae11_040 + ae12_045 + ae13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=31, value=ae14_999)


<Cell 'Hoja1'.AE14>

## 30  VIDA GRUPO  UNIDAD 1

In [21]:
# Filtrar y agrupar los datos donde 'Unidad' es 1
unidad1_VIDAGRUPO = bd[bd['UNIDAD'] == 1].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_VIDAGRUPO['TOTAL'] = unidad1_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 30, 4),
    ("Primas aceptadas en coaseguro", 30, 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 30, 6),
    ("Aceptadas Reaseguro interior", 30, 7),
    ("Aceptadas Reaseguro exterior", 30, 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 30, 9),
    ("Cedidas Reaseguro interior (resta)", 30, 10),
    ("Cedidas Reaseguro exterior (resta)", 30, 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 30, 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 30, 13)
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad1_VIDAGRUPO, concept, ramo, 32, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af4_005 = get_value(unidad1_VIDAGRUPO, "Primas Emitidas Directas", 30)
af5_010 = get_value(unidad1_VIDAGRUPO, "Primas aceptadas en coaseguro", 30)
af6_015 = get_value(unidad1_VIDAGRUPO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 30)
af7_020 = get_value(unidad1_VIDAGRUPO, "Aceptadas Reaseguro interior", 30)
af8_025 = get_value(unidad1_VIDAGRUPO, "Aceptadas Reaseguro exterior", 30)
af9_030 = get_value(unidad1_VIDAGRUPO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 30)
af10_035 = get_value(unidad1_VIDAGRUPO, "Cedidas Reaseguro interior (resta)", 30)
af11_040 = get_value(unidad1_VIDAGRUPO, "Cedidas Reaseguro exterior (resta)", 30)
af12_045 = get_value(unidad1_VIDAGRUPO, "Cancelaciones y/o anulaciones primas de reaseguro cedido", 30)
af13_050 = get_value(unidad1_VIDAGRUPO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 30)

# Aplicar la fórmula específica para calcular af14_999
af14_999 = (af4_005 + af5_010 - af6_015 + af7_020 + af8_025 - af9_030 - af10_035 - af11_040 + af12_045 + af13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=32, value=af14_999)


<Cell 'Hoja1'.AF14>

##  31 SALUD UNIDAD 1         

In [22]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 31
unidad1_SALUD = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 31)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_SALUD['TOTAL'] = unidad1_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_SALUD, concept, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag4_005 = get_value(unidad1_SALUD, "Primas Emitidas Directas")
ag5_010 = get_value(unidad1_SALUD, "Primas aceptadas en coaseguro")
ag6_015 = get_value(unidad1_SALUD, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ag7_020 = get_value(unidad1_SALUD, "Aceptadas Reaseguro interior")
ag8_025 = get_value(unidad1_SALUD, "Aceptadas Reaseguro exterior")
ag9_030 = get_value(unidad1_SALUD, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ag10_035 = get_value(unidad1_SALUD, "Cedidas Reaseguro interior (resta)")
ag11_040 = get_value(unidad1_SALUD, "Cedidas Reaseguro exterior (resta)")
ag12_045 = get_value(unidad1_SALUD, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ag13_050 = get_value(unidad1_SALUD, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ag14_999
ag14_999 = (ag4_005 + ag5_010 - ag6_015 + ag7_020 + ag8_025 - ag9_030 - ag10_035 - ag11_040 + ag12_045 + ag13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=33, value=ag14_999)



<Cell 'Hoja1'.AG14>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 1 


In [23]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 32
unidad1_ENFALTOCOSTO = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 32)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_ENFALTOCOSTO['TOTAL'] = unidad1_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_ENFALTOCOSTO, concept, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah4_005 = get_value(unidad1_ENFALTOCOSTO, "Primas Emitidas Directas")
ah5_010 = get_value(unidad1_ENFALTOCOSTO, "Primas aceptadas en coaseguro")
ah6_015 = get_value(unidad1_ENFALTOCOSTO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ah7_020 = get_value(unidad1_ENFALTOCOSTO, "Aceptadas Reaseguro interior")
ah8_025 = get_value(unidad1_ENFALTOCOSTO, "Aceptadas Reaseguro exterior")
ah9_030 = get_value(unidad1_ENFALTOCOSTO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ah10_035 = get_value(unidad1_ENFALTOCOSTO, "Cedidas Reaseguro interior (resta)")
ah11_040 = get_value(unidad1_ENFALTOCOSTO, "Cedidas Reaseguro exterior (resta)")
ah12_045 = get_value(unidad1_ENFALTOCOSTO, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ah13_050 = get_value(unidad1_ENFALTOCOSTO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ah14_999
ah14_999 = (ah4_005 + ah5_010 - ah6_015 + ah7_020 + ah8_025 - ah9_030 - ah10_035 - ah11_040 + ah12_045 + ah13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=34, value=ah14_999)


<Cell 'Hoja1'.AH14>

## 33 VIDA INDIVIDUAL UNIDAD 1 

In [24]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 33
unidad1_VIDAINDIVI = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 33)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_VIDAINDIVI['TOTAL'] = unidad1_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_VIDAINDIVI, concept, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai4_005 = get_value(unidad1_VIDAINDIVI, "Primas Emitidas Directas")
ai5_010 = get_value(unidad1_VIDAINDIVI, "Primas aceptadas en coaseguro")
ai6_015 = get_value(unidad1_VIDAINDIVI, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ai7_020 = get_value(unidad1_VIDAINDIVI, "Aceptadas Reaseguro interior")
ai8_025 = get_value(unidad1_VIDAINDIVI, "Aceptadas Reaseguro exterior")
ai9_030 = get_value(unidad1_VIDAINDIVI, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ai10_035 = get_value(unidad1_VIDAINDIVI, "Cedidas Reaseguro interior (resta)")
ai11_040 = get_value(unidad1_VIDAINDIVI, "Cedidas Reaseguro exterior (resta)")
ai12_045 = get_value(unidad1_VIDAINDIVI, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ai13_050 = get_value(unidad1_VIDAINDIVI, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ai14_999
ai14_999 = (ai4_005 + ai5_010 - ai6_015 + ai7_020 + ai8_025 - ai9_030 - ai10_035 - ai11_040 + ai12_045 + ai13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=35, value=ai14_999)


<Cell 'Hoja1'.AI14>

## 34 PREVISIONAL DE INVALIDEZ Y SOBREVIVENCIA UNIDAD 1 

In [25]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 34
unidad1_PREVIDENTIAL = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 34)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_PREVIDENTIAL['TOTAL'] = unidad1_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'


# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_PREVIDENTIAL, concept, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj4_005 = get_value(unidad1_PREVIDENTIAL, "Primas Emitidas Directas")
aj5_010 = get_value(unidad1_PREVIDENTIAL, "Primas aceptadas en coaseguro")
aj6_015 = get_value(unidad1_PREVIDENTIAL, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
aj7_020 = get_value(unidad1_PREVIDENTIAL, "Aceptadas Reaseguro interior")
aj8_025 = get_value(unidad1_PREVIDENTIAL, "Aceptadas Reaseguro exterior")
aj9_030 = get_value(unidad1_PREVIDENTIAL, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
aj10_035 = get_value(unidad1_PREVIDENTIAL, "Cedidas Reaseguro interior (resta)")
aj11_040 = get_value(unidad1_PREVIDENTIAL, "Cedidas Reaseguro exterior (resta)")
aj12_045 = get_value(unidad1_PREVIDENTIAL, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
aj13_050 = get_value(unidad1_PREVIDENTIAL, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular aj14_999
aj14_999 = (aj4_005 + aj5_010 - aj6_015 + aj7_020 + aj8_025 - aj9_030 - aj10_035 - aj11_040 + aj12_045 + aj13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=36, value=aj14_999)


<Cell 'Hoja1'.AJ14>

##  35  RIESGOS LABORALES UNIDAD 1 

In [26]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 35
unidad1_RIESGOSLAB = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 35)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_RIESGOSLAB['TOTAL'] = unidad1_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_RIESGOSLAB, concept, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak4_005 = get_value(unidad1_RIESGOSLAB, "Primas Emitidas Directas")
ak5_010 = get_value(unidad1_RIESGOSLAB, "Primas aceptadas en coaseguro")
ak6_015 = get_value(unidad1_RIESGOSLAB, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ak7_020 = get_value(unidad1_RIESGOSLAB, "Aceptadas Reaseguro interior")
ak8_025 = get_value(unidad1_RIESGOSLAB, "Aceptadas Reaseguro exterior")
ak9_030 = get_value(unidad1_RIESGOSLAB, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ak10_035 = get_value(unidad1_RIESGOSLAB, "Cedidas Reaseguro interior (resta)")
ak11_040 = get_value(unidad1_RIESGOSLAB, "Cedidas Reaseguro exterior (resta)")
ak12_045 = get_value(unidad1_RIESGOSLAB, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ak13_050 = get_value(unidad1_RIESGOSLAB, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ak14_999
ak14_999 = (ak4_005 + ak5_010 - ak6_015 + ak7_020 + ak8_025 - ak9_030 - ak10_035 - ak11_040 + ak12_045 + ak13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=37, value=ak14_999)


<Cell 'Hoja1'.AK14>

##  36 PENSIONES LEY 100 UNIDAD 1  

In [27]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 36
unidad1_PENSIONESLEY100 = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 36)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_PENSIONESLEY100['TOTAL'] = unidad1_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_PENSIONESLEY100, concept, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al4_005 = get_value(unidad1_PENSIONESLEY100, "Primas Emitidas Directas")
al5_010 = get_value(unidad1_PENSIONESLEY100, "Primas aceptadas en coaseguro")
al6_015 = get_value(unidad1_PENSIONESLEY100, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
al7_020 = get_value(unidad1_PENSIONESLEY100, "Aceptadas Reaseguro interior")
al8_025 = get_value(unidad1_PENSIONESLEY100, "Aceptadas Reaseguro exterior")
al9_030 = get_value(unidad1_PENSIONESLEY100, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
al10_035 = get_value(unidad1_PENSIONESLEY100, "Cedidas Reaseguro interior (resta)")
al11_040 = get_value(unidad1_PENSIONESLEY100, "Cedidas Reaseguro exterior (resta)")
al12_045 = get_value(unidad1_PENSIONESLEY100, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
al13_050 = get_value(unidad1_PENSIONESLEY100, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular al14_999
al14_999 = (al4_005 + al5_010 - al6_015 + al7_020 + al8_025 - al9_030 - al10_035 - al11_040 + al12_045 + al13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=38, value=al14_999)


<Cell 'Hoja1'.AL14>

##  37 PENSIONES VOLUNTARIAS UNIDAD 1 

In [28]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 37
unidad1_PENSIONESVOLUN = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 37)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_PENSIONESVOLUN['TOTAL'] = unidad1_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_PENSIONESVOLUN, concept, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am4_005 = get_value(unidad1_PENSIONESVOLUN, "Primas Emitidas Directas")
am5_010 = get_value(unidad1_PENSIONESVOLUN, "Primas aceptadas en coaseguro")
am6_015 = get_value(unidad1_PENSIONESVOLUN, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
am7_020 = get_value(unidad1_PENSIONESVOLUN, "Aceptadas Reaseguro interior")
am8_025 = get_value(unidad1_PENSIONESVOLUN, "Aceptadas Reaseguro exterior")
am9_030 = get_value(unidad1_PENSIONESVOLUN, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
am10_035 = get_value(unidad1_PENSIONESVOLUN, "Cedidas Reaseguro interior (resta)")
am11_040 = get_value(unidad1_PENSIONESVOLUN, "Cedidas Reaseguro exterior (resta)")
am12_045 = get_value(unidad1_PENSIONESVOLUN, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
am13_050 = get_value(unidad1_PENSIONESVOLUN, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular am14_999
am14_999 = (am4_005 + am5_010 - am6_015 + am7_020 + am8_025 - am9_030 - am10_035 - am11_040 + am12_045 + am13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=39, value=am14_999)


<Cell 'Hoja1'.AM14>

##   38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 1 

In [29]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 38
unidad1_PENSIONESCONMU = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 38)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_PENSIONESCONMU['TOTAL'] = unidad1_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_PENSIONESCONMU, concept, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an4_005 = get_value(unidad1_PENSIONESCONMU, "Primas Emitidas Directas")
an5_010 = get_value(unidad1_PENSIONESCONMU, "Primas aceptadas en coaseguro")
an6_015 = get_value(unidad1_PENSIONESCONMU, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
an7_020 = get_value(unidad1_PENSIONESCONMU, "Aceptadas Reaseguro interior")
an8_025 = get_value(unidad1_PENSIONESCONMU, "Aceptadas Reaseguro exterior")
an9_030 = get_value(unidad1_PENSIONESCONMU, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
an10_035 = get_value(unidad1_PENSIONESCONMU, "Cedidas Reaseguro interior (resta)")
an11_040 = get_value(unidad1_PENSIONESCONMU, "Cedidas Reaseguro exterior (resta)")
an12_045 = get_value(unidad1_PENSIONESCONMU, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
an13_050 = get_value(unidad1_PENSIONESCONMU, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular an14_999
an14_999 = (an4_005 + an5_010 - an6_015 + an7_020 + an8_025 - an9_030 - an10_035 - an11_040 + an12_045 + an13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=40, value=an14_999)


<Cell 'Hoja1'.AN14>

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 1 

In [30]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y COD_RAMO es 39
unidad1_PATRIAUTONOMO = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 39)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_PATRIAUTONOMO['TOTAL'] = unidad1_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_PATRIAUTONOMO, concept, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao4_005 = get_value(unidad1_PATRIAUTONOMO, "Primas Emitidas Directas")
ao5_010 = get_value(unidad1_PATRIAUTONOMO, "Primas aceptadas en coaseguro")
ao6_015 = get_value(unidad1_PATRIAUTONOMO, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ao7_020 = get_value(unidad1_PATRIAUTONOMO, "Aceptadas Reaseguro interior")
ao8_025 = get_value(unidad1_PATRIAUTONOMO, "Aceptadas Reaseguro exterior")
ao9_030 = get_value(unidad1_PATRIAUTONOMO, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ao10_035 = get_value(unidad1_PATRIAUTONOMO, "Cedidas Reaseguro interior (resta)")
ao11_040 = get_value(unidad1_PATRIAUTONOMO, "Cedidas Reaseguro exterior (resta)")
ao12_045 = get_value(unidad1_PATRIAUTONOMO, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ao13_050 = get_value(unidad1_PATRIAUTONOMO, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ao14_999
ao14_999 = (ao4_005 + ao5_010 - ao6_015 + ao7_020 + ao8_025 - ao9_030 - ao10_035 - ao11_040 + ao12_045 + ao13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=41, value=ao14_999)


<Cell 'Hoja1'.AO14>

## 40 RENTAS VOLUNTARIAS UNIDAD 1

In [31]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y 'COD_RAMO' es 40
unidad1_RENTASVOLUN = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 40)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_RENTASVOLUN['TOTAL'] = unidad1_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_RENTASVOLUN, concept, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap4_005 = get_value(unidad1_RENTASVOLUN, "Primas Emitidas Directas")
ap5_010 = get_value(unidad1_RENTASVOLUN, "Primas aceptadas en coaseguro")
ap6_015 = get_value(unidad1_RENTASVOLUN, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
ap7_020 = get_value(unidad1_RENTASVOLUN, "Aceptadas Reaseguro interior")
ap8_025 = get_value(unidad1_RENTASVOLUN, "Aceptadas Reaseguro exterior")
ap9_030 = get_value(unidad1_RENTASVOLUN, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
ap10_035 = get_value(unidad1_RENTASVOLUN, "Cedidas Reaseguro interior (resta)")
ap11_040 = get_value(unidad1_RENTASVOLUN, "Cedidas Reaseguro exterior (resta)")
ap12_045 = get_value(unidad1_RENTASVOLUN, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
ap13_050 = get_value(unidad1_RENTASVOLUN, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular ap14_999
ap14_999 = (ap4_005 + ap5_010 - ap6_015 + ap7_020 + ap8_025 - ap9_030 - ap10_035 - ap11_040 + ap12_045 + ap13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=42, value=ap14_999)


<Cell 'Hoja1'.AP14>

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 1

In [32]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y 'COD_RAMO' es 41
unidad1_BENECONOMICOS = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 41)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_BENECONOMICOS['TOTAL'] = unidad1_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_BENECONOMICOS, concept, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq4_005 = get_value(unidad1_BENECONOMICOS, "Primas Emitidas Directas")
aq5_010 = get_value(unidad1_BENECONOMICOS, "Primas aceptadas en coaseguro")
aq6_015 = get_value(unidad1_BENECONOMICOS, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
aq7_020 = get_value(unidad1_BENECONOMICOS, "Aceptadas Reaseguro interior")
aq8_025 = get_value(unidad1_BENECONOMICOS, "Aceptadas Reaseguro exterior")
aq9_030 = get_value(unidad1_BENECONOMICOS, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
aq10_035 = get_value(unidad1_BENECONOMICOS, "Cedidas Reaseguro interior (resta)")
aq11_040 = get_value(unidad1_BENECONOMICOS, "Cedidas Reaseguro exterior (resta)")
aq12_045 = get_value(unidad1_BENECONOMICOS, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
aq13_050 = get_value(unidad1_BENECONOMICOS, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular aq14_999
aq14_999 = (aq4_005 + aq5_010 - aq6_015 + aq7_020 + aq8_025 - aq9_030 - aq10_035 - aq11_040 + aq12_045 + aq13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=43, value=aq14_999)


<Cell 'Hoja1'.AQ14>

##  43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 1  

In [33]:
# Filtrar y agrupar los datos donde 'Unidad' es 1 y 'COD_RAMO' es 43
unidad1_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 1) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_OPERNOASIGNABLES['TOTAL'] = unidad1_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas Emitidas Directas", 4),
    ("Primas aceptadas en coaseguro", 5),
    ("Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)", 6),
    ("Aceptadas Reaseguro interior", 7),
    ("Aceptadas Reaseguro exterior", 8),
    ("Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)", 9),
    ("Cedidas Reaseguro interior (resta)", 10),
    ("Cedidas Reaseguro exterior (resta)", 11),
    ("Cancelaciones y/o anulaciones primas de reaseguro cedido", 12),
    ("Primas aceptadas y/o cedidas por Cámara de Compensación SOAT", 13)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_OPERNOASIGNABLES, concept, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as4_005 = get_value(unidad1_OPERNOASIGNABLES, "Primas Emitidas Directas")
as5_010 = get_value(unidad1_OPERNOASIGNABLES, "Primas aceptadas en coaseguro")
as6_015 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas emitidas directas y coaseguro (resta)")
as7_020 = get_value(unidad1_OPERNOASIGNABLES, "Aceptadas Reaseguro interior")
as8_025 = get_value(unidad1_OPERNOASIGNABLES, "Aceptadas Reaseguro exterior")
as9_030 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas de reaseguro aceptado (resta)")
as10_035 = get_value(unidad1_OPERNOASIGNABLES, "Cedidas Reaseguro interior (resta)")
as11_040 = get_value(unidad1_OPERNOASIGNABLES, "Cedidas Reaseguro exterior (resta)")
as12_045 = get_value(unidad1_OPERNOASIGNABLES, "Cancelaciones y/o anulaciones primas de reaseguro cedido")
as13_050 = get_value(unidad1_OPERNOASIGNABLES, "Primas aceptadas y/o cedidas por Cámara de Compensación SOAT")

# Aplicar la fórmula específica para calcular as14_999
as14_999 = (as4_005 + as5_010 - as6_015 + as7_020 + as8_025 - as9_030 - as10_035 - as11_040 + as12_045 + as13_050)

# Escribir el total general en Excel
ws.cell(row=14, column=45, value=as14_999)


<Cell 'Hoja1'.AS14>

# UNIDAD DE CAPTURA 2   

##  CONSULTA DE TOTALES RESERVAS TECNICAS Y MATEMATICAS UNIDAD 2

In [34]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales
unidad2_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad2_subtotal['TOTAL'] = unidad2_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Agrupar los datos por 'CUENTA' y 'COD_RAMO' para obtener los totales
unidad2_subtotal_a = bd.groupby(['CUENTA', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_subtotal_a['TOTAL'] = unidad2_subtotal_a['TOTAL'].abs()

# Definir la función para escribir en Excel
def write_to_excel(data_frame, column_name, value_name, start_row, start_col=3, cod_ramo=None):
    if cod_ramo:
        row = data_frame[(data_frame[column_name] == value_name) & (data_frame['COD_RAMO'] == cod_ramo)]
    else:
        row = data_frame[data_frame[column_name] == value_name]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Escribir cada valor en la hoja de Excel
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica de prima no devengada", 17)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica por insuficiencia de prima", 18)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática", 19)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales (resta)", 20)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs (resta)", 21)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Seguro de Vida Ahorro con Participa.", 22)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva de Insuficiencia de Activos", 23)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica de prima no devengada (resta)", 25)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica por insuficiencia de prima (resta)", 26)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática (resta)", 27)
write_to_excel(unidad2_subtotal_a, 'CUENTA', "6211001013", 28, cod_ramo=35)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs", 29)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva de Insuficiencia de Activos (resta)", 31)

# Definir la función get_value
def get_value(df, column_name, value_name, cod_ramo=None):
    if cod_ramo:
        row = df[(df[column_name] == value_name) & (df['COD_RAMO'] == cod_ramo)]
    else:
        row = df[df[column_name] == value_name]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
c17_006 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica de prima no devengada")
c18_007 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica por insuficiencia de prima")
c19_010 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática")
c20_015 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales (resta)")
c21_017 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs (resta)")
c22_020 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Seguro de Vida Ahorro con Participa.")
c23_022 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva de Insuficiencia de Activos")
c25_026 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica de prima no devengada (resta)")
c26_027 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica por insuficiencia de prima (resta)")
c27_030 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática (resta)")
c28_035 = get_value(unidad2_subtotal_a, 'CUENTA', "6211001013", cod_ramo=35)
c29_037 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs")
c30_040 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
c31_045 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular el total general según la fórmula dada
c32_999 = (c17_006 + c18_007 + c19_010 - c20_015 - c21_017 + c22_020 + c23_022 - c25_026 - c26_027 - c27_030 + c28_035 + c29_037 - c30_040 - c31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=3).value = c32_999

## CONSULTA SUBTOTAL - RAMOS RESERVAS TECNICAS Y MATEMATICAS UNIDAD 2  

In [35]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales
unidad2_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad2_subtotal['TOTAL'] = unidad2_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Agrupar los datos por 'CUENTA' y 'COD_RAMO' para obtener los totales
unidad2_subtotal_a = bd.groupby(['CUENTA', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_subtotal_a['TOTAL'] = unidad2_subtotal_a['TOTAL'].abs()

# Definir la función para escribir en Excel
def write_to_excel(data_frame, column_name, value_name, start_row, start_col=4, cod_ramo=None):
    if cod_ramo:
        row = data_frame[(data_frame[column_name] == value_name) & (data_frame['COD_RAMO'] == cod_ramo)]
    else:
        row = data_frame[data_frame[column_name] == value_name]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Escribir cada valor en la hoja de Excel
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica de prima no devengada", 17)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica por insuficiencia de prima", 18)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática", 19)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales (resta)", 20)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs (resta)", 21)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Seguro de Vida Ahorro con Participa.", 22)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva de Insuficiencia de Activos", 23)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica de prima no devengada (resta)", 25)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica por insuficiencia de prima (resta)", 26)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática (resta)", 27)
write_to_excel(unidad2_subtotal_a, 'CUENTA', "6211001013", 28, cod_ramo=35)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs", 29)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30)
write_to_excel(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva de Insuficiencia de Activos (resta)", 31)

# Definir la función get_value
def get_value(df, column_name, value_name, cod_ramo=None):
    if cod_ramo:
        row = df[(df[column_name] == value_name) & (df['COD_RAMO'] == cod_ramo)]
    else:
        row = df[df[column_name] == value_name]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
d17_006 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica de prima no devengada")
d18_007 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Técnica por insuficiencia de prima")
d19_010 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática")
d20_015 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales (resta)")
d21_017 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs (resta)")
d22_020 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva Seguro de Vida Ahorro con Participa.")
d23_022 = get_value(unidad2_subtotal, 'CONCEPTO', "Liberación Reserva de Insuficiencia de Activos")
d25_026 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica de prima no devengada (resta)")
d26_027 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Técnica por insuficiencia de prima (resta)")
d27_030 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática (resta)")
d28_035 = get_value(unidad2_subtotal_a, 'CUENTA', "6211001013", cod_ramo=35)
d29_037 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs")
d30_040 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
d31_045 = get_value(unidad2_subtotal, 'CONCEPTO', "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular el total general según la fórmula dada
d32_999 = (d17_006 + d18_007 + d19_010 - d20_015 - d21_017 + d22_020 + d23_022 - d25_026 - d26_027 - d27_030 + d28_035 + d29_037 - d30_040 - d31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=4).value = d32_999


##   26 EXEQUIAS UNIDAD 2 

In [36]:
# Filtrar y agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_EXEQUIAS
unidad2_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_EXEQUIAS['TOTAL'] = unidad2_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 26)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_EXEQUIAS, concept, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 26)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab17_006 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Técnica de prima no devengada")
ab18_007 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Técnica por insuficiencia de prima")
ab19_010 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Matemática")
ab20_015 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ab21_017 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Matemática BEPs (resta)")
ab22_020 = get_value(unidad2_EXEQUIAS, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ab23_022 = get_value(unidad2_EXEQUIAS, "Liberación Reserva de Insuficiencia de Activos")
ab25_026 = get_value(unidad2_EXEQUIAS, "Constitución Reserva Técnica de prima no devengada (resta)")
ab26_027 = get_value(unidad2_EXEQUIAS, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ab27_030 = get_value(unidad2_EXEQUIAS, "Constitución Reserva matemática (resta)")
ab28_035 = get_value(unidad2_EXEQUIAS, "Constitución Reserva matemática Riesgos Laborales")
ab29_037 = get_value(unidad2_EXEQUIAS, "Constitución Reserva matemática BEPs")
ab30_040 = get_value(unidad2_EXEQUIAS, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ab31_045 = get_value(unidad2_EXEQUIAS, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Aplicar la fórmula específica para calcular ab32_999
ab32_999 = (ab17_006 + ab18_007 + ab19_010 - ab20_015 - ab21_017 + ab22_020 + ab23_022 - ab25_026 - ab26_027 - ab27_030 + ab28_035 + ab29_037 - ab30_040 - ab31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=28, value=ab32_999)


<Cell 'Hoja1'.AB32>

## 27 ACCIDENTES PERSONALES UNIDAD 2 

In [37]:
# Filtrar y agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_ACCPERSONALES
unidad2_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_ACCPERSONALES['TOTAL'] = unidad2_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 27)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_ACCPERSONALES, concept, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 27)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac17_006 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Técnica de prima no devengada")
ac18_007 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Técnica por insuficiencia de prima")
ac19_010 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Matemática")
ac20_015 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ac21_017 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Matemática BEPs (resta)")
ac22_020 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ac23_022 = get_value(unidad2_ACCPERSONALES, "Liberación Reserva de Insuficiencia de Activos")
ac25_026 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva Técnica de prima no devengada (resta)")
ac26_027 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ac27_030 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva matemática (resta)")
ac28_035 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva matemática Riesgos Laborales")
ac29_037 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva matemática BEPs")
ac30_040 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ac31_045 = get_value(unidad2_ACCPERSONALES, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Aplicar la fórmula específica para calcular aC32_999
ac32_999 = (ac17_006 + ac18_007 + ac19_010 - ac20_015 - ac21_017 + ac22_020 + ac23_022 - ac25_026 - ac26_027 - ac27_030 + ac28_035 + ac29_037 - ac30_040 - ac31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=29, value=ac32_999)


<Cell 'Hoja1'.AC32>

## 28 COLECTIVO VIDA UNIDAD 2

In [38]:
# Filtrar y agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_COLECVI
unidad2_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_COLECVI['TOTAL'] = unidad2_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Definir la función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 28)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_COLECVI, concept, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 28)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad17_006 = get_value(unidad2_COLECVI, "Liberación Reserva Técnica de prima no devengada")
ad18_007 = get_value(unidad2_COLECVI, "Liberación Reserva Técnica por insuficiencia de prima")
ad19_010 = get_value(unidad2_COLECVI, "Liberación Reserva Matemática")
ad20_015 = get_value(unidad2_COLECVI, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ad21_017 = get_value(unidad2_COLECVI, "Liberación Reserva Matemática BEPs (resta)")
ad22_020 = get_value(unidad2_COLECVI, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ad23_022 = get_value(unidad2_COLECVI, "Liberación Reserva de Insuficiencia de Activos")
ad25_026 = get_value(unidad2_COLECVI, "Constitución Reserva Técnica de prima no devengada (resta)")
ad26_027 = get_value(unidad2_COLECVI, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ad27_030 = get_value(unidad2_COLECVI, "Constitución Reserva matemática (resta)")
ad28_035 = get_value(unidad2_COLECVI, "Constitución Reserva matemática Riesgos Laborales")
ad29_037 = get_value(unidad2_COLECVI, "Constitución Reserva matemática BEPs")
ad30_040 = get_value(unidad2_COLECVI, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ad31_045 = get_value(unidad2_COLECVI, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Aplicar la fórmula específica para calcular ad32_999
ad32_999 = (ad17_006 + ad18_007 + ad19_010 - ad20_015 - ad21_017 + ad22_020 + ad23_022 - ad25_026 - ad26_027 - ad27_030 + ad28_035 + ad29_037 - ad30_040 - ad31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=30, value=ad32_999)


<Cell 'Hoja1'.AD32>

## 29 EDUCATIVO  UNIDAD 2 

In [39]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_EDUCATIVO
unidad2_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_EDUCATIVO['TOTAL'] = unidad2_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 29)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_EDUCATIVO, concept, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 29)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae17_006 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Técnica de prima no devengada")
ae18_007 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Técnica por insuficiencia de prima")
ae19_010 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Matemática")
ae20_015 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ae21_017 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Matemática BEPs (resta)")
ae22_020 = get_value(unidad2_EDUCATIVO, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ae23_022 = get_value(unidad2_EDUCATIVO, "Liberación Reserva de Insuficiencia de Activos")
ae25_026 = get_value(unidad2_EDUCATIVO, "Constitución Reserva Técnica de prima no devengada (resta)")
ae26_027 = get_value(unidad2_EDUCATIVO, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ae27_030 = get_value(unidad2_EDUCATIVO, "Constitución Reserva matemática (resta)")
ae28_035 = get_value(unidad2_EDUCATIVO, "Constitución Reserva matemática Riesgos Laborales")
ae29_037 = get_value(unidad2_EDUCATIVO, "Constitución Reserva matemática BEPs")
ae30_040 = get_value(unidad2_EDUCATIVO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ae31_045 = get_value(unidad2_EDUCATIVO, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ae32_999 usando la fórmula especificada
ae32_999 = (ae17_006 + ae18_007 + ae19_010 - ae20_015 - ae21_017 + ae22_020 + ae23_022 - ae25_026 - ae26_027 - ae27_030 + ae28_035 + ae29_037 - ae30_040 - ae31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=31, value=ae32_999)


<Cell 'Hoja1'.AE32>

## 30  VIDA GRUPO  UNIDAD 2  

In [40]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_VIDAGRUPO
unidad2_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_VIDAGRUPO['TOTAL'] = unidad2_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 30)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_VIDAGRUPO, concept, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 30)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af17_006 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Técnica de prima no devengada")
af18_007 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Técnica por insuficiencia de prima")
af19_010 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Matemática")
af20_015 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Matemática Riesgos Laborales (resta)")
af21_017 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Matemática BEPs (resta)")
af22_020 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
af23_022 = get_value(unidad2_VIDAGRUPO, "Liberación Reserva de Insuficiencia de Activos")
af25_026 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva Técnica de prima no devengada (resta)")
af26_027 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
af27_030 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva matemática (resta)")
af28_035 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva matemática Riesgos Laborales")
af29_037 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva matemática BEPs")
af30_040 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
af31_045 = get_value(unidad2_VIDAGRUPO, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular af32_999 usando la fórmula especificada
af32_999 = (af17_006 + af18_007 + af19_010 - af20_015 - af21_017 + af22_020 + af23_022 - af25_026 - af26_027 - af27_030 + af28_035 + af29_037 - af30_040 - af31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=32, value=af32_999)


<Cell 'Hoja1'.AF32>

## 31 SALUD UNIDAD 2 

In [41]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_SALUD
unidad2_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_SALUD['TOTAL'] = unidad2_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 31)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_SALUD, concept, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 31)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag17_006 = get_value(unidad2_SALUD, "Liberación Reserva Técnica de prima no devengada")
ag18_007 = get_value(unidad2_SALUD, "Liberación Reserva Técnica por insuficiencia de prima")
ag19_010 = get_value(unidad2_SALUD, "Liberación Reserva Matemática")
ag20_015 = get_value(unidad2_SALUD, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ag21_017 = get_value(unidad2_SALUD, "Liberación Reserva Matemática BEPs (resta)")
ag22_020 = get_value(unidad2_SALUD, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ag23_022 = get_value(unidad2_SALUD, "Liberación Reserva de Insuficiencia de Activos")
ag25_026 = get_value(unidad2_SALUD, "Constitución Reserva Técnica de prima no devengada (resta)")
ag26_027 = get_value(unidad2_SALUD, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ag27_030 = get_value(unidad2_SALUD, "Constitución Reserva matemática (resta)")
ag28_035 = get_value(unidad2_SALUD, "Constitución Reserva matemática Riesgos Laborales")
ag29_037 = get_value(unidad2_SALUD, "Constitución Reserva matemática BEPs")
ag30_040 = get_value(unidad2_SALUD, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ag31_045 = get_value(unidad2_SALUD, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ag32_999 usando la fórmula especificada
ag32_999 = (ag17_006 + ag18_007 + ag19_010 - ag20_015 - ag21_017 + ag22_020 + ag23_022 - ag25_026 - ag26_027 - ag27_030 + ag28_035 + ag29_037 - ag30_040 - ag31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=33, value=ag32_999)


<Cell 'Hoja1'.AG32>

##  32 ENFERMEDADES DE ALTO COSTO UNIDAD 2 

In [42]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_ENFALTOCOSTO
unidad2_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_ENFALTOCOSTO['TOTAL'] = unidad2_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 32)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_ENFALTOCOSTO, concept, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 32)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah17_006 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Técnica de prima no devengada")
ah18_007 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Técnica por insuficiencia de prima")
ah19_010 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Matemática")
ah20_015 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ah21_017 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Matemática BEPs (resta)")
ah22_020 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ah23_022 = get_value(unidad2_ENFALTOCOSTO, "Liberación Reserva de Insuficiencia de Activos")
ah25_026 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva Técnica de prima no devengada (resta)")
ah26_027 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ah27_030 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva matemática (resta)")
ah28_035 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva matemática Riesgos Laborales")
ah29_037 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva matemática BEPs")
ah30_040 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ah31_045 = get_value(unidad2_ENFALTOCOSTO, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ah32_999 usando la fórmula especificada
ah32_999 = (ah17_006 + ah18_007 + ah19_010 - ah20_015 - ah21_017 + ah22_020 + ah23_022 - ah25_026 - ah26_027 - ah27_030 + ah28_035 + ah29_037 - ah30_040 - ah31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=34, value=ah32_999)


<Cell 'Hoja1'.AH32>

## 33 VIDA INDIVIDUAL UNIDAD 2

In [43]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_VIDAINDIVI
unidad2_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_VIDAINDIVI['TOTAL'] = unidad2_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 33)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_VIDAINDIVI, concept, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 33)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai17_006 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Técnica de prima no devengada")
ai18_007 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Técnica por insuficiencia de prima")
ai19_010 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Matemática")
ai20_015 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ai21_017 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Matemática BEPs (resta)")
ai22_020 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ai23_022 = get_value(unidad2_VIDAINDIVI, "Liberación Reserva de Insuficiencia de Activos")
ai25_026 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva Técnica de prima no devengada (resta)")
ai26_027 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ai27_030 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva matemática (resta)")
ai28_035 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva matemática Riesgos Laborales")
ai29_037 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva matemática BEPs")
ai30_040 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ai31_045 = get_value(unidad2_VIDAINDIVI, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ai32_999 usando la fórmula especificada
ai32_999 = (ai17_006 + ai18_007 + ai19_010 - ai20_015 - ai21_017 + ai22_020 + ai23_022 - ai25_026 - ai26_027 - ai27_030 + ai28_035 + ai29_037 - ai30_040 - ai31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=35, value=ai32_999)


<Cell 'Hoja1'.AI32>

## 34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 2

In [44]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_PREVIDINVAL
unidad2_PREVIDINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_PREVIDINVAL['TOTAL'] = unidad2_PREVIDINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 34)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_PREVIDINVAL, concept, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 34)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj17_006 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Técnica de prima no devengada")
aj18_007 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Técnica por insuficiencia de prima")
aj19_010 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Matemática")
aj20_015 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Matemática Riesgos Laborales (resta)")
aj21_017 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Matemática BEPs (resta)")
aj22_020 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
aj23_022 = get_value(unidad2_PREVIDINVAL, "Liberación Reserva de Insuficiencia de Activos")
aj25_026 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva Técnica de prima no devengada (resta)")
aj26_027 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
aj27_030 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva matemática (resta)")
aj28_035 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva matemática Riesgos Laborales")
aj29_037 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva matemática BEPs")
aj30_040 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
aj31_045 = get_value(unidad2_PREVIDINVAL, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular aj32_999 usando la fórmula especificada
aj32_999 = (aj17_006 + aj18_007 + aj19_010 - aj20_015 - aj21_017 + aj22_020 + aj23_022 - aj25_026 - aj26_027 - aj27_030 + aj28_035 + aj29_037 - aj30_040 - aj31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=36, value=aj32_999)


<Cell 'Hoja1'.AJ32>

## 35 RIESGOS LABORALES UNIDAD 2   

In [45]:
# Agrupar los datos según el concepto y el ramo
unidad2_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad2_RIESGOSLAB_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Función para escribir valores en Excel
def write_to_excel(data_frame, col, row, column_name='TOTAL'):
    if data_frame.empty:
        total = 0
    else:
        total = abs(data_frame[column_name].iloc[0])
    ws.cell(row=row, column=col).value = total

# Procesamiento similar al código R
ak17_006 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Técnica de prima no devengada") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak17_006, 37, 17)

ak18_007 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Técnica por insuficiencia de prima") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak18_007, 37, 18)

ak19_010 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Matemática") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak19_010, 37, 19)

ak20_015 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak20_015, 37, 20)

ak21_017 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Matemática BEPs (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak21_017, 37, 21)

ak22_020 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Seguro de Vida Ahorro con Participa.") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak22_020, 37, 22)

ak23_022 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva de Insuficiencia de Activos") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak23_022, 37, 23)

ak25_026 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva Técnica de prima no devengada (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak25_026, 37, 25)

ak26_027 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva Técnica por insuficiencia de prima (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak26_027, 37, 26)

ak27_030 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva matemática (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak27_030, 37, 27)

# Aquí se hace el filtro basado en CUENTA
ak28_035 = unidad2_RIESGOSLAB_a[(unidad2_RIESGOSLAB_a['CUENTA'] == "6211001013") & (unidad2_RIESGOSLAB_a['COD_RAMO'] == 35)]
write_to_excel(ak28_035, 37, 28, 'TOTAL')

ak29_037 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva matemática BEPs") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak29_037, 37, 29)

ak30_040 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak30_040, 37, 30)

ak31_045 = unidad2_RIESGOSLAB[(unidad2_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva de Insuficiencia de Activos (resta)") & (unidad2_RIESGOSLAB['COD_RAMO'] == 35)]
write_to_excel(ak31_045, 37, 31)

# Calcular ak32_999
ak32_999 = ak17_006['TOTAL'].sum() + ak18_007['TOTAL'].sum() + ak19_010['TOTAL'].sum() - ak20_015['TOTAL'].sum() - ak21_017['TOTAL'].sum() + \
           ak22_020['TOTAL'].sum() + ak23_022['TOTAL'].sum() - ak25_026['TOTAL'].sum() - ak26_027['TOTAL'].sum() - ak27_030['TOTAL'].sum() + \
           ak28_035['TOTAL'].sum() + ak29_037['TOTAL'].sum() - ak30_040['TOTAL'].sum() - ak31_045['TOTAL'].sum()

df_ak32_999 = pd.DataFrame({'total': [ak32_999]})
write_to_excel(df_ak32_999, 37, 32, 'total')


##   36  PENSIONES LEY 100 UNIDAD 2  

In [46]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_PENSIONESLEY100
unidad2_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_PENSIONESLEY100['TOTAL'] = unidad2_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 36)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_PENSIONESLEY100, concept, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 36)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al17_006 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Técnica de prima no devengada")
al18_007 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Técnica por insuficiencia de prima")
al19_010 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Matemática")
al20_015 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Matemática Riesgos Laborales (resta)")
al21_017 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Matemática BEPs (resta)")
al22_020 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
al23_022 = get_value(unidad2_PENSIONESLEY100, "Liberación Reserva de Insuficiencia de Activos")
al25_026 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva Técnica de prima no devengada (resta)")
al26_027 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
al27_030 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva matemática (resta)")
al28_035 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva matemática Riesgos Laborales")
al29_037 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva matemática BEPs")
al30_040 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
al31_045 = get_value(unidad2_PENSIONESLEY100, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular al32_999 usando la fórmula especificada
al32_999 = (al17_006 + al18_007 + al19_010 - al20_015 - al21_017 + al22_020 + al23_022 - al25_026 - al26_027 - al27_030 + al28_035 + al29_037 - al30_040 - al31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=38, value=al32_999)


<Cell 'Hoja1'.AL32>

##  37 PENSIONES VOLUNTARIAS UNIDAD 2 

In [47]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_PENSIONESVOLUN
unidad2_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_PENSIONESVOLUN['TOTAL'] = unidad2_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 37)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_PENSIONESVOLUN, concept, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 37)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am17_006 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Técnica de prima no devengada")
am18_007 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Técnica por insuficiencia de prima")
am19_010 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Matemática")
am20_015 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Matemática Riesgos Laborales (resta)")
am21_017 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Matemática BEPs (resta)")
am22_020 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
am23_022 = get_value(unidad2_PENSIONESVOLUN, "Liberación Reserva de Insuficiencia de Activos")
am25_026 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva Técnica de prima no devengada (resta)")
am26_027 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
am27_030 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva matemática (resta)")
am28_035 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva matemática Riesgos Laborales")
am29_037 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva matemática BEPs")
am30_040 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
am31_045 = get_value(unidad2_PENSIONESVOLUN, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular am32_999 usando la fórmula especificada
am32_999 = (am17_006 + am18_007 + am19_010 - am20_015 - am21_017 + am22_020 + am23_022 - am25_026 - am26_027 - am27_030 + am28_035 + am29_037 - am30_040 - am31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=39, value=am32_999)


<Cell 'Hoja1'.AM32>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 2 

In [48]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_PENSIONESCONMU
unidad2_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_PENSIONESCONMU['TOTAL'] = unidad2_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 38)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_PENSIONESCONMU, concept, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 38)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an17_006 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Técnica de prima no devengada")
an18_007 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Técnica por insuficiencia de prima")
an19_010 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Matemática")
an20_015 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Matemática Riesgos Laborales (resta)")
an21_017 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Matemática BEPs (resta)")
an22_020 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
an23_022 = get_value(unidad2_PENSIONESCONMU, "Liberación Reserva de Insuficiencia de Activos")
an25_026 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva Técnica de prima no devengada (resta)")
an26_027 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
an27_030 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva matemática (resta)")
an28_035 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva matemática Riesgos Laborales")
an29_037 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva matemática BEPs")
an30_040 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
an31_045 = get_value(unidad2_PENSIONESCONMU, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular an32_999 usando la fórmula especificada
an32_999 = (an17_006 + an18_007 + an19_010 - an20_015 - an21_017 + an22_020 + an23_022 - an25_026 - an26_027 - an27_030 + an28_035 + an29_037 - an30_040 - an31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=40, value=an32_999)


<Cell 'Hoja1'.AN32>

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 2 

In [49]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_PATRIAUTONOMO
unidad2_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_PATRIAUTONOMO['TOTAL'] = unidad2_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 39)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_PATRIAUTONOMO, concept, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 39)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao17_006 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Técnica de prima no devengada")
ao18_007 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Técnica por insuficiencia de prima")
ao19_010 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Matemática")
ao20_015 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ao21_017 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Matemática BEPs (resta)")
ao22_020 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ao23_022 = get_value(unidad2_PATRIAUTONOMO, "Liberación Reserva de Insuficiencia de Activos")
ao25_026 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva Técnica de prima no devengada (resta)")
ao26_027 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ao27_030 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva matemática (resta)")
ao28_035 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva matemática Riesgos Laborales")
ao29_037 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva matemática BEPs")
ao30_040 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ao31_045 = get_value(unidad2_PATRIAUTONOMO, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ao32_999 usando la fórmula especificada
ao32_999 = (ao17_006 + ao18_007 + ao19_010 - ao20_015 - ao21_017 + ao22_020 + ao23_022 - ao25_026 - ao26_027 - ao27_030 + ao28_035 + ao29_037 - ao30_040 - ao31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=41, value=ao32_999)


<Cell 'Hoja1'.AO32>

## 40 RENTAS VOLUNTARIAS UNIDAD 2 

In [50]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_RENTASVOLUN
unidad2_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_RENTASVOLUN['TOTAL'] = unidad2_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 40)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_RENTASVOLUN, concept, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 40)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap17_006 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Técnica de prima no devengada")
ap18_007 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Técnica por insuficiencia de prima")
ap19_010 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Matemática")
ap20_015 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Matemática Riesgos Laborales (resta)")
ap21_017 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Matemática BEPs (resta)")
ap22_020 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
ap23_022 = get_value(unidad2_RENTASVOLUN, "Liberación Reserva de Insuficiencia de Activos")
ap25_026 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva Técnica de prima no devengada (resta)")
ap26_027 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
ap27_030 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva matemática (resta)")
ap28_035 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva matemática Riesgos Laborales")
ap29_037 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva matemática BEPs")
ap30_040 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
ap31_045 = get_value(unidad2_RENTASVOLUN, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular ap32_999 usando la fórmula especificada
ap32_999 = (ap17_006 + ap18_007 + ap19_010 - ap20_015 - ap21_017 + ap22_020 + ap23_022 - ap25_026 - ap26_027 - ap27_030 + ap28_035 + ap29_037 - ap30_040 - ap31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=42, value=ap32_999)


<Cell 'Hoja1'.AP32>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 2 

In [51]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad1_BENECONOMICOS
unidad1_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad1_BENECONOMICOS['TOTAL'] = unidad1_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 41)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad1_BENECONOMICOS, concept, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 41)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq17_006 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Técnica de prima no devengada")
aq18_007 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Técnica por insuficiencia de prima")
aq19_010 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Matemática")
aq20_015 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Matemática Riesgos Laborales (resta)")
aq21_017 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Matemática BEPs (resta)")
aq22_020 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
aq23_022 = get_value(unidad1_BENECONOMICOS, "Liberación Reserva de Insuficiencia de Activos")
aq25_026 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva Técnica de prima no devengada (resta)")
aq26_027 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
aq27_030 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva matemática (resta)")
aq28_035 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva matemática Riesgos Laborales")
aq29_037 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva matemática BEPs")
aq30_040 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
aq31_045 = get_value(unidad1_BENECONOMICOS, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular aq32_999 usando la fórmula especificada
aq32_999 = (aq17_006 + aq18_007 + aq19_010 - aq20_015 - aq21_017 + aq22_020 + aq23_022 - aq25_026 - aq26_027 - aq27_030 + aq28_035 + aq29_037 - aq30_040 - aq31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=43, value=aq32_999)


<Cell 'Hoja1'.AQ32>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 2

In [52]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad2_OPERNOASIGNABLES
unidad2_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad2_OPERNOASIGNABLES['TOTAL'] = unidad2_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 43)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Técnica de prima no devengada", 17),
    ("Liberación Reserva Técnica por insuficiencia de prima", 18),
    ("Liberación Reserva Matemática", 19),
    ("Liberación Reserva Matemática Riesgos Laborales (resta)", 20),
    ("Liberación Reserva Matemática BEPs (resta)", 21),
    ("Liberación Reserva Seguro de Vida Ahorro con Participa.", 22),
    ("Liberación Reserva de Insuficiencia de Activos", 23),
    ("Constitución Reserva Técnica de prima no devengada (resta)", 25),
    ("Constitución Reserva Técnica por insuficiencia de prima (resta)", 26),
    ("Constitución Reserva matemática (resta)", 27),
    ("Constitución Reserva matemática Riesgos Laborales", 28),
    ("Constitución Reserva matemática BEPs", 29),
    ("Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)", 30),
    ("Constitución Reserva de Insuficiencia de Activos (resta)", 31)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad2_OPERNOASIGNABLES, concept, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 43)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as17_006 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Técnica de prima no devengada")
as18_007 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Técnica por insuficiencia de prima")
as19_010 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática")
as20_015 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática Riesgos Laborales (resta)")
as21_017 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Matemática BEPs (resta)")
as22_020 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva Seguro de Vida Ahorro con Participa.")
as23_022 = get_value(unidad2_OPERNOASIGNABLES, "Liberación Reserva de Insuficiencia de Activos")
as25_026 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Técnica de prima no devengada (resta)")
as26_027 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Técnica por insuficiencia de prima (resta)")
as27_030 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática (resta)")
as28_035 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática Riesgos Laborales")
as29_037 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva matemática BEPs")
as30_040 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva Seguro de Vida Ahorro con Participa.(resta)")
as31_045 = get_value(unidad2_OPERNOASIGNABLES, "Constitución Reserva de Insuficiencia de Activos (resta)")

# Calcular as32_999 usando la fórmula especificada
as32_999 = (as17_006 + as18_007 + as19_010 - as20_015 - as21_017 + as22_020 + as23_022 - as25_026 - as26_027 - as27_030 + as28_035 + as29_037 - as30_040 - as31_045)

# Escribir el total general en Excel
ws.cell(row=32, column=45, value=as32_999)


<Cell 'Hoja1'.AS32>

#   UNIDAD DE CAPTURA 3  

## CONSULTA DE TOTALES PRIMAS DEVENGADAS  UNIDAD 3

In [53]:
# Sumar c14_999 y c32_999, y escribir en la celda correspondiente

c34_999 = c14_999 + c32_999
df_c34_999 = pd.DataFrame({'total': [c34_999]})
if df_c34_999.empty or c34_999 == 0:
    df_c34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=3).value = df_c34_999['total'].iloc[0]

# Repetir el mismo proceso para las otras variables
ab34_999 = ab14_999 + ab32_999
df_ab34_999 = pd.DataFrame({'total': [ab34_999]})
if df_ab34_999.empty or ab34_999 == 0:
    df_ab34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=28).value = df_ab34_999['total'].iloc[0]

ac34_999 = ac14_999 + ac32_999
df_ac34_999 = pd.DataFrame({'total': [ac34_999]})
if df_ac34_999.empty or ac34_999 == 0:
    df_ac34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=29).value = df_ac34_999['total'].iloc[0]

ad34_999 = ad14_999 + ad32_999
df_ad34_999 = pd.DataFrame({'total': [ad34_999]})
if df_ad34_999.empty or ad34_999 == 0:
    df_ad34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=30).value = df_ad34_999['total'].iloc[0]

ae34_999 = ae14_999 + ae32_999
df_ae34_999 = pd.DataFrame({'total': [ae34_999]})
if df_ae34_999.empty or ae34_999 == 0:
    df_ae34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=31).value = df_ae34_999['total'].iloc[0]

af34_999 = af14_999 + af32_999
df_af34_999 = pd.DataFrame({'total': [af34_999]})
if df_af34_999.empty or af34_999 == 0:
    df_af34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=32).value = df_af34_999['total'].iloc[0]

ag34_999 = ag14_999 + ag32_999
df_ag34_999 = pd.DataFrame({'total': [ag34_999]})
if df_ag34_999.empty or ag34_999 == 0:
    df_ag34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=33).value = df_ag34_999['total'].iloc[0]

ah34_999 = ah14_999 + ah32_999
df_ah34_999 = pd.DataFrame({'total': [ah34_999]})
if df_ah34_999.empty or ah34_999 == 0:
    df_ah34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=34).value = df_ah34_999['total'].iloc[0]

ai34_999 = ai14_999 + ai32_999
df_ai34_999 = pd.DataFrame({'total': [ai34_999]})
if df_ai34_999.empty or ai34_999 == 0:
    df_ai34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=35).value = df_ai34_999['total'].iloc[0]

aj34_999 = aj14_999 + aj32_999
df_aj34_999 = pd.DataFrame({'total': [aj34_999]})
if df_aj34_999.empty or aj34_999 == 0:
    df_aj34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=36).value = df_aj34_999['total'].iloc[0]

ak34_999 = ak14_999 + ak32_999
df_ak34_999 = pd.DataFrame({'total': [ak34_999]})
if df_ak34_999.empty or ak34_999 == 0:
    df_ak34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=37).value = df_ak34_999['total'].iloc[0]

al34_999 = al14_999 + al32_999
df_al34_999 = pd.DataFrame({'total': [al34_999]})
if df_al34_999.empty or al34_999 == 0:
    df_al34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=38).value = df_al34_999['total'].iloc[0]

am34_999 = am14_999 + am32_999
df_am34_999 = pd.DataFrame({'total': [am34_999]})
if df_am34_999.empty or am34_999 == 0:
    df_am34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=39).value = df_am34_999['total'].iloc[0]

an34_999 = an14_999 + an32_999
df_an34_999 = pd.DataFrame({'total': [an34_999]})
if df_an34_999.empty or an34_999 == 0:
    df_an34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=40).value = df_an34_999['total'].iloc[0]

ao34_999 = ao14_999 + ao32_999
df_ao34_999 = pd.DataFrame({'total': [ao34_999]})
if df_ao34_999.empty or ao34_999 == 0:
    df_ao34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=41).value = df_ao34_999['total'].iloc[0]

ap34_999 = ap14_999 + ap32_999
df_ap34_999 = pd.DataFrame({'total': [ap34_999]})
if df_ap34_999.empty or ap34_999 == 0:
    df_ap34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=42).value = df_ap34_999['total'].iloc[0]

aq34_999 = aq14_999 + aq32_999
df_aq34_999 = pd.DataFrame({'total': [aq34_999]})
if df_aq34_999.empty or aq34_999 == 0:
    df_aq34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=43).value = df_aq34_999['total'].iloc[0]

as34_999 = as14_999 + as32_999
df_as34_999 = pd.DataFrame({'total': [as34_999]})
if df_as34_999.empty or as34_999 == 0:
    df_as34_999 = pd.DataFrame({'total': [0]})
ws.cell(row=34, column=45).value = df_as34_999['total'].iloc[0]


## CONSULTA DE SUBTOTAL - RAMOS PRIMAS DEVENGADAS UNIDAD 3

In [54]:
# Extrae el valor del diccionario d14_999
d14_val = d14_999.get('total', 0)

# Suma con el float d32_999
d34_999 = d14_val + d32_999

# Crear DataFrame
df_d34_999 = pd.DataFrame({'total': [d34_999]})

# Verifica si es vacío o 0
if df_d34_999.empty or d34_999 == 0:
    df_d34_999 = pd.DataFrame({'total': [0]})

# Escribir en Excel
ws.cell(row=34, column=4).value = df_d34_999['total'].iloc[0]

# UNIDAD DE CAPTURA 4 

## CONSULTA DE TOTALES LIBERACIÓN Y CONSTITUCION DE RESERVAS SINIESTROS UNIDAD 4

In [55]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales
unidad4_total = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad4_total['TOTAL'] = unidad4_total['TOTAL'].abs()

# Agrupar los datos por 'CUENTA' y 'COD_RAMO' para obtener los totales
unidad4_total_a = bd.groupby(['CUENTA', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_total_a['TOTAL'] = unidad4_total_a['TOTAL'].abs()

# Definir la función para escribir en Excel
def write_to_excel(data_frame, column_name, value_name, start_row, start_col=3, cod_ramo=None):
    if cod_ramo:
        row = data_frame[(data_frame[column_name] == value_name) & (data_frame['COD_RAMO'] == cod_ramo)]
    else:
        row = data_frame[data_frame[column_name] == value_name]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Escribir cada valor en la hoja de Excel
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales", 36)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reserva Matemática BEPs", 37)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reserva para Siniestros Avisados", 38)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reserva para Siniestros No Avisados", 39)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reservas desviación de siniestralidad", 40)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reservas especiales", 41)
write_to_excel(unidad4_total, 'CONCEPTO', "Liberación Reserva de Riesgos Catastróficos", 42)
write_to_excel(unidad4_total_a, 'CUENTA', "6211001013", 43, cod_ramo=35)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reserva Siniestros Avisados  (resta)", 44)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reserva matemática BEPs (resta)", 45)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reserva Siniestros No Avisados  (resta)", 46)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reserva desviación siniestralidad (resta)", 47)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reservas especiales (resta)", 48)
write_to_excel(unidad4_total, 'CONCEPTO', "Constitución Reserva de Riesgos Catastróficos (resta)", 49)

# Definir la función get_value
def get_value(df, column_name, value_name, cod_ramo=None):
    if cod_ramo:
        row = df[(df[column_name] == value_name) & (df['COD_RAMO'] == cod_ramo)]
    else:
        row = df[df[column_name] == value_name]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
c36_005 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales")
c37_007 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reserva Matemática BEPs")
c38_010 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reserva para Siniestros Avisados")
c39_015 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reserva para Siniestros No Avisados")
c40_020 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reservas desviación de siniestralidad")
c41_025 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reservas especiales")
c42_026 = get_value(unidad4_total, 'CONCEPTO', "Liberación Reserva de Riesgos Catastróficos")
c43_030 = get_value(unidad4_total_a, 'CUENTA', "6211001013", cod_ramo=35)
c44_035 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reserva Siniestros Avisados  (resta)")
c45_037 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reserva matemática BEPs (resta)")
c46_040 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reserva Siniestros No Avisados  (resta)")
c47_045 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reserva desviación siniestralidad (resta)")
c48_050 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reservas especiales (resta)")
c49_051 = get_value(unidad4_total, 'CONCEPTO', "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular el total general según la fórmula dada
c50_999 = (c36_005 + c37_007 + c38_010 + c39_015 + c40_020 + c41_025 + c42_026 - c43_030 - c44_035 - c45_037 - c46_040 - c47_045 - c48_050 - c49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=3).value = c50_999


## CONSULTA DE SUBTOTAL - RAMOS LIBERACIÓN Y CONSTITUCION DE RESERVAS SINIESTROS UNIDAD 4

In [56]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales
unidad4_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad4_subtotal['TOTAL'] = unidad4_subtotal['TOTAL'].abs()

# Agrupar los datos por 'CUENTA' y 'COD_RAMO' para obtener los totales
unidad4_subtotal_a = bd.groupby(['CUENTA', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_subtotal_a['TOTAL'] = unidad4_subtotal_a['TOTAL'].abs()

# Definir la función para escribir en Excel
def write_to_excel(data_frame, column_name, value_name, start_row, start_col=4, cod_ramo=None):
    if cod_ramo:
        row = data_frame[(data_frame[column_name] == value_name) & (data_frame['COD_RAMO'] == cod_ramo)]
    else:
        row = data_frame[data_frame[column_name] == value_name]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Escribir cada valor en la hoja de Excel
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales", 36)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs", 37)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva para Siniestros Avisados", 38)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva para Siniestros No Avisados", 39)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reservas desviación de siniestralidad", 40)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reservas especiales", 41)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva de Riesgos Catastróficos", 42)
write_to_excel(unidad4_subtotal_a, 'CUENTA', "6211001013", 43, cod_ramo=35)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva Siniestros Avisados  (resta)", 44)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs (resta)", 45)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva Siniestros No Avisados  (resta)", 46)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva desviación siniestralidad (resta)", 47)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reservas especiales (resta)", 48)
write_to_excel(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva de Riesgos Catastróficos (resta)", 49)

# Definir la función get_value
def get_value(df, column_name, value_name, cod_ramo=None):
    if cod_ramo:
        row = df[(df[column_name] == value_name) & (df['COD_RAMO'] == cod_ramo)]
    else:
        row = df[df[column_name] == value_name]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
d36_005 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva Matemática Riesgos Laborales")
d37_007 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva Matemática BEPs")
d38_010 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva para Siniestros Avisados")
d39_015 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva para Siniestros No Avisados")
d40_020 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reservas desviación de siniestralidad")
d41_025 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reservas especiales")
d42_026 = get_value(unidad4_subtotal, 'CONCEPTO', "Liberación Reserva de Riesgos Catastróficos")
d43_030 = get_value(unidad4_subtotal_a, 'CUENTA', "6211001013", cod_ramo=35)
d44_035 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva Siniestros Avisados  (resta)")
d45_037 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva matemática BEPs (resta)")
d46_040 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva Siniestros No Avisados  (resta)")
d47_045 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva desviación siniestralidad (resta)")
d48_050 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reservas especiales (resta)")
d49_051 = get_value(unidad4_subtotal, 'CONCEPTO', "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular el total general según la fórmula dada
d50_999 = (d36_005 + d37_007 + d38_010 + d39_015 + d40_020 + d41_025 + d42_026 - d43_030 - d44_035 - d45_037 - d46_040 - d47_045 - d48_050 - d49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=4).value = d50_999


##  26 EXEQUIAS UNIDAD 4 

In [57]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_EXEQUIAS
unidad4_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_EXEQUIAS['TOTAL'] = unidad4_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 26)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_EXEQUIAS, concept, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 26)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab36_005 = get_value(unidad4_EXEQUIAS, "Liberación Reserva Matemática Riesgos Laborales")
ab37_007 = get_value(unidad4_EXEQUIAS, "Liberación Reserva Matemática BEPs")
ab38_010 = get_value(unidad4_EXEQUIAS, "Liberación Reserva para Siniestros Avisados")
ab39_015 = get_value(unidad4_EXEQUIAS, "Liberación Reserva para Siniestros No Avisados")
ab40_020 = get_value(unidad4_EXEQUIAS, "Liberación Reservas desviación de siniestralidad")
ab41_025 = get_value(unidad4_EXEQUIAS, "Liberación Reservas especiales")
ab42_026 = get_value(unidad4_EXEQUIAS, "Liberación Reserva de Riesgos Catastróficos")
ab43_030 = get_value(unidad4_EXEQUIAS, "Constitución Reserva matemática Riesgos Laborales (resta)")
ab44_035 = get_value(unidad4_EXEQUIAS, "Constitución Reserva Siniestros Avisados  (resta)")
ab45_037 = get_value(unidad4_EXEQUIAS, "Constitución Reserva matemática BEPs (resta)")
ab46_040 = get_value(unidad4_EXEQUIAS, "Constitución Reserva Siniestros No Avisados  (resta)")
ab47_045 = get_value(unidad4_EXEQUIAS, "Constitución Reserva desviación siniestralidad (resta)")
ab48_050 = get_value(unidad4_EXEQUIAS, "Constitución Reservas especiales (resta)")
ab49_051 = get_value(unidad4_EXEQUIAS, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ab50_999 usando la fórmula especificada
ab50_999 = (ab36_005 + ab37_007 + ab38_010 + ab39_015 + ab40_020 + ab41_025 + ab42_026 -
            ab43_030 - ab44_035 - ab45_037 - ab46_040 - ab47_045 - ab48_050 - ab49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=28, value=ab50_999)


<Cell 'Hoja1'.AB50>

## 27 ACCIDENTES PERSONALES UNIDAD 4

In [58]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_ACCPERSONALES
unidad4_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_ACCPERSONALES['TOTAL'] = unidad4_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 27)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_ACCPERSONALES, concept, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 27)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac36_005 = get_value(unidad4_ACCPERSONALES, "Liberación Reserva Matemática Riesgos Laborales")
ac37_007 = get_value(unidad4_ACCPERSONALES, "Liberación Reserva Matemática BEPs")
ac38_010 = get_value(unidad4_ACCPERSONALES, "Liberación Reserva para Siniestros Avisados")
ac39_015 = get_value(unidad4_ACCPERSONALES, "Liberación Reserva para Siniestros No Avisados")
ac40_020 = get_value(unidad4_ACCPERSONALES, "Liberación Reservas desviación de siniestralidad")
ac41_025 = get_value(unidad4_ACCPERSONALES, "Liberación Reservas especiales")
ac42_026 = get_value(unidad4_ACCPERSONALES, "Liberación Reserva de Riesgos Catastróficos")
ac43_030 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva matemática Riesgos Laborales (resta)")
ac44_035 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva Siniestros Avisados  (resta)")
ac45_037 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva matemática BEPs (resta)")
ac46_040 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva Siniestros No Avisados  (resta)")
ac47_045 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva desviación siniestralidad (resta)")
ac48_050 = get_value(unidad4_ACCPERSONALES, "Constitución Reservas especiales (resta)")
ac49_051 = get_value(unidad4_ACCPERSONALES, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ac50_999 usando la fórmula especificada
ac50_999 = (ac36_005 + ac37_007 + ac38_010 + ac39_015 + ac40_020 + ac41_025 + ac42_026 -
            ac43_030 - ac44_035 - ac45_037 - ac46_040 - ac47_045 - ac48_050 - ac49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=29, value=ac50_999)


<Cell 'Hoja1'.AC50>

##  28 COLECTIVO VIDA UNIDAD 4 

In [59]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_COLECVI
unidad4_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_COLECVI['TOTAL'] = unidad4_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 28)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_COLECVI, concept, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 28)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad36_005 = get_value(unidad4_COLECVI, "Liberación Reserva Matemática Riesgos Laborales")
ad37_007 = get_value(unidad4_COLECVI, "Liberación Reserva Matemática BEPs")
ad38_010 = get_value(unidad4_COLECVI, "Liberación Reserva para Siniestros Avisados")
ad39_015 = get_value(unidad4_COLECVI, "Liberación Reserva para Siniestros No Avisados")
ad40_020 = get_value(unidad4_COLECVI, "Liberación Reservas desviación de siniestralidad")
ad41_025 = get_value(unidad4_COLECVI, "Liberación Reservas especiales")
ad42_026 = get_value(unidad4_COLECVI, "Liberación Reserva de Riesgos Catastróficos")
ad43_030 = get_value(unidad4_COLECVI, "Constitución Reserva matemática Riesgos Laborales (resta)")
ad44_035 = get_value(unidad4_COLECVI, "Constitución Reserva Siniestros Avisados  (resta)")
ad45_037 = get_value(unidad4_COLECVI, "Constitución Reserva matemática BEPs (resta)")
ad46_040 = get_value(unidad4_COLECVI, "Constitución Reserva Siniestros No Avisados  (resta)")
ad47_045 = get_value(unidad4_COLECVI, "Constitución Reserva desviación siniestralidad (resta)")
ad48_050 = get_value(unidad4_COLECVI, "Constitución Reservas especiales (resta)")
ad49_051 = get_value(unidad4_COLECVI, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ad50_999 usando la fórmula especificada
ad50_999 = (ad36_005 + ad37_007 + ad38_010 + ad39_015 + ad40_020 + ad41_025 + ad42_026 -
            ad43_030 - ad44_035 - ad45_037 - ad46_040 - ad47_045 - ad48_050 - ad49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=30, value=ad50_999)


<Cell 'Hoja1'.AD50>

## 29 EDUCATIVO  UNIDAD 4

In [60]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_EDUCATIVO
unidad4_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_EDUCATIVO['TOTAL'] = unidad4_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 29)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_EDUCATIVO, concept, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 29)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae36_005 = get_value(unidad4_EDUCATIVO, "Liberación Reserva Matemática Riesgos Laborales")
ae37_007 = get_value(unidad4_EDUCATIVO, "Liberación Reserva Matemática BEPs")
ae38_010 = get_value(unidad4_EDUCATIVO, "Liberación Reserva para Siniestros Avisados")
ae39_015 = get_value(unidad4_EDUCATIVO, "Liberación Reserva para Siniestros No Avisados")
ae40_020 = get_value(unidad4_EDUCATIVO, "Liberación Reservas desviación de siniestralidad")
ae41_025 = get_value(unidad4_EDUCATIVO, "Liberación Reservas especiales")
ae42_026 = get_value(unidad4_EDUCATIVO, "Liberación Reserva de Riesgos Catastróficos")
ae43_030 = get_value(unidad4_EDUCATIVO, "Constitución Reserva matemática Riesgos Laborales (resta)")
ae44_035 = get_value(unidad4_EDUCATIVO, "Constitución Reserva Siniestros Avisados  (resta)")
ae45_037 = get_value(unidad4_EDUCATIVO, "Constitución Reserva matemática BEPs (resta)")
ae46_040 = get_value(unidad4_EDUCATIVO, "Constitución Reserva Siniestros No Avisados  (resta)")
ae47_045 = get_value(unidad4_EDUCATIVO, "Constitución Reserva desviación siniestralidad (resta)")
ae48_050 = get_value(unidad4_EDUCATIVO, "Constitución Reservas especiales (resta)")
ae49_051 = get_value(unidad4_EDUCATIVO, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ae50_999 usando la fórmula especificada
ae50_999 = (ae36_005 + ae37_007 + ae38_010 + ae39_015 + ae40_020 + ae41_025 + ae42_026 -
            ae43_030 - ae44_035 - ae45_037 - ae46_040 - ae47_045 - ae48_050 - ae49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=31, value=ae50_999)


<Cell 'Hoja1'.AE50>

##  30  VIDA GRUPO  UNIDAD 4 

In [61]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_VIDAGRUPO
unidad4_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_VIDAGRUPO['TOTAL'] = unidad4_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 30)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_VIDAGRUPO, concept, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 30)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af36_005 = get_value(unidad4_VIDAGRUPO, "Liberación Reserva Matemática Riesgos Laborales")
af37_007 = get_value(unidad4_VIDAGRUPO, "Liberación Reserva Matemática BEPs")
af38_010 = get_value(unidad4_VIDAGRUPO, "Liberación Reserva para Siniestros Avisados")
af39_015 = get_value(unidad4_VIDAGRUPO, "Liberación Reserva para Siniestros No Avisados")
af40_020 = get_value(unidad4_VIDAGRUPO, "Liberación Reservas desviación de siniestralidad")
af41_025 = get_value(unidad4_VIDAGRUPO, "Liberación Reservas especiales")
af42_026 = get_value(unidad4_VIDAGRUPO, "Liberación Reserva de Riesgos Catastróficos")
af43_030 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva matemática Riesgos Laborales (resta)")
af44_035 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva Siniestros Avisados  (resta)")
af45_037 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva matemática BEPs (resta)")
af46_040 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva Siniestros No Avisados  (resta)")
af47_045 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva desviación siniestralidad (resta)")
af48_050 = get_value(unidad4_VIDAGRUPO, "Constitución Reservas especiales (resta)")
af49_051 = get_value(unidad4_VIDAGRUPO, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular af50_999 usando la fórmula especificada
af50_999 = (af36_005 + af37_007 + af38_010 + af39_015 + af40_020 + af41_025 + af42_026 -
            af43_030 - af44_035 - af45_037 - af46_040 - af47_045 - af48_050 - af49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=32, value=af50_999)


<Cell 'Hoja1'.AF50>

##  31 SALUD UNIDAD 4 

In [62]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_SALUD
unidad4_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_SALUD['TOTAL'] = unidad4_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 31)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_SALUD, concept, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 31)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag36_005 = get_value(unidad4_SALUD, "Liberación Reserva Matemática Riesgos Laborales")
ag37_007 = get_value(unidad4_SALUD, "Liberación Reserva Matemática BEPs")
ag38_010 = get_value(unidad4_SALUD, "Liberación Reserva para Siniestros Avisados")
ag39_015 = get_value(unidad4_SALUD, "Liberación Reserva para Siniestros No Avisados")
ag40_020 = get_value(unidad4_SALUD, "Liberación Reservas desviación de siniestralidad")
ag41_025 = get_value(unidad4_SALUD, "Liberación Reservas especiales")
ag42_026 = get_value(unidad4_SALUD, "Liberación Reserva de Riesgos Catastróficos")
ag43_030 = get_value(unidad4_SALUD, "Constitución Reserva matemática Riesgos Laborales (resta)")
ag44_035 = get_value(unidad4_SALUD, "Constitución Reserva Siniestros Avisados  (resta)")
ag45_037 = get_value(unidad4_SALUD, "Constitución Reserva matemática BEPs (resta)")
ag46_040 = get_value(unidad4_SALUD, "Constitución Reserva Siniestros No Avisados  (resta)")
ag47_045 = get_value(unidad4_SALUD, "Constitución Reserva desviación siniestralidad (resta)")
ag48_050 = get_value(unidad4_SALUD, "Constitución Reservas especiales (resta)")
ag49_051 = get_value(unidad4_SALUD, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ag50_999 usando la fórmula especificada
ag50_999 = (ag36_005 + ag37_007 + ag38_010 + ag39_015 + ag40_020 + ag41_025 + ag42_026 -
            ag43_030 - ag44_035 - ag45_037 - ag46_040 - ag47_045 - ag48_050 - ag49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=33, value=ag50_999)


<Cell 'Hoja1'.AG50>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 4  

In [63]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_ENFALTOCOSTO
unidad4_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_ENFALTOCOSTO['TOTAL'] = unidad4_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 32)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_ENFALTOCOSTO, concept, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 32)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah36_005 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reserva Matemática Riesgos Laborales")
ah37_007 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reserva Matemática BEPs")
ah38_010 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reserva para Siniestros Avisados")
ah39_015 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reserva para Siniestros No Avisados")
ah40_020 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reservas desviación de siniestralidad")
ah41_025 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reservas especiales")
ah42_026 = get_value(unidad4_ENFALTOCOSTO, "Liberación Reserva de Riesgos Catastróficos")
ah43_030 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva matemática Riesgos Laborales (resta)")
ah44_035 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva Siniestros Avisados  (resta)")
ah45_037 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva matemática BEPs (resta)")
ah46_040 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva Siniestros No Avisados  (resta)")
ah47_045 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva desviación siniestralidad (resta)")
ah48_050 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reservas especiales (resta)")
ah49_051 = get_value(unidad4_ENFALTOCOSTO, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ah50_999 usando la fórmula especificada
ah50_999 = (ah36_005 + ah37_007 + ah38_010 + ah39_015 + ah40_020 + ah41_025 + ah42_026 -
            ah43_030 - ah44_035 - ah45_037 - ah46_040 - ah47_045 - ah48_050 - ah49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=34, value=ah50_999)


<Cell 'Hoja1'.AH50>

## 33 VIDA INDIVIDUAL UNIDAD 4 

In [64]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_VIDAINDIVI
unidad4_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_VIDAINDIVI['TOTAL'] = unidad4_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 33)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_VIDAINDIVI, concept, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 33)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai36_005 = get_value(unidad4_VIDAINDIVI, "Liberación Reserva Matemática Riesgos Laborales")
ai37_007 = get_value(unidad4_VIDAINDIVI, "Liberación Reserva Matemática BEPs")
ai38_010 = get_value(unidad4_VIDAINDIVI, "Liberación Reserva para Siniestros Avisados")
ai39_015 = get_value(unidad4_VIDAINDIVI, "Liberación Reserva para Siniestros No Avisados")
ai40_020 = get_value(unidad4_VIDAINDIVI, "Liberación Reservas desviación de siniestralidad")
ai41_025 = get_value(unidad4_VIDAINDIVI, "Liberación Reservas especiales")
ai42_026 = get_value(unidad4_VIDAINDIVI, "Liberación Reserva de Riesgos Catastróficos")
ai43_030 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva matemática Riesgos Laborales (resta)")
ai44_035 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva Siniestros Avisados  (resta)")
ai45_037 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva matemática BEPs (resta)")
ai46_040 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva Siniestros No Avisados  (resta)")
ai47_045 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva desviación siniestralidad (resta)")
ai48_050 = get_value(unidad4_VIDAINDIVI, "Constitución Reservas especiales (resta)")
ai49_051 = get_value(unidad4_VIDAINDIVI, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ai50_999 usando la fórmula especificada
ai50_999 = (ai36_005 + ai37_007 + ai38_010 + ai39_015 + ai40_020 + ai41_025 + ai42_026 -
            ai43_030 - ai44_035 - ai45_037 - ai46_040 - ai47_045 - ai48_050 - ai49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=35, value=ai50_999)


<Cell 'Hoja1'.AI50>

## 34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 4

In [65]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_PREVIDEINVAL
unidad4_PREVIDEINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_PREVIDEINVAL['TOTAL'] = unidad4_PREVIDEINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 34)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_PREVIDEINVAL, concept, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 34)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj36_005 = get_value(unidad4_PREVIDEINVAL, "Liberación Reserva Matemática Riesgos Laborales")
aj37_007 = get_value(unidad4_PREVIDEINVAL, "Liberación Reserva Matemática BEPs")
aj38_010 = get_value(unidad4_PREVIDEINVAL, "Liberación Reserva para Siniestros Avisados")
aj39_015 = get_value(unidad4_PREVIDEINVAL, "Liberación Reserva para Siniestros No Avisados")
aj40_020 = get_value(unidad4_PREVIDEINVAL, "Liberación Reservas desviación de siniestralidad")
aj41_025 = get_value(unidad4_PREVIDEINVAL, "Liberación Reservas especiales")
aj42_026 = get_value(unidad4_PREVIDEINVAL, "Liberación Reserva de Riesgos Catastróficos")
aj43_030 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva matemática Riesgos Laborales (resta)")
aj44_035 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva Siniestros Avisados  (resta)")
aj45_037 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva matemática BEPs (resta)")
aj46_040 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva Siniestros No Avisados  (resta)")
aj47_045 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva desviación siniestralidad (resta)")
aj48_050 = get_value(unidad4_PREVIDEINVAL, "Constitución Reservas especiales (resta)")
aj49_051 = get_value(unidad4_PREVIDEINVAL, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular aj50_999 usando la fórmula especificada
aj50_999 = (aj36_005 + aj37_007 + aj38_010 + aj39_015 + aj40_020 + aj41_025 + aj42_026 -
            aj43_030 - aj44_035 - aj45_037 - aj46_040 - aj47_045 - aj48_050 - aj49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=36, value=aj50_999)


<Cell 'Hoja1'.AJ50>

## 35 RIESGOS LABORALES UNIDAD 4

In [66]:
# Suponiendo que 'bd' es un DataFrame ya cargado
unidad4_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_RIESGOSLAB_a = bd.groupby(['CUENTA', 'COD_RAMO'])['TOTAL'].sum().reset_index()

# Función para escribir en Excel y manejo de valores vacíos
def write_to_excel(data_frame, start_col, start_row):
    if data_frame.empty:
        total = 0
    else:
        total = data_frame.iloc[0, 0]  # Mantener el signo original, NO usar abs()
    ws.cell(row=start_row, column=start_col).value = total

# Filtrar y escribir los datos en la hoja de Excel, aplicando abs() en las sumas positivas
ak36_005 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Matemática Riesgos Laborales") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak36_005, 37, 36)

ak37_007 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva Matemática BEPs") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak37_007, 37, 37)

ak38_010 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva para Siniestros Avisados") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak38_010, 37, 38)

ak39_015 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva para Siniestros No Avisados") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak39_015, 37, 39)

ak40_020 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reservas desviación de siniestralidad") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak40_020, 37, 40)

ak41_025 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reservas especiales") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak41_025, 37, 41)

ak42_026 = abs(unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Liberación Reserva de Riesgos Catastróficos") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']])
write_to_excel(ak42_026, 37, 42)

# No usar abs() en las siguientes filas, ya que son restas y deben mantener su signo negativo
ak43_030 = unidad4_RIESGOSLAB_a.loc[
    (unidad4_RIESGOSLAB_a['CUENTA'] == "6211001013") &
    (unidad4_RIESGOSLAB_a['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak43_030, 37, 43)

ak44_035 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva Siniestros Avisados  (resta)") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak44_035, 37, 44)

ak45_037 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva matemática BEPs (resta)") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak45_037, 37, 45)

ak46_040 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva Siniestros No Avisados  (resta)") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak46_040, 37, 46)

ak47_045 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva desviación siniestralidad (resta))") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak47_045, 37, 47)

ak48_050 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reservas especiales (resta)") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak48_050, 37, 48)

ak49_051 = unidad4_RIESGOSLAB.loc[
    (unidad4_RIESGOSLAB['CONCEPTO'] == "Constitución Reserva de Riesgos Catastróficos (resta)") &
    (unidad4_RIESGOSLAB['COD_RAMO'] == 35), ['TOTAL']]
write_to_excel(ak49_051, 37, 49)

# Realizar las operaciones con los datos filtrados
ak50_999 = (ak36_005['TOTAL'].sum() + ak37_007['TOTAL'].sum() + ak38_010['TOTAL'].sum() + 
            ak39_015['TOTAL'].sum() + ak40_020['TOTAL'].sum() + ak41_025['TOTAL'].sum() + 
            ak42_026['TOTAL'].sum() - ak43_030['TOTAL'].sum() - ak44_035['TOTAL'].sum() - 
            ak45_037['TOTAL'].sum() - ak46_040['TOTAL'].sum() - ak47_045['TOTAL'].sum() - 
            ak48_050['TOTAL'].sum() - ak49_051['TOTAL'].sum())

# Convertir el resultado a DataFrame y escribir en Excel
ak50_999_df = pd.DataFrame({'total': [ak50_999]})
write_to_excel(ak50_999_df, 37, 50)


##  36  PENSIONES LEY 100 UNIDAD 4 

In [67]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_PENSIONESLEY100
unidad4_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_PENSIONESLEY100['TOTAL'] = unidad4_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 36)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_PENSIONESLEY100, concept, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 36)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al36_005 = get_value(unidad4_PENSIONESLEY100, "Liberación Reserva Matemática Riesgos Laborales")
al37_007 = get_value(unidad4_PENSIONESLEY100, "Liberación Reserva Matemática BEPs")
al38_010 = get_value(unidad4_PENSIONESLEY100, "Liberación Reserva para Siniestros Avisados")
al39_015 = get_value(unidad4_PENSIONESLEY100, "Liberación Reserva para Siniestros No Avisados")
al40_020 = get_value(unidad4_PENSIONESLEY100, "Liberación Reservas desviación de siniestralidad")
al41_025 = get_value(unidad4_PENSIONESLEY100, "Liberación Reservas especiales")
al42_026 = get_value(unidad4_PENSIONESLEY100, "Liberación Reserva de Riesgos Catastróficos")
al43_030 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva matemática Riesgos Laborales (resta)")
al44_035 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva Siniestros Avisados  (resta)")
al45_037 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva matemática BEPs (resta)")
al46_040 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva Siniestros No Avisados  (resta)")
al47_045 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva desviación siniestralidad (resta)")
al48_050 = get_value(unidad4_PENSIONESLEY100, "Constitución Reservas especiales (resta)")
al49_051 = get_value(unidad4_PENSIONESLEY100, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular al50_999 usando la fórmula especificada
al50_999 = (al36_005 + al37_007 + al38_010 + al39_015 + al40_020 + al41_025 + al42_026 -
            al43_030 - al44_035 - al45_037 - al46_040 - al47_045 - al48_050 - al49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=38, value=al50_999)


<Cell 'Hoja1'.AL50>

## 37 PENSIONES VOLUNTARIAS UNIDAD 4

In [68]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_PENSIONESVOLUN
unidad4_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_PENSIONESVOLUN['TOTAL'] = unidad4_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 37)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_PENSIONESVOLUN, concept, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 37)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am36_005 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reserva Matemática Riesgos Laborales")
am37_007 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reserva Matemática BEPs")
am38_010 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reserva para Siniestros Avisados")
am39_015 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reserva para Siniestros No Avisados")
am40_020 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reservas desviación de siniestralidad")
am41_025 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reservas especiales")
am42_026 = get_value(unidad4_PENSIONESVOLUN, "Liberación Reserva de Riesgos Catastróficos")
am43_030 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva matemática Riesgos Laborales (resta)")
am44_035 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva Siniestros Avisados  (resta)")
am45_037 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva matemática BEPs (resta)")
am46_040 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva Siniestros No Avisados  (resta)")
am47_045 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva desviación siniestralidad (resta)")
am48_050 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reservas especiales (resta)")
am49_051 = get_value(unidad4_PENSIONESVOLUN, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular am50_999 usando la fórmula especificada
am50_999 = (am36_005 + am37_007 + am38_010 + am39_015 + am40_020 + am41_025 + am42_026 -
            am43_030 - am44_035 - am45_037 - am46_040 - am47_045 - am48_050 - am49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=39, value=am50_999)


<Cell 'Hoja1'.AM50>

##  38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 4  

In [69]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_PENSIONESCONMU
unidad4_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_PENSIONESCONMU['TOTAL'] = unidad4_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 38)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_PENSIONESCONMU, concept, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 38)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an36_005 = get_value(unidad4_PENSIONESCONMU, "Liberación Reserva Matemática Riesgos Laborales")
an37_007 = get_value(unidad4_PENSIONESCONMU, "Liberación Reserva Matemática BEPs")
an38_010 = get_value(unidad4_PENSIONESCONMU, "Liberación Reserva para Siniestros Avisados")
an39_015 = get_value(unidad4_PENSIONESCONMU, "Liberación Reserva para Siniestros No Avisados")
an40_020 = get_value(unidad4_PENSIONESCONMU, "Liberación Reservas desviación de siniestralidad")
an41_025 = get_value(unidad4_PENSIONESCONMU, "Liberación Reservas especiales")
an42_026 = get_value(unidad4_PENSIONESCONMU, "Liberación Reserva de Riesgos Catastróficos")
an43_030 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva matemática Riesgos Laborales (resta)")
an44_035 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva Siniestros Avisados  (resta)")
an45_037 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva matemática BEPs (resta)")
an46_040 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva Siniestros No Avisados  (resta)")
an47_045 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva desviación siniestralidad (resta)")
an48_050 = get_value(unidad4_PENSIONESCONMU, "Constitución Reservas especiales (resta)")
an49_051 = get_value(unidad4_PENSIONESCONMU, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular an50_999 usando la fórmula especificada
an50_999 = (an36_005 + an37_007 + an38_010 + an39_015 + an40_020 + an41_025 + an42_026 -
            an43_030 - an44_035 - an45_037 - an46_040 - an47_045 - an48_050 - an49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=40, value=an50_999)


<Cell 'Hoja1'.AN50>

## 39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 4

In [70]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_PATRIAUTONOMO
unidad4_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_PATRIAUTONOMO['TOTAL'] = unidad4_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 39)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_PATRIAUTONOMO, concept, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 39)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao36_005 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reserva Matemática Riesgos Laborales")
ao37_007 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reserva Matemática BEPs")
ao38_010 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reserva para Siniestros Avisados")
ao39_015 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reserva para Siniestros No Avisados")
ao40_020 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reservas desviación de siniestralidad")
ao41_025 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reservas especiales")
ao42_026 = get_value(unidad4_PATRIAUTONOMO, "Liberación Reserva de Riesgos Catastróficos")
ao43_030 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva matemática Riesgos Laborales (resta)")
ao44_035 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva Siniestros Avisados  (resta)")
ao45_037 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva matemática BEPs (resta)")
ao46_040 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva Siniestros No Avisados  (resta)")
ao47_045 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva desviación siniestralidad (resta)")
ao48_050 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reservas especiales (resta)")
ao49_051 = get_value(unidad4_PATRIAUTONOMO, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ao50_999 usando la fórmula especificada
ao50_999 = (ao36_005 + ao37_007 + ao38_010 + ao39_015 + ao40_020 + ao41_025 + ao42_026 -
            ao43_030 - ao44_035 - ao45_037 - ao46_040 - ao47_045 - ao48_050 - ao49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=41, value=ao50_999)


<Cell 'Hoja1'.AO50>

## 40 RENTAS VOLUNTARIAS UNIDAD 4

In [71]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_RENTASVOLUN
unidad4_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_RENTASVOLUN['TOTAL'] = unidad4_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 40)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_RENTASVOLUN, concept, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 40)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap36_005 = get_value(unidad4_RENTASVOLUN, "Liberación Reserva Matemática Riesgos Laborales")
ap37_007 = get_value(unidad4_RENTASVOLUN, "Liberación Reserva Matemática BEPs")
ap38_010 = get_value(unidad4_RENTASVOLUN, "Liberación Reserva para Siniestros Avisados")
ap39_015 = get_value(unidad4_RENTASVOLUN, "Liberación Reserva para Siniestros No Avisados")
ap40_020 = get_value(unidad4_RENTASVOLUN, "Liberación Reservas desviación de siniestralidad")
ap41_025 = get_value(unidad4_RENTASVOLUN, "Liberación Reservas especiales")
ap42_026 = get_value(unidad4_RENTASVOLUN, "Liberación Reserva de Riesgos Catastróficos")
ap43_030 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva matemática Riesgos Laborales (resta)")
ap44_035 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva Siniestros Avisados  (resta)")
ap45_037 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva matemática BEPs (resta)")
ap46_040 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva Siniestros No Avisados  (resta)")
ap47_045 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva desviación siniestralidad (resta)")
ap48_050 = get_value(unidad4_RENTASVOLUN, "Constitución Reservas especiales (resta)")
ap49_051 = get_value(unidad4_RENTASVOLUN, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular ap50_999 usando la fórmula especificada
ap50_999 = (ap36_005 + ap37_007 + ap38_010 + ap39_015 + ap40_020 + ap41_025 + ap42_026 -
            ap43_030 - ap44_035 - ap45_037 - ap46_040 - ap47_045 - ap48_050 - ap49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=42, value=ap50_999)


<Cell 'Hoja1'.AP50>

##   41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 4 

In [72]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_BENECONOMICOS
unidad4_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_BENECONOMICOS['TOTAL'] = unidad4_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 41)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_BENECONOMICOS, concept, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 41)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq36_005 = get_value(unidad4_BENECONOMICOS, "Liberación Reserva Matemática Riesgos Laborales")
aq37_007 = get_value(unidad4_BENECONOMICOS, "Liberación Reserva Matemática BEPs")
aq38_010 = get_value(unidad4_BENECONOMICOS, "Liberación Reserva para Siniestros Avisados")
aq39_015 = get_value(unidad4_BENECONOMICOS, "Liberación Reserva para Siniestros No Avisados")
aq40_020 = get_value(unidad4_BENECONOMICOS, "Liberación Reservas desviación de siniestralidad")
aq41_025 = get_value(unidad4_BENECONOMICOS, "Liberación Reservas especiales")
aq42_026 = get_value(unidad4_BENECONOMICOS, "Liberación Reserva de Riesgos Catastróficos")
aq43_030 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva matemática Riesgos Laborales (resta)")
aq44_035 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva Siniestros Avisados  (resta)")
aq45_037 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva matemática BEPs (resta)")
aq46_040 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva Siniestros No Avisados  (resta)")
aq47_045 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva desviación siniestralidad (resta)")
aq48_050 = get_value(unidad4_BENECONOMICOS, "Constitución Reservas especiales (resta)")
aq49_051 = get_value(unidad4_BENECONOMICOS, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular aq50_999 usando la fórmula especificada
aq50_999 = (aq36_005 + aq37_007 + aq38_010 + aq39_015 + aq40_020 + aq41_025 + aq42_026 -
            aq43_030 - aq44_035 - aq45_037 - aq46_040 - aq47_045 - aq48_050 - aq49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=43, value=aq50_999)


<Cell 'Hoja1'.AQ50>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 4

In [73]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad4_OPERNOASIGNABLES
unidad4_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad4_OPERNOASIGNABLES['TOTAL'] = unidad4_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 43)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Liberación Reserva Matemática Riesgos Laborales", 36),
    ("Liberación Reserva Matemática BEPs", 37),
    ("Liberación Reserva para Siniestros Avisados", 38),
    ("Liberación Reserva para Siniestros No Avisados", 39),
    ("Liberación Reservas desviación de siniestralidad", 40),
    ("Liberación Reservas especiales", 41),
    ("Liberación Reserva de Riesgos Catastróficos", 42),
    ("Constitución Reserva matemática Riesgos Laborales (resta)", 43),
    ("Constitución Reserva Siniestros Avisados  (resta)", 44),
    ("Constitución Reserva matemática BEPs (resta)", 45),
    ("Constitución Reserva Siniestros No Avisados  (resta)", 46),
    ("Constitución Reserva desviación siniestralidad (resta)", 47),
    ("Constitución Reservas especiales (resta)", 48),
    ("Constitución Reserva de Riesgos Catastróficos (resta)", 49)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad4_OPERNOASIGNABLES, concept, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 43)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as36_005 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva Matemática Riesgos Laborales")
as37_007 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva Matemática BEPs")
as38_010 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva para Siniestros Avisados")
as39_015 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva para Siniestros No Avisados")
as40_020 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reservas desviación de siniestralidad")
as41_025 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reservas especiales")
as42_026 = get_value(unidad4_OPERNOASIGNABLES, "Liberación Reserva de Riesgos Catastróficos")
as43_030 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva matemática Riesgos Laborales (resta)")
as44_035 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva Siniestros Avisados  (resta)")
as45_037 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva matemática BEPs (resta)")
as46_040 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva Siniestros No Avisados  (resta)")
as47_045 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva desviación siniestralidad (resta)")
as48_050 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reservas especiales (resta)")
as49_051 = get_value(unidad4_OPERNOASIGNABLES, "Constitución Reserva de Riesgos Catastróficos (resta)")

# Calcular as50_999 usando la fórmula especificada
as50_999 = (as36_005 + as37_007 + as38_010 + as39_015 + as40_020 + as41_025 + as42_026 -
            as43_030 - as44_035 - as45_037 - as46_040 - as47_045 - as48_050 - as49_051)

# Escribir el total general en Excel
ws.cell(row=50, column=45, value=as50_999)


<Cell 'Hoja1'.AS50>

# UNIDAD DE CAPTURA 5

##  CONSULTA DE TOTALES SINIESTROS LIQUIDADOS UNIDAD 5

In [74]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad5
unidad5_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad5_totales['TOTAL'] = unidad5_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=3).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad5_totales, "Siniestros liquidados", 52)
write_to_excel(unidad5_totales, "Siniestros de aceptaciones reaseguro interior", 53)
write_to_excel(unidad5_totales, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54)
write_to_excel(unidad5_totales, "Siniestros de aceptaciones reaseguro exterior", 55)
write_to_excel(unidad5_totales, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c52_005 = get_value(unidad5_totales, "Siniestros liquidados")
c53_010 = get_value(unidad5_totales, "Siniestros de aceptaciones reaseguro interior")
c54_015 = get_value(unidad5_totales, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior")
c55_020 = get_value(unidad5_totales, "Siniestros de aceptaciones reaseguro exterior")
c56_025 = get_value(unidad5_totales, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior")

# Calcular el total general según la fórmula dada
c57_999 = c52_005 + c53_010 + c54_015 + c55_020 + c56_025

# Escribir el total general en Excel
ws.cell(row=57, column=3).value = c57_999


##  CONSULTA DE SUBTOTALES SINIESTROS LIQUIDADOS UNIDAD 5

In [75]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad5
unidad5_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad5_subtotal['TOTAL'] = unidad5_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=4).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad5_subtotal, "Siniestros liquidados", 52)
write_to_excel(unidad5_subtotal, "Siniestros de aceptaciones reaseguro interior", 53)
write_to_excel(unidad5_subtotal, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54)
write_to_excel(unidad5_subtotal, "Siniestros de aceptaciones reaseguro exterior", 55)
write_to_excel(unidad5_subtotal, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d52_005 = get_value(unidad5_subtotal, "Siniestros liquidados")
d53_010 = get_value(unidad5_subtotal, "Siniestros de aceptaciones reaseguro interior")
d54_015 = get_value(unidad5_subtotal, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior")
d55_020 = get_value(unidad5_subtotal, "Siniestros de aceptaciones reaseguro exterior")
d56_025 = get_value(unidad5_subtotal, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior")

# Calcular el total general según la fórmula dada
d57_999 = d52_005 + d53_010 + d54_015 + d55_020 + d56_025

# Escribir el total general en Excel
ws.cell(row=57, column=4).value = d57_999


##  26 EXEQUIAS UNIDAD 5 

In [76]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_EXEQUIAS
unidad5_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_EXEQUIAS['TOTAL'] = unidad5_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 26)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_EXEQUIAS, concept, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 26)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab52_005 = get_value(unidad5_EXEQUIAS, "Siniestros liquidados")
ab53_010 = get_value(unidad5_EXEQUIAS, "Siniestros de aceptaciones reaseguro interior")
ab54_015 = get_value(unidad5_EXEQUIAS, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior")
ab55_020 = get_value(unidad5_EXEQUIAS, "Siniestros de aceptaciones reaseguro exterior")
ab56_025 = get_value(unidad5_EXEQUIAS, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior")

# Calcular ab57_999 usando la fórmula especificada
ab57_999 = (ab52_005 + ab53_010 + ab54_015 + ab55_020 + ab56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=28, value=ab57_999)


<Cell 'Hoja1'.AB57>

## 27 ACCIDENTES PERSONALES UNIDAD 5

In [77]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_ACCPERSONALES
unidad5_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_ACCPERSONALES['TOTAL'] = unidad5_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 27)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_ACCPERSONALES, concept, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 27)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac52_005 = get_value(unidad5_ACCPERSONALES, "Siniestros liquidados")
ac53_010 = get_value(unidad5_ACCPERSONALES, "Siniestros de aceptaciones reaseguro interior")
ac54_015 = get_value(unidad5_ACCPERSONALES, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior")
ac55_020 = get_value(unidad5_ACCPERSONALES, "Siniestros de aceptaciones reaseguro exterior")
ac56_025 = get_value(unidad5_ACCPERSONALES, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior")

# Calcular ac57_999 usando la fórmula especificada
ac57_999 = (ac52_005 + ac53_010 + ac54_015 + ac55_020 + ac56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=29, value=ac57_999)


<Cell 'Hoja1'.AC57>

## 28 COLECTIVO VIDA UNIDAD 5 

In [78]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_COLECVI
unidad5_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_COLECVI['TOTAL'] = unidad5_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == 28)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_COLECVI, concept, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == 28)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad52_005 = get_value(unidad5_COLECVI, "Siniestros liquidados")
ad53_010 = get_value(unidad5_COLECVI, "Siniestros de aceptaciones reaseguro interior")
ad54_015 = get_value(unidad5_COLECVI, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior")
ad55_020 = get_value(unidad5_COLECVI, "Siniestros de aceptaciones reaseguro exterior")
ad56_025 = get_value(unidad5_COLECVI, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior")

# Calcular ad57_999 usando la fórmula especificada
ad57_999 = (ad52_005 + ad53_010 + ad54_015 + ad55_020 + ad56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=30, value=ad57_999)


<Cell 'Hoja1'.AD57>

## 29 EDUCATIVO  UNIDAD 5 

In [79]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_EDUCATIVO
unidad5_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_EDUCATIVO['TOTAL'] = unidad5_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae52_005 = get_value(unidad5_EDUCATIVO, "Siniestros liquidados", 29)
ae53_010 = get_value(unidad5_EDUCATIVO, "Siniestros de aceptaciones reaseguro interior", 29)
ae54_015 = get_value(unidad5_EDUCATIVO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 29)
ae55_020 = get_value(unidad5_EDUCATIVO, "Siniestros de aceptaciones reaseguro exterior", 29)
ae56_025 = get_value(unidad5_EDUCATIVO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 29)

# Calcular ae57_999 usando la fórmula especificada
ae57_999 = (ae52_005 + ae53_010 + ae54_015 + ae55_020 + ae56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=31, value=ae57_999)


<Cell 'Hoja1'.AE57>

##  30  VIDA GRUPO  UNIDAD 5 

In [80]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_VIDAGRUPO
unidad5_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_VIDAGRUPO['TOTAL'] = unidad5_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af52_005 = get_value(unidad5_VIDAGRUPO, "Siniestros liquidados", 30)
af53_010 = get_value(unidad5_VIDAGRUPO, "Siniestros de aceptaciones reaseguro interior", 30)
af54_015 = get_value(unidad5_VIDAGRUPO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 30)
af55_020 = get_value(unidad5_VIDAGRUPO, "Siniestros de aceptaciones reaseguro exterior", 30)
af56_025 = get_value(unidad5_VIDAGRUPO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 30)

# Calcular af57_999 usando la fórmula especificada
af57_999 = (af52_005 + af53_010 + af54_015 + af55_020 + af56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=32, value=af57_999)


<Cell 'Hoja1'.AF57>

##  31 SALUD UNIDAD 5  

In [81]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_SALUD
unidad5_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_SALUD['TOTAL'] = unidad5_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag52_005 = get_value(unidad5_SALUD, "Siniestros liquidados", 31)
ag53_010 = get_value(unidad5_SALUD, "Siniestros de aceptaciones reaseguro interior", 31)
ag54_015 = get_value(unidad5_SALUD, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 31)
ag55_020 = get_value(unidad5_SALUD, "Siniestros de aceptaciones reaseguro exterior", 31)
ag56_025 = get_value(unidad5_SALUD, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 31)

# Calcular ag57_999 usando la fórmula especificada
ag57_999 = (ag52_005 + ag53_010 + ag54_015 + ag55_020 + ag56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=33, value=ag57_999)


<Cell 'Hoja1'.AG57>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 5 

In [82]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_ENFALTOCOSTO
unidad5_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_ENFALTOCOSTO['TOTAL'] = unidad5_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah52_005 = get_value(unidad5_ENFALTOCOSTO, "Siniestros liquidados", 32)
ah53_010 = get_value(unidad5_ENFALTOCOSTO, "Siniestros de aceptaciones reaseguro interior", 32)
ah54_015 = get_value(unidad5_ENFALTOCOSTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 32)
ah55_020 = get_value(unidad5_ENFALTOCOSTO, "Siniestros de aceptaciones reaseguro exterior", 32)
ah56_025 = get_value(unidad5_ENFALTOCOSTO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 32)

# Calcular ah57_999 usando la fórmula especificada
ah57_999 = (ah52_005 + ah53_010 + ah54_015 + ah55_020 + ah56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=34, value=ah57_999)


<Cell 'Hoja1'.AH57>

## 33 VIDA INDIVIDUAL UNIDAD 5 

In [83]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_VIDAINDIVI
unidad5_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_VIDAINDIVI['TOTAL'] = unidad5_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai52_005 = get_value(unidad5_VIDAINDIVI, "Siniestros liquidados", 33)
ai53_010 = get_value(unidad5_VIDAINDIVI, "Siniestros de aceptaciones reaseguro interior", 33)
ai54_015 = get_value(unidad5_VIDAINDIVI, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 33)
ai55_020 = get_value(unidad5_VIDAINDIVI, "Siniestros de aceptaciones reaseguro exterior", 33)
ai56_025 = get_value(unidad5_VIDAINDIVI, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 33)

# Calcular ai57_999 usando la fórmula especificada
ai57_999 = (ai52_005 + ai53_010 + ai54_015 + ai55_020 + ai56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=35, value=ai57_999)


<Cell 'Hoja1'.AI57>

##  34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 5

In [84]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_PREVIDEINVAL
unidad5_PREVIDENTIAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_PREVIDENTIAL['TOTAL'] = unidad5_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_PREVIDENTIAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj52_005 = get_value(unidad5_PREVIDENTIAL, "Siniestros liquidados", 34)
aj53_010 = get_value(unidad5_PREVIDENTIAL, "Siniestros de aceptaciones reaseguro interior", 34)
aj54_015 = get_value(unidad5_PREVIDENTIAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 34)
aj55_020 = get_value(unidad5_PREVIDENTIAL, "Siniestros de aceptaciones reaseguro exterior", 34)
aj56_025 = get_value(unidad5_PREVIDENTIAL, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 34)

# Calcular aj57_999 usando la fórmula especificada
aj57_999 = (aj52_005 + aj53_010 + aj54_015 + aj55_020 + aj56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=36, value=aj57_999)


<Cell 'Hoja1'.AJ57>

## 35 RIESGOS LABORALES UNIDAD 5 

In [85]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_RIESGOSLAB
unidad5_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_RIESGOSLAB['TOTAL'] = unidad5_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak52_005 = get_value(unidad5_RIESGOSLAB, "Siniestros liquidados", 35)
ak53_010 = get_value(unidad5_RIESGOSLAB, "Siniestros de aceptaciones reaseguro interior", 35)
ak54_015 = get_value(unidad5_RIESGOSLAB, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 35)
ak55_020 = get_value(unidad5_RIESGOSLAB, "Siniestros de aceptaciones reaseguro exterior", 35)
ak56_025 = get_value(unidad5_RIESGOSLAB, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 35)

# Calcular ak57_999 usando la fórmula especificada
ak57_999 = (ak52_005 + ak53_010 + ak54_015 + ak55_020 + ak56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=37, value=ak57_999)


<Cell 'Hoja1'.AK57>

##  36  PENSIONES LEY 100 UNIDAD 5

In [86]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_PENSIONESLEY100
unidad5_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_PENSIONESLEY100['TOTAL'] = unidad5_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al52_005 = get_value(unidad5_PENSIONESLEY100, "Siniestros liquidados", 36)
al53_010 = get_value(unidad5_PENSIONESLEY100, "Siniestros de aceptaciones reaseguro interior", 36)
al54_015 = get_value(unidad5_PENSIONESLEY100, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 36)
al55_020 = get_value(unidad5_PENSIONESLEY100, "Siniestros de aceptaciones reaseguro exterior", 36)
al56_025 = get_value(unidad5_PENSIONESLEY100, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 36)

# Calcular al57_999 usando la fórmula especificada
al57_999 = (al52_005 + al53_010 + al54_015 + al55_020 + al56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=38, value=al57_999)


<Cell 'Hoja1'.AL57>

## 37 PENSIONES VOLUNTARIAS UNIDAD 5 

In [87]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_PENSIONESVOLUN
unidad5_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_PENSIONESVOLUN['TOTAL'] = unidad5_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am52_005 = get_value(unidad5_PENSIONESVOLUN, "Siniestros liquidados", 37)
am53_010 = get_value(unidad5_PENSIONESVOLUN, "Siniestros de aceptaciones reaseguro interior", 37)
am54_015 = get_value(unidad5_PENSIONESVOLUN, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 37)
am55_020 = get_value(unidad5_PENSIONESVOLUN, "Siniestros de aceptaciones reaseguro exterior", 37)
am56_025 = get_value(unidad5_PENSIONESVOLUN, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 37)

# Calcular am57_999 usando la fórmula especificada
am57_999 = (am52_005 + am53_010 + am54_015 + am55_020 + am56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=39, value=am57_999)


<Cell 'Hoja1'.AM57>

##  38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 5 

In [88]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_PENSIONESCONMU
unidad5_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_PENSIONESCONMU['TOTAL'] = unidad5_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an52_005 = get_value(unidad5_PENSIONESCONMU, "Siniestros liquidados", 38)
an53_010 = get_value(unidad5_PENSIONESCONMU, "Siniestros de aceptaciones reaseguro interior", 38)
an54_015 = get_value(unidad5_PENSIONESCONMU, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 38)
an55_020 = get_value(unidad5_PENSIONESCONMU, "Siniestros de aceptaciones reaseguro exterior", 38)
an56_025 = get_value(unidad5_PENSIONESCONMU, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 38)

# Calcular an57_999 usando la fórmula especificada
an57_999 = (an52_005 + an53_010 + an54_015 + an55_020 + an56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=40, value=an57_999)


<Cell 'Hoja1'.AN57>

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 5 

In [89]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_PATRIAUTONOMO
unidad5_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_PATRIAUTONOMO['TOTAL'] = unidad5_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao52_005 = get_value(unidad5_PATRIAUTONOMO, "Siniestros liquidados", 39)
ao53_010 = get_value(unidad5_PATRIAUTONOMO, "Siniestros de aceptaciones reaseguro interior", 39)
ao54_015 = get_value(unidad5_PATRIAUTONOMO, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 39)
ao55_020 = get_value(unidad5_PATRIAUTONOMO, "Siniestros de aceptaciones reaseguro exterior", 39)
ao56_025 = get_value(unidad5_PATRIAUTONOMO, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 39)

# Calcular ao57_999 usando la fórmula especificada
ao57_999 = (ao52_005 + ao53_010 + ao54_015 + ao55_020 + ao56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=41, value=ao57_999)


<Cell 'Hoja1'.AO57>

## 40 RENTAS VOLUNTARIAS UNIDAD 5   

In [90]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_RENTASVOLUN
unidad5_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_RENTASVOLUN['TOTAL'] = unidad5_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap52_005 = get_value(unidad5_RENTASVOLUN, "Siniestros liquidados", 40)
ap53_010 = get_value(unidad5_RENTASVOLUN, "Siniestros de aceptaciones reaseguro interior", 40)
ap54_015 = get_value(unidad5_RENTASVOLUN, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 40)
ap55_020 = get_value(unidad5_RENTASVOLUN, "Siniestros de aceptaciones reaseguro exterior", 40)
ap56_025 = get_value(unidad5_RENTASVOLUN, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 40)

# Calcular ap57_999 usando la fórmula especificada
ap57_999 = (ap52_005 + ap53_010 + ap54_015 + ap55_020 + ap56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=42, value=ap57_999)


<Cell 'Hoja1'.AP57>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 5 

In [91]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_BENECONOMICOS
unidad5_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_BENECONOMICOS['TOTAL'] = unidad5_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq52_005 = get_value(unidad5_BENECONOMICOS, "Siniestros liquidados", 41)
aq53_010 = get_value(unidad5_BENECONOMICOS, "Siniestros de aceptaciones reaseguro interior", 41)
aq54_015 = get_value(unidad5_BENECONOMICOS, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 41)
aq55_020 = get_value(unidad5_BENECONOMICOS, "Siniestros de aceptaciones reaseguro exterior", 41)
aq56_025 = get_value(unidad5_BENECONOMICOS, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 41)

# Calcular aq57_999 usando la fórmula especificada
aq57_999 = (aq52_005 + aq53_010 + aq54_015 + aq55_020 + aq56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=43, value=aq57_999)


<Cell 'Hoja1'.AQ57>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 5

In [92]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad5_OPERNOASIGNABLES
unidad5_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad5_OPERNOASIGNABLES['TOTAL'] = unidad5_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Siniestros liquidados", 52),
    ("Siniestros de aceptaciones reaseguro interior", 53),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 54),
    ("Siniestros de aceptaciones reaseguro exterior", 55),
    ("Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 56)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad5_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as52_005 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros liquidados", 43)
as53_010 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones reaseguro interior", 43)
as54_015 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones contratos no proporcionales reaseguros interior", 43)
as55_020 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones reaseguro exterior", 43)
as56_025 = get_value(unidad5_OPERNOASIGNABLES, "Siniestros de aceptaciones contratos no proporcionales reaseguros exterior", 43)

# Calcular as57_999 usando la fórmula especificada
as57_999 = (as52_005 + as53_010 + as54_015 + as55_020 + as56_025)

# Escribir el total general en Excel
ws.cell(row=57, column=45, value=as57_999)


<Cell 'Hoja1'.AS57>

#  UNIDAD DE CAPTURA 6   

## CONSULTA DE TOTALES REEMBOLSO DE SINIESTROS UNIDAD 6

In [93]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad6
unidad6_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad6_totales['TOTAL'] = unidad6_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=3).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad6_totales, "Reembolso de siniestros sobre cesiones reaseguro interior", 59)
write_to_excel(unidad6_totales, "Reembolso siniestros contratos no proporcionales reaseguro interior", 60)
write_to_excel(unidad6_totales, "Reembolso de siniestros sobre cesiones reaseguro exterior", 61)
write_to_excel(unidad6_totales, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62)
write_to_excel(unidad6_totales, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63)
write_to_excel(unidad6_totales, "Reembolsos por enfermedad profesional", 64)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c59_005 = get_value(unidad6_totales, "Reembolso de siniestros sobre cesiones reaseguro interior")
c60_010 = get_value(unidad6_totales, "Reembolso siniestros contratos no proporcionales reaseguro interior")
c61_015 = get_value(unidad6_totales, "Reembolso de siniestros sobre cesiones reaseguro exterior")
c62_020 = get_value(unidad6_totales, "Reembolso de siniestros contratos no proporcionales reaseguro exterior")
c63_025 = get_value(unidad6_totales, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.")
c64_030 = get_value(unidad6_totales, "Reembolsos por enfermedad profesional")

# Calcular el total general según la fórmula dada
c65_999 = c59_005 + c60_010 + c61_015 + c62_020 + c63_025 + c64_030

# Escribir el total general en Excel
ws.cell(row=65, column=3).value = c65_999


## CONSULTA DE SUBTOTALES SINIESTROS LIQUIDADOS UNIDAD 6

In [94]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad6
unidad6_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad6_subtotal['TOTAL'] = unidad6_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=4):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad6_subtotal, "Reembolso de siniestros sobre cesiones reaseguro interior", 59)
write_to_excel(unidad6_subtotal, "Reembolso siniestros contratos no proporcionales reaseguro interior", 60)
write_to_excel(unidad6_subtotal, "Reembolso de siniestros sobre cesiones reaseguro exterior", 61)
write_to_excel(unidad6_subtotal, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62)
write_to_excel(unidad6_subtotal, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63)
write_to_excel(unidad6_subtotal, "Reembolsos por enfermedad profesional", 64)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d59_005 = get_value(unidad6_subtotal, "Reembolso de siniestros sobre cesiones reaseguro interior")
d60_010 = get_value(unidad6_subtotal, "Reembolso siniestros contratos no proporcionales reaseguro interior")
d61_015 = get_value(unidad6_subtotal, "Reembolso de siniestros sobre cesiones reaseguro exterior")
d62_020 = get_value(unidad6_subtotal, "Reembolso de siniestros contratos no proporcionales reaseguro exterior")
d63_025 = get_value(unidad6_subtotal, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.")
d64_030 = get_value(unidad6_subtotal, "Reembolsos por enfermedad profesional")

# Calcular el total general según la fórmula dada
d65_999 = d59_005 + d60_010 + d61_015 + d62_020 + d63_025 + d64_030

# Escribir el total general en Excel
ws.cell(row=65, column=4).value = d65_999


##  26 EXEQUIAS UNIDAD 6 

In [95]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_EXEQUIAS
unidad6_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_EXEQUIAS['TOTAL'] = unidad6_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab59_005 = get_value(unidad6_EXEQUIAS, "Reembolso de siniestros sobre cesiones reaseguro interior", 26)
ab60_010 = get_value(unidad6_EXEQUIAS, "Reembolso siniestros contratos no proporcionales reaseguro interior", 26)
ab61_015 = get_value(unidad6_EXEQUIAS, "Reembolso de siniestros sobre cesiones reaseguro exterior", 26)
ab62_020 = get_value(unidad6_EXEQUIAS, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 26)
ab63_025 = get_value(unidad6_EXEQUIAS, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 26)
ab64_030 = get_value(unidad6_EXEQUIAS, "Reembolsos por enfermedad profesional", 26)

# Calcular ab65_999 usando la fórmula especificada
ab65_999 = (ab59_005 + ab60_010 + ab61_015 + ab62_020 + ab63_025 + ab64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=28, value=ab65_999)


<Cell 'Hoja1'.AB65>

## 27 ACCIDENTES PERSONALES UNIDAD 6 

In [96]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_ACCPERSONALES
unidad6_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_ACCPERSONALES['TOTAL'] = unidad6_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac59_005 = get_value(unidad6_ACCPERSONALES, "Reembolso de siniestros sobre cesiones reaseguro interior", 27)
ac60_010 = get_value(unidad6_ACCPERSONALES, "Reembolso siniestros contratos no proporcionales reaseguro interior", 27)
ac61_015 = get_value(unidad6_ACCPERSONALES, "Reembolso de siniestros sobre cesiones reaseguro exterior", 27)
ac62_020 = get_value(unidad6_ACCPERSONALES, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 27)
ac63_025 = get_value(unidad6_ACCPERSONALES, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 27)
ac64_030 = get_value(unidad6_ACCPERSONALES, "Reembolsos por enfermedad profesional", 27)

# Calcular ac65_999 usando la fórmula especificada
ac65_999 = (ac59_005 + ac60_010 + ac61_015 + ac62_020 + ac63_025 + ac64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=29, value=ac65_999)


<Cell 'Hoja1'.AC65>

##   28 COLECTIVO VIDA UNIDAD 6 

In [97]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_COLECVI
unidad6_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_COLECVI['TOTAL'] = unidad6_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad59_005 = get_value(unidad6_COLECVI, "Reembolso de siniestros sobre cesiones reaseguro interior", 28)
ad60_010 = get_value(unidad6_COLECVI, "Reembolso siniestros contratos no proporcionales reaseguro interior", 28)
ad61_015 = get_value(unidad6_COLECVI, "Reembolso de siniestros sobre cesiones reaseguro exterior", 28)
ad62_020 = get_value(unidad6_COLECVI, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 28)
ad63_025 = get_value(unidad6_COLECVI, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 28)
ad64_030 = get_value(unidad6_COLECVI, "Reembolsos por enfermedad profesional", 28)

# Calcular ad65_999 usando la fórmula especificada
ad65_999 = (ad59_005 + ad60_010 + ad61_015 + ad62_020 + ad63_025 + ad64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=30, value=ad65_999)


<Cell 'Hoja1'.AD65>

## 29 EDUCATIVO  UNIDAD 6  

In [98]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_EDUCATIVO
unidad6_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_EDUCATIVO['TOTAL'] = unidad6_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae59_005 = get_value(unidad6_EDUCATIVO, "Reembolso de siniestros sobre cesiones reaseguro interior", 29)
ae60_010 = get_value(unidad6_EDUCATIVO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 29)
ae61_015 = get_value(unidad6_EDUCATIVO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 29)
ae62_020 = get_value(unidad6_EDUCATIVO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 29)
ae63_025 = get_value(unidad6_EDUCATIVO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 29)
ae64_030 = get_value(unidad6_EDUCATIVO, "Reembolsos por enfermedad profesional", 29)

# Calcular ae65_999 usando la fórmula especificada
ae65_999 = (ae59_005 + ae60_010 + ae61_015 + ae62_020 + ae63_025 + ae64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=31, value=ae65_999)


<Cell 'Hoja1'.AE65>

## 30  VIDA GRUPO  UNIDAD 6

In [99]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_VIDAGRUPO
unidad6_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_VIDAGRUPO['TOTAL'] = unidad6_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af59_005 = get_value(unidad6_VIDAGRUPO, "Reembolso de siniestros sobre cesiones reaseguro interior", 30)
af60_010 = get_value(unidad6_VIDAGRUPO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 30)
af61_015 = get_value(unidad6_VIDAGRUPO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 30)
af62_020 = get_value(unidad6_VIDAGRUPO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 30)
af63_025 = get_value(unidad6_VIDAGRUPO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 30)
af64_030 = get_value(unidad6_VIDAGRUPO, "Reembolsos por enfermedad profesional", 30)

# Calcular af65_999 usando la fórmula especificada
af65_999 = (af59_005 + af60_010 + af61_015 + af62_020 + af63_025 + af64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=32, value=af65_999)


<Cell 'Hoja1'.AF65>

##   31 SALUD UNIDAD 6

In [100]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_SALUD
unidad6_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_SALUD['TOTAL'] = unidad6_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag59_005 = get_value(unidad6_SALUD, "Reembolso de siniestros sobre cesiones reaseguro interior", 31)
ag60_010 = get_value(unidad6_SALUD, "Reembolso siniestros contratos no proporcionales reaseguro interior", 31)
ag61_015 = get_value(unidad6_SALUD, "Reembolso de siniestros sobre cesiones reaseguro exterior", 31)
ag62_020 = get_value(unidad6_SALUD, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 31)
ag63_025 = get_value(unidad6_SALUD, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 31)
ag64_030 = get_value(unidad6_SALUD, "Reembolsos por enfermedad profesional", 31)

# Calcular ag65_999 usando la fórmula especificada
ag65_999 = (ag59_005 + ag60_010 + ag61_015 + ag62_020 + ag63_025 + ag64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=33, value=ag65_999)


<Cell 'Hoja1'.AG65>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 6 

In [101]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_ENFALTOCOSTO
unidad6_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_ENFALTOCOSTO['TOTAL'] = unidad6_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah59_005 = get_value(unidad6_ENFALTOCOSTO, "Reembolso de siniestros sobre cesiones reaseguro interior", 32)
ah60_010 = get_value(unidad6_ENFALTOCOSTO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 32)
ah61_015 = get_value(unidad6_ENFALTOCOSTO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 32)
ah62_020 = get_value(unidad6_ENFALTOCOSTO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 32)
ah63_025 = get_value(unidad6_ENFALTOCOSTO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 32)
ah64_030 = get_value(unidad6_ENFALTOCOSTO, "Reembolsos por enfermedad profesional", 32)

# Calcular ah65_999 usando la fórmula especificada
ah65_999 = (ah59_005 + ah60_010 + ah61_015 + ah62_020 + ah63_025 + ah64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=34, value=ah65_999)


<Cell 'Hoja1'.AH65>

##  33 VIDA INDIVIDUAL UNIDAD 6

In [102]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_VIDAINDIVI
unidad6_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_VIDAINDIVI['TOTAL'] = unidad6_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai59_005 = get_value(unidad6_VIDAINDIVI, "Reembolso de siniestros sobre cesiones reaseguro interior", 33)
ai60_010 = get_value(unidad6_VIDAINDIVI, "Reembolso siniestros contratos no proporcionales reaseguro interior", 33)
ai61_015 = get_value(unidad6_VIDAINDIVI, "Reembolso de siniestros sobre cesiones reaseguro exterior", 33)
ai62_020 = get_value(unidad6_VIDAINDIVI, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 33)
ai63_025 = get_value(unidad6_VIDAINDIVI, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 33)
ai64_030 = get_value(unidad6_VIDAINDIVI, "Reembolsos por enfermedad profesional", 33)

# Calcular ai65_999 usando la fórmula especificada
ai65_999 = (ai59_005 + ai60_010 + ai61_015 + ai62_020 + ai63_025 + ai64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=35, value=ai65_999)


<Cell 'Hoja1'.AI65>

## 34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 6 

In [103]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_PREVIDINVAL
unidad6_PREVIDINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_PREVIDINVAL['TOTAL'] = unidad6_PREVIDINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_PREVIDINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj59_005 = get_value(unidad6_PREVIDINVAL, "Reembolso de siniestros sobre cesiones reaseguro interior", 34)
aj60_010 = get_value(unidad6_PREVIDINVAL, "Reembolso siniestros contratos no proporcionales reaseguro interior", 34)
aj61_015 = get_value(unidad6_PREVIDINVAL, "Reembolso de siniestros sobre cesiones reaseguro exterior", 34)
aj62_020 = get_value(unidad6_PREVIDINVAL, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 34)
aj63_025 = get_value(unidad6_PREVIDINVAL, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 34)
aj64_030 = get_value(unidad6_PREVIDINVAL, "Reembolsos por enfermedad profesional", 34)

# Calcular aj65_999 usando la fórmula especificada
aj65_999 = (aj59_005 + aj60_010 + aj61_015 + aj62_020 + aj63_025 + aj64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=36, value=aj65_999)


<Cell 'Hoja1'.AJ65>

## 35 RIESGOS LABORALES UNIDAD 6 

In [104]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_RIESGOSLAB
unidad6_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_RIESGOSLAB['TOTAL'] = unidad6_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak59_005 = get_value(unidad6_RIESGOSLAB, "Reembolso de siniestros sobre cesiones reaseguro interior", 35)
ak60_010 = get_value(unidad6_RIESGOSLAB, "Reembolso siniestros contratos no proporcionales reaseguro interior", 35)
ak61_015 = get_value(unidad6_RIESGOSLAB, "Reembolso de siniestros sobre cesiones reaseguro exterior", 35)
ak62_020 = get_value(unidad6_RIESGOSLAB, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 35)
ak63_025 = get_value(unidad6_RIESGOSLAB, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 35)
ak64_030 = get_value(unidad6_RIESGOSLAB, "Reembolsos por enfermedad profesional", 35)

# Calcular ak65_999 usando la fórmula especificada
ak65_999 = (ak59_005 + ak60_010 + ak61_015 + ak62_020 + ak63_025 + ak64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=37, value=ak65_999)


<Cell 'Hoja1'.AK65>

## 36  PENSIONES LEY 100 UNIDAD 6  

In [105]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_PENSIONESLEY100
unidad6_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_PENSIONESLEY100['TOTAL'] = unidad6_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al59_005 = get_value(unidad6_PENSIONESLEY100, "Reembolso de siniestros sobre cesiones reaseguro interior", 36)
al60_010 = get_value(unidad6_PENSIONESLEY100, "Reembolso siniestros contratos no proporcionales reaseguro interior", 36)
al61_015 = get_value(unidad6_PENSIONESLEY100, "Reembolso de siniestros sobre cesiones reaseguro exterior", 36)
al62_020 = get_value(unidad6_PENSIONESLEY100, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 36)
al63_025 = get_value(unidad6_PENSIONESLEY100, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 36)
al64_030 = get_value(unidad6_PENSIONESLEY100, "Reembolsos por enfermedad profesional", 36)

# Calcular al65_999 usando la fórmula especificada
al65_999 = (al59_005 + al60_010 + al61_015 + al62_020 + al63_025 + al64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=38, value=al65_999)


<Cell 'Hoja1'.AL65>

## 37 PENSIONES VOLUNTARIAS UNIDAD 6 

In [106]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_PENSIONESVOLUN
unidad6_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_PENSIONESVOLUN['TOTAL'] = unidad6_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am59_005 = get_value(unidad6_PENSIONESVOLUN, "Reembolso de siniestros sobre cesiones reaseguro interior", 37)
am60_010 = get_value(unidad6_PENSIONESVOLUN, "Reembolso siniestros contratos no proporcionales reaseguro interior", 37)
am61_015 = get_value(unidad6_PENSIONESVOLUN, "Reembolso de siniestros sobre cesiones reaseguro exterior", 37)
am62_020 = get_value(unidad6_PENSIONESVOLUN, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 37)
am63_025 = get_value(unidad6_PENSIONESVOLUN, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 37)
am64_030 = get_value(unidad6_PENSIONESVOLUN, "Reembolsos por enfermedad profesional", 37)

# Calcular am65_999 usando la fórmula especificada
am65_999 = (am59_005 + am60_010 + am61_015 + am62_020 + am63_025 + am64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=39, value=am65_999)


<Cell 'Hoja1'.AM65>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 6

In [107]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_PENSIONESCONMU
unidad6_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_PENSIONESCONMU['TOTAL'] = unidad6_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an59_005 = get_value(unidad6_PENSIONESCONMU, "Reembolso de siniestros sobre cesiones reaseguro interior", 38)
an60_010 = get_value(unidad6_PENSIONESCONMU, "Reembolso siniestros contratos no proporcionales reaseguro interior", 38)
an61_015 = get_value(unidad6_PENSIONESCONMU, "Reembolso de siniestros sobre cesiones reaseguro exterior", 38)
an62_020 = get_value(unidad6_PENSIONESCONMU, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 38)
an63_025 = get_value(unidad6_PENSIONESCONMU, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 38)
an64_030 = get_value(unidad6_PENSIONESCONMU, "Reembolsos por enfermedad profesional", 38)

# Calcular an65_999 usando la fórmula especificada
an65_999 = (an59_005 + an60_010 + an61_015 + an62_020 + an63_025 + an64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=40, value=an65_999)


<Cell 'Hoja1'.AN65>

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 6  

In [108]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_PATRIAUTONOMO
unidad6_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_PATRIAUTONOMO['TOTAL'] = unidad6_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao59_005 = get_value(unidad6_PATRIAUTONOMO, "Reembolso de siniestros sobre cesiones reaseguro interior", 39)
ao60_010 = get_value(unidad6_PATRIAUTONOMO, "Reembolso siniestros contratos no proporcionales reaseguro interior", 39)
ao61_015 = get_value(unidad6_PATRIAUTONOMO, "Reembolso de siniestros sobre cesiones reaseguro exterior", 39)
ao62_020 = get_value(unidad6_PATRIAUTONOMO, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 39)
ao63_025 = get_value(unidad6_PATRIAUTONOMO, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 39)
ao64_030 = get_value(unidad6_PATRIAUTONOMO, "Reembolsos por enfermedad profesional", 39)

# Calcular ao65_999 usando la fórmula especificada
ao65_999 = (ao59_005 + ao60_010 + ao61_015 + ao62_020 + ao63_025 + ao64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=41, value=ao65_999)


<Cell 'Hoja1'.AO65>

## 40 RENTAS VOLUNTARIAS UNIDAD 6

In [109]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_RENTASVOLUN
unidad6_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_RENTASVOLUN['TOTAL'] = unidad6_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap59_005 = get_value(unidad6_RENTASVOLUN, "Reembolso de siniestros sobre cesiones reaseguro interior", 40)
ap60_010 = get_value(unidad6_RENTASVOLUN, "Reembolso siniestros contratos no proporcionales reaseguro interior", 40)
ap61_015 = get_value(unidad6_RENTASVOLUN, "Reembolso de siniestros sobre cesiones reaseguro exterior", 40)
ap62_020 = get_value(unidad6_RENTASVOLUN, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 40)
ap63_025 = get_value(unidad6_RENTASVOLUN, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 40)
ap64_030 = get_value(unidad6_RENTASVOLUN, "Reembolsos por enfermedad profesional", 40)

# Calcular ap65_999 usando la fórmula especificada
ap65_999 = (ap59_005 + ap60_010 + ap61_015 + ap62_020 + ap63_025 + ap64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=42, value=ap65_999)


<Cell 'Hoja1'.AP65>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 6  

In [110]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_BENECONOMICOS
unidad6_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_BENECONOMICOS['TOTAL'] = unidad6_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq59_005 = get_value(unidad6_BENECONOMICOS, "Reembolso de siniestros sobre cesiones reaseguro interior", 41)
aq60_010 = get_value(unidad6_BENECONOMICOS, "Reembolso siniestros contratos no proporcionales reaseguro interior", 41)
aq61_015 = get_value(unidad6_BENECONOMICOS, "Reembolso de siniestros sobre cesiones reaseguro exterior", 41)
aq62_020 = get_value(unidad6_BENECONOMICOS, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 41)
aq63_025 = get_value(unidad6_BENECONOMICOS, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 41)
aq64_030 = get_value(unidad6_BENECONOMICOS, "Reembolsos por enfermedad profesional", 41)

# Calcular aq65_999 usando la fórmula especificada
aq65_999 = (aq59_005 + aq60_010 + aq61_015 + aq62_020 + aq63_025 + aq64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=43, value=aq65_999)


<Cell 'Hoja1'.AQ65>

##  43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 6 

In [111]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_OPERNOASIGNABLES
unidad6_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_OPERNOASIGNABLES['TOTAL'] = unidad6_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Reembolso de siniestros sobre cesiones reaseguro interior", 59),
    ("Reembolso siniestros contratos no proporcionales reaseguro interior", 60),
    ("Reembolso de siniestros sobre cesiones reaseguro exterior", 61),
    ("Reembolso de siniestros contratos no proporcionales reaseguro exterior", 62),
    ("Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 63),
    ("Reembolsos por enfermedad profesional", 64)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as59_005 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros sobre cesiones reaseguro interior", 43)
as60_010 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso siniestros contratos no proporcionales reaseguro interior", 43)
as61_015 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros sobre cesiones reaseguro exterior", 43)
as62_020 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso de siniestros contratos no proporcionales reaseguro exterior", 43)
as63_025 = get_value(unidad6_OPERNOASIGNABLES, "Reembolso stros. garantiz. por la Nación rgos. polít y extraor.", 43)
as64_030 = get_value(unidad6_OPERNOASIGNABLES, "Reembolsos por enfermedad profesional", 43)

# Calcular as65_999 usando la fórmula especificada
as65_999 = (as59_005 + as60_010 + as61_015 + as62_020 + as63_025 + as64_030)

# Escribir el total general en Excel
ws.cell(row=65, column=45, value=as65_999)


<Cell 'Hoja1'.AS65>

#    UNIDAD DE CAPTURA 7  

## CONSULTA DE TOTALES SALVAMENTOS Y RECOBROS UNIDAD 7

In [112]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad7
unidad7_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad7_totales['TOTAL'] = unidad7_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=3):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad7_totales, "Salvamentos de aceptaciones reaseguro interior", 67)
write_to_excel(unidad7_totales, "Salvamentos de aceptaciones reaseguro exterior", 68)
write_to_excel(unidad7_totales, "Salvamentos liquidados y realizados", 69)
write_to_excel(unidad7_totales, "Recobros", 70)
write_to_excel(unidad7_totales, "Gastos Salvamentos parte reasegurada interior (resta)", 71)
write_to_excel(unidad7_totales, "Gastos Salvamentos parte reasegurada exterior (resta)", 72)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c67_005 = get_value(unidad7_totales, "Salvamentos de aceptaciones reaseguro interior")
c68_010 = get_value(unidad7_totales, "Salvamentos de aceptaciones reaseguro exterior")
c69_015 = get_value(unidad7_totales, "Salvamentos liquidados y realizados")
c70_020 = get_value(unidad7_totales, "Recobros")
c71_025 = get_value(unidad7_totales, "Gastos Salvamentos parte reasegurada interior (resta)")
c72_030 = get_value(unidad7_totales, "Gastos Salvamentos parte reasegurada exterior (resta)")

# Calcular el total general según la fórmula dada
c73_999 = c67_005 + c68_010 + c69_015 + c70_020 - c71_025 - c72_030

# Escribir el total general en Excel
ws.cell(row=73, column=3).value = c73_999


## CONSULTA DE SUBTOTALES SINIESTROS LIQUIDADOS 

In [113]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad7
unidad7_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad7_subtotal['TOTAL'] = unidad7_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=4):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad7_subtotal, "Salvamentos de aceptaciones reaseguro interior", 67)
write_to_excel(unidad7_subtotal, "Salvamentos de aceptaciones reaseguro exterior", 68)
write_to_excel(unidad7_subtotal, "Salvamentos liquidados y realizados", 69)
write_to_excel(unidad7_subtotal, "Recobros", 70)
write_to_excel(unidad7_subtotal, "Gastos Salvamentos parte reasegurada interior (resta)", 71)
write_to_excel(unidad7_subtotal, "Gastos Salvamentos parte reasegurada exterior (resta)", 72)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d67_005 = get_value(unidad7_subtotal, "Salvamentos de aceptaciones reaseguro interior")
d68_010 = get_value(unidad7_subtotal, "Salvamentos de aceptaciones reaseguro exterior")
d69_015 = get_value(unidad7_subtotal, "Salvamentos liquidados y realizados")
d70_020 = get_value(unidad7_subtotal, "Recobros")
d71_025 = get_value(unidad7_subtotal, "Gastos Salvamentos parte reasegurada interior (resta)")
d72_030 = get_value(unidad7_subtotal, "Gastos Salvamentos parte reasegurada exterior (resta)")

# Calcular el total general según la fórmula dada
d73_999 = d67_005 + d68_010 + d69_015 + d70_020 - d71_025 - d72_030

# Escribir el total general en Excel
ws.cell(row=73, column=4).value = d73_999


## 26 EXEQUIAS UNIDAD 7 

In [114]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_EXEQUIAS
unidad7_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_EXEQUIAS['TOTAL'] = unidad7_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab67_005 = get_value(unidad7_EXEQUIAS, "Salvamentos de aceptaciones reaseguro interior", 26)
ab68_010 = get_value(unidad7_EXEQUIAS, "Salvamentos de aceptaciones reaseguro exterior", 26)
ab69_015 = get_value(unidad7_EXEQUIAS, "Salvamentos liquidados y realizados", 26)
ab70_020 = get_value(unidad7_EXEQUIAS, "Recobros", 26)
ab71_025 = get_value(unidad7_EXEQUIAS, "Gastos Salvamentos parte reasegurada interior (resta)", 26)
ab72_030 = get_value(unidad7_EXEQUIAS, "Gastos Salvamentos parte reasegurada exterior (resta)", 26)

# Calcular ab73_999 usando la fórmula especificada
ab73_999 = (ab67_005 + ab68_010 + ab69_015 + ab70_020 - ab71_025 - ab72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=28, value=ab73_999)


<Cell 'Hoja1'.AB73>

## 27 ACCIDENTES PERSONALES UNIDAD 7 

In [115]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_ACCPERSONALES
unidad7_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_ACCPERSONALES['TOTAL'] = unidad7_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac67_005 = get_value(unidad7_ACCPERSONALES, "Salvamentos de aceptaciones reaseguro interior", 27)
ac68_010 = get_value(unidad7_ACCPERSONALES, "Salvamentos de aceptaciones reaseguro exterior", 27)
ac69_015 = get_value(unidad7_ACCPERSONALES, "Salvamentos liquidados y realizados", 27)
ac70_020 = get_value(unidad7_ACCPERSONALES, "Recobros", 27)
ac71_025 = get_value(unidad7_ACCPERSONALES, "Gastos Salvamentos parte reasegurada interior (resta)", 27)
ac72_030 = get_value(unidad7_ACCPERSONALES, "Gastos Salvamentos parte reasegurada exterior (resta)", 27)

# Calcular ac73_999 usando la fórmula especificada
ac73_999 = (ac67_005 + ac68_010 + ac69_015 + ac70_020 - ac71_025 - ac72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=29, value=ac73_999)


<Cell 'Hoja1'.AC73>

## 28 COLECTIVO VIDA UNIDAD 7 

In [116]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad6_COLECVI
unidad6_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad6_COLECVI['TOTAL'] = unidad6_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad6_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad67_005 = get_value(unidad6_COLECVI, "Salvamentos de aceptaciones reaseguro interior", 28)
ad68_010 = get_value(unidad6_COLECVI, "Salvamentos de aceptaciones reaseguro exterior", 28)
ad69_015 = get_value(unidad6_COLECVI, "Salvamentos liquidados y realizados", 28)
ad70_020 = get_value(unidad6_COLECVI, "Recobros", 28)
ad71_025 = get_value(unidad6_COLECVI, "Gastos Salvamentos parte reasegurada interior (resta)", 28)
ad72_030 = get_value(unidad6_COLECVI, "Gastos Salvamentos parte reasegurada exterior (resta)", 28)

# Calcular ad73_999 usando la fórmula especificada
ad73_999 = (ad67_005 + ad68_010 + ad69_015 + ad70_020 - ad71_025 - ad72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=30, value=ad73_999)


<Cell 'Hoja1'.AD73>

## 29 EDUCATIVO  UNIDAD 7 

In [117]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_EDUCATIVO
unidad7_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_EDUCATIVO['TOTAL'] = unidad7_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae67_005 = get_value(unidad7_EDUCATIVO, "Salvamentos de aceptaciones reaseguro interior", 29)
ae68_010 = get_value(unidad7_EDUCATIVO, "Salvamentos de aceptaciones reaseguro exterior", 29)
ae69_015 = get_value(unidad7_EDUCATIVO, "Salvamentos liquidados y realizados", 29)
ae70_020 = get_value(unidad7_EDUCATIVO, "Recobros", 29)
ae71_025 = get_value(unidad7_EDUCATIVO, "Gastos Salvamentos parte reasegurada interior (resta)", 29)
ae72_030 = get_value(unidad7_EDUCATIVO, "Gastos Salvamentos parte reasegurada exterior (resta)", 29)

# Calcular ae73_999 usando la fórmula especificada
ae73_999 = (ae67_005 + ae68_010 + ae69_015 + ae70_020 - ae71_025 - ae72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=31, value=ae73_999)


<Cell 'Hoja1'.AE73>

## 30  VIDA GRUPO  UNIDAD 7

In [118]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_VIDAGRUPO
unidad7_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_VIDAGRUPO['TOTAL'] = unidad7_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af67_005 = get_value(unidad7_VIDAGRUPO, "Salvamentos de aceptaciones reaseguro interior", 30)
af68_010 = get_value(unidad7_VIDAGRUPO, "Salvamentos de aceptaciones reaseguro exterior", 30)
af69_015 = get_value(unidad7_VIDAGRUPO, "Salvamentos liquidados y realizados", 30)
af70_020 = get_value(unidad7_VIDAGRUPO, "Recobros", 30)
af71_025 = get_value(unidad7_VIDAGRUPO, "Gastos Salvamentos parte reasegurada interior (resta)", 30)
af72_030 = get_value(unidad7_VIDAGRUPO, "Gastos Salvamentos parte reasegurada exterior (resta)", 30)

# Calcular af73_999 usando la fórmula especificada
af73_999 = (af67_005 + af68_010 + af69_015 + af70_020 - af71_025 - af72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=32, value=af73_999)


<Cell 'Hoja1'.AF73>

##  31 SALUD UNIDAD 7 

In [119]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_SALUD
unidad7_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_SALUD['TOTAL'] = unidad7_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag67_005 = get_value(unidad7_SALUD, "Salvamentos de aceptaciones reaseguro interior", 31)
ag68_010 = get_value(unidad7_SALUD, "Salvamentos de aceptaciones reaseguro exterior", 31)
ag69_015 = get_value(unidad7_SALUD, "Salvamentos liquidados y realizados", 31)
ag70_020 = get_value(unidad7_SALUD, "Recobros", 31)
ag71_025 = get_value(unidad7_SALUD, "Gastos Salvamentos parte reasegurada interior (resta)", 31)
ag72_030 = get_value(unidad7_SALUD, "Gastos Salvamentos parte reasegurada exterior (resta)", 31)

# Calcular ag73_999 usando la fórmula especificada
ag73_999 = (ag67_005 + ag68_010 + ag69_015 + ag70_020 - ag71_025 - ag72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=33, value=ag73_999)


<Cell 'Hoja1'.AG73>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 7  

In [120]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_ENFALTOCOSTO
unidad7_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_ENFALTOCOSTO['TOTAL'] = unidad7_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah67_005 = get_value(unidad7_ENFALTOCOSTO, "Salvamentos de aceptaciones reaseguro interior", 32)
ah68_010 = get_value(unidad7_ENFALTOCOSTO, "Salvamentos de aceptaciones reaseguro exterior", 32)
ah69_015 = get_value(unidad7_ENFALTOCOSTO, "Salvamentos liquidados y realizados", 32)
ah70_020 = get_value(unidad7_ENFALTOCOSTO, "Recobros", 32)
ah71_025 = get_value(unidad7_ENFALTOCOSTO, "Gastos Salvamentos parte reasegurada interior (resta)", 32)
ah72_030 = get_value(unidad7_ENFALTOCOSTO, "Gastos Salvamentos parte reasegurada exterior (resta)", 32)

# Calcular ah73_999 usando la fórmula especificada
ah73_999 = (ah67_005 + ah68_010 + ah69_015 + ah70_020 - ah71_025 - ah72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=34, value=ah73_999)


<Cell 'Hoja1'.AH73>

## 33 VIDA INDIVIDUAL UNIDAD 7 

In [121]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_VIDAINDIVI
unidad7_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_VIDAINDIVI['TOTAL'] = unidad7_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai67_005 = get_value(unidad7_VIDAINDIVI, "Salvamentos de aceptaciones reaseguro interior", 33)
ai68_010 = get_value(unidad7_VIDAINDIVI, "Salvamentos de aceptaciones reaseguro exterior", 33)
ai69_015 = get_value(unidad7_VIDAINDIVI, "Salvamentos liquidados y realizados", 33)
ai70_020 = get_value(unidad7_VIDAINDIVI, "Recobros", 33)
ai71_025 = get_value(unidad7_VIDAINDIVI, "Gastos Salvamentos parte reasegurada interior (resta)", 33)
ai72_030 = get_value(unidad7_VIDAINDIVI, "Gastos Salvamentos parte reasegurada exterior (resta)", 33)

# Calcular ai73_999 usando la fórmula especificada
ai73_999 = (ai67_005 + ai68_010 + ai69_015 + ai70_020 - ai71_025 - ai72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=35, value=ai73_999)


<Cell 'Hoja1'.AI73>

## 34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 7 

In [122]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_PREVIDENTIAL
unidad7_PREVIDENTIAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_PREVIDENTIAL['TOTAL'] = unidad7_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_PREVIDENTIAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj67_005 = get_value(unidad7_PREVIDENTIAL, "Salvamentos de aceptaciones reaseguro interior", 34)
aj68_010 = get_value(unidad7_PREVIDENTIAL, "Salvamentos de aceptaciones reaseguro exterior", 34)
aj69_015 = get_value(unidad7_PREVIDENTIAL, "Salvamentos liquidados y realizados", 34)
aj70_020 = get_value(unidad7_PREVIDENTIAL, "Recobros", 34)
aj71_025 = get_value(unidad7_PREVIDENTIAL, "Gastos Salvamentos parte reasegurada interior (resta)", 34)
aj72_030 = get_value(unidad7_PREVIDENTIAL, "Gastos Salvamentos parte reasegurada exterior (resta)", 34)

# Calcular aj73_999 usando la fórmula especificada
aj73_999 = (aj67_005 + aj68_010 + aj69_015 + aj70_020 - aj71_025 - aj72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=36, value=aj73_999)


<Cell 'Hoja1'.AJ73>

## 35 RIESGOS LABORALES UNIDAD 7

In [123]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_RIESGOSLAB
unidad7_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_RIESGOSLAB['TOTAL'] = unidad7_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak67_005 = get_value(unidad7_RIESGOSLAB, "Salvamentos de aceptaciones reaseguro interior", 35)
ak68_010 = get_value(unidad7_RIESGOSLAB, "Salvamentos de aceptaciones reaseguro exterior", 35)
ak69_015 = get_value(unidad7_RIESGOSLAB, "Salvamentos liquidados y realizados", 35)
ak70_020 = get_value(unidad7_RIESGOSLAB, "Recobros", 35)
ak71_025 = get_value(unidad7_RIESGOSLAB, "Gastos Salvamentos parte reasegurada interior (resta)", 35)
ak72_030 = get_value(unidad7_RIESGOSLAB, "Gastos Salvamentos parte reasegurada exterior (resta)", 35)

# Calcular ak73_999 usando la fórmula especificada
ak73_999 = (ak67_005 + ak68_010 + ak69_015 + ak70_020 - ak71_025 - ak72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=37, value=ak73_999)


<Cell 'Hoja1'.AK73>

## 36  PENSIONES LEY 100 UNIDAD 7  

In [124]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_PENSIONESLEY100
unidad7_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_PENSIONESLEY100['TOTAL'] = unidad7_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al67_005 = get_value(unidad7_PENSIONESLEY100, "Salvamentos de aceptaciones reaseguro interior", 36)
al68_010 = get_value(unidad7_PENSIONESLEY100, "Salvamentos de aceptaciones reaseguro exterior", 36)
al69_015 = get_value(unidad7_PENSIONESLEY100, "Salvamentos liquidados y realizados", 36)
al70_020 = get_value(unidad7_PENSIONESLEY100, "Recobros", 36)
al71_025 = get_value(unidad7_PENSIONESLEY100, "Gastos Salvamentos parte reasegurada interior (resta)", 36)
al72_030 = get_value(unidad7_PENSIONESLEY100, "Gastos Salvamentos parte reasegurada exterior (resta)", 36)

# Calcular al73_999 usando la fórmula especificada
al73_999 = (al67_005 + al68_010 + al69_015 + al70_020 - al71_025 - al72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=38, value=al73_999)


<Cell 'Hoja1'.AL73>

## 37 PENSIONES VOLUNTARIAS UNIDAD 7 

In [125]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_PENSIONESVOLUN
unidad7_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_PENSIONESVOLUN['TOTAL'] = unidad7_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am67_005 = get_value(unidad7_PENSIONESVOLUN, "Salvamentos de aceptaciones reaseguro interior", 37)
am68_010 = get_value(unidad7_PENSIONESVOLUN, "Salvamentos de aceptaciones reaseguro exterior", 37)
am69_015 = get_value(unidad7_PENSIONESVOLUN, "Salvamentos liquidados y realizados", 37)
am70_020 = get_value(unidad7_PENSIONESVOLUN, "Recobros", 37)
am71_025 = get_value(unidad7_PENSIONESVOLUN, "Gastos Salvamentos parte reasegurada interior (resta)", 37)
am72_030 = get_value(unidad7_PENSIONESVOLUN, "Gastos Salvamentos parte reasegurada exterior (resta)", 37)

# Calcular am73_999 usando la fórmula especificada
am73_999 = (am67_005 + am68_010 + am69_015 + am70_020 - am71_025 - am72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=39, value=am73_999)


<Cell 'Hoja1'.AM73>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 7

In [126]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_PENSIONESCONMU
unidad7_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_PENSIONESCONMU['TOTAL'] = unidad7_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an67_005 = get_value(unidad7_PENSIONESCONMU, "Salvamentos de aceptaciones reaseguro interior", 38)
an68_010 = get_value(unidad7_PENSIONESCONMU, "Salvamentos de aceptaciones reaseguro exterior", 38)
an69_015 = get_value(unidad7_PENSIONESCONMU, "Salvamentos liquidados y realizados", 38)
an70_020 = get_value(unidad7_PENSIONESCONMU, "Recobros", 38)
an71_025 = get_value(unidad7_PENSIONESCONMU, "Gastos Salvamentos parte reasegurada interior (resta)", 38)
an72_030 = get_value(unidad7_PENSIONESCONMU, "Gastos Salvamentos parte reasegurada exterior (resta)", 38)

# Calcular an73_999 usando la fórmula especificada
an73_999 = (an67_005 + an68_010 + an69_015 + an70_020 - an71_025 - an72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=40, value=an73_999)


<Cell 'Hoja1'.AN73>

## 39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 7 

In [127]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_PATRIAUTONOMO
unidad7_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_PATRIAUTONOMO['TOTAL'] = unidad7_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao67_005 = get_value(unidad7_PATRIAUTONOMO, "Salvamentos de aceptaciones reaseguro interior", 39)
ao68_010 = get_value(unidad7_PATRIAUTONOMO, "Salvamentos de aceptaciones reaseguro exterior", 39)
ao69_015 = get_value(unidad7_PATRIAUTONOMO, "Salvamentos liquidados y realizados", 39)
ao70_020 = get_value(unidad7_PATRIAUTONOMO, "Recobros", 39)
ao71_025 = get_value(unidad7_PATRIAUTONOMO, "Gastos Salvamentos parte reasegurada interior (resta)", 39)
ao72_030 = get_value(unidad7_PATRIAUTONOMO, "Gastos Salvamentos parte reasegurada exterior (resta)", 39)

# Calcular ao73_999 usando la fórmula especificada
ao73_999 = (ao67_005 + ao68_010 + ao69_015 + ao70_020 - ao71_025 - ao72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=41, value=ao73_999)


<Cell 'Hoja1'.AO73>

## 40 RENTAS VOLUNTARIAS UNIDAD 7 

In [128]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_RENTASVOLUN
unidad7_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_RENTASVOLUN['TOTAL'] = unidad7_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap67_005 = get_value(unidad7_RENTASVOLUN, "Salvamentos de aceptaciones reaseguro interior", 40)
ap68_010 = get_value(unidad7_RENTASVOLUN, "Salvamentos de aceptaciones reaseguro exterior", 40)
ap69_015 = get_value(unidad7_RENTASVOLUN, "Salvamentos liquidados y realizados", 40)
ap70_020 = get_value(unidad7_RENTASVOLUN, "Recobros", 40)
ap71_025 = get_value(unidad7_RENTASVOLUN, "Gastos Salvamentos parte reasegurada interior (resta)", 40)
ap72_030 = get_value(unidad7_RENTASVOLUN, "Gastos Salvamentos parte reasegurada exterior (resta)", 40)

# Calcular ap73_999 usando la fórmula especificada
ap73_999 = (ap67_005 + ap68_010 + ap69_015 + ap70_020 - ap71_025 - ap72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=42, value=ap73_999)


<Cell 'Hoja1'.AP73>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 7  

In [129]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_BENECONOMICOS
unidad7_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_BENECONOMICOS['TOTAL'] = unidad7_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq67_005 = get_value(unidad7_BENECONOMICOS, "Salvamentos de aceptaciones reaseguro interior", 41)
aq68_010 = get_value(unidad7_BENECONOMICOS, "Salvamentos de aceptaciones reaseguro exterior", 41)
aq69_015 = get_value(unidad7_BENECONOMICOS, "Salvamentos liquidados y realizados", 41)
aq70_020 = get_value(unidad7_BENECONOMICOS, "Recobros", 41)
aq71_025 = get_value(unidad7_BENECONOMICOS, "Gastos Salvamentos parte reasegurada interior (resta)", 41)
aq72_030 = get_value(unidad7_BENECONOMICOS, "Gastos Salvamentos parte reasegurada exterior (resta)", 41)

# Calcular aq73_999 usando la fórmula especificada
aq73_999 = (aq67_005 + aq68_010 + aq69_015 + aq70_020 - aq71_025 - aq72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=43, value=aq73_999)


<Cell 'Hoja1'.AQ73>

##  43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 7  

In [130]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad7_OPERNOASIGNABLES
unidad7_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad7_OPERNOASIGNABLES['TOTAL'] = unidad7_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Salvamentos de aceptaciones reaseguro interior", 67),
    ("Salvamentos de aceptaciones reaseguro exterior", 68),
    ("Salvamentos liquidados y realizados", 69),
    ("Recobros", 70),
    ("Gastos Salvamentos parte reasegurada interior (resta)", 71),
    ("Gastos Salvamentos parte reasegurada exterior (resta)", 72)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad7_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as67_005 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos de aceptaciones reaseguro interior", 43)
as68_010 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos de aceptaciones reaseguro exterior", 43)
as69_015 = get_value(unidad7_OPERNOASIGNABLES, "Salvamentos liquidados y realizados", 43)
as70_020 = get_value(unidad7_OPERNOASIGNABLES, "Recobros", 43)
as71_025 = get_value(unidad7_OPERNOASIGNABLES, "Gastos Salvamentos parte reasegurada interior (resta)", 43)
as72_030 = get_value(unidad7_OPERNOASIGNABLES, "Gastos Salvamentos parte reasegurada exterior (resta)", 43)

# Calcular as73_999 usando la fórmula especificada
as73_999 = (as67_005 + as68_010 + as69_015 + as70_020 - as71_025 - as72_030)

# Escribir el total general en Excel
ws.cell(row=73, column=45, value=as73_999)


<Cell 'Hoja1'.AS73>

#  UNIDAD DE CAPTURA 8  

##  CONSULTA DE TOTALES PRIMAS DEVENGADAS 8 

In [131]:
# Sumar c50_999, c57_999, c65_999 y c73_999, y escribir en la celda correspondiente

c75_999 = c50_999 - c57_999 + c65_999 + c73_999
df_c75_999 = pd.DataFrame({'total': [c75_999]})
if df_c75_999.empty or c75_999 == 0:
    df_c75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=3).value = df_c75_999['total'].iloc[0]

# Repetir el mismo proceso para las otras variables
ab75_999 = ab50_999 - ab57_999 + ab65_999 + ab73_999
df_ab75_999 = pd.DataFrame({'total': [ab75_999]})
if df_ab75_999.empty or ab75_999 == 0:
    df_ab75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=28).value = df_ab75_999['total'].iloc[0]

ac75_999 = ac50_999 - ac57_999 + ac65_999 + ac73_999
df_ac75_999 = pd.DataFrame({'total': [ac75_999]})
if df_ac75_999.empty or ac75_999 == 0:
    df_ac75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=29).value = df_ac75_999['total'].iloc[0]

ad75_999 = ad50_999 - ad57_999 + ad65_999 + ad73_999
df_ad75_999 = pd.DataFrame({'total': [ad75_999]})
if df_ad75_999.empty or ad75_999 == 0:
    df_ad75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=30).value = df_ad75_999['total'].iloc[0]

ae75_999 = ae50_999 - ae57_999 + ae65_999 + ae73_999
df_ae75_999 = pd.DataFrame({'total': [ae75_999]})
if df_ae75_999.empty or ae75_999 == 0:
    df_ae75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=31).value = df_ae75_999['total'].iloc[0]

af75_999 = af50_999 - af57_999 + af65_999 + af73_999
df_af75_999 = pd.DataFrame({'total': [af75_999]})
if df_af75_999.empty or af75_999 == 0:
    df_af75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=32).value = df_af75_999['total'].iloc[0]

ag75_999 = ag50_999 - ag57_999 + ag65_999 + ag73_999
df_ag75_999 = pd.DataFrame({'total': [ag75_999]})
if df_ag75_999.empty or ag75_999 == 0:
    df_ag75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=33).value = df_ag75_999['total'].iloc[0]

ah75_999 = ah50_999 - ah57_999 + ah65_999 + ah73_999
df_ah75_999 = pd.DataFrame({'total': [ah75_999]})
if df_ah75_999.empty or ah75_999 == 0:
    df_ah75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=34).value = df_ah75_999['total'].iloc[0]

ai75_999 = ai50_999 - ai57_999 + ai65_999 + ai73_999
df_ai75_999 = pd.DataFrame({'total': [ai75_999]})
if df_ai75_999.empty or ai75_999 == 0:
    df_ai75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=35).value = df_ai75_999['total'].iloc[0]

aj75_999 = aj50_999 - aj57_999 + aj65_999 + aj73_999
df_aj75_999 = pd.DataFrame({'total': [aj75_999]})
if df_aj75_999.empty or aj75_999 == 0:
    df_aj75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=36).value = df_aj75_999['total'].iloc[0]

ak75_999 = ak50_999 - ak57_999 + ak65_999 + ak73_999
df_ak75_999 = pd.DataFrame({'total': [ak75_999]})
if df_ak75_999.empty or ak75_999 == 0:
    df_ak75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=37).value = df_ak75_999['total'].iloc[0]

al75_999 = al50_999 - al57_999 + al65_999 + al73_999
df_al75_999 = pd.DataFrame({'total': [al75_999]})
if df_al75_999.empty or al75_999 == 0:
    df_al75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=38).value = df_al75_999['total'].iloc[0]

am75_999 = am50_999 - am57_999 + am65_999 + am73_999
df_am75_999 = pd.DataFrame({'total': [am75_999]})
if df_am75_999.empty or am75_999 == 0:
    df_am75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=39).value = df_am75_999['total'].iloc[0]

an75_999 = an50_999 - an57_999 + an65_999 + an73_999
df_an75_999 = pd.DataFrame({'total': [an75_999]})
if df_an75_999.empty or an75_999 == 0:
    df_an75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=40).value = df_an75_999['total'].iloc[0]

ao75_999 = ao50_999 - ao57_999 + ao65_999 + ao73_999
df_ao75_999 = pd.DataFrame({'total': [ao75_999]})
if df_ao75_999.empty or ao75_999 == 0:
    df_ao75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=41).value = df_ao75_999['total'].iloc[0]

ap75_999 = ap50_999 - ap57_999 + ap65_999 + ap73_999
df_ap75_999 = pd.DataFrame({'total': [ap75_999]})
if df_ap75_999.empty or ap75_999 == 0:
    df_ap75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=42).value = df_ap75_999['total'].iloc[0]

aq75_999 = aq50_999 - aq57_999 + aq65_999 + aq73_999
df_aq75_999 = pd.DataFrame({'total': [aq75_999]})
if df_aq75_999.empty or aq75_999 == 0:
    df_aq75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=43).value = df_aq75_999['total'].iloc[0]

as75_999 = as50_999 - as57_999 + as65_999 + as73_999
df_as75_999 = pd.DataFrame({'total': [as75_999]})
if df_as75_999.empty or as75_999 == 0:
    df_as75_999 = pd.DataFrame({'total': [0]})
ws.cell(row=75, column=45).value = df_as75_999['total'].iloc[0]

##  CONSULTA DE SUBTOTAL - RAMOS PRIMAS DEVENGADAS 8

In [132]:
# Realiza la suma para d75_999
d75_999 = d50_999 - d57_999 + d65_999 + d73_999
df_d75_999 = pd.DataFrame({'total': [d75_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d75_999.empty or d75_999 == 0:
    df_d75_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=75, column=4).value = df_d75_999['total'].iloc[0]


# UNIDAD DE CAPTURA 9 

##  CONSULTA DE TOTALES OTROS INGRESOS Y/O GASTOS NETOS DE REASEGURO 9 

In [133]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad9
unidad9_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad9_totales['TOTAL'] = unidad9_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=3):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad9_totales, "Comisiones de Reaseguro Cedido", 77)
write_to_excel(unidad9_totales, "Ingresos Exceso de Pérdida", 78)
write_to_excel(unidad9_totales, "Participación Utilidades Reaseguro Cedido", 79)
write_to_excel(unidad9_totales, "Intereses y Gastos Reconocidos por Reaseguradores", 80)
write_to_excel(unidad9_totales, "Intereses Reconocidos a Reaseguradores (resta)", 81)
write_to_excel(unidad9_totales, "Participación de Utilidades y Otros (resta)", 82)
write_to_excel(unidad9_totales, "Costo Exceso de Pérdida (resta)", 83)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c77_005 = get_value(unidad9_totales, "Comisiones de Reaseguro Cedido")
c78_010 = get_value(unidad9_totales, "Ingresos Exceso de Pérdida")
c79_015 = get_value(unidad9_totales, "Participación Utilidades Reaseguro Cedido")
c80_020 = get_value(unidad9_totales, "Intereses y Gastos Reconocidos por Reaseguradores")
c81_025 = get_value(unidad9_totales, "Intereses Reconocidos a Reaseguradores (resta)")
c82_030 = get_value(unidad9_totales, "Participación de Utilidades y Otros (resta)")
c83_035 = get_value(unidad9_totales, "Costo Exceso de Pérdida (resta)")

# Calcular el total general según la fórmula dada
c84_999 = c77_005 + c78_010 + c79_015 + c80_020 - c81_025 - c82_030 - c83_035

# Escribir el total general en Excel
ws.cell(row=84, column=3).value = c84_999


## CONSULTA DE SUBTOTALES SINIESTROS LIQUIDADOS UNIDAD 9

In [134]:
# Agrupar los datos por 'CONCEPTO' para obtener los subtotales en unidad9
unidad9_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad9_subtotal['TOTAL'] = unidad9_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=4):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad9_subtotal, "Comisiones de Reaseguro Cedido", 77)
write_to_excel(unidad9_subtotal, "Ingresos Exceso de Pérdida", 78)
write_to_excel(unidad9_subtotal, "Participación Utilidades Reaseguro Cedido", 79)
write_to_excel(unidad9_subtotal, "Intereses y Gastos Reconocidos por Reaseguradores", 80)
write_to_excel(unidad9_subtotal, "Intereses Reconocidos a Reaseguradores (resta)", 81)
write_to_excel(unidad9_subtotal, "Participación de Utilidades y Otros (resta)", 82)
write_to_excel(unidad9_subtotal, "Costo Exceso de Pérdida (resta)", 83)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d77_005 = get_value(unidad9_subtotal, "Comisiones de Reaseguro Cedido")
d78_010 = get_value(unidad9_subtotal, "Ingresos Exceso de Pérdida")
d79_015 = get_value(unidad9_subtotal, "Participación Utilidades Reaseguro Cedido")
d80_020 = get_value(unidad9_subtotal, "Intereses y Gastos Reconocidos por Reaseguradores")
d81_025 = get_value(unidad9_subtotal, "Intereses Reconocidos a Reaseguradores (resta)")
d82_030 = get_value(unidad9_subtotal, "Participación de Utilidades y Otros (resta)")
d83_035 = get_value(unidad9_subtotal, "Costo Exceso de Pérdida (resta)")

# Calcular el total general según la fórmula dada
d84_999 = d77_005 + d78_010 + d79_015 + d80_020 - d81_025 - d82_030 - d83_035

# Escribir el total general en Excel
ws.cell(row=84, column=4).value = d84_999


##  26 EXEQUIAS UNIDAD 9 

In [135]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_EXEQUIAS
unidad9_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_EXEQUIAS['TOTAL'] = unidad9_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab77_005 = get_value(unidad9_EXEQUIAS, "Comisiones de Reaseguro Cedido", 26)
ab78_010 = get_value(unidad9_EXEQUIAS, "Ingresos Exceso de Pérdida", 26)
ab79_015 = get_value(unidad9_EXEQUIAS, "Participación Utilidades Reaseguro Cedido", 26)
ab80_020 = get_value(unidad9_EXEQUIAS, "Intereses y Gastos Reconocidos por Reaseguradores", 26)
ab81_025 = get_value(unidad9_EXEQUIAS, "Intereses Reconocidos a Reaseguradores (resta)", 26)
ab82_030 = get_value(unidad9_EXEQUIAS, "Participación de Utilidades y Otros (resta)", 26)
ab83_035 = get_value(unidad9_EXEQUIAS, "Costo Exceso de Pérdida (resta)", 26)

# Calcular ab84_999 usando la fórmula especificada
ab84_999 = (ab77_005 + ab78_010 + ab79_015 + ab80_020 - ab81_025 - ab82_030 - ab83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=28, value=ab84_999)


<Cell 'Hoja1'.AB84>

## 27 ACCIDENTES PERSONALES UNIDAD 9  

In [136]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_ACCPERSONALES
unidad9_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_ACCPERSONALES['TOTAL'] = unidad9_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac77_005 = get_value(unidad9_ACCPERSONALES, "Comisiones de Reaseguro Cedido", 27)
ac78_010 = get_value(unidad9_ACCPERSONALES, "Ingresos Exceso de Pérdida", 27)
ac79_015 = get_value(unidad9_ACCPERSONALES, "Participación Utilidades Reaseguro Cedido", 27)
ac80_020 = get_value(unidad9_ACCPERSONALES, "Intereses y Gastos Reconocidos por Reaseguradores", 27)
ac81_025 = get_value(unidad9_ACCPERSONALES, "Intereses Reconocidos a Reaseguradores (resta)", 27)
ac82_030 = get_value(unidad9_ACCPERSONALES, "Participación de Utilidades y Otros (resta)", 27)
ac83_035 = get_value(unidad9_ACCPERSONALES, "Costo Exceso de Pérdida (resta)", 27)

# Calcular ac84_999 usando la fórmula especificada
ac84_999 = (ac77_005 + ac78_010 + ac79_015 + ac80_020 - ac81_025 - ac82_030 - ac83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=29, value=ac84_999)


<Cell 'Hoja1'.AC84>

##  28 COLECTIVO VIDA UNIDAD 9  

In [137]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_COLECVI
unidad9_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_COLECVI['TOTAL'] = unidad9_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad77_005 = get_value(unidad9_COLECVI, "Comisiones de Reaseguro Cedido", 28)
ad78_010 = get_value(unidad9_COLECVI, "Ingresos Exceso de Pérdida", 28)
ad79_015 = get_value(unidad9_COLECVI, "Participación Utilidades Reaseguro Cedido", 28)
ad80_020 = get_value(unidad9_COLECVI, "Intereses y Gastos Reconocidos por Reaseguradores", 28)
ad81_025 = get_value(unidad9_COLECVI, "Intereses Reconocidos a Reaseguradores (resta)", 28)
ad82_030 = get_value(unidad9_COLECVI, "Participación de Utilidades y Otros (resta)", 28)
ad83_035 = get_value(unidad9_COLECVI, "Costo Exceso de Pérdida (resta)", 28)

# Calcular ad84_999 usando la fórmula especificada
ad84_999 = (ad77_005 + ad78_010 + ad79_015 + ad80_020 - ad81_025 - ad82_030 - ad83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=30, value=ad84_999)


<Cell 'Hoja1'.AD84>

## 29 EDUCATIVO  UNIDAD 9 

In [138]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_EDUCATIVO
unidad9_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_EDUCATIVO['TOTAL'] = unidad9_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae77_005 = get_value(unidad9_EDUCATIVO, "Comisiones de Reaseguro Cedido", 29)
ae78_010 = get_value(unidad9_EDUCATIVO, "Ingresos Exceso de Pérdida", 29)
ae79_015 = get_value(unidad9_EDUCATIVO, "Participación Utilidades Reaseguro Cedido", 29)
ae80_020 = get_value(unidad9_EDUCATIVO, "Intereses y Gastos Reconocidos por Reaseguradores", 29)
ae81_025 = get_value(unidad9_EDUCATIVO, "Intereses Reconocidos a Reaseguradores (resta)", 29)
ae82_030 = get_value(unidad9_EDUCATIVO, "Participación de Utilidades y Otros (resta)", 29)
ae83_035 = get_value(unidad9_EDUCATIVO, "Costo Exceso de Pérdida (resta)", 29)

# Calcular ae84_999 usando la fórmula especificada
ae84_999 = (ae77_005 + ae78_010 + ae79_015 + ae80_020 - ae81_025 - ae82_030 - ae83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=31, value=ae84_999)


<Cell 'Hoja1'.AE84>

## 30  VIDA GRUPO  UNIDAD 9  

In [139]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_VIDAGRUPO
unidad9_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_VIDAGRUPO['TOTAL'] = unidad9_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af77_005 = get_value(unidad9_VIDAGRUPO, "Comisiones de Reaseguro Cedido", 30)
af78_010 = get_value(unidad9_VIDAGRUPO, "Ingresos Exceso de Pérdida", 30)
af79_015 = get_value(unidad9_VIDAGRUPO, "Participación Utilidades Reaseguro Cedido", 30)
af80_020 = get_value(unidad9_VIDAGRUPO, "Intereses y Gastos Reconocidos por Reaseguradores", 30)
af81_025 = get_value(unidad9_VIDAGRUPO, "Intereses Reconocidos a Reaseguradores (resta)", 30)
af82_030 = get_value(unidad9_VIDAGRUPO, "Participación de Utilidades y Otros (resta)", 30)
af83_035 = get_value(unidad9_VIDAGRUPO, "Costo Exceso de Pérdida (resta)", 30)

# Calcular af84_999 usando la fórmula especificada
af84_999 = (af77_005 + af78_010 + af79_015 + af80_020 - af81_025 - af82_030 - af83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=32, value=af84_999)


<Cell 'Hoja1'.AF84>

##  31 SALUD UNIDAD 9 

In [140]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_SALUD
unidad9_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_SALUD['TOTAL'] = unidad9_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag77_005 = get_value(unidad9_SALUD, "Comisiones de Reaseguro Cedido", 31)
ag78_010 = get_value(unidad9_SALUD, "Ingresos Exceso de Pérdida", 31)
ag79_015 = get_value(unidad9_SALUD, "Participación Utilidades Reaseguro Cedido", 31)
ag80_020 = get_value(unidad9_SALUD, "Intereses y Gastos Reconocidos por Reaseguradores", 31)
ag81_025 = get_value(unidad9_SALUD, "Intereses Reconocidos a Reaseguradores (resta)", 31)
ag82_030 = get_value(unidad9_SALUD, "Participación de Utilidades y Otros (resta)", 31)
ag83_035 = get_value(unidad9_SALUD, "Costo Exceso de Pérdida (resta)", 31)

# Calcular ag84_999 usando la fórmula especificada
ag84_999 = (ag77_005 + ag78_010 + ag79_015 + ag80_020 - ag81_025 - ag82_030 - ag83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=33, value=ag84_999)


<Cell 'Hoja1'.AG84>

##  32 ENFERMEDADES DE ALTO COSTO UNIDAD 9

In [141]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_ENFALTOCOSTO
unidad9_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_ENFALTOCOSTO['TOTAL'] = unidad9_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah77_005 = get_value(unidad9_ENFALTOCOSTO, "Comisiones de Reaseguro Cedido", 32)
ah78_010 = get_value(unidad9_ENFALTOCOSTO, "Ingresos Exceso de Pérdida", 32)
ah79_015 = get_value(unidad9_ENFALTOCOSTO, "Participación Utilidades Reaseguro Cedido", 32)
ah80_020 = get_value(unidad9_ENFALTOCOSTO, "Intereses y Gastos Reconocidos por Reaseguradores", 32)
ah81_025 = get_value(unidad9_ENFALTOCOSTO, "Intereses Reconocidos a Reaseguradores (resta)", 32)
ah82_030 = get_value(unidad9_ENFALTOCOSTO, "Participación de Utilidades y Otros (resta)", 32)
ah83_035 = get_value(unidad9_ENFALTOCOSTO, "Costo Exceso de Pérdida (resta)", 32)

# Calcular ah84_999 usando la fórmula especificada
ah84_999 = (ah77_005 + ah78_010 + ah79_015 + ah80_020 - ah81_025 - ah82_030 - ah83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=34, value=ah84_999)


<Cell 'Hoja1'.AH84>

## 33 VIDA INDIVIDUAL UNIDAD 9  

In [142]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_VIDAINDIVI
unidad9_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_VIDAINDIVI['TOTAL'] = unidad9_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai77_005 = get_value(unidad9_VIDAINDIVI, "Comisiones de Reaseguro Cedido", 33)
ai78_010 = get_value(unidad9_VIDAINDIVI, "Ingresos Exceso de Pérdida", 33)
ai79_015 = get_value(unidad9_VIDAINDIVI, "Participación Utilidades Reaseguro Cedido", 33)
ai80_020 = get_value(unidad9_VIDAINDIVI, "Intereses y Gastos Reconocidos por Reaseguradores", 33)
ai81_025 = get_value(unidad9_VIDAINDIVI, "Intereses Reconocidos a Reaseguradores (resta)", 33)
ai82_030 = get_value(unidad9_VIDAINDIVI, "Participación de Utilidades y Otros (resta)", 33)
ai83_035 = get_value(unidad9_VIDAINDIVI, "Costo Exceso de Pérdida (resta)", 33)

# Calcular ai84_999 usando la fórmula especificada
ai84_999 = (ai77_005 + ai78_010 + ai79_015 + ai80_020 - ai81_025 - ai82_030 - ai83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=35, value=ai84_999)


<Cell 'Hoja1'.AI84>

##  34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 9 

In [143]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_PREVIDEINVAL
unidad9_PREVIDENTIAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_PREVIDENTIAL['TOTAL'] = unidad9_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_PREVIDENTIAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj77_005 = get_value(unidad9_PREVIDENTIAL, "Comisiones de Reaseguro Cedido", 34)
aj78_010 = get_value(unidad9_PREVIDENTIAL, "Ingresos Exceso de Pérdida", 34)
aj79_015 = get_value(unidad9_PREVIDENTIAL, "Participación Utilidades Reaseguro Cedido", 34)
aj80_020 = get_value(unidad9_PREVIDENTIAL, "Intereses y Gastos Reconocidos por Reaseguradores", 34)
aj81_025 = get_value(unidad9_PREVIDENTIAL, "Intereses Reconocidos a Reaseguradores (resta)", 34)
aj82_030 = get_value(unidad9_PREVIDENTIAL, "Participación de Utilidades y Otros (resta)", 34)
aj83_035 = get_value(unidad9_PREVIDENTIAL, "Costo Exceso de Pérdida (resta)", 34)

# Calcular aj84_999 usando la fórmula especificada
aj84_999 = (aj77_005 + aj78_010 + aj79_015 + aj80_020 - aj81_025 - aj82_030 - aj83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=36, value=aj84_999)


<Cell 'Hoja1'.AJ84>

## 35 RIESGOS LABORALES UNIDAD 9 

In [144]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_RIESGOSLAB
unidad9_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_RIESGOSLAB['TOTAL'] = unidad9_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak77_005 = get_value(unidad9_RIESGOSLAB, "Comisiones de Reaseguro Cedido", 35)
ak78_010 = get_value(unidad9_RIESGOSLAB, "Ingresos Exceso de Pérdida", 35)
ak79_015 = get_value(unidad9_RIESGOSLAB, "Participación Utilidades Reaseguro Cedido", 35)
ak80_020 = get_value(unidad9_RIESGOSLAB, "Intereses y Gastos Reconocidos por Reaseguradores", 35)
ak81_025 = get_value(unidad9_RIESGOSLAB, "Intereses Reconocidos a Reaseguradores (resta)", 35)
ak82_030 = get_value(unidad9_RIESGOSLAB, "Participación de Utilidades y Otros (resta)", 35)
ak83_035 = get_value(unidad9_RIESGOSLAB, "Costo Exceso de Pérdida (resta)", 35)

# Calcular ak84_999 usando la fórmula especificada
ak84_999 = (ak77_005 + ak78_010 + ak79_015 + ak80_020 - ak81_025 - ak82_030 - ak83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=37, value=ak84_999)


<Cell 'Hoja1'.AK84>

## 36  PENSIONES LEY 100 UNIDAD 9 

In [145]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_PENSIONESLEY100
unidad9_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_PENSIONESLEY100['TOTAL'] = unidad9_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al77_005 = get_value(unidad9_PENSIONESLEY100, "Comisiones de Reaseguro Cedido", 36)
al78_010 = get_value(unidad9_PENSIONESLEY100, "Ingresos Exceso de Pérdida", 36)
al79_015 = get_value(unidad9_PENSIONESLEY100, "Participación Utilidades Reaseguro Cedido", 36)
al80_020 = get_value(unidad9_PENSIONESLEY100, "Intereses y Gastos Reconocidos por Reaseguradores", 36)
al81_025 = get_value(unidad9_PENSIONESLEY100, "Intereses Reconocidos a Reaseguradores (resta)", 36)
al82_030 = get_value(unidad9_PENSIONESLEY100, "Participación de Utilidades y Otros (resta)", 36)
al83_035 = get_value(unidad9_PENSIONESLEY100, "Costo Exceso de Pérdida (resta)", 36)

# Calcular al84_999 usando la fórmula especificada
al84_999 = (al77_005 + al78_010 + al79_015 + al80_020 - al81_025 - al82_030 - al83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=38, value=al84_999)


<Cell 'Hoja1'.AL84>

## 37 PENSIONES VOLUNTARIAS UNIDAD 9

In [146]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_PENSIONESVOLUN
unidad9_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_PENSIONESVOLUN['TOTAL'] = unidad9_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am77_005 = get_value(unidad9_PENSIONESVOLUN, "Comisiones de Reaseguro Cedido", 37)
am78_010 = get_value(unidad9_PENSIONESVOLUN, "Ingresos Exceso de Pérdida", 37)
am79_015 = get_value(unidad9_PENSIONESVOLUN, "Participación Utilidades Reaseguro Cedido", 37)
am80_020 = get_value(unidad9_PENSIONESVOLUN, "Intereses y Gastos Reconocidos por Reaseguradores", 37)
am81_025 = get_value(unidad9_PENSIONESVOLUN, "Intereses Reconocidos a Reaseguradores (resta)", 37)
am82_030 = get_value(unidad9_PENSIONESVOLUN, "Participación de Utilidades y Otros (resta)", 37)
am83_035 = get_value(unidad9_PENSIONESVOLUN, "Costo Exceso de Pérdida (resta)", 37)

# Calcular am84_999 usando la fórmula especificada
am84_999 = (am77_005 + am78_010 + am79_015 + am80_020 - am81_025 - am82_030 - am83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=39, value=am84_999)


<Cell 'Hoja1'.AM84>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 9 

In [147]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_PENSIONESCONMU
unidad9_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_PENSIONESCONMU['TOTAL'] = unidad9_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an77_005 = get_value(unidad9_PENSIONESCONMU, "Comisiones de Reaseguro Cedido", 38)
an78_010 = get_value(unidad9_PENSIONESCONMU, "Ingresos Exceso de Pérdida", 38)
an79_015 = get_value(unidad9_PENSIONESCONMU, "Participación Utilidades Reaseguro Cedido", 38)
an80_020 = get_value(unidad9_PENSIONESCONMU, "Intereses y Gastos Reconocidos por Reaseguradores", 38)
an81_025 = get_value(unidad9_PENSIONESCONMU, "Intereses Reconocidos a Reaseguradores (resta)", 38)
an82_030 = get_value(unidad9_PENSIONESCONMU, "Participación de Utilidades y Otros (resta)", 38)
an83_035 = get_value(unidad9_PENSIONESCONMU, "Costo Exceso de Pérdida (resta)", 38)

# Calcular an84_999 usando la fórmula especificada
an84_999 = (an77_005 + an78_010 + an79_015 + an80_020 - an81_025 - an82_030 - an83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=40, value=an84_999)


<Cell 'Hoja1'.AN84>

## 39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 9 

In [148]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_PATRIAUTONOMO
unidad9_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_PATRIAUTONOMO['TOTAL'] = unidad9_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao77_005 = get_value(unidad9_PATRIAUTONOMO, "Comisiones de Reaseguro Cedido", 39)
ao78_010 = get_value(unidad9_PATRIAUTONOMO, "Ingresos Exceso de Pérdida", 39)
ao79_015 = get_value(unidad9_PATRIAUTONOMO, "Participación Utilidades Reaseguro Cedido", 39)
ao80_020 = get_value(unidad9_PATRIAUTONOMO, "Intereses y Gastos Reconocidos por Reaseguradores", 39)
ao81_025 = get_value(unidad9_PATRIAUTONOMO, "Intereses Reconocidos a Reaseguradores (resta)", 39)
ao82_030 = get_value(unidad9_PATRIAUTONOMO, "Participación de Utilidades y Otros (resta)", 39)
ao83_035 = get_value(unidad9_PATRIAUTONOMO, "Costo Exceso de Pérdida (resta)", 39)

# Calcular ao84_999 usando la fórmula especificada
ao84_999 = (ao77_005 + ao78_010 + ao79_015 + ao80_020 - ao81_025 - ao82_030 - ao83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=41, value=ao84_999)


<Cell 'Hoja1'.AO84>

## 40 RENTAS VOLUNTARIAS UNIDAD 9

In [149]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_RENTASVOLUN
unidad9_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_RENTASVOLUN['TOTAL'] = unidad9_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap77_005 = get_value(unidad9_RENTASVOLUN, "Comisiones de Reaseguro Cedido", 40)
ap78_010 = get_value(unidad9_RENTASVOLUN, "Ingresos Exceso de Pérdida", 40)
ap79_015 = get_value(unidad9_RENTASVOLUN, "Participación Utilidades Reaseguro Cedido", 40)
ap80_020 = get_value(unidad9_RENTASVOLUN, "Intereses y Gastos Reconocidos por Reaseguradores", 40)
ap81_025 = get_value(unidad9_RENTASVOLUN, "Intereses Reconocidos a Reaseguradores (resta)", 40)
ap82_030 = get_value(unidad9_RENTASVOLUN, "Participación de Utilidades y Otros (resta)", 40)
ap83_035 = get_value(unidad9_RENTASVOLUN, "Costo Exceso de Pérdida (resta)", 40)

# Calcular ap84_999 usando la fórmula especificada
ap84_999 = (ap77_005 + ap78_010 + ap79_015 + ap80_020 - ap81_025 - ap82_030 - ap83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=42, value=ap84_999)


<Cell 'Hoja1'.AP84>

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 9

In [150]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_BENECONOMICOS
unidad9_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_BENECONOMICOS['TOTAL'] = unidad9_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq77_005 = get_value(unidad9_BENECONOMICOS, "Comisiones de Reaseguro Cedido", 41)
aq78_010 = get_value(unidad9_BENECONOMICOS, "Ingresos Exceso de Pérdida", 41)
aq79_015 = get_value(unidad9_BENECONOMICOS, "Participación Utilidades Reaseguro Cedido", 41)
aq80_020 = get_value(unidad9_BENECONOMICOS, "Intereses y Gastos Reconocidos por Reaseguradores", 41)
aq81_025 = get_value(unidad9_BENECONOMICOS, "Intereses Reconocidos a Reaseguradores (resta)", 41)
aq82_030 = get_value(unidad9_BENECONOMICOS, "Participación de Utilidades y Otros (resta)", 41)
aq83_035 = get_value(unidad9_BENECONOMICOS, "Costo Exceso de Pérdida (resta)", 41)

# Calcular aq84_999 usando la fórmula especificada
aq84_999 = (aq77_005 + aq78_010 + aq79_015 + aq80_020 - aq81_025 - aq82_030 - aq83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=43, value=aq84_999)


<Cell 'Hoja1'.AQ84>

##  43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 9

In [151]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad9_OPERNOASIGNABLES
unidad9_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad9_OPERNOASIGNABLES['TOTAL'] = unidad9_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Comisiones de Reaseguro Cedido", 77),
    ("Ingresos Exceso de Pérdida", 78),
    ("Participación Utilidades Reaseguro Cedido", 79),
    ("Intereses y Gastos Reconocidos por Reaseguradores", 80),
    ("Intereses Reconocidos a Reaseguradores (resta)", 81),
    ("Participación de Utilidades y Otros (resta)", 82),
    ("Costo Exceso de Pérdida (resta)", 83)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad9_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as77_005 = get_value(unidad9_OPERNOASIGNABLES, "Comisiones de Reaseguro Cedido", 43)
as78_010 = get_value(unidad9_OPERNOASIGNABLES, "Ingresos Exceso de Pérdida", 43)
as79_015 = get_value(unidad9_OPERNOASIGNABLES, "Participación Utilidades Reaseguro Cedido", 43)
as80_020 = get_value(unidad9_OPERNOASIGNABLES, "Intereses y Gastos Reconocidos por Reaseguradores", 43)
as81_025 = get_value(unidad9_OPERNOASIGNABLES, "Intereses Reconocidos a Reaseguradores (resta)", 43)
as82_030 = get_value(unidad9_OPERNOASIGNABLES, "Participación de Utilidades y Otros (resta)", 43)
as83_035 = get_value(unidad9_OPERNOASIGNABLES, "Costo Exceso de Pérdida (resta)", 43)

# Calcular as84_999 usando la fórmula especificada
as84_999 = (as77_005 + as78_010 + as79_015 + as80_020 - as81_025 - as82_030 - as83_035)

# Escribir el total general en Excel
ws.cell(row=84, column=45, value=as84_999)


<Cell 'Hoja1'.AS84>

#  UNIDAD DE CAPTURA 10  

##  CONSULTA DE TOTALES OTROS INGRESOS Y/O GASTOS NETOS DE SEGUROS 10 

In [152]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad10
unidad10_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad10_totales['TOTAL'] = unidad10_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=3):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad10_totales, "Otros productos de seguros", 86)
write_to_excel(unidad10_totales, "Remuneración de administración de coaseguro", 87)
write_to_excel(unidad10_totales, "Gastos de Administración de Coaseguro Aceptado (resta)", 88)
write_to_excel(unidad10_totales, "Provisión cuentas por cobrar actividad aseguradora (resta)", 89)
write_to_excel(unidad10_totales, "Costos Riesgos Profesionales (resta)", 90)
write_to_excel(unidad10_totales, "Contribuciones de Seguros (resta)", 91)
write_to_excel(unidad10_totales, "Otros Egresos (resta)", 92)
write_to_excel(unidad10_totales, "Recuperación provisión cuentas por cobrar actividad aseguradora", 93)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c86_005 = get_value(unidad10_totales, "Otros productos de seguros")
c87_010 = get_value(unidad10_totales, "Remuneración de administración de coaseguro")
c88_015 = get_value(unidad10_totales, "Gastos de Administración de Coaseguro Aceptado (resta)")
c89_020 = get_value(unidad10_totales, "Provisión cuentas por cobrar actividad aseguradora (resta)")
c90_025 = get_value(unidad10_totales, "Costos Riesgos Profesionales (resta)")
c91_030 = get_value(unidad10_totales, "Contribuciones de Seguros (resta)")
c92_035 = get_value(unidad10_totales, "Otros Egresos (resta)")
c93_040 = get_value(unidad10_totales, "Recuperación provisión cuentas por cobrar actividad aseguradora")

# Calcular el total general según la fórmula dada
c94_999 = c86_005 + c87_010 - c88_015 - c89_020 - c90_025 - c91_030 - c92_035 + c93_040

# Escribir el total general en Excel
ws.cell(row=94, column=3).value = c94_999


## CONSULTA DE SUBTOTALES OTROS INGRESOS Y/O GASTOS NETOS DE SEGUROS 10

In [153]:
# Agrupar los datos por 'CONCEPTO' para obtener los subtotales en unidad10
unidad10_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad10_subtotal['TOTAL'] = unidad10_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=4):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad10_subtotal, "Otros productos de seguros", 86)
write_to_excel(unidad10_subtotal, "Remuneración de administración de coaseguro", 87)
write_to_excel(unidad10_subtotal, "Gastos de Administración de Coaseguro Aceptado (resta)", 88)
write_to_excel(unidad10_subtotal, "Provisión cuentas por cobrar actividad aseguradora (resta)", 89)
write_to_excel(unidad10_subtotal, "Costos Riesgos Profesionales (resta)", 90)
write_to_excel(unidad10_subtotal, "Contribuciones de Seguros (resta)", 91)
write_to_excel(unidad10_subtotal, "Otros Egresos (resta)", 92)
write_to_excel(unidad10_subtotal, "Recuperación provisión cuentas por cobrar actividad aseguradora", 93)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d86_005 = get_value(unidad10_subtotal, "Otros productos de seguros")
d87_010 = get_value(unidad10_subtotal, "Remuneración de administración de coaseguro")
d88_015 = get_value(unidad10_subtotal, "Gastos de Administración de Coaseguro Aceptado (resta)")
d89_020 = get_value(unidad10_subtotal, "Provisión cuentas por cobrar actividad aseguradora (resta)")
d90_025 = get_value(unidad10_subtotal, "Costos Riesgos Profesionales (resta)")
d91_030 = get_value(unidad10_subtotal, "Contribuciones de Seguros (resta)")
d92_035 = get_value(unidad10_subtotal, "Otros Egresos (resta)")
d93_040 = get_value(unidad10_subtotal, "Recuperación provisión cuentas por cobrar actividad aseguradora")

# Calcular el total general según la fórmula dada
d94_999 = d86_005 + d87_010 - d88_015 - d89_020 - d90_025 - d91_030 - d92_035 + d93_040

# Escribir el total general en Excel
ws.cell(row=94, column=4).value = d94_999


##   26 EXEQUIAS UNIDAD 10 

In [154]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EXEQUIAS
unidad10_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_EXEQUIAS['TOTAL'] = unidad10_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab86_005 = get_value(unidad10_EXEQUIAS, "Otros productos de seguros", 26)
ab87_010 = get_value(unidad10_EXEQUIAS, "Remuneración de administración de coaseguro", 26)
ab88_015 = get_value(unidad10_EXEQUIAS, "Gastos de Administración de Coaseguro Aceptado (resta)", 26)
ab89_020 = get_value(unidad10_EXEQUIAS, "Provisión cuentas por cobrar actividad aseguradora (resta)", 26)
ab90_025 = get_value(unidad10_EXEQUIAS, "Costos Riesgos Profesionales (resta)", 26)
ab91_030 = get_value(unidad10_EXEQUIAS, "Contribuciones de Seguros (resta)", 26)
ab92_035 = get_value(unidad10_EXEQUIAS, "Otros Egresos (resta)", 26)
ab93_040 = get_value(unidad10_EXEQUIAS, "Recuperación provisión cuentas por cobrar actividad aseguradora", 26)

# Calcular ab94_999 usando la fórmula especificada
ab94_999 = (ab86_005 + ab87_010 - ab88_015 - ab89_020 - ab90_025 - ab91_030 - ab92_035 + ab93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=28, value=ab94_999)


<Cell 'Hoja1'.AB94>

##  27 ACCIDENTES PERSONALES UNIDAD 10 

In [155]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_ACCPERSONALES
unidad10_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_ACCPERSONALES['TOTAL'] = unidad10_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac86_005 = get_value(unidad10_ACCPERSONALES, "Otros productos de seguros", 27)
ac87_010 = get_value(unidad10_ACCPERSONALES, "Remuneración de administración de coaseguro", 27)
ac88_015 = get_value(unidad10_ACCPERSONALES, "Gastos de Administración de Coaseguro Aceptado (resta)", 27)
ac89_020 = get_value(unidad10_ACCPERSONALES, "Provisión cuentas por cobrar actividad aseguradora (resta)", 27)
ac90_025 = get_value(unidad10_ACCPERSONALES, "Costos Riesgos Profesionales (resta)", 27)
ac91_030 = get_value(unidad10_ACCPERSONALES, "Contribuciones de Seguros (resta)", 27)
ac92_035 = get_value(unidad10_ACCPERSONALES, "Otros Egresos (resta)", 27)
ac93_040 = get_value(unidad10_ACCPERSONALES, "Recuperación provisión cuentas por cobrar actividad aseguradora", 27)

# Calcular ac94_999 usando la fórmula especificada
ac94_999 = (ac86_005 + ac87_010 - ac88_015 - ac89_020 - ac90_025 - ac91_030 - ac92_035 + ac93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=29, value=ac94_999)


<Cell 'Hoja1'.AC94>

## 28 COLECTIVO VIDA UNIDAD 10  

In [156]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_COLECVI
unidad10_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_COLECVI['TOTAL'] = unidad10_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad86_005 = get_value(unidad10_COLECVI, "Otros productos de seguros", 28)
ad87_010 = get_value(unidad10_COLECVI, "Remuneración de administración de coaseguro", 28)
ad88_015 = get_value(unidad10_COLECVI, "Gastos de Administración de Coaseguro Aceptado (resta)", 28)
ad89_020 = get_value(unidad10_COLECVI, "Provisión cuentas por cobrar actividad aseguradora (resta)", 28)
ad90_025 = get_value(unidad10_COLECVI, "Costos Riesgos Profesionales (resta)", 28)
ad91_030 = get_value(unidad10_COLECVI, "Contribuciones de Seguros (resta)", 28)
ad92_035 = get_value(unidad10_COLECVI, "Otros Egresos (resta)", 28)
ad93_040 = get_value(unidad10_COLECVI, "Recuperación provisión cuentas por cobrar actividad aseguradora", 28)

# Calcular ad94_999 usando la fórmula especificada
ad94_999 = (ad86_005 + ad87_010 - ad88_015 - ad89_020 - ad90_025 - ad91_030 - ad92_035 + ad93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=30, value=ad94_999)


<Cell 'Hoja1'.AD94>

## 29 EDUCATIVO  UNIDAD 10

In [157]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_EDUCATIVO
unidad10_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_EDUCATIVO['TOTAL'] = unidad10_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae86_005 = get_value(unidad10_EDUCATIVO, "Otros productos de seguros", 29)
ae87_010 = get_value(unidad10_EDUCATIVO, "Remuneración de administración de coaseguro", 29)
ae88_015 = get_value(unidad10_EDUCATIVO, "Gastos de Administración de Coaseguro Aceptado (resta)", 29)
ae89_020 = get_value(unidad10_EDUCATIVO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 29)
ae90_025 = get_value(unidad10_EDUCATIVO, "Costos Riesgos Profesionales (resta)", 29)
ae91_030 = get_value(unidad10_EDUCATIVO, "Contribuciones de Seguros (resta)", 29)
ae92_035 = get_value(unidad10_EDUCATIVO, "Otros Egresos (resta)", 29)
ae93_040 = get_value(unidad10_EDUCATIVO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 29)

# Calcular ae94_999 usando la fórmula especificada
ae94_999 = (ae86_005 + ae87_010 - ae88_015 - ae89_020 - ae90_025 - ae91_030 - ae92_035 + ae93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=31, value=ae94_999)


<Cell 'Hoja1'.AE94>

##  30  VIDA GRUPO  UNIDAD 10

In [158]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_VIDAGRUPO
unidad10_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_VIDAGRUPO['TOTAL'] = unidad10_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af86_005 = get_value(unidad10_VIDAGRUPO, "Otros productos de seguros", 30)
af87_010 = get_value(unidad10_VIDAGRUPO, "Remuneración de administración de coaseguro", 30)
af88_015 = get_value(unidad10_VIDAGRUPO, "Gastos de Administración de Coaseguro Aceptado (resta)", 30)
af89_020 = get_value(unidad10_VIDAGRUPO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 30)
af90_025 = get_value(unidad10_VIDAGRUPO, "Costos Riesgos Profesionales (resta)", 30)
af91_030 = get_value(unidad10_VIDAGRUPO, "Contribuciones de Seguros (resta)", 30)
af92_035 = get_value(unidad10_VIDAGRUPO, "Otros Egresos (resta)", 30)
af93_040 = get_value(unidad10_VIDAGRUPO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 30)

# Calcular af94_999 usando la fórmula especificada
af94_999 = (af86_005 + af87_010 - af88_015 - af89_020 - af90_025 - af91_030 - af92_035 + af93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=32, value=af94_999)


<Cell 'Hoja1'.AF94>

## 31 SALUD UNIDAD 10 

In [159]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_SALUD
unidad10_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_SALUD['TOTAL'] = unidad10_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag86_005 = get_value(unidad10_SALUD, "Otros productos de seguros", 31)
ag87_010 = get_value(unidad10_SALUD, "Remuneración de administración de coaseguro", 31)
ag88_015 = get_value(unidad10_SALUD, "Gastos de Administración de Coaseguro Aceptado (resta)", 31)
ag89_020 = get_value(unidad10_SALUD, "Provisión cuentas por cobrar actividad aseguradora (resta)", 31)
ag90_025 = get_value(unidad10_SALUD, "Costos Riesgos Profesionales (resta)", 31)
ag91_030 = get_value(unidad10_SALUD, "Contribuciones de Seguros (resta)", 31)
ag92_035 = get_value(unidad10_SALUD, "Otros Egresos (resta)", 31)
ag93_040 = get_value(unidad10_SALUD, "Recuperación provisión cuentas por cobrar actividad aseguradora", 31)

# Calcular ag94_999 usando la fórmula especificada
ag94_999 = (ag86_005 + ag87_010 - ag88_015 - ag89_020 - ag90_025 - ag91_030 - ag92_035 + ag93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=33, value=ag94_999)


<Cell 'Hoja1'.AG94>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 10

In [160]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_ENFALTOCOSTO
unidad10_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_ENFALTOCOSTO['TOTAL'] = unidad10_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah86_005 = get_value(unidad10_ENFALTOCOSTO, "Otros productos de seguros", 32)
ah87_010 = get_value(unidad10_ENFALTOCOSTO, "Remuneración de administración de coaseguro", 32)
ah88_015 = get_value(unidad10_ENFALTOCOSTO, "Gastos de Administración de Coaseguro Aceptado (resta)", 32)
ah89_020 = get_value(unidad10_ENFALTOCOSTO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 32)
ah90_025 = get_value(unidad10_ENFALTOCOSTO, "Costos Riesgos Profesionales (resta)", 32)
ah91_030 = get_value(unidad10_ENFALTOCOSTO, "Contribuciones de Seguros (resta)", 32)
ah92_035 = get_value(unidad10_ENFALTOCOSTO, "Otros Egresos (resta)", 32)
ah93_040 = get_value(unidad10_ENFALTOCOSTO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 32)

# Calcular ah94_999 usando la fórmula especificada
ah94_999 = (ah86_005 + ah87_010 - ah88_015 - ah89_020 - ah90_025 - ah91_030 - ah92_035 + ah93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=34, value=ah94_999)


<Cell 'Hoja1'.AH94>

##  33 VIDA INDIVIDUAL UNIDAD 10 

In [161]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_VIDAINDIVI
unidad10_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_VIDAINDIVI['TOTAL'] = unidad10_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai86_005 = get_value(unidad10_VIDAINDIVI, "Otros productos de seguros", 33)
ai87_010 = get_value(unidad10_VIDAINDIVI, "Remuneración de administración de coaseguro", 33)
ai88_015 = get_value(unidad10_VIDAINDIVI, "Gastos de Administración de Coaseguro Aceptado (resta)", 33)
ai89_020 = get_value(unidad10_VIDAINDIVI, "Provisión cuentas por cobrar actividad aseguradora (resta)", 33)
ai90_025 = get_value(unidad10_VIDAINDIVI, "Costos Riesgos Profesionales (resta)", 33)
ai91_030 = get_value(unidad10_VIDAINDIVI, "Contribuciones de Seguros (resta)", 33)
ai92_035 = get_value(unidad10_VIDAINDIVI, "Otros Egresos (resta)", 33)
ai93_040 = get_value(unidad10_VIDAINDIVI, "Recuperación provisión cuentas por cobrar actividad aseguradora", 33)

# Calcular ai94_999 usando la fórmula especificada
ai94_999 = (ai86_005 + ai87_010 - ai88_015 - ai89_020 - ai90_025 - ai91_030 - ai92_035 + ai93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=35, value=ai94_999)


<Cell 'Hoja1'.AI94>

## 34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 10 

In [162]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_PREVIDINVAL
unidad10_PREVIDINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_PREVIDINVAL['TOTAL'] = unidad10_PREVIDINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_PREVIDINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj86_005 = get_value(unidad10_PREVIDINVAL, "Otros productos de seguros", 34)
aj87_010 = get_value(unidad10_PREVIDINVAL, "Remuneración de administración de coaseguro", 34)
aj88_015 = get_value(unidad10_PREVIDINVAL, "Gastos de Administración de Coaseguro Aceptado (resta)", 34)
aj89_020 = get_value(unidad10_PREVIDINVAL, "Provisión cuentas por cobrar actividad aseguradora (resta)", 34)
aj90_025 = get_value(unidad10_PREVIDINVAL, "Costos Riesgos Profesionales (resta)", 34)
aj91_030 = get_value(unidad10_PREVIDINVAL, "Contribuciones de Seguros (resta)", 34)
aj92_035 = get_value(unidad10_PREVIDINVAL, "Otros Egresos (resta)", 34)
aj93_040 = get_value(unidad10_PREVIDINVAL, "Recuperación provisión cuentas por cobrar actividad aseguradora", 34)

# Calcular aj94_999 usando la fórmula especificada
aj94_999 = (aj86_005 + aj87_010 - aj88_015 - aj89_020 - aj90_025 - aj91_030 - aj92_035 + aj93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=36, value=aj94_999)


<Cell 'Hoja1'.AJ94>

## 35 RIESGOS LABORALES UNIDAD 10

In [163]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_RIESGOSLAB
unidad10_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_RIESGOSLAB['TOTAL'] = unidad10_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak86_005 = get_value(unidad10_RIESGOSLAB, "Otros productos de seguros", 35)
ak87_010 = get_value(unidad10_RIESGOSLAB, "Remuneración de administración de coaseguro", 35)
ak88_015 = get_value(unidad10_RIESGOSLAB, "Gastos de Administración de Coaseguro Aceptado (resta)", 35)
ak89_020 = get_value(unidad10_RIESGOSLAB, "Provisión cuentas por cobrar actividad aseguradora (resta)", 35)
ak90_025 = get_value(unidad10_RIESGOSLAB, "Costos Riesgos Profesionales (resta)", 35)
ak91_030 = get_value(unidad10_RIESGOSLAB, "Contribuciones de Seguros (resta)", 35)
ak92_035 = get_value(unidad10_RIESGOSLAB, "Otros Egresos (resta)", 35)
ak93_040 = get_value(unidad10_RIESGOSLAB, "Recuperación provisión cuentas por cobrar actividad aseguradora", 35)

# Calcular ak94_999 usando la fórmula especificada
ak94_999 = (ak86_005 + ak87_010 - ak88_015 - ak89_020 - ak90_025 - ak91_030 - ak92_035 + ak93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=37, value=ak94_999)


<Cell 'Hoja1'.AK94>

##  36  PENSIONES LEY 100 UNIDAD 10

In [164]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_PENSIONESLEY100
unidad10_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_PENSIONESLEY100['TOTAL'] = unidad10_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al86_005 = get_value(unidad10_PENSIONESLEY100, "Otros productos de seguros", 36)
al87_010 = get_value(unidad10_PENSIONESLEY100, "Remuneración de administración de coaseguro", 36)
al88_015 = get_value(unidad10_PENSIONESLEY100, "Gastos de Administración de Coaseguro Aceptado (resta)", 36)
al89_020 = get_value(unidad10_PENSIONESLEY100, "Provisión cuentas por cobrar actividad aseguradora (resta)", 36)
al90_025 = get_value(unidad10_PENSIONESLEY100, "Costos Riesgos Profesionales (resta)", 36)
al91_030 = get_value(unidad10_PENSIONESLEY100, "Contribuciones de Seguros (resta)", 36)
al92_035 = get_value(unidad10_PENSIONESLEY100, "Otros Egresos (resta)", 36)
al93_040 = get_value(unidad10_PENSIONESLEY100, "Recuperación provisión cuentas por cobrar actividad aseguradora", 36)

# Calcular al94_999 usando la fórmula especificada
al94_999 = (al86_005 + al87_010 - al88_015 - al89_020 - al90_025 - al91_030 - al92_035 + al93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=38, value=al94_999)


<Cell 'Hoja1'.AL94>

##  37 PENSIONES VOLUNTARIAS UNIDAD 10 

In [165]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_PENSIONESVOLUN
unidad10_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_PENSIONESVOLUN['TOTAL'] = unidad10_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am86_005 = get_value(unidad10_PENSIONESVOLUN, "Otros productos de seguros", 37)
am87_010 = get_value(unidad10_PENSIONESVOLUN, "Remuneración de administración de coaseguro", 37)
am88_015 = get_value(unidad10_PENSIONESVOLUN, "Gastos de Administración de Coaseguro Aceptado (resta)", 37)
am89_020 = get_value(unidad10_PENSIONESVOLUN, "Provisión cuentas por cobrar actividad aseguradora (resta)", 37)
am90_025 = get_value(unidad10_PENSIONESVOLUN, "Costos Riesgos Profesionales (resta)", 37)
am91_030 = get_value(unidad10_PENSIONESVOLUN, "Contribuciones de Seguros (resta)", 37)
am92_035 = get_value(unidad10_PENSIONESVOLUN, "Otros Egresos (resta)", 37)
am93_040 = get_value(unidad10_PENSIONESVOLUN, "Recuperación provisión cuentas por cobrar actividad aseguradora", 37)

# Calcular am94_999 usando la fórmula especificada
am94_999 = (am86_005 + am87_010 - am88_015 - am89_020 - am90_025 - am91_030 - am92_035 + am93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=39, value=am94_999)


<Cell 'Hoja1'.AM94>

##  38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 10

In [166]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_PENSIONESCONMU
unidad10_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_PENSIONESCONMU['TOTAL'] = unidad10_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an86_005 = get_value(unidad10_PENSIONESCONMU, "Otros productos de seguros", 38)
an87_010 = get_value(unidad10_PENSIONESCONMU, "Remuneración de administración de coaseguro", 38)
an88_015 = get_value(unidad10_PENSIONESCONMU, "Gastos de Administración de Coaseguro Aceptado (resta)", 38)
an89_020 = get_value(unidad10_PENSIONESCONMU, "Provisión cuentas por cobrar actividad aseguradora (resta)", 38)
an90_025 = get_value(unidad10_PENSIONESCONMU, "Costos Riesgos Profesionales (resta)", 38)
an91_030 = get_value(unidad10_PENSIONESCONMU, "Contribuciones de Seguros (resta)", 38)
an92_035 = get_value(unidad10_PENSIONESCONMU, "Otros Egresos (resta)", 38)
an93_040 = get_value(unidad10_PENSIONESCONMU, "Recuperación provisión cuentas por cobrar actividad aseguradora", 38)

# Calcular an94_999 usando la fórmula especificada
an94_999 = (an86_005 + an87_010 - an88_015 - an89_020 - an90_025 - an91_030 - an92_035 + an93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=40, value=an94_999)


<Cell 'Hoja1'.AN94>

## 39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 10

In [167]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_PATRIAUTONOMO
unidad10_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_PATRIAUTONOMO['TOTAL'] = unidad10_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao86_005 = get_value(unidad10_PATRIAUTONOMO, "Otros productos de seguros", 39)
ao87_010 = get_value(unidad10_PATRIAUTONOMO, "Remuneración de administración de coaseguro", 39)
ao88_015 = get_value(unidad10_PATRIAUTONOMO, "Gastos de Administración de Coaseguro Aceptado (resta)", 39)
ao89_020 = get_value(unidad10_PATRIAUTONOMO, "Provisión cuentas por cobrar actividad aseguradora (resta)", 39)
ao90_025 = get_value(unidad10_PATRIAUTONOMO, "Costos Riesgos Profesionales (resta)", 39)
ao91_030 = get_value(unidad10_PATRIAUTONOMO, "Contribuciones de Seguros (resta)", 39)
ao92_035 = get_value(unidad10_PATRIAUTONOMO, "Otros Egresos (resta)", 39)
ao93_040 = get_value(unidad10_PATRIAUTONOMO, "Recuperación provisión cuentas por cobrar actividad aseguradora", 39)

# Calcular ao94_999 usando la fórmula especificada
ao94_999 = (ao86_005 + ao87_010 - ao88_015 - ao89_020 - ao90_025 - ao91_030 - ao92_035 + ao93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=41, value=ao94_999)


<Cell 'Hoja1'.AO94>

##  40 RENTAS VOLUNTARIAS UNIDAD 10 

In [168]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_RENTASVOLUN
unidad10_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_RENTASVOLUN['TOTAL'] = unidad10_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap86_005 = get_value(unidad10_RENTASVOLUN, "Otros productos de seguros", 40)
ap87_010 = get_value(unidad10_RENTASVOLUN, "Remuneración de administración de coaseguro", 40)
ap88_015 = get_value(unidad10_RENTASVOLUN, "Gastos de Administración de Coaseguro Aceptado (resta)", 40)
ap89_020 = get_value(unidad10_RENTASVOLUN, "Provisión cuentas por cobrar actividad aseguradora (resta)", 40)
ap90_025 = get_value(unidad10_RENTASVOLUN, "Costos Riesgos Profesionales (resta)", 40)
ap91_030 = get_value(unidad10_RENTASVOLUN, "Contribuciones de Seguros (resta)", 40)
ap92_035 = get_value(unidad10_RENTASVOLUN, "Otros Egresos (resta)", 40)
ap93_040 = get_value(unidad10_RENTASVOLUN, "Recuperación provisión cuentas por cobrar actividad aseguradora", 40)

# Calcular ap94_999 usando la fórmula especificada
ap94_999 = (ap86_005 + ap87_010 - ap88_015 - ap89_020 - ap90_025 - ap91_030 - ap92_035 + ap93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=42, value=ap94_999)


<Cell 'Hoja1'.AP94>

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 10

In [169]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_BENECONOMICOS
unidad10_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_BENECONOMICOS['TOTAL'] = unidad10_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq86_005 = get_value(unidad10_BENECONOMICOS, "Otros productos de seguros", 41)
aq87_010 = get_value(unidad10_BENECONOMICOS, "Remuneración de administración de coaseguro", 41)
aq88_015 = get_value(unidad10_BENECONOMICOS, "Gastos de Administración de Coaseguro Aceptado (resta)", 41)
aq89_020 = get_value(unidad10_BENECONOMICOS, "Provisión cuentas por cobrar actividad aseguradora (resta)", 41)
aq90_025 = get_value(unidad10_BENECONOMICOS, "Costos Riesgos Profesionales (resta)", 41)
aq91_030 = get_value(unidad10_BENECONOMICOS, "Contribuciones de Seguros (resta)", 41)
aq92_035 = get_value(unidad10_BENECONOMICOS, "Otros Egresos (resta)", 41)
aq93_040 = get_value(unidad10_BENECONOMICOS, "Recuperación provisión cuentas por cobrar actividad aseguradora", 41)

# Calcular aq94_999 usando la fórmula especificada
aq94_999 = (aq86_005 + aq87_010 - aq88_015 - aq89_020 - aq90_025 - aq91_030 - aq92_035 + aq93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=43, value=aq94_999)


<Cell 'Hoja1'.AQ94>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 10 

In [170]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad10_OPERNOASIGNABLES
unidad10_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad10_OPERNOASIGNABLES['TOTAL'] = unidad10_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otros productos de seguros", 86),
    ("Remuneración de administración de coaseguro", 87),
    ("Gastos de Administración de Coaseguro Aceptado (resta)", 88),
    ("Provisión cuentas por cobrar actividad aseguradora (resta)", 89),
    ("Costos Riesgos Profesionales (resta)", 90),
    ("Contribuciones de Seguros (resta)", 91),
    ("Otros Egresos (resta)", 92),
    ("Recuperación provisión cuentas por cobrar actividad aseguradora", 93)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad10_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as86_005 = get_value(unidad10_OPERNOASIGNABLES, "Otros productos de seguros", 43)
as87_010 = get_value(unidad10_OPERNOASIGNABLES, "Remuneración de administración de coaseguro", 43)
as88_015 = get_value(unidad10_OPERNOASIGNABLES, "Gastos de Administración de Coaseguro Aceptado (resta)", 43)
as89_020 = get_value(unidad10_OPERNOASIGNABLES, "Provisión cuentas por cobrar actividad aseguradora (resta)", 43)
as90_025 = get_value(unidad10_OPERNOASIGNABLES, "Costos Riesgos Profesionales (resta)", 43)
as91_030 = get_value(unidad10_OPERNOASIGNABLES, "Contribuciones de Seguros (resta)", 43)
as92_035 = get_value(unidad10_OPERNOASIGNABLES, "Otros Egresos (resta)", 43)
as93_040 = get_value(unidad10_OPERNOASIGNABLES, "Recuperación provisión cuentas por cobrar actividad aseguradora", 43)

# Calcular as94_999 usando la fórmula especificada
as94_999 = (as86_005 + as87_010 - as88_015 - as89_020 - as90_025 - as91_030 - as92_035 + as93_040)

# Escribir el total general en Excel
ws.cell(row=94, column=45, value=as94_999)


<Cell 'Hoja1'.AS94>

# UNIDAD DE CAPTURA 11 

## 26 EXEQUIAS UNIDAD 11 

In [171]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_EXEQUIAS = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '26 EXEQUIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ab98_015 = get_value_super_ramo(unidad11_EXEQUIAS, "Gastos de administración directos ramo")
ab99_020 = get_value_super_ramo(unidad11_EXEQUIAS, "Gastos de personal directos ramo")
ab100_025 = get_value_super_ramo(unidad11_EXEQUIAS, "Gastos de administración indirectos asignables a cada ramo")
ab101_030 = get_value_super_ramo(unidad11_EXEQUIAS, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=28, value=ab98_015)   
ws.cell(row=99, column=28, value=ab99_020)  
ws.cell(row=100, column=28, value=ab100_025) 
ws.cell(row=101, column=28, value=ab101_030) 

# Calcular e102_999 usando la fórmula especificada
ab102_999 = (ab98_015 + ab99_020 + ab100_025 + ab101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=28). value = ab102_999

##  27 ACCIDENTES PERSONALES UNIDAD 11

In [172]:
# Asegurar que VALOR es numérico
Base['VALOR'] = pd.to_numeric(Base['VALOR'], errors='coerce')

# Agrupar
unidad11_ACCPERSONALES = Base[
    (Base['SOCIEDAD_DEST'] != 'GRAL') & 
    (Base['RAMO_SUPER']  == '27 ACCIDENTES PERSONALES')
].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener valores
ac98_015 = get_value_super_ramo(unidad11_ACCPERSONALES, "Gastos de administración directos ramo")
ac99_020 = get_value_super_ramo(unidad11_ACCPERSONALES, "Gastos de personal directos ramo")
ac100_025 = get_value_super_ramo(unidad11_ACCPERSONALES, "Gastos de administración indirectos asignables a cada ramo")
ac101_030 = get_value_super_ramo(unidad11_ACCPERSONALES, "Gastos de personal indirectos asignables a cada ramo")

# Escribir en Excel
ws.cell(row=98, column=29, value=ac98_015)
ws.cell(row=99, column=29, value=ac99_020)
ws.cell(row=100, column=29, value=ac100_025)
ws.cell(row=101, column=29, value=ac101_030)

# Total
ac102_999 = ac98_015 + ac99_020 + ac100_025 + ac101_030
ws.cell(row=102, column=29, value=ac102_999)


<Cell 'Hoja1'.AC102>

##  28 COLECTIVO VIDA UNIDAD 11

In [173]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_COLECVI = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '28 COLECTIVO VIDA')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ad98_015 = get_value_super_ramo(unidad11_COLECVI, "Gastos de administración directos ramo")
ad99_020 = get_value_super_ramo(unidad11_COLECVI, "Gastos de personal directos ramo")
ad100_025 = get_value_super_ramo(unidad11_COLECVI, "Gastos de administración indirectos asignables a cada ramo")
ad101_030 = get_value_super_ramo(unidad11_COLECVI, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=30, value=ad98_015)   
ws.cell(row=99, column=30, value=ad99_020)  
ws.cell(row=100, column=30, value=ad100_025) 
ws.cell(row=101, column=30, value=ad101_030) 

# Calcular e102_999 usando la fórmula especificada
ad102_999 = (ad98_015 + ad99_020 + ad100_025 + ad101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=30, value=ad102_999)

<Cell 'Hoja1'.AD102>

##  29 EDUCATIVO  UNIDAD 11

In [174]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_EDUCATIVO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '29 EDUCATIVO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ae98_015 = get_value_super_ramo(unidad11_EDUCATIVO, "Gastos de administración directos ramo")
ae99_020 = get_value_super_ramo(unidad11_EDUCATIVO, "Gastos de personal directos ramo")
ae100_025 = get_value_super_ramo(unidad11_EDUCATIVO, "Gastos de administración indirectos asignables a cada ramo")
ae101_030 = get_value_super_ramo(unidad11_EDUCATIVO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=31, value=ae98_015)   
ws.cell(row=99, column=31, value=ae99_020)  
ws.cell(row=100, column=31, value=ae100_025) 
ws.cell(row=101, column=31, value=ae101_030) 

# Calcular e102_999 usando la fórmula especificada
ae102_999 = (ae98_015 + ae99_020 + ae100_025 + ae101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=31, value=ae102_999)

<Cell 'Hoja1'.AE102>

## 30  VIDA GRUPO  UNIDAD 11

In [175]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_VIDAGRUPO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '30 VIDA GRUPO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

af98_015 = get_value_super_ramo(unidad11_VIDAGRUPO, "Gastos de administración directos ramo")
af99_020 = get_value_super_ramo(unidad11_VIDAGRUPO, "Gastos de personal directos ramo")
af100_025 = get_value_super_ramo(unidad11_VIDAGRUPO, "Gastos de administración indirectos asignables a cada ramo")
af101_030 = get_value_super_ramo(unidad11_VIDAGRUPO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=32, value=af98_015)   
ws.cell(row=99, column=32, value=af99_020)  
ws.cell(row=100, column=32, value=af100_025) 
ws.cell(row=101, column=32, value=af101_030) 

# Calcular e102_999 usando la fórmula especificada
af102_999 = (af98_015 + af99_020 + af100_025 + af101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=32, value=af102_999)

<Cell 'Hoja1'.AF102>

## 31 SALUD UNIDAD 11 

In [176]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_SALUD = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '31 SALUD')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ag98_015 = get_value_super_ramo(unidad11_SALUD, "Gastos de administración directos ramo")
ag99_020 = get_value_super_ramo(unidad11_SALUD, "Gastos de personal directos ramo")
ag100_025 = get_value_super_ramo(unidad11_SALUD, "Gastos de administración indirectos asignables a cada ramo")
ag101_030 = get_value_super_ramo(unidad11_SALUD, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=33, value=ag98_015)   
ws.cell(row=99, column=33, value=ag99_020)  
ws.cell(row=100, column=33, value=ag100_025) 
ws.cell(row=101, column=33, value=ag101_030) 

# Calcular e102_999 usando la fórmula especificada
ag102_999 = (ag98_015 + ag99_020 + ag100_025 + ag101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=33, value=ag102_999)

<Cell 'Hoja1'.AG102>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 11 

In [177]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_ENFALTOCOSTO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '32 ENFERMEDADES DE ALTO COSTO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ah98_015 = get_value_super_ramo(unidad11_ENFALTOCOSTO, "Gastos de administración directos ramo")
ah99_020 = get_value_super_ramo(unidad11_ENFALTOCOSTO, "Gastos de personal directos ramo")
ah100_025 = get_value_super_ramo(unidad11_ENFALTOCOSTO, "Gastos de administración indirectos asignables a cada ramo")
ah101_030 = get_value_super_ramo(unidad11_ENFALTOCOSTO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=34, value=ah98_015)   
ws.cell(row=99, column=34, value=ah99_020)  
ws.cell(row=100, column=34, value=ah100_025) 
ws.cell(row=101, column=34, value=ah101_030) 

# Calcular e102_999 usando la fórmula especificada
ah102_999 = (ah98_015 + ah99_020 + ah100_025 + ah101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=34, value=ah102_999)

<Cell 'Hoja1'.AH102>

## 33 VIDA INDIVIDUAL UNIDAD 11 

In [178]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_VIDAINDIVI = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '33 VIDA INDIVIDUAL')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ai98_015 = get_value_super_ramo(unidad11_VIDAINDIVI, "Gastos de administración directos ramo")
ai99_020 = get_value_super_ramo(unidad11_VIDAINDIVI, "Gastos de personal directos ramo")
ai100_025 = get_value_super_ramo(unidad11_VIDAINDIVI, "Gastos de administración indirectos asignables a cada ramo")
ai101_030 = get_value_super_ramo(unidad11_VIDAINDIVI, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=35, value=ai98_015)   
ws.cell(row=99, column=35, value=ai99_020)  
ws.cell(row=100, column=35, value=ai100_025) 
ws.cell(row=101, column=35, value=ai101_030) 

# Calcular e102_999 usando la fórmula especificada
ai102_999 = (ai98_015 + ai99_020 + ai100_025 + ai101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=35, value=ai102_999)

<Cell 'Hoja1'.AI102>

##  34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 11 

In [179]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_PREVIDENTIAL = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '34 PREVISIONAL DE INVALIDEZ.Y SOBREVIVENCIA')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aj98_015 = get_value_super_ramo(unidad11_PREVIDENTIAL, "Gastos de administración directos ramo")
aj99_020 = get_value_super_ramo(unidad11_PREVIDENTIAL, "Gastos de personal directos ramo")
aj100_025 = get_value_super_ramo(unidad11_PREVIDENTIAL, "Gastos de administración indirectos asignables a cada ramo")
aj101_030 = get_value_super_ramo(unidad11_PREVIDENTIAL, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=36, value=aj98_015)   
ws.cell(row=99, column=36, value=aj99_020)  
ws.cell(row=100, column=36, value=aj100_025) 
ws.cell(row=101, column=36, value=aj101_030) 

# Calcular e102_999 usando la fórmula especificada
aj102_999 = (aj98_015 + aj99_020 + aj100_025 + aj101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=36, value=aj102_999)

<Cell 'Hoja1'.AJ102>

##  35 RIESGOS LABORALES UNIDAD 11

In [180]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_RIESGOSLAB = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '35 RIESGOS LABORALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ak98_015 = get_value_super_ramo(unidad11_RIESGOSLAB, "Gastos de administración directos ramo")
ak99_020 = get_value_super_ramo(unidad11_RIESGOSLAB, "Gastos de personal directos ramo")
ak100_025 = get_value_super_ramo(unidad11_RIESGOSLAB, "Gastos de administración indirectos asignables a cada ramo")
ak101_030 = get_value_super_ramo(unidad11_RIESGOSLAB, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=37, value=ak98_015)   
ws.cell(row=99, column=37, value=ak99_020)  
ws.cell(row=100, column=37, value=ak100_025) 
ws.cell(row=101, column=37, value=ak101_030) 

# Calcular e102_999 usando la fórmula especificada
ak102_999 = (ak98_015 + ak99_020 + ak100_025 + ak101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=37, value=ak102_999)

<Cell 'Hoja1'.AK102>

##  36  PENSIONES LEY 100 UNIDAD 11

In [181]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_PENSIONESLEY100 = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '36 PENSIONES LEY 100')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

al98_015 = get_value_super_ramo(unidad11_PENSIONESLEY100, "Gastos de administración directos ramo")
al99_020 = get_value_super_ramo(unidad11_PENSIONESLEY100, "Gastos de personal directos ramo")
al100_025 = get_value_super_ramo(unidad11_PENSIONESLEY100, "Gastos de administración indirectos asignables a cada ramo")
al101_030 = get_value_super_ramo(unidad11_PENSIONESLEY100, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=38, value=al98_015)   
ws.cell(row=99, column=38, value=al99_020)  
ws.cell(row=100, column=38, value=al100_025) 
ws.cell(row=101, column=38, value=al101_030) 

# Calcular e102_999 usando la fórmula especificada
al102_999 = (al98_015 + al99_020 + al100_025 + al101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=38, value=al102_999)

<Cell 'Hoja1'.AL102>

## 37 PENSIONES VOLUNTARIAS UNIDAD 11   

In [182]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_PENSIONESVOLUN = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '37 PENSIONES VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

am98_015 = get_value_super_ramo(unidad11_PENSIONESVOLUN, "Gastos de administración directos ramo")
am99_020 = get_value_super_ramo(unidad11_PENSIONESVOLUN, "Gastos de personal directos ramo")
am100_025 = get_value_super_ramo(unidad11_PENSIONESVOLUN, "Gastos de administración indirectos asignables a cada ramo")
am101_030 = get_value_super_ramo(unidad11_PENSIONESVOLUN, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=39, value=am98_015)   
ws.cell(row=99, column=39, value=am99_020)  
ws.cell(row=100, column=39, value=am100_025) 
ws.cell(row=101, column=39, value=am101_030) 

# Calcular e102_999 usando la fórmula especificada
am102_999 = (am98_015 + am99_020 + am100_025 + am101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=39, value=am102_999)

<Cell 'Hoja1'.AM102>

##   38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 11

In [183]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_PENSIONESCONMU = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '38 PENSIONES CON CONMUTACIÓN PENSIONAL')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

an98_015 = get_value_super_ramo(unidad11_PENSIONESCONMU, "Gastos de administración directos ramo")
an99_020 = get_value_super_ramo(unidad11_PENSIONESCONMU, "Gastos de personal directos ramo")
an100_025 = get_value_super_ramo(unidad11_PENSIONESCONMU, "Gastos de administración indirectos asignables a cada ramo")
an101_030 = get_value_super_ramo(unidad11_PENSIONESCONMU, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=40, value=an98_015)   
ws.cell(row=99, column=40, value=an99_020)  
ws.cell(row=100, column=40, value=an100_025) 
ws.cell(row=101, column=40, value=an101_030) 

# Calcular e102_999 usando la fórmula especificada
an102_999 = (an98_015 + an99_020 + an100_025 + an101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=40, value=an102_999)

<Cell 'Hoja1'.AN102>

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 11 

In [184]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_PATRIAUTONOMO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ao98_015 = get_value_super_ramo(unidad11_PATRIAUTONOMO, "Gastos de administración directos ramo")
ao99_020 = get_value_super_ramo(unidad11_PATRIAUTONOMO, "Gastos de personal directos ramo")
ao100_025 = get_value_super_ramo(unidad11_PATRIAUTONOMO, "Gastos de administración indirectos asignables a cada ramo")
ao101_030 = get_value_super_ramo(unidad11_PATRIAUTONOMO, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=41, value=ao98_015)   
ws.cell(row=99, column=41, value=ao99_020)  
ws.cell(row=100, column=41, value=ao100_025) 
ws.cell(row=101, column=41, value=ao101_030) 

# Calcular e102_999 usando la fórmula especificada
ao102_999 = (ao98_015 + ao99_020 + ao100_025 + ao101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=41, value=ao102_999)

<Cell 'Hoja1'.AO102>

##   40 RENTAS VOLUNTARIAS UNIDAD 11

In [185]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_RENTASVOLUN = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '40 RENTAS VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ap98_015 = get_value_super_ramo(unidad11_RENTASVOLUN, "Gastos de administración directos ramo")
ap99_020 = get_value_super_ramo(unidad11_RENTASVOLUN, "Gastos de personal directos ramo")
ap100_025 = get_value_super_ramo(unidad11_RENTASVOLUN, "Gastos de administración indirectos asignables a cada ramo")
ap101_030 = get_value_super_ramo(unidad11_RENTASVOLUN, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=42, value=ap98_015)   
ws.cell(row=99, column=42, value=ap99_020)  
ws.cell(row=100, column=42, value=ap100_025) 
ws.cell(row=101, column=42, value=ap101_030) 

# Calcular e102_999 usando la fórmula especificada
ap102_999 = (ap98_015 + ap99_020 + ap100_025 + ap101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=42, value=ap102_999)

<Cell 'Hoja1'.AP102>

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 11

In [186]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_BENECONOMICOS = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '41 BENEFICIOS ECONÓMICOS PERIÓDICOS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aq98_015 = get_value_super_ramo(unidad11_BENECONOMICOS, "Gastos de administración directos ramo")
aq99_020 = get_value_super_ramo(unidad11_BENECONOMICOS, "Gastos de personal directos ramo")
aq100_025 = get_value_super_ramo(unidad11_BENECONOMICOS, "Gastos de administración indirectos asignables a cada ramo")
aq101_030 = get_value_super_ramo(unidad11_BENECONOMICOS, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=43, value=aq98_015)   
ws.cell(row=99, column=43, value=aq99_020)  
ws.cell(row=100, column=43, value=aq100_025) 
ws.cell(row=101, column=43, value=aq101_030) 

# Calcular e102_999 usando la fórmula especificada
aq102_999 = (aq98_015 + aq99_020 + aq100_025 + aq101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=43, value=aq102_999)

<Cell 'Hoja1'.AQ102>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 11

In [187]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad11_OPERNOASIGNABLES = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '43 OPERACIONES NO ASIGANBLES A RAMOS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

as98_015 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de administración directos ramo")
as99_020 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de personal directos ramo")
as100_025 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de administración indirectos asignables a cada ramo")
as101_030 = get_value_super_ramo(unidad11_OPERNOASIGNABLES, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=45, value=as98_015)   
ws.cell(row=99, column=45, value=as99_020)  
ws.cell(row=100, column=45, value=as100_025) 
ws.cell(row=101, column=45, value=as101_030) 

# Calcular e102_999 usando la fórmula especificada
as102_999 = (as98_015 + as99_020 + as100_025 + as101_030)

# Escribir el total general en Excel
ws.cell(row=102, column=45, value=as102_999)

<Cell 'Hoja1'.AS102>

##  CONSULTA DE TOTALES GASTOS DE ADMINISTRACIÓN Y PERSONAL UNIDAD 11  

In [188]:
# Definir la función para obtener el valor total de un concepto específico
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    return abs(row['VALOR'].sum()) if not row.empty else 0

# Agrupar la base de datos sin filtrar por RAMO_SUPER
unidad11_total = Base[Base['SOCIEDAD_DEST'] != 'GRAL'].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores totales de cada concepto
c98_015 = get_value_total(unidad11_total, "Gastos de administración directos ramo")
c99_020 = get_value_total(unidad11_total, "Gastos de personal directos ramo")
c100_025 = get_value_total(unidad11_total, "Gastos de administración indirectos asignables a cada ramo")
c101_030 = get_value_total(unidad11_total, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=3, value=c98_015)   
ws.cell(row=99, column=3, value=c99_020)  
ws.cell(row=100, column=3, value=c100_025) 
ws.cell(row=101, column=3, value=c101_030)

# Calcular el total general
c102_999 = c98_015 + c99_020 + c100_025 + c101_030

# Escribir el total general en Excel en la fila 102, columna 53 (columna BA)
ws.cell(row=102, column=3, value=c102_999)

<Cell 'Hoja1'.C102>

## CONSULTA DE SUBTOTALES GASTOS DE ADMINISTRACIÓN Y PERSONAL UNIDAD 11 

In [189]:
# Definir la función para obtener el valor total de un concepto específico
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    return abs(row['VALOR'].sum()) if not row.empty else 0

# Agrupar la base de datos sin filtrar por RAMO_SUPER
unidad11_subtotal = Base[Base['SOCIEDAD_DEST'] != 'GRAL'].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores totales de cada concepto
d98_015 = get_value_total(unidad11_subtotal, "Gastos de administración directos ramo")
d99_020 = get_value_total(unidad11_subtotal, "Gastos de personal directos ramo")
d100_025 = get_value_total(unidad11_subtotal, "Gastos de administración indirectos asignables a cada ramo")
d101_030 = get_value_total(unidad11_subtotal, "Gastos de personal indirectos asignables a cada ramo")

ws.cell(row=98, column=4, value=d98_015)   
ws.cell(row=99, column=4, value=d99_020)  
ws.cell(row=100, column=4, value=d100_025) 
ws.cell(row=101, column=4, value=d101_030)

# Calcular el total general
d102_999 = d98_015 + d99_020 + d100_025 + d101_030

# Escribir el total general en Excel en la fila 102, columna 53 (columna BA)
ws.cell(row=102, column=4, value=d102_999)

<Cell 'Hoja1'.D102>

#  UNIDAD DE CAPTURA 12  

##  26 EXEQUIAS UNIDAD 12 

In [190]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_EXEQUIAS = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '26 EXEQUIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ab104_005 = get_value_super_ramo(unidad12_EXEQUIAS, "Remuneración a favor de intermediarios dependientes e independientes")
ab105_010 = get_value_super_ramo(unidad12_EXEQUIAS, "Comisiones AFP")
ab107_020 = get_value_super_ramo(unidad12_EXEQUIAS, "Gastos en tarifación")
ab108_025 = get_value_super_ramo(unidad12_EXEQUIAS, "Remuneraciones por uso de red y a canales de comercialización")
ab109_030 = get_value_super_ramo(unidad12_EXEQUIAS, "Papelería y gastos de entrega")

ws.cell(row=104, column=28, value=ab104_005)   
ws.cell(row=105, column=28, value=ab105_010)  
ws.cell(row=107, column=28, value=ab107_020) 
ws.cell(row=108, column=28, value=ab108_025) 
ws.cell(row=109, column=28, value=ab109_030) 

# Calcular e102_999 usando la fórmula especificada
ab110_999 = (ab104_005 + ab105_010 + ab107_020 + ab108_025 + ab109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=28). value = ab110_999

##  27 ACCIDENTES PERSONALES UNIDAD 12 

In [191]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_ACCPERSONALES = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '27 ACCIDENTES PERSONALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ac104_005 = get_value_super_ramo(unidad12_ACCPERSONALES, "Remuneración a favor de intermediarios dependientes e independientes")
ac105_010 = get_value_super_ramo(unidad12_ACCPERSONALES, "Comisiones AFP")
ac107_020 = get_value_super_ramo(unidad12_ACCPERSONALES, "Gastos en tarifación")
ac108_025 = get_value_super_ramo(unidad12_ACCPERSONALES, "Remuneraciones por uso de red y a canales de comercialización")
ac109_030 = get_value_super_ramo(unidad12_ACCPERSONALES, "Papelería y gastos de entrega")

ws.cell(row=104, column=29, value=ac104_005)   
ws.cell(row=105, column=29, value=ac105_010)  
ws.cell(row=107, column=29, value=ac107_020) 
ws.cell(row=108, column=29, value=ac108_025) 
ws.cell(row=109, column=29, value=ac109_030) 

# Calcular e102_999 usando la fórmula especificada
ac110_999 = (ac104_005 + ac105_010 + ac107_020 + ac108_025 + ac109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=29). value = ac110_999

##  28 COLECTIVO VIDA UNIDAD 12 

In [192]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_COLECVI = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '28 COLECTIVO VIDA')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ad104_005 = get_value_super_ramo(unidad12_COLECVI, "Remuneración a favor de intermediarios dependientes e independientes")
ad105_010 = get_value_super_ramo(unidad12_COLECVI, "Comisiones AFP")
ad107_020 = get_value_super_ramo(unidad12_COLECVI, "Gastos en tarifación")
ad108_025 = get_value_super_ramo(unidad12_COLECVI, "Remuneraciones por uso de red y a canales de comercialización")
ad109_030 = get_value_super_ramo(unidad12_COLECVI, "Papelería y gastos de entrega")

ws.cell(row=104, column=30, value=ad104_005)   
ws.cell(row=105, column=30, value=ad105_010)  
ws.cell(row=107, column=30, value=ad107_020) 
ws.cell(row=108, column=30, value=ad108_025) 
ws.cell(row=109, column=30, value=ad109_030) 

# Calcular e102_999 usando la fórmula especificada
ad110_999 = (ad104_005 + ad105_010 + ad107_020 + ad108_025 + ad109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=30). value = ad110_999

##  29 EDUCATIVO  UNIDAD 12 

In [193]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_EDUCATIVO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '29 EDUCATIVO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ae104_005 = get_value_super_ramo(unidad12_EDUCATIVO, "Remuneración a favor de intermediarios dependientes e independientes")
ae105_010 = get_value_super_ramo(unidad12_EDUCATIVO, "Comisiones AFP")
ae107_020 = get_value_super_ramo(unidad12_EDUCATIVO, "Gastos en tarifación")
ae108_025 = get_value_super_ramo(unidad12_EDUCATIVO, "Remuneraciones por uso de red y a canales de comercialización")
ae109_030 = get_value_super_ramo(unidad12_EDUCATIVO, "Papelería y gastos de entrega")

ws.cell(row=104, column=31, value=ae104_005)   
ws.cell(row=105, column=31, value=ae105_010)  
ws.cell(row=107, column=31, value=ae107_020) 
ws.cell(row=108, column=31, value=ae108_025) 
ws.cell(row=109, column=31, value=ae109_030) 

# Calcular e102_999 usando la fórmula especificada
ae110_999 = (ae104_005 + ae105_010 + ae107_020 + ae108_025 + ae109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=31). value = ae110_999

## 30  VIDA GRUPO  UNIDAD 12 

In [194]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_VIDAGRUPO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '30 VIDA GRUPO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

af104_005 = get_value_super_ramo(unidad12_VIDAGRUPO, "Remuneración a favor de intermediarios dependientes e independientes")
af105_010 = get_value_super_ramo(unidad12_VIDAGRUPO, "Comisiones AFP")
af107_020 = get_value_super_ramo(unidad12_VIDAGRUPO, "Gastos en tarifación")
af108_025 = get_value_super_ramo(unidad12_VIDAGRUPO, "Remuneraciones por uso de red y a canales de comercialización")
af109_030 = get_value_super_ramo(unidad12_VIDAGRUPO, "Papelería y gastos de entrega")

ws.cell(row=104, column=32, value=af104_005)   
ws.cell(row=105, column=32, value=af105_010)  
ws.cell(row=107, column=32, value=af107_020) 
ws.cell(row=108, column=32, value=af108_025) 
ws.cell(row=109, column=32, value=af109_030) 

# Calcular e102_999 usando la fórmula especificada
af110_999 = (af104_005 + af105_010 + af107_020 + af108_025 + af109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=32). value = af110_999

## 31 SALUD UNIDAD 12 

In [195]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_SALUD = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '31 SALUD')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ag104_005 = get_value_super_ramo(unidad12_SALUD, "Remuneración a favor de intermediarios dependientes e independientes")
ag105_010 = get_value_super_ramo(unidad12_SALUD, "Comisiones AFP")
ag107_020 = get_value_super_ramo(unidad12_SALUD, "Gastos en tarifación")
ag108_025 = get_value_super_ramo(unidad12_SALUD, "Remuneraciones por uso de red y a canales de comercialización")
ag109_030 = get_value_super_ramo(unidad12_SALUD, "Papelería y gastos de entrega")

ws.cell(row=104, column=33, value=ag104_005)   
ws.cell(row=105, column=33, value=ag105_010)  
ws.cell(row=107, column=33, value=ag107_020) 
ws.cell(row=108, column=33, value=ag108_025) 
ws.cell(row=109, column=33, value=ag109_030) 

# Calcular e102_999 usando la fórmula especificada
ag110_999 = (ag104_005 + ag105_010 + ag107_020 + ag108_025 + ag109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=33). value = ag110_999

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 12

In [196]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_ENFALTOCOSTO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '32 ENFERMEDADES DE ALTO COSTO')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ah104_005 = get_value_super_ramo(unidad12_ENFALTOCOSTO, "Remuneración a favor de intermediarios dependientes e independientes")
ah105_010 = get_value_super_ramo(unidad12_ENFALTOCOSTO, "Comisiones AFP")
ah107_020 = get_value_super_ramo(unidad12_ENFALTOCOSTO, "Gastos en tarifación")
ah108_025 = get_value_super_ramo(unidad12_ENFALTOCOSTO, "Remuneraciones por uso de red y a canales de comercialización")
ah109_030 = get_value_super_ramo(unidad12_ENFALTOCOSTO, "Papelería y gastos de entrega")

ws.cell(row=104, column=34, value=ah104_005)   
ws.cell(row=105, column=34, value=ah105_010)  
ws.cell(row=107, column=34, value=ah107_020) 
ws.cell(row=108, column=34, value=ah108_025) 
ws.cell(row=109, column=34, value=ah109_030) 

# Calcular e102_999 usando la fórmula especificada
ah110_999 = (ah104_005 + ah105_010 + ah107_020 + ah108_025 + ah109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=34). value = ah110_999

##   33 VIDA INDIVIDUAL UNIDAD 12 

In [197]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_VIDAINDIVI = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '33 VIDA INDIVIDUAL')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ai104_005 = get_value_super_ramo(unidad12_VIDAINDIVI, "Remuneración a favor de intermediarios dependientes e independientes")
ai105_010 = get_value_super_ramo(unidad12_VIDAINDIVI, "Comisiones AFP")
ai107_020 = get_value_super_ramo(unidad12_VIDAINDIVI, "Gastos en tarifación")
ai108_025 = get_value_super_ramo(unidad12_VIDAINDIVI, "Remuneraciones por uso de red y a canales de comercialización")
ai109_030 = get_value_super_ramo(unidad12_VIDAINDIVI, "Papelería y gastos de entrega")

ws.cell(row=104, column=35, value=ai104_005)   
ws.cell(row=105, column=35, value=ai105_010)  
ws.cell(row=107, column=35, value=ai107_020) 
ws.cell(row=108, column=35, value=ai108_025) 
ws.cell(row=109, column=35, value=ai109_030) 

# Calcular e102_999 usando la fórmula especificada
ai110_999 = (ai104_005 + ai105_010 + ai107_020 + ai108_025 + ai109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=35). value = ai110_999

##   34 PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 12 

In [198]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_PREVIDENTIAL = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '34 PREVISIONAL DE INVALIDEZ.Y SOBREVIVENCIA')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aj104_005 = get_value_super_ramo(unidad12_PREVIDENTIAL, "Remuneración a favor de intermediarios dependientes e independientes")
aj105_010 = get_value_super_ramo(unidad12_PREVIDENTIAL, "Comisiones AFP")
aj107_020 = get_value_super_ramo(unidad12_PREVIDENTIAL, "Gastos en tarifación")
aj108_025 = get_value_super_ramo(unidad12_PREVIDENTIAL, "Remuneraciones por uso de red y a canales de comercialización")
aj109_030 = get_value_super_ramo(unidad12_PREVIDENTIAL, "Papelería y gastos de entrega")

ws.cell(row=104, column=36, value=aj104_005)   
ws.cell(row=105, column=36, value=aj105_010)  
ws.cell(row=107, column=36, value=aj107_020) 
ws.cell(row=108, column=36, value=aj108_025) 
ws.cell(row=109, column=36, value=aj109_030) 

# Calcular e102_999 usando la fórmula especificada
aj110_999 = (aj104_005 + aj105_010 + aj107_020 + aj108_025 + aj109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=36). value = aj110_999

##  35 RIESGOS LABORALES UNIDAD 12 

In [199]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_RIESGOSLAB = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '35 RIESGOS LABORALES')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ak104_005 = get_value_super_ramo(unidad12_RIESGOSLAB, "Remuneración a favor de intermediarios dependientes e independientes")
ak105_010 = get_value_super_ramo(unidad12_RIESGOSLAB, "Comisiones AFP")
ak107_020 = get_value_super_ramo(unidad12_RIESGOSLAB, "Gastos en tarifación")
ak108_025 = get_value_super_ramo(unidad12_RIESGOSLAB, "Remuneraciones por uso de red y a canales de comercialización")
ak109_030 = get_value_super_ramo(unidad12_RIESGOSLAB, "Papelería y gastos de entrega")

ws.cell(row=104, column=37, value=ak104_005)   
ws.cell(row=105, column=37, value=ak105_010)  
ws.cell(row=107, column=37, value=ak107_020) 
ws.cell(row=108, column=37, value=ak108_025) 
ws.cell(row=109, column=37, value=ak109_030) 

# Calcular e102_999 usando la fórmula especificada
ak110_999 = (ak104_005 + ak105_010 + ak107_020 + ak108_025 + ak109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=37). value = ak110_999

## 36  PENSIONES LEY 100 UNIDAD 12  

In [200]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_PENSIONESLEY100 = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '36 PENSIONES LEY 100')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

al104_005 = get_value_super_ramo(unidad12_PENSIONESLEY100, "Remuneración a favor de intermediarios dependientes e independientes")
al105_010 = get_value_super_ramo(unidad12_PENSIONESLEY100, "Comisiones AFP")
al107_020 = get_value_super_ramo(unidad12_PENSIONESLEY100, "Gastos en tarifación")
al108_025 = get_value_super_ramo(unidad12_PENSIONESLEY100, "Remuneraciones por uso de red y a canales de comercialización")
al109_030 = get_value_super_ramo(unidad12_PENSIONESLEY100, "Papelería y gastos de entrega")

ws.cell(row=104, column=38, value=al104_005)   
ws.cell(row=105, column=38, value=al105_010)  
ws.cell(row=107, column=38, value=al107_020) 
ws.cell(row=108, column=38, value=al108_025) 
ws.cell(row=109, column=38, value=al109_030) 

# Calcular e102_999 usando la fórmula especificada
al110_999 = (al104_005 + al105_010 + al107_020 + al108_025 + al109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=38). value = al110_999

##  37 PENSIONES VOLUNTARIAS UNIDAD 12 

In [201]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_PENSIONESVOLUN = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '37 PENSIONES VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

am104_005 = get_value_super_ramo(unidad12_PENSIONESVOLUN, "Remuneración a favor de intermediarios dependientes e independientes")
am105_010 = get_value_super_ramo(unidad12_PENSIONESVOLUN, "Comisiones AFP")
am107_020 = get_value_super_ramo(unidad12_PENSIONESVOLUN, "Gastos en tarifación")
am108_025 = get_value_super_ramo(unidad12_PENSIONESVOLUN, "Remuneraciones por uso de red y a canales de comercialización")
am109_030 = get_value_super_ramo(unidad12_PENSIONESVOLUN, "Papelería y gastos de entrega")

ws.cell(row=104, column=39, value=am104_005)   
ws.cell(row=105, column=39, value=am105_010)  
ws.cell(row=107, column=39, value=am107_020) 
ws.cell(row=108, column=39, value=am108_025) 
ws.cell(row=109, column=39, value=am109_030) 

# Calcular e102_999 usando la fórmula especificada
am110_999 = (am104_005 + am105_010 + am107_020 + am108_025 + am109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=39). value = am110_999

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 12 

In [202]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_PENSIONESCONMU = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '38 PENSIONES CON CONMUTACIÓN PENSIONAL')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

an104_005 = get_value_super_ramo(unidad12_PENSIONESCONMU, "Remuneración a favor de intermediarios dependientes e independientes")
an105_010 = get_value_super_ramo(unidad12_PENSIONESCONMU, "Comisiones AFP")
an107_020 = get_value_super_ramo(unidad12_PENSIONESCONMU, "Gastos en tarifación")
an108_025 = get_value_super_ramo(unidad12_PENSIONESCONMU, "Remuneraciones por uso de red y a canales de comercialización")
an109_030 = get_value_super_ramo(unidad12_PENSIONESCONMU, "Papelería y gastos de entrega")

ws.cell(row=104, column=40, value=an104_005)   
ws.cell(row=105, column=40, value=an105_010)  
ws.cell(row=107, column=40, value=an107_020) 
ws.cell(row=108, column=40, value=an108_025) 
ws.cell(row=109, column=40, value=an109_030) 

# Calcular e102_999 usando la fórmula especificada
an110_999 = (an104_005 + an105_010 + an107_020 + an108_025 + an109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=40). value = an110_999

##  39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 12

In [203]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_PATRIAUTONOMO = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ao104_005 = get_value_super_ramo(unidad12_PATRIAUTONOMO, "Remuneración a favor de intermediarios dependientes e independientes")
ao105_010 = get_value_super_ramo(unidad12_PATRIAUTONOMO, "Comisiones AFP")
ao107_020 = get_value_super_ramo(unidad12_PATRIAUTONOMO, "Gastos en tarifación")
ao108_025 = get_value_super_ramo(unidad12_PATRIAUTONOMO, "Remuneraciones por uso de red y a canales de comercialización")
ao109_030 = get_value_super_ramo(unidad12_PATRIAUTONOMO, "Papelería y gastos de entrega")

ws.cell(row=104, column=41, value=ao104_005)   
ws.cell(row=105, column=41, value=ao105_010)  
ws.cell(row=107, column=41, value=ao107_020) 
ws.cell(row=108, column=41, value=ao108_025) 
ws.cell(row=109, column=41, value=ao109_030) 

# Calcular e102_999 usando la fórmula especificada
ao110_999 = (ao104_005 + ao105_010 + ao107_020 + ao108_025 + ao109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=41). value = ao110_999

## 40 RENTAS VOLUNTARIAS UNIDAD 12 

In [204]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_RENTASVOLUN = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '40 RENTAS VOLUNTARIAS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

ap104_005 = get_value_super_ramo(unidad12_RENTASVOLUN, "Remuneración a favor de intermediarios dependientes e independientes")
ap105_010 = get_value_super_ramo(unidad12_RENTASVOLUN, "Comisiones AFP")
ap107_020 = get_value_super_ramo(unidad12_RENTASVOLUN, "Gastos en tarifación")
ap108_025 = get_value_super_ramo(unidad12_RENTASVOLUN, "Remuneraciones por uso de red y a canales de comercialización")
ap109_030 = get_value_super_ramo(unidad12_RENTASVOLUN, "Papelería y gastos de entrega")

ws.cell(row=104, column=42, value=ap104_005)   
ws.cell(row=105, column=42, value=ap105_010)  
ws.cell(row=107, column=42, value=ap107_020) 
ws.cell(row=108, column=42, value=ap108_025) 
ws.cell(row=109, column=42, value=ap109_030) 

# Calcular e102_999 usando la fórmula especificada
ap110_999 = (ap104_005 + ap105_010 + ap107_020 + ap108_025 + ap109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=42). value = ap110_999

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 12 

In [205]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_BENECONOMICOS = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '41 BENEFICIOS ECONÓMICOS PERIÓDICOS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

aq104_005 = get_value_super_ramo(unidad12_BENECONOMICOS, "Remuneración a favor de intermediarios dependientes e independientes")
aq105_010 = get_value_super_ramo(unidad12_BENECONOMICOS, "Comisiones AFP")
aq107_020 = get_value_super_ramo(unidad12_BENECONOMICOS, "Gastos en tarifación")
aq108_025 = get_value_super_ramo(unidad12_BENECONOMICOS, "Remuneraciones por uso de red y a canales de comercialización")
aq109_030 = get_value_super_ramo(unidad12_BENECONOMICOS, "Papelería y gastos de entrega")

ws.cell(row=104, column=43, value=aq104_005)   
ws.cell(row=105, column=43, value=aq105_010)  
ws.cell(row=107, column=43, value=aq107_020) 
ws.cell(row=108, column=43, value=aq108_025) 
ws.cell(row=109, column=43, value=aq109_030) 

# Calcular e102_999 usando la fórmula especificada
aq110_999 = (aq104_005 + aq105_010 + aq107_020 + aq108_025 + aq109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=43). value = aq110_999

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 12

In [206]:
# Definir la función para obtener el valor de un concepto específico
def get_value_super_ramo(df, concept):
    row = df[(df['CONCEPTO_290'] == concept) ]
    return abs(row['VALOR'].iloc[0]) if not row.empty else 0

unidad12_OPERNOASIGNABLES = Base[(Base['SOCIEDAD_DEST'] != 'GRAL') & (Base['RAMO_SUPER']  == '43 OPERACIONES NO ASIGANBLES A RAMOS')].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

as104_005 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Remuneración a favor de intermediarios dependientes e independientes")
as105_010 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Comisiones AFP")
as107_020 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Gastos en tarifación")
as108_025 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Remuneraciones por uso de red y a canales de comercialización")
as109_030 = get_value_super_ramo(unidad12_OPERNOASIGNABLES, "Papelería y gastos de entrega")

ws.cell(row=104, column=45, value=as104_005)   
ws.cell(row=105, column=45, value=as105_010)  
ws.cell(row=107, column=45, value=as107_020) 
ws.cell(row=108, column=45, value=as108_025) 
ws.cell(row=109, column=45, value=as109_030) 

# Calcular e102_999 usando la fórmula especificada
as110_999 = (as104_005 + as105_010 + as107_020 + as108_025 + as109_030)

# Escribir el total general en Excel
ws.cell(row=110, column=45). value = as110_999

##  CONSULTA DE TOTALES 12  

In [207]:
# Definir la función para obtener el valor total de un concepto específico
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    return abs(row['VALOR'].sum()) if not row.empty else 0

# Agrupar la base de datos sin filtrar por RAMO_SUPER
unidad12_total = Base[Base['SOCIEDAD_DEST'] != 'GRAL'].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores totales de cada concepto
c104_005 = get_value_total(unidad12_total, "Remuneración a favor de intermediarios dependientes e independientes")
c105_010 = get_value_total(unidad12_total, "Comisiones AFP")
c107_020 = get_value_total(unidad12_total, "Gastos en tarifación")
c108_025 = get_value_total(unidad12_total, "Remuneraciones por uso de red y a canales de comercialización")
c109_030 = get_value_total(unidad12_total, "Papelería y gastos de entrega")

ws.cell(row=104, column=3, value=c104_005)   
ws.cell(row=105, column=3, value=c105_010)  
ws.cell(row=107, column=3, value=c107_020) 
ws.cell(row=108, column=3, value=c108_025)
ws.cell(row=109, column=3, value=c109_030)

# Calcular el total general
c110_999 = c104_005 + c105_010 + c107_020 + c108_025 + c109_030

# Escribir el total general en Excel en la fila 102, columna 53 (columna BA)
ws.cell(row=110, column=3, value=c110_999)

<Cell 'Hoja1'.C110>

## CONSULTA DE SUBTOTALES 12 

In [208]:
# Definir la función para obtener el valor total de un concepto específico
def get_value_total(df, concept):
    row = df[df['CONCEPTO_290'] == concept]
    return abs(row['VALOR'].sum()) if not row.empty else 0

# Agrupar la base de datos sin filtrar por RAMO_SUPER
unidad12_subtotal = Base[Base['SOCIEDAD_DEST'] != 'GRAL'].groupby(['CONCEPTO_290'])['VALOR'].sum().reset_index()

# Obtener los valores totales de cada concepto
d104_005 = get_value_total(unidad12_subtotal, "Remuneración a favor de intermediarios dependientes e independientes")
d105_010 = get_value_total(unidad12_subtotal, "Comisiones AFP")
d107_020 = get_value_total(unidad12_subtotal, "Gastos en tarifación")
d108_025 = get_value_total(unidad12_subtotal, "Remuneraciones por uso de red y a canales de comercialización")
d109_030 = get_value_total(unidad12_subtotal, "Papelería y gastos de entrega")

ws.cell(row=104, column=4, value=d104_005)   
ws.cell(row=105, column=4, value=d105_010)  
ws.cell(row=107, column=4, value=d107_020) 
ws.cell(row=108, column=4, value=d108_025)
ws.cell(row=109, column=4, value=d109_030)

# Calcular el total general
d110_999 = d104_005 + d105_010 + d107_020 + d108_025 + d109_030

# Escribir el total general en Excel en la fila 102, columna 53 (columna BA)
ws.cell(row=110, column=4, value=d110_999)

<Cell 'Hoja1'.D110>

#  UNIDAD DE CAPTURA 13 

## CONSULTA DE TOTALES UNIDAD DE CAPTURA 13

In [209]:
c113_999 = c84_999 + c94_999 - c102_999 - c110_999
df_c113_999 = pd.DataFrame({'total': [c113_999]})
if df_c113_999.empty or c113_999 == 0:
    df_c113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=3).value = df_c113_999['total'].iloc[0]

ab113_999 = ab84_999 + ab94_999 - ab102_999 - ab110_999
df_ab113_999 = pd.DataFrame({'total': [ab113_999]})
if df_ab113_999.empty or ab113_999 == 0:
    df_ab113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=28).value = df_ab113_999['total'].iloc[0]

ac113_999 = ac84_999 + ac94_999 - ac102_999 - ac110_999
df_ac113_999 = pd.DataFrame({'total': [ac113_999]})
if df_ac113_999.empty or ac113_999 == 0:
    df_ac113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=29).value = df_ac113_999['total'].iloc[0]

ad113_999 = ad84_999 + ad94_999 - ad102_999 - ad110_999
df_ad113_999 = pd.DataFrame({'total': [ad113_999]})
if df_ad113_999.empty or ad113_999 == 0:
    df_ad113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=30).value = df_ad113_999['total'].iloc[0]

ae113_999 = ae84_999 + ae94_999 - ae102_999 - ae110_999
df_ae113_999 = pd.DataFrame({'total': [ae113_999]})
if df_ae113_999.empty or ae113_999 == 0:
    df_ae113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=31).value = df_ae113_999['total'].iloc[0]

af113_999 = af84_999 + af94_999 - af102_999 - af110_999
df_af113_999 = pd.DataFrame({'total': [af113_999]})
if df_af113_999.empty or af113_999 == 0:
    df_af113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=32).value = df_af113_999['total'].iloc[0]

ag113_999 = ag84_999 + ag94_999 - ag102_999 - ag110_999
df_ag113_999 = pd.DataFrame({'total': [ag113_999]})
if df_ag113_999.empty or ag113_999 == 0:
    df_ag113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=33).value = df_ag113_999['total'].iloc[0]

ah113_999 = ah84_999 + ah94_999 - ah102_999 - ah110_999
df_ah113_999 = pd.DataFrame({'total': [ah113_999]})
if df_ah113_999.empty or ah113_999 == 0:
    df_ah113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=34).value = df_ah113_999['total'].iloc[0]

ai113_999 = ai84_999 + ai94_999 - ai102_999 - ai110_999
df_ai113_999 = pd.DataFrame({'total': [ai113_999]})
if df_ai113_999.empty or ai113_999 == 0:
    df_ai113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=35).value = df_ai113_999['total'].iloc[0]

aj113_999 = aj84_999 + aj94_999 - aj102_999 - aj110_999
df_aj113_999 = pd.DataFrame({'total': [aj113_999]})
if df_aj113_999.empty or aj113_999 == 0:
    df_aj113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=36).value = df_aj113_999['total'].iloc[0]

ak113_999 = ak84_999 + ak94_999 - ak102_999 - ak110_999
df_ak113_999 = pd.DataFrame({'total': [ak113_999]})
if df_ak113_999.empty or ak113_999 == 0:
    df_ak113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=37).value = df_ak113_999['total'].iloc[0]

al113_999 = al84_999 + al94_999 - al102_999 - al110_999
df_al113_999 = pd.DataFrame({'total': [al113_999]})
if df_al113_999.empty or al113_999 == 0:
    df_al113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=38).value = df_al113_999['total'].iloc[0]

am113_999 = am84_999 + am94_999 - am102_999 - am110_999
df_am113_999 = pd.DataFrame({'total': [am113_999]})
if df_am113_999.empty or am113_999 == 0:
    df_am113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=39).value = df_am113_999['total'].iloc[0]

an113_999 = an84_999 + an94_999 - an102_999 - an110_999
df_an113_999 = pd.DataFrame({'total': [an113_999]})
if df_an113_999.empty or an113_999 == 0:
    df_an113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=40).value = df_an113_999['total'].iloc[0]

ao113_999 = ao84_999 + ao94_999 - ao102_999 - ao110_999
df_ao113_999 = pd.DataFrame({'total': [ao113_999]})
if df_ao113_999.empty or ao113_999 == 0:
    df_ao113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=41).value = df_ao113_999['total'].iloc[0]

ap113_999 = ap84_999 + ap94_999 - ap102_999 - ap110_999
df_ap113_999 = pd.DataFrame({'total': [ap113_999]})
if df_ap113_999.empty or ap113_999 == 0:
    df_ap113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=42).value = df_ap113_999['total'].iloc[0]

aq113_999 = aq84_999 + aq94_999 - aq102_999 - aq110_999
df_aq113_999 = pd.DataFrame({'total': [aq113_999]})
if df_aq113_999.empty or aq113_999 == 0:
    df_aq113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=43).value = df_aq113_999['total'].iloc[0]

as113_999 = as84_999 + as94_999 - as102_999 - as110_999
df_as113_999 = pd.DataFrame({'total': [as113_999]})
if df_as113_999.empty or as113_999 == 0:
    df_as113_999 = pd.DataFrame({'total': [0]})
ws.cell(row=113, column=45).value = df_as113_999['total'].iloc[0]


## CONSULTA DE SUBTOTALES UNIDAD DE CAPTURA 13

In [210]:
# Realiza la suma para d75_999
d113_999 = d84_999 + d94_999 - d102_999 - d110_999
df_d113_999 = pd.DataFrame({'total': [d113_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d113_999.empty or d113_999 == 0:
    df_d113_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=113, column=4).value = df_d113_999['total'].iloc[0]

# UNIDAD DE CAPTURA 14

## CONSULTA DE TOTALES 14

In [211]:
c115_999 = c34_999 + c75_999 + c113_999
df_c115_999 = pd.DataFrame({'total': [c115_999]})
if df_c115_999.empty or c115_999 == 0:
    df_c115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=3).value = df_c115_999['total'].iloc[0]

ab115_999 = ab34_999 + ab75_999 + ab113_999
df_ab115_999 = pd.DataFrame({'total': [ab115_999]})
if df_ab115_999.empty or ab115_999 == 0:
    df_ab115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=28).value = df_ab115_999['total'].iloc[0]

ac115_999 = ac34_999 + ac75_999 + ac113_999
df_ac115_999 = pd.DataFrame({'total': [ac115_999]})
if df_ac115_999.empty or ac115_999 == 0:
    df_ac115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=29).value = df_ac115_999['total'].iloc[0]

ad115_999 = ad34_999 + ad75_999 + ad113_999
df_ad115_999 = pd.DataFrame({'total': [ad115_999]})
if df_ad115_999.empty or ad115_999 == 0:
    df_ad115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=30).value = df_ad115_999['total'].iloc[0]

ae115_999 = ae34_999 + ae75_999 + ae113_999
df_ae115_999 = pd.DataFrame({'total': [ae115_999]})
if df_ae115_999.empty or ae115_999 == 0:
    df_ae115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=31).value = df_ae115_999['total'].iloc[0]

af115_999 = af34_999 + af75_999 + af113_999
df_af115_999 = pd.DataFrame({'total': [af115_999]})
if df_af115_999.empty or af115_999 == 0:
    df_af115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=32).value = df_af115_999['total'].iloc[0]

ag115_999 = ag34_999 + ag75_999 + ag113_999
df_ag115_999 = pd.DataFrame({'total': [ag115_999]})
if df_ag115_999.empty or ag115_999 == 0:
    df_ag115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=33).value = df_ag115_999['total'].iloc[0]

ah115_999 = ah34_999 + ah75_999 + ah113_999
df_ah115_999 = pd.DataFrame({'total': [ah115_999]})
if df_ah115_999.empty or ah115_999 == 0:
    df_ah115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=34).value = df_ah115_999['total'].iloc[0]

ai115_999 = ai34_999 + ai75_999 + ai113_999
df_ai115_999 = pd.DataFrame({'total': [ai115_999]})
if df_ai115_999.empty or ai115_999 == 0:
    df_ai115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=35).value = df_ai115_999['total'].iloc[0]

aj115_999 = aj34_999 + aj75_999 + aj113_999
df_aj115_999 = pd.DataFrame({'total': [aj115_999]})
if df_aj115_999.empty or aj115_999 == 0:
    df_aj115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=36).value = df_aj115_999['total'].iloc[0]

ak115_999 = ak34_999 + ak75_999 + ak113_999
df_ak115_999 = pd.DataFrame({'total': [ak115_999]})
if df_ak115_999.empty or ak115_999 == 0:
    df_ak115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=37).value = df_ak115_999['total'].iloc[0]

al115_999 = al34_999 + al75_999 + al113_999
df_al115_999 = pd.DataFrame({'total': [al115_999]})
if df_al115_999.empty or al115_999 == 0:
    df_al115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=38).value = df_al115_999['total'].iloc[0]

am115_999 = am34_999 + am75_999 + am113_999
df_am115_999 = pd.DataFrame({'total': [am115_999]})
if df_am115_999.empty or am115_999 == 0:
    df_am115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=39).value = df_am115_999['total'].iloc[0]

an115_999 = an34_999 + an75_999 + an113_999
df_an115_999 = pd.DataFrame({'total': [an115_999]})
if df_an115_999.empty or an115_999 == 0:
    df_an115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=40).value = df_an115_999['total'].iloc[0]

ao115_999 = ao34_999 + ao75_999 + ao113_999
df_ao115_999 = pd.DataFrame({'total': [ao115_999]})
if df_ao115_999.empty or ao115_999 == 0:
    df_ao115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=41).value = df_ao115_999['total'].iloc[0]

ap115_999 = ap34_999 + ap75_999 + ap113_999
df_ap115_999 = pd.DataFrame({'total': [ap115_999]})
if df_ap115_999.empty or ap115_999 == 0:
    df_ap115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=42).value = df_ap115_999['total'].iloc[0]

aq115_999 = aq34_999 + aq75_999 + aq113_999
df_aq115_999 = pd.DataFrame({'total': [aq115_999]})
if df_aq115_999.empty or aq115_999 == 0:
    df_aq115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=43).value = df_aq115_999['total'].iloc[0]

as115_999 = as34_999 + as75_999 + as113_999
df_as115_999 = pd.DataFrame({'total': [as115_999]})
if df_as115_999.empty or as115_999 == 0:
    df_as115_999 = pd.DataFrame({'total': [0]})
ws.cell(row=115, column=45).value = df_as115_999['total'].iloc[0]


## CONSULTA DE SUBTOTALES RESULTADO TECNICO 14

In [212]:
# Realiza la suma para d75_999
d115_999 = d34_999 + d75_999 + d113_999
df_d115_999 = pd.DataFrame({'total': [d115_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d115_999.empty or d115_999 == 0:
    df_d115_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=115, column=4).value = df_d115_999['total'].iloc[0]

#  UNIDAD DE CAPTURA 15 

##  26  EXEQUIAS UNIDAD 15

In [213]:
unidad15_EXEQUIAS = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 26)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_EXEQUIAS['TOTAL'] = unidad15_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 26, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 26, 119),
    ("Otros ingresos y Egresos", 26, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 26, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 26, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 26, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 26, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 26, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_EXEQUIAS, concept, ramo, 28, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ab118_010 = get_value(unidad15_EXEQUIAS, "Otras Provisiones (resta)", 26, ignore_subaccounts)
ab119_015 = get_value(unidad15_EXEQUIAS, "Ingresos y/o Egresos reportes programas especiales", 26, ignore_subaccounts)
ab120_020 = get_value(unidad15_EXEQUIAS, "Otros ingresos y Egresos", 26, ignore_subaccounts)
ab121_025 = get_value(unidad15_EXEQUIAS, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 26, ignore_subaccounts)
ab122_030 = get_value(unidad15_EXEQUIAS, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 26, ignore_subaccounts)
ab123_035 = get_value(unidad15_EXEQUIAS, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 26, ignore_subaccounts)
ab124_040 = get_value(unidad15_EXEQUIAS, "Egresos inversiones de las reservas técnicas (resta)", 26, ignore_subaccounts)
ab125_045 = get_value(unidad15_EXEQUIAS, "Otros gastos de administración y personal no asignables a ramos", 26, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ab126_999 = abs (ab118_010 + ab119_015 - ab120_020 - ab121_025 - ab122_030 - ab123_035 - ab124_040 + ab125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=28, value=ab126_999)

<Cell 'Hoja1'.AB126>

## 27  ACCIDENTES PERSONALES UNIDAD 15

In [214]:
unidad15_ACCPERSONALES = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 27)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_ACCPERSONALES['TOTAL'] = unidad15_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 27, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 27, 119),
    ("Otros ingresos y Egresos", 27, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 27, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 27, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 27, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 27, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 27, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_ACCPERSONALES, concept, ramo, 29, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ac118_010 = get_value(unidad15_ACCPERSONALES, "Otras Provisiones (resta)", 27, ignore_subaccounts)
ac119_015 = get_value(unidad15_ACCPERSONALES, "Ingresos y/o Egresos reportes programas especiales", 27, ignore_subaccounts)
ac120_020 = get_value(unidad15_ACCPERSONALES, "Otros ingresos y Egresos", 27, ignore_subaccounts)
ac121_025 = get_value(unidad15_ACCPERSONALES, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 27, ignore_subaccounts)
ac122_030 = get_value(unidad15_ACCPERSONALES, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 27, ignore_subaccounts)
ac123_035 = get_value(unidad15_ACCPERSONALES, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 27, ignore_subaccounts)
ac124_040 = get_value(unidad15_ACCPERSONALES, "Egresos inversiones de las reservas técnicas (resta)", 27, ignore_subaccounts)
ac125_045 = get_value(unidad15_ACCPERSONALES, "Otros gastos de administración y personal no asignables a ramos", 27, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ac126_999 = abs (ac118_010 + ac119_015 - ac120_020 - ac121_025 - ac122_030 - ac123_035 - ac124_040 + ac125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=29, value=ac126_999)

<Cell 'Hoja1'.AC126>

##  28  COLECTIVO VIDA UNIDAD 15

In [215]:
unidad15_COLECVI = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 28)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_COLECVI['TOTAL'] = unidad15_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 28, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 28, 119),
    ("Otros ingresos y Egresos", 28, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 28, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 28, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 28, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 28, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 28, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_COLECVI, concept, ramo, 30, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ad118_010 = get_value(unidad15_COLECVI, "Otras Provisiones (resta)", 28, ignore_subaccounts)
ad119_015 = get_value(unidad15_COLECVI, "Ingresos y/o Egresos reportes programas especiales", 28, ignore_subaccounts)
ad120_020 = get_value(unidad15_COLECVI, "Otros ingresos y Egresos", 28, ignore_subaccounts)
ad121_025 = get_value(unidad15_COLECVI, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 28, ignore_subaccounts)
ad122_030 = get_value(unidad15_COLECVI, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 28, ignore_subaccounts)
ad123_035 = get_value(unidad15_COLECVI, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 28, ignore_subaccounts)
ad124_040 = get_value(unidad15_COLECVI, "Egresos inversiones de las reservas técnicas (resta)", 28, ignore_subaccounts)
ad125_045 = get_value(unidad15_COLECVI, "Otros gastos de administración y personal no asignables a ramos", 28, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ad126_999 = abs (ad118_010 + ad119_015 - ad120_020 - ad121_025 - ad122_030 - ad123_035 - ad124_040 + ad125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=30, value=ad126_999)

<Cell 'Hoja1'.AD126>

##  29  EDUCATIVO UNIDAD 15

In [216]:
unidad15_EDUCATIVO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 29)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_EDUCATIVO['TOTAL'] = unidad15_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 29, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 29, 119),
    ("Otros ingresos y Egresos", 29, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 29, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 29, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 29, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 29, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 29, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_EDUCATIVO, concept, ramo, 31, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ae118_010 = get_value(unidad15_EDUCATIVO, "Otras Provisiones (resta)", 29, ignore_subaccounts)
ae119_015 = get_value(unidad15_EDUCATIVO, "Ingresos y/o Egresos reportes programas especiales", 29, ignore_subaccounts)
ae120_020 = get_value(unidad15_EDUCATIVO, "Otros ingresos y Egresos", 29, ignore_subaccounts)
ae121_025 = get_value(unidad15_EDUCATIVO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 29, ignore_subaccounts)
ae122_030 = get_value(unidad15_EDUCATIVO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 29, ignore_subaccounts)
ae123_035 = get_value(unidad15_EDUCATIVO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 29, ignore_subaccounts)
ae124_040 = get_value(unidad15_EDUCATIVO, "Egresos inversiones de las reservas técnicas (resta)", 29, ignore_subaccounts)
ae125_045 = get_value(unidad15_EDUCATIVO, "Otros gastos de administración y personal no asignables a ramos", 29, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ae126_999 = abs (ae118_010 + ae119_015 - ae120_020 - ae121_025 - ae122_030 - ae123_035 - ae124_040 + ae125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=31, value=ae126_999)

<Cell 'Hoja1'.AE126>

## 30  VIDA GRUPO UNIDAD 15

In [217]:
unidad15_VIDAGRUPO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 30)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_VIDAGRUPO['TOTAL'] = unidad15_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 30, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 30, 119),
    ("Otros ingresos y Egresos", 30, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 30, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 30, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 30, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 30, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 30, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_VIDAGRUPO, concept, ramo, 32, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
af118_010 = get_value(unidad15_VIDAGRUPO, "Otras Provisiones (resta)", 30, ignore_subaccounts)
af119_015 = get_value(unidad15_VIDAGRUPO, "Ingresos y/o Egresos reportes programas especiales", 30, ignore_subaccounts)
af120_020 = get_value(unidad15_VIDAGRUPO, "Otros ingresos y Egresos", 30, ignore_subaccounts)
af121_025 = get_value(unidad15_VIDAGRUPO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 30, ignore_subaccounts)
af122_030 = get_value(unidad15_VIDAGRUPO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 30, ignore_subaccounts)
af123_035 = get_value(unidad15_VIDAGRUPO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 30, ignore_subaccounts)
af124_040 = get_value(unidad15_VIDAGRUPO, "Egresos inversiones de las reservas técnicas (resta)", 30, ignore_subaccounts)
af125_045 = get_value(unidad15_VIDAGRUPO, "Otros gastos de administración y personal no asignables a ramos", 30, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
af126_999 = abs (af118_010 + af119_015 - af120_020 - af121_025 - af122_030 - af123_035 - af124_040 + af125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=32, value=af126_999)

<Cell 'Hoja1'.AF126>

##  31  SALUD UNIDAD 15

In [218]:
unidad15_SALUD = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 31)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_SALUD['TOTAL'] = unidad15_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 31, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 31, 119),
    ("Otros ingresos y Egresos", 31, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 31, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 31, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 31, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 31, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 31, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_SALUD, concept, ramo, 33, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ag118_010 = get_value(unidad15_SALUD, "Otras Provisiones (resta)", 31, ignore_subaccounts)
ag119_015 = get_value(unidad15_SALUD, "Ingresos y/o Egresos reportes programas especiales", 31, ignore_subaccounts)
ag120_020 = get_value(unidad15_SALUD, "Otros ingresos y Egresos", 31, ignore_subaccounts)
ag121_025 = get_value(unidad15_SALUD, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 31, ignore_subaccounts)
ag122_030 = get_value(unidad15_SALUD, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 31, ignore_subaccounts)
ag123_035 = get_value(unidad15_SALUD, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 31, ignore_subaccounts)
ag124_040 = get_value(unidad15_SALUD, "Egresos inversiones de las reservas técnicas (resta)", 31, ignore_subaccounts)
ag125_045 = get_value(unidad15_SALUD, "Otros gastos de administración y personal no asignables a ramos", 31, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ag126_999 = abs (ag118_010 + ag119_015 - ag120_020 - ag121_025 - ag122_030 - ag123_035 - ag124_040 + ag125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=33, value=ag126_999)

<Cell 'Hoja1'.AG126>

##  32  ENFERMEDADES DE ALTO COSTO UNIDAD 15

In [219]:
unidad15_ENFALTOCOSTO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 32)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_ENFALTOCOSTO['TOTAL'] = unidad15_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 32, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 32, 119),
    ("Otros ingresos y Egresos", 32, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 32, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 32, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 32, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 32, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 32, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_ENFALTOCOSTO, concept, ramo, 34, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ah118_010 = get_value(unidad15_ENFALTOCOSTO, "Otras Provisiones (resta)", 32, ignore_subaccounts)
ah119_015 = get_value(unidad15_ENFALTOCOSTO, "Ingresos y/o Egresos reportes programas especiales", 32, ignore_subaccounts)
ah120_020 = get_value(unidad15_ENFALTOCOSTO, "Otros ingresos y Egresos", 32, ignore_subaccounts)
ah121_025 = get_value(unidad15_ENFALTOCOSTO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 32, ignore_subaccounts)
ah122_030 = get_value(unidad15_ENFALTOCOSTO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 32, ignore_subaccounts)
ah123_035 = get_value(unidad15_ENFALTOCOSTO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 32, ignore_subaccounts)
ah124_040 = get_value(unidad15_ENFALTOCOSTO, "Egresos inversiones de las reservas técnicas (resta)", 32, ignore_subaccounts)
ah125_045 = get_value(unidad15_ENFALTOCOSTO, "Otros gastos de administración y personal no asignables a ramos", 32, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ah126_999 = abs (ah118_010 + ah119_015 - ah120_020 - ah121_025 - ah122_030 - ah123_035 - ah124_040 + ah125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=34, value=ah126_999)

<Cell 'Hoja1'.AH126>

##  33  VIDA INDIVIDUAL UNIDAD 15

In [220]:
unidad15_VIDAINDIVI = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 33)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_VIDAINDIVI['TOTAL'] = unidad15_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 33, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 33, 119),
    ("Otros ingresos y Egresos", 33, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 33, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 33, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 33, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 33, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 33, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_VIDAINDIVI, concept, ramo, 35, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ai118_010 = get_value(unidad15_VIDAINDIVI, "Otras Provisiones (resta)", 33, ignore_subaccounts)
ai119_015 = get_value(unidad15_VIDAINDIVI, "Ingresos y/o Egresos reportes programas especiales", 33, ignore_subaccounts)
ai120_020 = get_value(unidad15_VIDAINDIVI, "Otros ingresos y Egresos", 33, ignore_subaccounts)
ai121_025 = get_value(unidad15_VIDAINDIVI, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 33, ignore_subaccounts)
ai122_030 = get_value(unidad15_VIDAINDIVI, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 33, ignore_subaccounts)
ai123_035 = get_value(unidad15_VIDAINDIVI, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 33, ignore_subaccounts)
ai124_040 = get_value(unidad15_VIDAINDIVI, "Egresos inversiones de las reservas técnicas (resta)", 33, ignore_subaccounts)
ai125_045 = get_value(unidad15_VIDAINDIVI, "Otros gastos de administración y personal no asignables a ramos", 33, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ai126_999 = abs (ai118_010 + ai119_015 - ai120_020 - ai121_025 - ai122_030 - ai123_035 - ai124_040 + ai125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=35, value=ai126_999)

<Cell 'Hoja1'.AI126>

##  34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 15

In [221]:
unidad15_PREVIDENTIAL = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 34)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_PREVIDENTIAL['TOTAL'] = unidad15_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 34, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 34, 119),
    ("Otros ingresos y Egresos", 34, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 34, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 34, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 34, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 34, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 34, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_PREVIDENTIAL, concept, ramo, 36, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
aj118_010 = get_value(unidad15_PREVIDENTIAL, "Otras Provisiones (resta)", 34, ignore_subaccounts)
aj119_015 = get_value(unidad15_PREVIDENTIAL, "Ingresos y/o Egresos reportes programas especiales", 34, ignore_subaccounts)
aj120_020 = get_value(unidad15_PREVIDENTIAL, "Otros ingresos y Egresos", 34, ignore_subaccounts)
aj121_025 = get_value(unidad15_PREVIDENTIAL, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 34, ignore_subaccounts)
aj122_030 = get_value(unidad15_PREVIDENTIAL, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 34, ignore_subaccounts)
aj123_035 = get_value(unidad15_PREVIDENTIAL, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 34, ignore_subaccounts)
aj124_040 = get_value(unidad15_PREVIDENTIAL, "Egresos inversiones de las reservas técnicas (resta)", 34, ignore_subaccounts)
aj125_045 = get_value(unidad15_PREVIDENTIAL, "Otros gastos de administración y personal no asignables a ramos", 34, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
aj126_999 = abs (aj118_010 + aj119_015 - aj120_020 - aj121_025 - aj122_030 - aj123_035 - aj124_040 + aj125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=36, value=aj126_999)

<Cell 'Hoja1'.AJ126>

##  35  RIESGOS LABORALES UNIDAD 15

In [222]:
unidad15_RIESGOSLAB = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 35)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_RIESGOSLAB['TOTAL'] = unidad15_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 35, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 35, 119),
    ("Otros ingresos y Egresos", 35, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 35, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 35, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 35, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 35, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 35, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_RIESGOSLAB, concept, ramo, 37, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ak118_010 = get_value(unidad15_RIESGOSLAB, "Otras Provisiones (resta)", 35, ignore_subaccounts)
ak119_015 = get_value(unidad15_RIESGOSLAB, "Ingresos y/o Egresos reportes programas especiales", 35, ignore_subaccounts)
ak120_020 = get_value(unidad15_RIESGOSLAB, "Otros ingresos y Egresos", 35, ignore_subaccounts)
ak121_025 = get_value(unidad15_RIESGOSLAB, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 35, ignore_subaccounts)
ak122_030 = get_value(unidad15_RIESGOSLAB, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 35, ignore_subaccounts)
ak123_035 = get_value(unidad15_RIESGOSLAB, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 35, ignore_subaccounts)
ak124_040 = get_value(unidad15_RIESGOSLAB, "Egresos inversiones de las reservas técnicas (resta)", 35, ignore_subaccounts)
ak125_045 = get_value(unidad15_RIESGOSLAB, "Otros gastos de administración y personal no asignables a ramos", 35, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ak126_999 = abs (ak118_010 + ak119_015 - ak120_020 - ak121_025 - ak122_030 - ak123_035 - ak124_040 + ak125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=37, value=ak126_999)

<Cell 'Hoja1'.AK126>

##  36  PENSIONES LEY 100

In [223]:
unidad15_PENSIONESLEY100 = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 36)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_PENSIONESLEY100['TOTAL'] = unidad15_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 36, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 36, 119),
    ("Otros ingresos y Egresos", 36, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 36, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 36, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 36, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 36, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 36, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_PENSIONESLEY100, concept, ramo, 38, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
al118_010 = get_value(unidad15_PENSIONESLEY100, "Otras Provisiones (resta)", 36, ignore_subaccounts)
al119_015 = get_value(unidad15_PENSIONESLEY100, "Ingresos y/o Egresos reportes programas especiales", 36, ignore_subaccounts)
al120_020 = get_value(unidad15_PENSIONESLEY100, "Otros ingresos y Egresos", 36, ignore_subaccounts)
al121_025 = get_value(unidad15_PENSIONESLEY100, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 36, ignore_subaccounts)
al122_030 = get_value(unidad15_PENSIONESLEY100, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 36, ignore_subaccounts)
al123_035 = get_value(unidad15_PENSIONESLEY100, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 36, ignore_subaccounts)
al124_040 = get_value(unidad15_PENSIONESLEY100, "Egresos inversiones de las reservas técnicas (resta)", 36, ignore_subaccounts)
al125_045 = get_value(unidad15_PENSIONESLEY100, "Otros gastos de administración y personal no asignables a ramos", 36, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
al126_999 = abs (al118_010 + al119_015 - al120_020 - al121_025 - al122_030 - al123_035 - al124_040 + al125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=38, value=al126_999)

<Cell 'Hoja1'.AL126>

## 37  PENSIONES VOLUNTARIAS

In [224]:
unidad15_PENSIONESVOLUN = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 37)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_PENSIONESVOLUN['TOTAL'] = unidad15_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 37, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 37, 119),
    ("Otros ingresos y Egresos", 37, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 37, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 37, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 37, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 37, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 37, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_PENSIONESVOLUN, concept, ramo, 39, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
am118_010 = get_value(unidad15_PENSIONESVOLUN, "Otras Provisiones (resta)", 37, ignore_subaccounts)
am119_015 = get_value(unidad15_PENSIONESVOLUN, "Ingresos y/o Egresos reportes programas especiales", 37, ignore_subaccounts)
am120_020 = get_value(unidad15_PENSIONESVOLUN, "Otros ingresos y Egresos", 37, ignore_subaccounts)
am121_025 = get_value(unidad15_PENSIONESVOLUN, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 37, ignore_subaccounts)
am122_030 = get_value(unidad15_PENSIONESVOLUN, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 37, ignore_subaccounts)
am123_035 = get_value(unidad15_PENSIONESVOLUN, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 37, ignore_subaccounts)
am124_040 = get_value(unidad15_PENSIONESVOLUN, "Egresos inversiones de las reservas técnicas (resta)", 37, ignore_subaccounts)
am125_045 = get_value(unidad15_PENSIONESVOLUN, "Otros gastos de administración y personal no asignables a ramos", 37, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
am126_999 = abs (am118_010 + am119_015 - am120_020 - am121_025 - am122_030 - am123_035 - am124_040 + am125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=39, value=am126_999)

<Cell 'Hoja1'.AM126>

##  38  PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 15

In [225]:
unidad15_PENSIONESCONMU = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 38)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_PENSIONESCONMU['TOTAL'] = unidad15_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 38, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 38, 119),
    ("Otros ingresos y Egresos", 38, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 38, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 38, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 38, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 38, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 38, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_PENSIONESCONMU, concept, ramo, 40, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
an118_010 = get_value(unidad15_PENSIONESCONMU, "Otras Provisiones (resta)", 38, ignore_subaccounts)
an119_015 = get_value(unidad15_PENSIONESCONMU, "Ingresos y/o Egresos reportes programas especiales", 38, ignore_subaccounts)
an120_020 = get_value(unidad15_PENSIONESCONMU, "Otros ingresos y Egresos", 38, ignore_subaccounts)
an121_025 = get_value(unidad15_PENSIONESCONMU, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 38, ignore_subaccounts)
an122_030 = get_value(unidad15_PENSIONESCONMU, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 38, ignore_subaccounts)
an123_035 = get_value(unidad15_PENSIONESCONMU, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 38, ignore_subaccounts)
an124_040 = get_value(unidad15_PENSIONESCONMU, "Egresos inversiones de las reservas técnicas (resta)", 38, ignore_subaccounts)
an125_045 = get_value(unidad15_PENSIONESCONMU, "Otros gastos de administración y personal no asignables a ramos", 38, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
an126_999 = abs (an118_010 + an119_015 - an120_020 - an121_025 - an122_030 - an123_035 - an124_040 + an125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=40, value=an126_999)

<Cell 'Hoja1'.AN126>

##  39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 15

In [226]:
unidad15_PATRIAUTONOMO = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 39)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_PATRIAUTONOMO['TOTAL'] = unidad15_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 39, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 39, 119),
    ("Otros ingresos y Egresos", 39, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 39, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 39, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 39, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 39, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 39, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_PATRIAUTONOMO, concept, ramo, 41, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ao118_010 = get_value(unidad15_PATRIAUTONOMO, "Otras Provisiones (resta)", 39, ignore_subaccounts)
ao119_015 = get_value(unidad15_PATRIAUTONOMO, "Ingresos y/o Egresos reportes programas especiales", 39, ignore_subaccounts)
ao120_020 = get_value(unidad15_PATRIAUTONOMO, "Otros ingresos y Egresos", 39, ignore_subaccounts)
ao121_025 = get_value(unidad15_PATRIAUTONOMO, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 39, ignore_subaccounts)
ao122_030 = get_value(unidad15_PATRIAUTONOMO, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 39, ignore_subaccounts)
ao123_035 = get_value(unidad15_PATRIAUTONOMO, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 39, ignore_subaccounts)
ao124_040 = get_value(unidad15_PATRIAUTONOMO, "Egresos inversiones de las reservas técnicas (resta)", 39, ignore_subaccounts)
ao125_045 = get_value(unidad15_PATRIAUTONOMO, "Otros gastos de administración y personal no asignables a ramos", 39, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ao126_999 = abs (ao118_010 + ao119_015 - ao120_020 - ao121_025 - ao122_030 - ao123_035 - ao124_040 + ao125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=41, value=ao126_999)

<Cell 'Hoja1'.AO126>

## 40 RENTAS VOLUNTARIAS UNIDAD 15

In [227]:
unidad15_RENTASVOLUN = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 40)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_RENTASVOLUN['TOTAL'] = unidad15_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 40, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 40, 119),
    ("Otros ingresos y Egresos", 40, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 40, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 40, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 40, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 40, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 40, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_RENTASVOLUN, concept, ramo, 42, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
ap118_010 = get_value(unidad15_RENTASVOLUN, "Otras Provisiones (resta)", 40, ignore_subaccounts)
ap119_015 = get_value(unidad15_RENTASVOLUN, "Ingresos y/o Egresos reportes programas especiales", 40, ignore_subaccounts)
ap120_020 = get_value(unidad15_RENTASVOLUN, "Otros ingresos y Egresos", 40, ignore_subaccounts)
ap121_025 = get_value(unidad15_RENTASVOLUN, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 40, ignore_subaccounts)
ap122_030 = get_value(unidad15_RENTASVOLUN, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 40, ignore_subaccounts)
ap123_035 = get_value(unidad15_RENTASVOLUN, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 40, ignore_subaccounts)
ap124_040 = get_value(unidad15_RENTASVOLUN, "Egresos inversiones de las reservas técnicas (resta)", 40, ignore_subaccounts)
ap125_045 = get_value(unidad15_RENTASVOLUN, "Otros gastos de administración y personal no asignables a ramos", 40, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
ap126_999 = abs (ap118_010 + ap119_015 - ap120_020 - ap121_025 - ap122_030 - ap123_035 - ap124_040 + ap125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=42, value=ap126_999)

<Cell 'Hoja1'.AP126>

## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 15

In [228]:
unidad15_BENECONOMICOS = bd[(bd['UNIDAD'] == 15) & (bd['COD_RAMO'] == 41)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad15_BENECONOMICOS['TOTAL'] = unidad15_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        total = 0  # Si el concepto está en las subcuentas ignoradas, asignar 0
    else:
        row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
        total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Otras Provisiones (resta)", 41, 118),
    ("Ingresos y/o Egresos reportes programas especiales", 41, 119),
    ("Otros ingresos y Egresos", 41, 120),
    ("Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 41, 121),
    ("Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 41, 122),
    ("Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 41, 123),
    ("Egresos inversiones de las reservas técnicas (resta)", 41, 124),
    ("Otros gastos de administración y personal no asignables a ramos", 41, 125)
]

# Subcuentas que deben ser ignoradas y tener un valor de 0
ignore_subaccounts = ["Otras Provisiones (resta)", "Otros ingresos y Egresos", "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", "Otros gastos de administración y personal no asignables a ramos"]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad15_BENECONOMICOS, concept, ramo, 43, row, ignore_subaccounts)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo, ignore_subaccounts=[]):
    if concept in ignore_subaccounts:
        return 0  # Si el concepto está en las subcuentas ignoradas, devolver 0
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto con la restricción de las subcuentas ignoradas
aq118_010 = get_value(unidad15_BENECONOMICOS, "Otras Provisiones (resta)", 41, ignore_subaccounts)
aq119_015 = get_value(unidad15_BENECONOMICOS, "Ingresos y/o Egresos reportes programas especiales", 41, ignore_subaccounts)
aq120_020 = get_value(unidad15_BENECONOMICOS, "Otros ingresos y Egresos", 41, ignore_subaccounts)
aq121_025 = get_value(unidad15_BENECONOMICOS, "Ingresos realizados de activos a valor razonable respaldando reservas técnicas", 41, ignore_subaccounts)
aq122_030 = get_value(unidad15_BENECONOMICOS, "Ingresos causados de activos a costo amortizado respaldando reservas técnicas", 41, ignore_subaccounts)
aq123_035 = get_value(unidad15_BENECONOMICOS, "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas", 41, ignore_subaccounts)
aq124_040 = get_value(unidad15_BENECONOMICOS, "Egresos inversiones de las reservas técnicas (resta)", 41, ignore_subaccounts)
aq125_045 = get_value(unidad15_BENECONOMICOS, "Otros gastos de administración y personal no asignables a ramos", 41, ignore_subaccounts)

# Calcular ab94_999 usando la fórmula especificada
aq126_999 = abs (aq118_010 + aq119_015 - aq120_020 - aq121_025 - aq122_030 - aq123_035 - aq124_040 + aq125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=43, value=aq126_999)

<Cell 'Hoja1'.AQ126>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 15

In [229]:
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otras Provisiones (resta)") ]
as118_010 = cons['TOTAL'].sum()
ws.cell(row=118, column=45, value=as118_010)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos y/o Egresos reportes programas especiales") ]
as119_015 = cons['TOTAL'].sum()
ws.cell(row=119, column=45, value=as119_015)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15)].groupby(['CONCEPTO', 'COD_RAMO', 'LEDGER'])['TOTAL'].sum().reset_index()
cons = unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otros ingresos y Egresos")]
as120_020 = cons['TOTAL'].sum() * -1  # Cambiar el signo del valor
ws.cell(row=120, column=45, value=as120_020)

unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos realizados de activos a valor razonable respaldando reservas técnicas") ]
as121_025 = cons['TOTAL'].sum()
as121_025 = 0
ws.cell(row=121, column=45, value=as121_025)


unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Ingresos causados de activos a costo amortizado respaldando reservas técnicas") ]
as122_030 = cons['TOTAL'].sum()
as122_030 = 0
ws.cell(row=122, column=45, value=as122_030)

 
as121_025_030 = bd[
    (bd['UNIDAD'] == 15) &
    (bd['LINE OF BUSS'].isin(['598', '998'])) &  # Corrección aquí
    (bd['CONCEPTO'].isin([
        "Ingresos realizados de activos a valor razonable respaldando reservas técnicas",
        "Ingresos causados de activos a costo amortizado respaldando reservas técnicas"
    ]))
]['TOTAL'].sum()
 
 
as123_035_1 = bd[(bd['UNIDAD'] == 15) &
                 (bd['CONCEPTO'] == "Ingresos y egresos Portafolio de Inversiones diferente a reservas técnicas")]['TOTAL'].sum()
as123_035 = as123_035_1 + as121_025_030
# Invertir el signo
as123_035 *= -1  
ws.cell(row=123, column=45, value=as123_035)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Egresos inversiones de las reservas técnicas (resta)") ]
as124_040 = cons['TOTAL'].sum()
ws.cell(row=124, column=45, value=as124_040)
 
unidad15_OPERNOASIGNABLES = bd[(bd['UNIDAD'] == 15) ].groupby(['CONCEPTO', 'COD_RAMO','LEDGER'])['TOTAL'].sum().reset_index()
cons=unidad15_OPERNOASIGNABLES[(unidad15_OPERNOASIGNABLES['CONCEPTO'] == "Otros gastos de administración y personal no asignables a ramos") ]
as125_045 = cons['TOTAL'].sum()
ws.cell(row=125, column=45, value=as125_045)
 
as126_999 = abs (as118_010 + as119_015 - as120_020 - as121_025 - as122_030 - as123_035 - as124_040 + as125_045)
ws.cell(row=126, column=45, value=as126_999)

<Cell 'Hoja1'.AS126>

#  1   TOTAL UNIDAD 15

In [230]:
c118_010 = sum([ws.cell(row=118, column=col).value if ws.cell(row=118, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=118, column=3).value = c118_010

c119_015 = sum([ws.cell(row=119, column=col).value if ws.cell(row=119, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=119, column=3).value = c119_015

c120_020 = sum([ws.cell(row=120, column=col).value if ws.cell(row=120, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=120, column=3).value = c120_020

c121_025 = sum([ws.cell(row=121, column=col).value if ws.cell(row=121, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=121, column=3).value = c121_025

c122_030 = sum([ws.cell(row=122, column=col).value if ws.cell(row=122, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=122, column=3).value = c122_030

c123_035 = sum([ws.cell(row=123, column=col).value if ws.cell(row=123, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=123, column=3).value = c123_035

c124_040 = sum([ws.cell(row=124, column=col).value if ws.cell(row=124, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=124, column=3).value = c124_040

c125_045 = sum([ws.cell(row=125, column=col).value if ws.cell(row=125, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=125, column=3).value = c125_045

# Calcular el total general según la fórmula dada

c126_999 = abs(c118_010 + c119_015 - c120_020 - c121_025 - c122_030 - c123_035 - c124_040 + c125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=3).value = c126_999

#  2   SUBTOTAL - RAMOS UNIDAD 15

In [231]:
d118_010 = sum([ws.cell(row=118, column=col).value if ws.cell(row=118, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=118, column=4).value = d118_010

d119_015 = sum([ws.cell(row=119, column=col).value if ws.cell(row=119, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=119, column=4).value = d119_015

d120_020 = sum([ws.cell(row=120, column=col).value if ws.cell(row=120, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=120, column=4).value = d120_020

d121_025 = sum([ws.cell(row=121, column=col).value if ws.cell(row=121, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=121, column=4).value = d121_025

d122_030 = sum([ws.cell(row=122, column=col).value if ws.cell(row=122, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=122, column=4).value = d122_030

d123_035 = sum([ws.cell(row=123, column=col).value if ws.cell(row=123, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=123, column=4).value = d123_035

d124_040 = sum([ws.cell(row=124, column=col).value if ws.cell(row=124, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=124, column=4).value = d124_040

d125_045 = sum([ws.cell(row=125, column=col).value if ws.cell(row=125, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=125, column=4).value = d125_045

# Calcular el total general según la fórmula dada

d126_999 = abs(d118_010 + d119_015 - d120_020 - d121_025 - d122_030 - d123_035 - d124_040 + d125_045)

# Escribir el total general en Excel
ws.cell(row=126, column=4).value = d126_999


# UNIDAD DE CAPTURA 16

## CONSULTA DE TOTALES UNIDAD 16

In [232]:
c128_999 = c115_999 + c126_999
df_c128_999 = pd.DataFrame({'total': [c128_999]})
if df_c128_999.empty or c128_999 == 0:
    df_c128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=3).value = df_c128_999['total'].iloc[0]

ab128_999 = ab115_999 + ab126_999
df_ab128_999 = pd.DataFrame({'total': [ab128_999]})
if df_ab128_999.empty or ab128_999 == 0:
    df_ab128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=28).value = df_ab128_999['total'].iloc[0]

ac128_999 = ac115_999 + ac126_999
df_ac128_999 = pd.DataFrame({'total': [ac128_999]})
if df_ac128_999.empty or ac128_999 == 0:
    df_ac128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=29).value = df_ac128_999['total'].iloc[0]

ad128_999 = ad115_999 + ad126_999
df_ad128_999 = pd.DataFrame({'total': [ad128_999]})
if df_ad128_999.empty or ad128_999 == 0:
    df_ad128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=30).value = df_ad128_999['total'].iloc[0]

ae128_999 = ae115_999 + ae126_999
df_ae128_999 = pd.DataFrame({'total': [ae128_999]})
if df_ae128_999.empty or ae128_999 == 0:
    df_ae128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=31).value = df_ae128_999['total'].iloc[0]

af128_999 = af115_999 + af126_999
df_af128_999 = pd.DataFrame({'total': [af128_999]})
if df_af128_999.empty or af128_999 == 0:
    df_af128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=32).value = df_af128_999['total'].iloc[0]

ag128_999 = ag115_999 + ag126_999
df_ag128_999 = pd.DataFrame({'total': [ag128_999]})
if df_ag128_999.empty or ag128_999 == 0:
    df_ag128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=33).value = df_ag128_999['total'].iloc[0]

ah128_999 = ah115_999 + ah126_999
df_ah128_999 = pd.DataFrame({'total': [ah128_999]})
if df_ah128_999.empty or ah128_999 == 0:
    df_ah128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=34).value = df_ah128_999['total'].iloc[0]

ai128_999 = ai115_999 + ai126_999
df_ai128_999 = pd.DataFrame({'total': [ai128_999]})
if df_ai128_999.empty or ai128_999 == 0:
    df_ai128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=35).value = df_ai128_999['total'].iloc[0]

aj128_999 = aj115_999 + aj126_999
df_aj128_999 = pd.DataFrame({'total': [aj128_999]})
if df_aj128_999.empty or aj128_999 == 0:
    df_aj128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=36).value = df_aj128_999['total'].iloc[0]

ak128_999 = ak115_999 + ak126_999
df_ak128_999 = pd.DataFrame({'total': [ak128_999]})
if df_ak128_999.empty or ak128_999 == 0:
    df_ak128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=37).value = df_ak128_999['total'].iloc[0]

al128_999 = al115_999 + al126_999
df_al128_999 = pd.DataFrame({'total': [al128_999]})
if df_al128_999.empty or al128_999 == 0:
    df_al128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=38).value = df_al128_999['total'].iloc[0]

am128_999 = am115_999 + am126_999
df_am128_999 = pd.DataFrame({'total': [am128_999]})
if df_am128_999.empty or am128_999 == 0:
    df_am128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=39).value = df_am128_999['total'].iloc[0]

an128_999 = an115_999 + an126_999
df_an128_999 = pd.DataFrame({'total': [an128_999]})
if df_an128_999.empty or an128_999 == 0:
    df_an128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=40).value = df_an128_999['total'].iloc[0]

ao128_999 = ao115_999 + ao126_999
df_ao128_999 = pd.DataFrame({'total': [ao128_999]})
if df_ao128_999.empty or ao128_999 == 0:
    df_ao128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=41).value = df_ao128_999['total'].iloc[0]

ap128_999 = ap115_999 + ap126_999
df_ap128_999 = pd.DataFrame({'total': [ap128_999]})
if df_ap128_999.empty or ap128_999 == 0:
    df_ap128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=42).value = df_ap128_999['total'].iloc[0]

aq128_999 = aq115_999 + aq126_999
df_aq128_999 = pd.DataFrame({'total': [aq128_999]})
if df_aq128_999.empty or aq128_999 == 0:
    df_aq128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=43).value = df_aq128_999['total'].iloc[0]

as128_999 = as115_999 + as126_999
df_as128_999 = pd.DataFrame({'total': [as128_999]})
if df_as128_999.empty or as128_999 == 0:
    df_as128_999 = pd.DataFrame({'total': [0]})
ws.cell(row=128, column=45).value = df_as128_999['total'].iloc[0]


## CONSULTA DE SUBTOTALES UNIDAD 16

In [233]:
# Realiza la suma para d75_999
d128_999 = d115_999 + d126_999
df_d128_999 = pd.DataFrame({'total': [d128_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d128_999.empty or d128_999 == 0:
    df_d128_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=128, column=4).value = df_d128_999['total'].iloc[0]

## UNIDAD DE CAPTURA 17

#   26  EXEQUIAS UNIDAD 17

In [234]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_EXEQUIAS = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 26)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_EXEQUIAS['TOTAL'] = unidad17_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",26, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_EXEQUIAS, concept, ramo, 28, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ab130_999 = get_value(unidad17_EXEQUIAS, "Impuesto de renta y complementarios", 26)

# Escribir el total general en Excel
ws.cell(row=130, column=28, value=ab130_999)

<Cell 'Hoja1'.AB130>

# 27  ACCIDENTES PERSONALES UNIDAD 17

In [235]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_ACCPERSONALES = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 27)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_ACCPERSONALES['TOTAL'] = unidad17_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",27, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_ACCPERSONALES, concept, ramo, 29, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ac130_999 = get_value(unidad17_ACCPERSONALES, "Impuesto de renta y complementarios", 27)

# Escribir el total general en Excel
ws.cell(row=130, column=29, value=ac130_999)

<Cell 'Hoja1'.AC130>

#  28  COLECTIVO VIDA UNIDAD 17

In [236]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_COLECVI = bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 28)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_COLECVI['TOTAL'] = unidad17_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",28, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_COLECVI, concept, ramo, 30, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ad130_999 = get_value(unidad17_COLECVI, "Impuesto de renta y complementarios", 28)

# Escribir el total general en Excel
ws.cell(row=130, column=30, value=ad130_999)

<Cell 'Hoja1'.AD130>

#  29  EDUCATIVO UNIDAD 17

In [237]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_EDUCATIVO= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 29)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_EDUCATIVO['TOTAL'] = unidad17_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",29, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_EDUCATIVO, concept, ramo, 31, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ae130_999 = get_value(unidad17_EDUCATIVO, "Impuesto de renta y complementarios", 29)

# Escribir el total general en Excel
ws.cell(row=130, column=31, value=ae130_999)

<Cell 'Hoja1'.AE130>

#  30  VIDA GRUPO UNIDAD 17

In [238]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_VIDAGRUPO= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 30)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_VIDAGRUPO['TOTAL'] = unidad17_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",30, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_VIDAGRUPO, concept, ramo, 32, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
af130_999 = get_value(unidad17_VIDAGRUPO, "Impuesto de renta y complementarios", 30)

# Escribir el total general en Excel
ws.cell(row=130, column=32, value=af130_999)

<Cell 'Hoja1'.AF130>

#  31  SALUD UNIDAD 17

In [239]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_SALUD= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 31)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_SALUD['TOTAL'] = unidad17_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",31, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_SALUD, concept, ramo, 33, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ag130_999 = get_value(unidad17_SALUD, "Impuesto de renta y complementarios", 31)

# Escribir el total general en Excel
ws.cell(row=130, column=33, value=ag130_999)

<Cell 'Hoja1'.AG130>

#  32  ENFERMEDADES DE ALTO COSTO UNIDAD 17

In [240]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_ENFALTOCOSTO= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 32)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_ENFALTOCOSTO['TOTAL'] = unidad17_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",32, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_ENFALTOCOSTO, concept, ramo, 34, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ah130_999 = get_value(unidad17_ENFALTOCOSTO, "Impuesto de renta y complementarios", 32)

# Escribir el total general en Excel
ws.cell(row=130, column=34, value=ah130_999)

<Cell 'Hoja1'.AH130>

#  33  VIDA INDIVIDUAL UNIDAD 17

In [241]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_VIDAINDIVI= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 33)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_VIDAINDIVI['TOTAL'] = unidad17_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",33, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_VIDAINDIVI, concept, ramo, 35, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ai130_999 = get_value(unidad17_VIDAINDIVI, "Impuesto de renta y complementarios", 33)

# Escribir el total general en Excel
ws.cell(row=130, column=35, value=ai130_999)

<Cell 'Hoja1'.AI130>

#  34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 17

In [242]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_PREVIDENTIAL= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 34)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_PREVIDENTIAL['TOTAL'] = unidad17_PREVIDENTIAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",34, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_PREVIDENTIAL, concept, ramo, 36, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aj130_999 = get_value(unidad17_PREVIDENTIAL, "Impuesto de renta y complementarios", 34)

# Escribir el total general en Excel
ws.cell(row=130, column=36, value=aj130_999)

<Cell 'Hoja1'.AJ130>

#  35  RIESGOS LABORALES UNIDAD 17

In [243]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_RIESGOSLAB= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 35)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_RIESGOSLAB['TOTAL'] = unidad17_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",35, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_RIESGOSLAB, concept, ramo, 37, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ak130_999 = get_value(unidad17_RIESGOSLAB, "Impuesto de renta y complementarios", 35)

# Escribir el total general en Excel
ws.cell(row=130, column=37, value=ak130_999)

<Cell 'Hoja1'.AK130>

#  36  PENSIONES LEY 100 UNIDAD 17

In [244]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_PENSIONESLEY100= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 36)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_PENSIONESLEY100['TOTAL'] = unidad17_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",36, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_PENSIONESLEY100, concept, ramo, 38, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
al130_999 = get_value(unidad17_PENSIONESLEY100, "Impuesto de renta y complementarios", 36)

# Escribir el total general en Excel
ws.cell(row=130, column=38, value=al130_999)

<Cell 'Hoja1'.AL130>

#  37  PENSIONES VOLUNTARIAS UNIDAD 17

In [245]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_PENSIONESVOLUN= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 37)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_PENSIONESVOLUN['TOTAL'] = unidad17_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",37, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_PENSIONESVOLUN, concept, ramo, 39, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
am130_999 = get_value(unidad17_PENSIONESVOLUN, "Impuesto de renta y complementarios", 37)

# Escribir el total general en Excel
ws.cell(row=130, column=39, value=am130_999)

<Cell 'Hoja1'.AM130>

#  38  PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 17

In [246]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_PENSIONESCONMU= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 38)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_PENSIONESCONMU['TOTAL'] = unidad17_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",38, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_PENSIONESCONMU, concept, ramo, 40, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
an130_999 = get_value(unidad17_PENSIONESCONMU, "Impuesto de renta y complementarios", 38)

# Escribir el total general en Excel
ws.cell(row=130, column=40, value=an130_999)

<Cell 'Hoja1'.AN130>

#  39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 17

In [247]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_PATRIAUTONOMO= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 39)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_PATRIAUTONOMO['TOTAL'] = unidad17_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",39, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_PATRIAUTONOMO, concept, ramo, 41, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ao130_999 = get_value(unidad17_PATRIAUTONOMO, "Impuesto de renta y complementarios", 39)

# Escribir el total general en Excel
ws.cell(row=130, column=41, value=ao130_999)

<Cell 'Hoja1'.AO130>

# 40 RENTAS VOLUNTARIAS UNIDAD 17

In [248]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_RENTASVOLUN= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 40)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_RENTASVOLUN['TOTAL'] = unidad17_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",40, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_RENTASVOLUN, concept, ramo, 42, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
ap130_999 = get_value(unidad17_RENTASVOLUN, "Impuesto de renta y complementarios", 40)

# Escribir el total general en Excel
ws.cell(row=130, column=42, value=ap130_999)

<Cell 'Hoja1'.AP130>

# 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 17

In [249]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_BENECONOMICOS= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 41)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_BENECONOMICOS['TOTAL'] = unidad17_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",41, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_BENECONOMICOS, concept, ramo, 43, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
aq130_999 = get_value(unidad17_BENECONOMICOS, "Impuesto de renta y complementarios", 41)

# Escribir el total general en Excel
ws.cell(row=130, column=43, value=aq130_999)

<Cell 'Hoja1'.AQ130>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 17

In [250]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad17_OPERNOASIGNABLES= bd[(bd['UNIDAD'] == 17) & (bd['COD_RAMO'] == 43)].groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad17_OPERNOASIGNABLES['TOTAL'] = unidad17_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Impuesto de renta y complementarios",43, 130),
]

# Escribir cada valor en la hoja de Excel
for concept, ramo, row in concepts_info:
    write_to_excel(unidad17_OPERNOASIGNABLES, concept, ramo, 45, row)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0


# Obtener los valores individuales de cada concepto
as130_999 = get_value(unidad17_OPERNOASIGNABLES, "Impuesto de renta y complementarios", 43)

# Escribir el total general en Excel
ws.cell(row=130, column=45, value=as130_999)

<Cell 'Hoja1'.AS130>

#  1   TOTAL UNIDAD 17

In [251]:
c130_999 = sum([ws.cell(row=130, column=col).value if ws.cell(row=130, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=130, column=3).value = c130_999

#  2   SUBTOTAL - RAMOS UNIDAD 17

In [252]:
d130_999 = sum([ws.cell(row=130, column=col).value if ws.cell(row=130, column=col).value is not None else 0 for col in range(5, 41)])
ws.cell(row=130, column=4).value = d130_999

## UNIDAD DE CAPTURA 18

#  CONSULTA DE TOTALES UNIDAD 18

In [253]:
c132_999 = c128_999 - c130_999
df_c132_999 = pd.DataFrame({'total': [c132_999]})
if df_c132_999.empty or c132_999 == 0:
    df_c132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=3).value = df_c132_999['total'].iloc[0]

ab132_999 = ab128_999 - ab130_999
df_ab132_999 = pd.DataFrame({'total': [ab132_999]})
if df_ab132_999.empty or ab132_999 == 0:
    df_ab132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=28).value = df_ab132_999['total'].iloc[0]

ac132_999 = ac128_999 - ac130_999
df_ac132_999 = pd.DataFrame({'total': [ac132_999]})
if df_ac132_999.empty or ac132_999 == 0:
    df_ac132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=29).value = df_ac132_999['total'].iloc[0]

ad132_999 = ad128_999 - ad130_999
df_ad132_999 = pd.DataFrame({'total': [ad132_999]})
if df_ad132_999.empty or ad132_999 == 0:
    df_ad132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=30).value = df_ad132_999['total'].iloc[0]

ae132_999 = ae128_999 - ae130_999
df_ae132_999 = pd.DataFrame({'total': [ae132_999]})
if df_ae132_999.empty or ae132_999 == 0:
    df_ae132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=31).value = df_ae132_999['total'].iloc[0]

af132_999 = af128_999 - af130_999
df_af132_999 = pd.DataFrame({'total': [af132_999]})
if df_af132_999.empty or af132_999 == 0:
    df_af132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=32).value = df_af132_999['total'].iloc[0]

ag132_999 = ag128_999 - ag130_999
df_ag132_999 = pd.DataFrame({'total': [ag132_999]})
if df_ag132_999.empty or ag132_999 == 0:
    df_ag132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=33).value = df_ag132_999['total'].iloc[0]

ah132_999 = ah128_999 - ah130_999
df_ah132_999 = pd.DataFrame({'total': [ah132_999]})
if df_ah132_999.empty or ah132_999 == 0:
    df_ah132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=34).value = df_ah132_999['total'].iloc[0]

ai132_999 = ai128_999 - ai130_999
df_ai132_999 = pd.DataFrame({'total': [ai132_999]})
if df_ai132_999.empty or ai132_999 == 0:
    df_ai132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=35).value = df_ai132_999['total'].iloc[0]

aj132_999 = aj128_999 - aj130_999
df_aj132_999 = pd.DataFrame({'total': [aj132_999]})
if df_aj132_999.empty or aj132_999 == 0:
    df_aj132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=36).value = df_aj132_999['total'].iloc[0]

ak132_999 = ak128_999 - ak130_999
df_ak132_999 = pd.DataFrame({'total': [ak132_999]})
if df_ak132_999.empty or ak132_999 == 0:
    df_ak132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=37).value = df_ak132_999['total'].iloc[0]

al132_999 = al128_999 - al130_999
df_al132_999 = pd.DataFrame({'total': [al132_999]})
if df_al132_999.empty or al132_999 == 0:
    df_al132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=38).value = df_al132_999['total'].iloc[0]

am132_999 = am128_999 - am130_999
df_am132_999 = pd.DataFrame({'total': [am132_999]})
if df_am132_999.empty or am132_999 == 0:
    df_am132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=39).value = df_am132_999['total'].iloc[0]

an132_999 = an128_999 - an130_999
df_an132_999 = pd.DataFrame({'total': [an132_999]})
if df_an132_999.empty or an132_999 == 0:
    df_an132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=40).value = df_an132_999['total'].iloc[0]

ao132_999 = ao128_999 - ao130_999
df_ao132_999 = pd.DataFrame({'total': [ao132_999]})
if df_ao132_999.empty or ao132_999 == 0:
    df_ao132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=41).value = df_ao132_999['total'].iloc[0]

ap132_999 = ap128_999 - ap130_999
df_ap132_999 = pd.DataFrame({'total': [ap132_999]})
if df_ap132_999.empty or ap132_999 == 0:
    df_ap132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=42).value = df_ap132_999['total'].iloc[0]

aq132_999 = aq128_999 - aq130_999
df_aq132_999 = pd.DataFrame({'total': [aq132_999]})
if df_aq132_999.empty or aq132_999 == 0:
    df_aq132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=43).value = df_aq132_999['total'].iloc[0]

as132_999 = as128_999 - as130_999
df_as132_999 = pd.DataFrame({'total': [as132_999]})
if df_as132_999.empty or as132_999 == 0:
    df_as132_999 = pd.DataFrame({'total': [0]})
ws.cell(row=132, column=45).value = df_as132_999['total'].iloc[0]


# CONSULTA DE SUBTOTALES - RAMOS UNIDAD 18

In [254]:
# Realiza la suma para d75_999
d132_999 = d128_999 - d130_999
df_d132_999 = pd.DataFrame({'total': [d132_999]})

# Verifica si el resultado es vacío o 0 y ajusta en consecuencia
if df_d132_999.empty or d132_999 == 0:
    df_d132_999 = pd.DataFrame({'total': [0]})

# Escribir en la celda especificada
ws.cell(row=132, column=4).value = df_d132_999['total'].iloc[0]

# UNIDAD DE CAPTURA 19  

##  CONSULTA DE TOTALES VALORES ASEGURADOS (Millones de pesos) UNIDAD 19

In [255]:
# Agrupar los datos por 'CONCEPTO' para obtener los totales en unidad19
unidad19_totales = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad19_totales['TOTAL'] = unidad19_totales['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=3):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad19_totales, "Directos", 134)
write_to_excel(unidad19_totales, "Aceptados en reaseguro", 135)
write_to_excel(unidad19_totales, "Parte reaseguro al interior (resta)", 136)
write_to_excel(unidad19_totales, "Parte reaseguro al exterior (resta)", 137)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

c134_005 = get_value(unidad19_totales, "Directos")
c135_010 = get_value(unidad19_totales, "Aceptados en reaseguro")
c136_015 = get_value(unidad19_totales, "Parte reaseguro al interior (resta)")
c137_020 = get_value(unidad19_totales, "Parte reaseguro al exterior (resta)")

# Calcular el total general según la fórmula dada
c138_999 = c134_005 + c135_010 - c136_015 - c137_020

# Escribir el total general en Excel
ws.cell(row=138, column=3).value = c138_999


##  CONSULTA DE SUBTOTALES VALORES ASEGURADOS (Millones de pesos) UNIDAD 19

In [256]:
# Agrupar los datos por 'CONCEPTO' para obtener los subtotales en unidad19
unidad19_subtotal = bd.groupby('CONCEPTO')['TOTAL'].sum().reset_index()
unidad19_subtotal['TOTAL'] = unidad19_subtotal['TOTAL'].abs()  # Tomamos el valor absoluto de los totales

# Función para escribir en Excel
def write_to_excel(data_frame, concept, start_row, start_col=4):
    row = data_frame[data_frame['CONCEPTO'] == concept]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col).value = total

# Llamadas a la función para escribir en Excel
write_to_excel(unidad19_subtotal, "Directos", 134)
write_to_excel(unidad19_subtotal, "Aceptados en reaseguro", 135)
write_to_excel(unidad19_subtotal, "Parte reaseguro al interior (resta)", 136)
write_to_excel(unidad19_subtotal, "Parte reaseguro al exterior (resta)", 137)

# Obtener los valores individuales de cada concepto
def get_value(df, concept):
    row = df[df['CONCEPTO'] == concept]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

d134_005 = get_value(unidad19_subtotal, "Directos")
d135_010 = get_value(unidad19_subtotal, "Aceptados en reaseguro")
d136_015 = get_value(unidad19_subtotal, "Parte reaseguro al interior (resta)")
d137_020 = get_value(unidad19_subtotal, "Parte reaseguro al exterior (resta)")

# Calcular el total general según la fórmula dada
d138_999 = d134_005 + d135_010 - d136_015 - d137_020

# Escribir el total general en Excel
ws.cell(row=138, column=4).value = d138_999


## 26 EXEQUIAS UNIDAD 19  

In [257]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_EXEQUIAS
unidad19_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_EXEQUIAS['TOTAL'] = unidad19_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab134_005 = get_value(unidad19_EXEQUIAS, "Directos", 26)
ab135_010 = get_value(unidad19_EXEQUIAS, "Aceptados en reaseguro", 26)
ab136_015 = get_value(unidad19_EXEQUIAS, "Parte reaseguro al interior (resta)", 26)
ab137_020 = get_value(unidad19_EXEQUIAS, "Parte reaseguro al exterior (resta)", 26)

# Calcular ab138_999 usando la fórmula especificada
ab138_999 = (ab134_005 + ab135_010 - ab136_015 - ab137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=28, value=ab138_999)


<Cell 'Hoja1'.AB138>

##  27 ACCIDENTES PERSONALES UNIDAD 19 

In [258]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_ACCPERSONALES
unidad19_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_ACCPERSONALES['TOTAL'] = unidad19_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac134_005 = get_value(unidad19_ACCPERSONALES, "Directos", 27)
ac135_010 = get_value(unidad19_ACCPERSONALES, "Aceptados en reaseguro", 27)
ac136_015 = get_value(unidad19_ACCPERSONALES, "Parte reaseguro al interior (resta)", 27)
ac137_020 = get_value(unidad19_ACCPERSONALES, "Parte reaseguro al exterior (resta)", 27)

# Calcular ac138_999 usando la fórmula especificada
ac138_999 = (ac134_005 + ac135_010 - ac136_015 - ac137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=29, value=ac138_999)


<Cell 'Hoja1'.AC138>

## 28 COLECTIVO VIDA UNIDAD 19

In [259]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_COLECVI
unidad19_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_COLECVI['TOTAL'] = unidad19_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad134_005 = get_value(unidad19_COLECVI, "Directos", 28)
ad135_010 = get_value(unidad19_COLECVI, "Aceptados en reaseguro", 28)
ad136_015 = get_value(unidad19_COLECVI, "Parte reaseguro al interior (resta)", 28)
ad137_020 = get_value(unidad19_COLECVI, "Parte reaseguro al exterior (resta)", 28)

# Calcular ad138_999 usando la fórmula especificada
ad138_999 = (ad134_005 + ad135_010 - ad136_015 - ad137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=30, value=ad138_999)


<Cell 'Hoja1'.AD138>

##  29 EDUCATIVO  UNIDAD 19 

In [260]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_EDUCATIVO
unidad19_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_EDUCATIVO['TOTAL'] = unidad19_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae134_005 = get_value(unidad19_EDUCATIVO, "Directos", 29)
ae135_010 = get_value(unidad19_EDUCATIVO, "Aceptados en reaseguro", 29)
ae136_015 = get_value(unidad19_EDUCATIVO, "Parte reaseguro al interior (resta)", 29)
ae137_020 = get_value(unidad19_EDUCATIVO, "Parte reaseguro al exterior (resta)", 29)

# Calcular ae138_999 usando la fórmula especificada
ae138_999 = (ae134_005 + ae135_010 - ae136_015 - ae137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=31, value=ae138_999)


<Cell 'Hoja1'.AE138>

## 30  VIDA GRUPO  UNIDAD 19

In [261]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_VIDAGRUPO
unidad19_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_VIDAGRUPO['TOTAL'] = unidad19_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af134_005 = get_value(unidad19_VIDAGRUPO, "Directos", 30)
af135_010 = get_value(unidad19_VIDAGRUPO, "Aceptados en reaseguro", 30)
af136_015 = get_value(unidad19_VIDAGRUPO, "Parte reaseguro al interior (resta)", 30)
af137_020 = get_value(unidad19_VIDAGRUPO, "Parte reaseguro al exterior (resta)", 30)

# Calcular af138_999 usando la fórmula especificada
af138_999 = (af134_005 + af135_010 - af136_015 - af137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=32, value=af138_999)


<Cell 'Hoja1'.AF138>

##  31 SALUD UNIDAD 19 

In [262]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_SALUD
unidad19_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_SALUD['TOTAL'] = unidad19_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag134_005 = get_value(unidad19_SALUD, "Directos", 31)
ag135_010 = get_value(unidad19_SALUD, "Aceptados en reaseguro", 31)
ag136_015 = get_value(unidad19_SALUD, "Parte reaseguro al interior (resta)", 31)
ag137_020 = get_value(unidad19_SALUD, "Parte reaseguro al exterior (resta)", 31)

# Calcular ag138_999 usando la fórmula especificada
ag138_999 = (ag134_005 + ag135_010 - ag136_015 - ag137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=33, value=ag138_999)


<Cell 'Hoja1'.AG138>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 19 

In [263]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_ENFALTOCOSTO
unidad19_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_ENFALTOCOSTO['TOTAL'] = unidad19_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah134_005 = get_value(unidad19_ENFALTOCOSTO, "Directos", 32)
ah135_010 = get_value(unidad19_ENFALTOCOSTO, "Aceptados en reaseguro", 32)
ah136_015 = get_value(unidad19_ENFALTOCOSTO, "Parte reaseguro al interior (resta)", 32)
ah137_020 = get_value(unidad19_ENFALTOCOSTO, "Parte reaseguro al exterior (resta)", 32)

# Calcular ah138_999 usando la fórmula especificada
ah138_999 = (ah134_005 + ah135_010 - ah136_015 - ah137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=34, value=ah138_999)


<Cell 'Hoja1'.AH138>

##  33 VIDA INDIVIDUAL UNIDAD 19 

In [264]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_VIDAINDIVI
unidad19_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_VIDAINDIVI['TOTAL'] = unidad19_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai134_005 = get_value(unidad19_VIDAINDIVI, "Directos", 33)
ai135_010 = get_value(unidad19_VIDAINDIVI, "Aceptados en reaseguro", 33)
ai136_015 = get_value(unidad19_VIDAINDIVI, "Parte reaseguro al interior (resta)", 33)
ai137_020 = get_value(unidad19_VIDAINDIVI, "Parte reaseguro al exterior (resta)", 33)

# Calcular ai138_999 usando la fórmula especificada
ai138_999 = (ai134_005 + ai135_010 - ai136_015 - ai137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=35, value=ai138_999)


<Cell 'Hoja1'.AI138>

##    34 PREVISIONAL DE INVALIDEZ Y SOBREVIVENCIA UNIDAD 19 

In [265]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_PREVIDINVAL
unidad19_PREVIDINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PREVIDINVAL['TOTAL'] = unidad19_PREVIDINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PREVIDINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj134_005 = get_value(unidad19_PREVIDINVAL, "Directos", 34)
aj135_010 = get_value(unidad19_PREVIDINVAL, "Aceptados en reaseguro", 34)
aj136_015 = get_value(unidad19_PREVIDINVAL, "Parte reaseguro al interior (resta)", 34)
aj137_020 = get_value(unidad19_PREVIDINVAL, "Parte reaseguro al exterior (resta)", 34)

# Calcular aj138_999 usando la fórmula especificada
aj138_999 = (aj134_005 + aj135_010 - aj136_015 - aj137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=36, value=aj138_999)


<Cell 'Hoja1'.AJ138>

## 35  RIESGOS LABORALES UNIDAD 19 

In [266]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_RIESGOSLAB
unidad19_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_RIESGOSLAB['TOTAL'] = unidad19_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak134_005 = get_value(unidad19_RIESGOSLAB, "Directos", 35)
ak135_010 = get_value(unidad19_RIESGOSLAB, "Aceptados en reaseguro", 35)
ak136_015 = get_value(unidad19_RIESGOSLAB, "Parte reaseguro al interior (resta)", 35)
ak137_020 = get_value(unidad19_RIESGOSLAB, "Parte reaseguro al exterior (resta)", 35)

# Calcular ak138_999 usando la fórmula especificada
ak138_999 = (ak134_005 + ak135_010 - ak136_015 - ak137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=37, value=ak138_999)


<Cell 'Hoja1'.AK138>

##  36 PENSIONES LEY 100 UNIDAD 19 

In [267]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_PENSIONESLEY100
unidad19_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PENSIONESLEY100['TOTAL'] = unidad19_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al134_005 = get_value(unidad19_PENSIONESLEY100, "Directos", 36)
al135_010 = get_value(unidad19_PENSIONESLEY100, "Aceptados en reaseguro", 36)
al136_015 = get_value(unidad19_PENSIONESLEY100, "Parte reaseguro al interior (resta)", 36)
al137_020 = get_value(unidad19_PENSIONESLEY100, "Parte reaseguro al exterior (resta)", 36)

# Calcular al138_999 usando la fórmula especificada
al138_999 = (al134_005 + al135_010 - al136_015 - al137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=38, value=al138_999)


<Cell 'Hoja1'.AL138>

##  37 PENSIONES VOLUNTARIAS UNIDAD 19 

In [268]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_PENSIONESVOLUN
unidad19_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PENSIONESVOLUN['TOTAL'] = unidad19_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am134_005 = get_value(unidad19_PENSIONESVOLUN, "Directos", 37)
am135_010 = get_value(unidad19_PENSIONESVOLUN, "Aceptados en reaseguro", 37)
am136_015 = get_value(unidad19_PENSIONESVOLUN, "Parte reaseguro al interior (resta)", 37)
am137_020 = get_value(unidad19_PENSIONESVOLUN, "Parte reaseguro al exterior (resta)", 37)

# Calcular am138_999 usando la fórmula especificada
am138_999 = (am134_005 + am135_010 - am136_015 - am137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=39, value=am138_999)


<Cell 'Hoja1'.AM138>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 19 

In [269]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_PENSIONESCONMU
unidad19_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PENSIONESCONMU['TOTAL'] = unidad19_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an134_005 = get_value(unidad19_PENSIONESCONMU, "Directos", 38)
an135_010 = get_value(unidad19_PENSIONESCONMU, "Aceptados en reaseguro", 38)
an136_015 = get_value(unidad19_PENSIONESCONMU, "Parte reaseguro al interior (resta)", 38)
an137_020 = get_value(unidad19_PENSIONESCONMU, "Parte reaseguro al exterior (resta)", 38)

# Calcular an138_999 usando la fórmula especificada
an138_999 = (an134_005 + an135_010 - an136_015 - an137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=40, value=an138_999)


<Cell 'Hoja1'.AN138>

##   39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 19  

In [270]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_PATRIAUTONOMO
unidad19_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PATRIAUTONOMO['TOTAL'] = unidad19_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao134_005 = get_value(unidad19_PATRIAUTONOMO, "Directos", 39)
ao135_010 = get_value(unidad19_PATRIAUTONOMO, "Aceptados en reaseguro", 39)
ao136_015 = get_value(unidad19_PATRIAUTONOMO, "Parte reaseguro al interior (resta)", 39)
ao137_020 = get_value(unidad19_PATRIAUTONOMO, "Parte reaseguro al exterior (resta)", 39)

# Calcular ao138_999 usando la fórmula especificada
ao138_999 = (ao134_005 + ao135_010 - ao136_015 - ao137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=41, value=ao138_999)


<Cell 'Hoja1'.AO138>

##   40 RENTAS VOLUNTARIAS UNIDAD 19  

In [271]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_RENTASVOLUN
unidad19_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_RENTASVOLUN['TOTAL'] = unidad19_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap134_005 = get_value(unidad19_RENTASVOLUN, "Directos", 40)
ap135_010 = get_value(unidad19_RENTASVOLUN, "Aceptados en reaseguro", 40)
ap136_015 = get_value(unidad19_RENTASVOLUN, "Parte reaseguro al interior (resta)", 40)
ap137_020 = get_value(unidad19_RENTASVOLUN, "Parte reaseguro al exterior (resta)", 40)

# Calcular ap138_999 usando la fórmula especificada
ap138_999 = (ap134_005 + ap135_010 - ap136_015 - ap137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=42, value=ap138_999)


<Cell 'Hoja1'.AP138>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 19

In [272]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_BENECONOMICOS
unidad19_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_BENECONOMICOS['TOTAL'] = unidad19_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq134_005 = get_value(unidad19_BENECONOMICOS, "Directos", 41)
aq135_010 = get_value(unidad19_BENECONOMICOS, "Aceptados en reaseguro", 41)
aq136_015 = get_value(unidad19_BENECONOMICOS, "Parte reaseguro al interior (resta)", 41)
aq137_020 = get_value(unidad19_BENECONOMICOS, "Parte reaseguro al exterior (resta)", 41)

# Calcular aq138_999 usando la fórmula especificada
aq138_999 = (aq134_005 + aq135_010 - aq136_015 - aq137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=43, value=aq138_999)


<Cell 'Hoja1'.AQ138>

##   43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 19

In [273]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' para unidad19_OPERNOASIGNABLES
unidad19_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_OPERNOASIGNABLES['TOTAL'] = unidad19_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Directos", 134),
    ("Aceptados en reaseguro", 135),
    ("Parte reaseguro al interior (resta)", 136),
    ("Parte reaseguro al exterior (resta)", 137)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as134_005 = get_value(unidad19_OPERNOASIGNABLES, "Directos", 43)
as135_010 = get_value(unidad19_OPERNOASIGNABLES, "Aceptados en reaseguro", 43)
as136_015 = get_value(unidad19_OPERNOASIGNABLES, "Parte reaseguro al interior (resta)", 43)
as137_020 = get_value(unidad19_OPERNOASIGNABLES, "Parte reaseguro al exterior (resta)", 43)

# Calcular as138_999 usando la fórmula especificada
as138_999 = (as134_005 + as135_010 - as136_015 - as137_020)

# Escribir el total general en Excel
ws.cell(row=138, column=45, value=as138_999)


<Cell 'Hoja1'.AS138>

#   UNIDAD DE CAPTURA 20   

## 26 EXEQUIAS  UNIDAD  20 

In [274]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Filtrar la fila correspondiente
posicion = POLIZAS[
    (POLIZAS['RAMOS'] == "26  EXEQUIAS") &
    (POLIZAS['AÑO'] == max_date[0]) &
    (POLIZAS['MES'] == max_date[1])
].index

# Obtener los valores de las columnas relevantes en esas posiciones
ab140_005 = POLIZAS.iloc[posicion, 2]  # Columna 3 (índice 2)
ab141_010 = POLIZAS.iloc[posicion, 3]  # Columna 4 (índice 3)
ab142_015 = POLIZAS.iloc[posicion, 4]  # Columna 5 (índice 4)
ab143_020 = POLIZAS.iloc[posicion, 5]  # Columna 6 (índice 5)

# Función auxiliar para extraer el valor numérico o devolver 0
def get_numeric_value(series):
    if series.empty:
        return 0
    return pd.to_numeric(series.values[0], errors='coerce') or 0

# Convertir todos los valores a numéricos seguros
val_140 = get_numeric_value(ab140_005)
val_141 = get_numeric_value(ab141_010)
val_142 = get_numeric_value(ab142_015)
val_143 = get_numeric_value(ab143_020)

# Escribir cada valor individual en Excel
ws.cell(row=140, column=28, value=val_140)
ws.cell(row=141, column=28, value=val_141)
ws.cell(row=142, column=28, value=val_142)
ws.cell(row=143, column=28, value=val_143)

# Calcular y escribir el total
ab144_999 = (val_140 + val_141) - (val_142 + val_143)
ws.cell(row=144, column=28, value=ab144_999)


<Cell 'Hoja1'.AB144>

##  27  ACCIDENTES PERSONALES UNIDAD 20

In [275]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función para encontrar la posición y obtener el valor convertido a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    
    if len(posicion) > 0:
        valor_str = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor_str, errors='coerce') or 0
    else:
        return 0

# Obtener valores y escribir en Excel
ac140_005 = obtener_valor("27  ACCIDENTES PERSONALES", 2)
ws.cell(row=140, column=29).value = ac140_005

ac141_010 = obtener_valor("27  ACCIDENTES PERSONALES", 3)
ws.cell(row=141, column=29).value = ac141_010

ac142_015 = obtener_valor("27  ACCIDENTES PERSONALES", 4)
ws.cell(row=142, column=29).value = ac142_015

ac143_020 = obtener_valor("27  ACCIDENTES PERSONALES", 5)
ws.cell(row=143, column=29).value = ac143_020

# Calcular el valor final
ac144_999 = ac140_005 + ac141_010 - ac142_015 - ac143_020
ws.cell(row=144, column=29).value = ac144_999


##   28  COLECTIVO VIDA UNIDAD 20 

In [276]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'Periodo' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función para encontrar la posición y obtener los valores convertidos a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Obtener los valores convertidos
ad140_005 = obtener_valor("28  COLECTIVO VIDA", 2)
ws.cell(row=140, column=30).value = ad140_005

ad141_010 = obtener_valor("28  COLECTIVO VIDA", 3)
ws.cell(row=141, column=30).value = ad141_010

ad142_015 = obtener_valor("28  COLECTIVO VIDA", 4)
ws.cell(row=142, column=30).value = ad142_015

ad143_020 = obtener_valor("28  COLECTIVO VIDA", 5)
ws.cell(row=143, column=30).value = ad143_020

# Calcular el valor final
ad144_999 = ad140_005 + ad141_010 - ad142_015 - ad143_020
ws.cell(row=144, column=30).value = ad144_999


## 29  EDUCATIVO UNIDAD 20

In [277]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función para encontrar la posición y obtener el valor como número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Obtener los valores convertidos a numéricos
ae140_005 = obtener_valor("29  EDUCATIVO", 2)
ws.cell(row=140, column=31).value = ae140_005

ae141_010 = obtener_valor("29  EDUCATIVO", 3)
ws.cell(row=141, column=31).value = ae141_010

ae142_015 = obtener_valor("29  EDUCATIVO", 4)
ws.cell(row=142, column=31).value = ae142_015

ae143_020 = obtener_valor("29  EDUCATIVO", 5)
ws.cell(row=143, column=31).value = ae143_020

# Calcular el valor final como número
ae144_999 = ae140_005 + ae141_010 - ae142_015 - ae143_020
ws.cell(row=144, column=31).value = ae144_999


##   30  VIDA GRUPO UNIDAD 20   

In [278]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función para obtener valores convertidos a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Obtener los valores como números
af140_005 = obtener_valor("30  VIDA GRUPO", 2)
ws.cell(row=140, column=32).value = af140_005

af141_010 = obtener_valor("30  VIDA GRUPO", 3)
ws.cell(row=141, column=32).value = af141_010

af142_015 = obtener_valor("30  VIDA GRUPO", 4)
ws.cell(row=142, column=32).value = af142_015

af143_020 = obtener_valor("30  VIDA GRUPO", 5)
ws.cell(row=143, column=32).value = af143_020

# Calcular y escribir total
af144_999 = af140_005 + af141_010 - af142_015 - af143_020
ws.cell(row=144, column=32).value = af144_999


##  31  SALUD UNIDAD 20 

In [279]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función segura para obtener valores numéricos
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo: "31  SALUD", columna Excel 33
col_excel = 33
ramo = "31  SALUD"

# Obtener los valores asegurando que sean numéricos
ag140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ag140_005

ag141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ag141_010

ag142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ag142_015

ag143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ag143_020

# Calcular total final
ag144_999 = ag140_005 + ag141_010 - ag142_015 - ag143_020
ws.cell(row=144, column=col_excel).value = ag144_999


## 32  ENFERMEDADES DE ALTO COSTO UNIDAD 20

In [280]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida para obtener valores numéricos
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna de Excel
ramo = "32  ENFERMEDADES DE ALTO COSTO"
col_excel = 34

# Obtener valores ya convertidos a número
ah140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ah140_005

ah141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ah141_010

ah142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ah142_015

ah143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ah143_020

# Calcular total
ah144_999 = ah140_005 + ah141_010 - ah142_015 - ah143_020
ws.cell(row=144, column=col_excel).value = ah144_999


## 33  VIDA INDIVIDUAL UNIDAD 20 

In [281]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función segura para obtener valores numéricos
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna de Excel
ramo = "33  VIDA INDIVIDUAL"
col_excel = 35

# Obtener los valores convertidos a número
ai140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ai140_005

ai141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ai141_010

ai142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ai142_015

ai143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ai143_020

# Calcular el total final
ai144_999 = ai140_005 + ai141_010 - ai142_015 - ai143_020
ws.cell(row=144, column=col_excel).value = ai144_999


## 34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 20

In [282]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida para obtener valores como números
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna destino
ramo = "34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA"
col_excel = 36

# Obtener valores
aj140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = aj140_005

aj141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = aj141_010

aj142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = aj142_015

aj143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = aj143_020

# Calcular total
aj144_999 = aj140_005 + aj141_010 - aj142_015 - aj143_020
ws.cell(row=144, column=col_excel).value = aj144_999


##  35  RIESGOS LABORALESUNIDAD 20

In [283]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida para devolver siempre un valor numérico
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Parámetros del ramo
ramo = "35  RIESGOS LABORALES"
col_excel = 37

# Obtener valores convertidos a número
ak140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ak140_005

ak141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ak141_010

ak142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ak142_015

ak143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ak143_020

# Calcular y escribir total
ak144_999 = ak140_005 + ak141_010 - ak142_015 - ak143_020
ws.cell(row=144, column=col_excel).value = ak144_999


##  36  PENSIONES LEY 100 UNIDAD 20

In [284]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida para convertir a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna
ramo = "36  PENSIONES LEY 100"
col_excel = 38

# Obtener los valores como números
al140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = al140_005

al141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = al141_010

al142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = al142_015

al143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = al143_020

# Calcular total
al144_999 = al140_005 + al141_010 - al142_015 - al143_020
ws.cell(row=144, column=col_excel).value = al144_999


##  37  PENSIONES VOLUNTARIAS UNIDAD 20

In [285]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida que convierte a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Parámetros del ramo
ramo = "37  PENSIONES VOLUNTARIAS"
col_excel = 39

# Obtener los valores como números
am140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = am140_005

am141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = am141_010

am142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = am142_015

am143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = am143_020

# Calcular total
am144_999 = am140_005 + am141_010 - am142_015 - am143_020
ws.cell(row=144, column=col_excel).value = am144_999



## 38  PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 20

In [286]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida: convierte los valores a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna
ramo = "38  PENSIONES CON CONMUTACIÓN PENSIONAL"
col_excel = 40

# Obtener los valores convertidos
an140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = an140_005

an141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = an141_010

an142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = an142_015

an143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = an143_020

# Calcular total
an144_999 = an140_005 + an141_010 - an142_015 - an143_020
ws.cell(row=144, column=col_excel).value = an144_999


## 39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 20

In [287]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida: convierte los valores a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna en Excel
ramo = "39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS"
col_excel = 41

# Obtener los valores convertidos
ao140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ao140_005

ao141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ao141_010

ao142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ao142_015

ao143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ao143_020

# Calcular total final
ao144_999 = ao140_005 + ao141_010 - ao142_015 - ao143_020
ws.cell(row=144, column=col_excel).value = ao144_999


## 40 RENTAS VOLUNTARIAS UNIDAD 20

In [288]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida que convierte a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Parámetros del ramo
ramo = "40 RENTAS VOLUNTARIAS"
col_excel = 42

# Obtener los valores convertidos
ap140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = ap140_005

ap141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = ap141_010

ap142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = ap142_015

ap143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = ap143_020

# Calcular total
ap144_999 = ap140_005 + ap141_010 - ap142_015 - ap143_020
ws.cell(row=144, column=col_excel).value = ap144_999


## 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 20 

In [289]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida: convierte los valores a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index

    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Ramo y columna de Excel
ramo = "41 BENEFICIOS ECONÓMICOS PERIÓDICOS"
col_excel = 43

# Obtener los valores convertidos
aq140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = aq140_005

aq141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = aq141_010

aq142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = aq142_015

aq143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = aq143_020

# Calcular total
aq144_999 = aq140_005 + aq141_010 - aq142_015 - aq143_020
ws.cell(row=144, column=col_excel).value = aq144_999


##  43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 20

In [290]:
# Obtener la fecha máxima y separarla en Año, Mes y Día
max_date = str(POLIZAS['PERIODO'].max()).split("-")

# Convertir la columna 'PERIODO' en Año, Mes y Día
POLIZAS[['AÑO', 'MES', 'DIA']] = POLIZAS['PERIODO'].astype(str).str.split("-", expand=True)

# Función corregida: convierte los valores a número
def obtener_valor(ramo, col):
    posicion = POLIZAS[
        (POLIZAS['RAMOS'] == ramo) &
        (POLIZAS['AÑO'] == max_date[0]) &
        (POLIZAS['MES'] == max_date[1])
    ].index
    if len(posicion) > 0:
        valor = POLIZAS.iloc[posicion[0], col]
        return pd.to_numeric(valor, errors='coerce') or 0
    else:
        return 0

# Parámetros del ramo
ramo = "43 OPERACIONES NO ASIGANBLES A RAMOS"
col_excel = 45

# Obtener valores convertidos a número
as140_005 = obtener_valor(ramo, 2)
ws.cell(row=140, column=col_excel).value = as140_005

as141_010 = obtener_valor(ramo, 3)
ws.cell(row=141, column=col_excel).value = as141_010

as142_015 = obtener_valor(ramo, 4)
ws.cell(row=142, column=col_excel).value = as142_015

as143_020 = obtener_valor(ramo, 5)
ws.cell(row=143, column=col_excel).value = as143_020

# Calcular el valor final
as144_999 = as140_005 + as141_010 - as142_015 - as143_020
ws.cell(row=144, column=col_excel).value = as144_999


# UNIDAD DE CAPTURA 21

## 26 EXEQUIAS UNIDAD 21  

In [291]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_EXEQUIAS
unidad21_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_EXEQUIAS['TOTAL'] = unidad21_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab146_005 = get_value(unidad21_EXEQUIAS, "Primas cedidas contratos automáticos interior", 26)
ab147_010 = get_value(unidad21_EXEQUIAS, "Primas cedidas contratos facultativos interior", 26)
ab148_015 = get_value(unidad21_EXEQUIAS, "Primas cedidas contratos automáticos exterior", 26)
ab149_020 = get_value(unidad21_EXEQUIAS, "Primas cedidas contratos facultativos exterior", 26)

# Calcular c150_999 usando la fórmula especificada
ab150_999 = (ab146_005 + ab147_010 - ab148_015 - ab149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=28, value=ab150_999)


<Cell 'Hoja1'.AB150>

##  27 ACCIDENTES PERSONALES UNIDAD 21

In [292]:
# Supongo que 'bd' es un DataFrame de pandas previamente cargado
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_ACCPERSONALES_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ac146_005 = abs(unidad21_ACCPERSONALES[(unidad21_ACCPERSONALES['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                       (unidad21_ACCPERSONALES['COD_RAMO'] == 27)]['TOTAL']).sum()

if ac146_005 == 0:
    ac146_005 = 0

ws.cell(row=146, column=29, value=float(ac146_005))

# Primas cedidas contratos facultativos interior
ac147_010 = abs(unidad21_ACCPERSONALES[(unidad21_ACCPERSONALES['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                       (unidad21_ACCPERSONALES['COD_RAMO'] == 27)]['TOTAL']).sum()

if ac147_010 == 0:
    ac147_010 = 0

ws.cell(row=147, column=29, value=float(ac147_010))

# Parte reaseguro al exterior
ac148_015 = abs(unidad21_ACCPERSONALES[(unidad21_ACCPERSONALES['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                       (unidad21_ACCPERSONALES['COD_RAMO'] == 27)]['TOTAL']).sum()

ac148_015 = ac148_015 if ac148_015 != 0 else 0
ws.cell(row=148, column=29, value=float(ac148_015))


# Parte reaseguro al interior con cuentas específicas
cuentas = ["6232101372", "6232101432"]
ac149_020 = abs(unidad21_ACCPERSONALES_a[(unidad21_ACCPERSONALES_a['CUENTA'].isin(cuentas)) &
                                         (unidad21_ACCPERSONALES_a['COD_RAMO'] == 27)]['TOTAL']).sum()

ac149_020 = ac149_020 if ac149_020 != 0 else 0
ws.cell(row=149, column=29, value=float(ac149_020))

# Realizar la suma de los valores individuales extraídos
ac150_999 = abs(float(ac146_005) + float(ac147_010) - float(ac148_015) - float(ac149_020))

# Escribir el resultado en la celda
ws.cell(row=150, column=29, value=ac150_999)


<Cell 'Hoja1'.AC150>

##  28 COLECTIVO VIDA UNIDAD 21

In [293]:
# Supongamos que 'bd' es el DataFrame de pandas previamente cargado
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ad146_005 = abs(unidad21_COLECVI[(unidad21_COLECVI['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                 (unidad21_COLECVI['COD_RAMO'] == 28)]['TOTAL']).sum()

ad146_005 = ad146_005 if ad146_005 != 0 else 0
ws.cell(row=146, column=30, value=float(ad146_005))

# Primas cedidas contratos facultativos interior
ad147_010 = abs(unidad21_COLECVI[(unidad21_COLECVI['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                 (unidad21_COLECVI['COD_RAMO'] == 28)]['TOTAL']).sum()

ad147_010 = ad147_010 if ad147_010 != 0 else 0
ws.cell(row=147, column=30, value=float(ad147_010))

# Parte reaseguro al interior
ad148_015 = abs(unidad21_COLECVI[(unidad21_COLECVI['CONCEPTO'] == "Parte reaseguro al interior (resta)") &
                                 (unidad21_COLECVI['COD_RAMO'] == 28)]['TOTAL']).sum()

ad148_015 = ad148_015 if ad148_015 != 0 else 0
ws.cell(row=148, column=30, value=float(ad148_015))

# Parte reaseguro al exterior
ad149_020 = abs(unidad21_COLECVI[(unidad21_COLECVI['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                 (unidad21_COLECVI['COD_RAMO'] == 28)]['TOTAL']).sum()

ad149_020 = ad149_020 if ad149_020 != 0 else 0
ws.cell(row=149, column=30, value=float(ad149_020))

# Realizar la operación para calcular ad150_999
ad150_999 = abs(ad146_005 + ad147_010 - ad148_015 - ad149_020)

# Escribir el resultado final en la celda correspondiente
ws.cell(row=150, column=30, value=float(ad150_999))


<Cell 'Hoja1'.AD150>

##  29 EDUCATIVO  UNIDAD 21

In [294]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ae146_005 = abs(unidad21_EDUCATIVO[(unidad21_EDUCATIVO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                   (unidad21_EDUCATIVO['COD_RAMO'] == 29)]['TOTAL']).sum()
ae146_005 = ae146_005 if ae146_005 != 0 else 0
ws.cell(row=146, column=31, value=float(ae146_005))

# Primas cedidas contratos facultativos interior
ae147_010 = abs(unidad21_EDUCATIVO[(unidad21_EDUCATIVO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                   (unidad21_EDUCATIVO['COD_RAMO'] == 29)]['TOTAL']).sum()
ae147_010 = ae147_010 if ae147_010 != 0 else 0
ws.cell(row=147, column=31, value=float(ae147_010))

# Parte reaseguro al interior
ae148_015 = abs(unidad21_EDUCATIVO[(unidad21_EDUCATIVO['CONCEPTO'] == "Parte reaseguro al interior (resta)") &
                                   (unidad21_EDUCATIVO['COD_RAMO'] == 29)]['TOTAL']).sum()
ae148_015 = ae148_015 if ae148_015 != 0 else 0
ws.cell(row=148, column=31, value=float(ae148_015))

# Parte reaseguro al exterior
ae149_020 = abs(unidad21_EDUCATIVO[(unidad21_EDUCATIVO['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                   (unidad21_EDUCATIVO['COD_RAMO'] == 29)]['TOTAL']).sum()
ae149_020 = ae149_020 if ae149_020 != 0 else 0
ws.cell(row=149, column=31, value=float(ae149_020))

# Calcular ae150_999
ae150_999 = abs(ae146_005 + ae147_010 - ae148_015 - ae149_020)
ws.cell(row=150, column=31, value=float(ae150_999))


<Cell 'Hoja1'.AE150>

##  30  VIDA GRUPO  UNIDAD 21

In [295]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_VIDAGRUPO_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
af146_005 = abs(unidad21_VIDAGRUPO[(unidad21_VIDAGRUPO['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                       (unidad21_VIDAGRUPO['COD_RAMO'] == 30)]['TOTAL']).sum()

if af146_005 == 0:
    af146_005 = 0

ws.cell(row=146, column=32, value=float(af146_005))

# Primas cedidas contratos facultativos interior
af147_010 = abs(unidad21_VIDAGRUPO[(unidad21_VIDAGRUPO['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                       (unidad21_VIDAGRUPO['COD_RAMO'] == 30)]['TOTAL']).sum()

if af147_010 == 0:
    af147_010 = 0

ws.cell(row=147, column=32, value=float(af147_010))

# Parte reaseguro al exterior (manteniendo abs para obtener el valor absoluto)
af148_015 = abs(unidad21_VIDAGRUPO[(unidad21_VIDAGRUPO['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                       (unidad21_VIDAGRUPO['COD_RAMO'] == 30)]['TOTAL']).sum()

af148_015 = af148_015 if af148_015 != 0 else 0
ws.cell(row=148, column=32, value=float(af148_015))


# Parte reaseguro al interior con cuentas específicas (sin abs para respetar el signo)
cuentas = ["6232101372", "6232101432"]
af149_020 = unidad21_VIDAGRUPO_a[(unidad21_VIDAGRUPO_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_VIDAGRUPO_a['COD_RAMO'] == 30)]['TOTAL'].sum()

af149_020 = af149_020 if af149_020 != 0 else 0
ws.cell(row=149, column=32, value=float(af149_020))

# Realizar la suma de los valores individuales extraídos, y aplicar abs() para evitar resultados negativos
af150_999 = abs(float(af146_005) + float(af147_010) - float(af148_015) - float(af149_020))

# Escribir el resultado en la celda
ws.cell(row=150, column=32, value=af150_999)

<Cell 'Hoja1'.AF150>

## 31 SALUD UNIDAD 21

In [296]:
# Agrupar los datos por CONCEPTO y COD_RAMO
unidad21_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_SALUD_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ag146_005 = abs(unidad21_SALUD[(unidad21_SALUD['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                       (unidad21_SALUD['COD_RAMO'] == 31)]['TOTAL']).sum()

if ag146_005 == 0:
    ag146_005 = 0

ws.cell(row=146, column=33, value=float(ag146_005))

# Primas cedidas contratos facultativos interior
ag147_010 = abs(unidad21_SALUD[(unidad21_SALUD['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                       (unidad21_SALUD['COD_RAMO'] == 31)]['TOTAL']).sum()

if ag147_010 == 0:
    ag147_010 = 0

ws.cell(row=147, column=33, value=float(ag147_010))

# Parte reaseguro al exterior (manteniendo abs para obtener el valor absoluto)
ag148_015 = abs(unidad21_SALUD[(unidad21_SALUD['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                       (unidad21_SALUD['COD_RAMO'] == 31)]['TOTAL']).sum()

ag148_015 = ag148_015 if ag148_015 != 0 else 0
ws.cell(row=148, column=33, value=float(ag148_015))

# Parte reaseguro al interior con cuentas específicas (sin abs para respetar el signo)
cuentas = ["6232101133", "6232101243", "6232101283"]
ag149_020 = unidad21_SALUD_a[(unidad21_SALUD_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_SALUD_a['COD_RAMO'] == 31)]['TOTAL'].sum()

ag149_020 = ag149_020 if ag149_020 != 0 else 0
ws.cell(row=149, column=33, value=float(ag149_020))

# Realizar la suma de los valores individuales extraídos, y aplicar abs() para evitar resultados negativos
ag150_999 = abs(float(ag146_005) + float(ag147_010) - float(ag148_015) - float(ag149_020))

# Escribir el resultado en la celda
ws.cell(row=150, column=33, value=ag150_999)

<Cell 'Hoja1'.AG150>

## 32 ENFERMEDADES DE ALTO COSTO UNIDAD 21

In [297]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_ENFALTOCOSTO
unidad21_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_ENFALTOCOSTO['TOTAL'] = unidad21_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah146_005 = get_value(unidad21_ENFALTOCOSTO, "Primas cedidas contratos automáticos interior", 32)
ah147_010 = get_value(unidad21_ENFALTOCOSTO, "Primas cedidas contratos facultativos interior", 32)
ah148_015 = get_value(unidad21_ENFALTOCOSTO, "Primas cedidas contratos automáticos exterior", 32)
ah149_020 = get_value(unidad21_ENFALTOCOSTO, "Primas cedidas contratos facultativos exterior", 32)

# Calcular c150_999 usando la fórmula especificada
ah150_999 = (ah146_005 + ah147_010 - ah148_015 - ah149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=34, value=ah150_999)

<Cell 'Hoja1'.AH150>

## 33 VIDA INDIVIDUAL UNIDAD 21

In [298]:
# Agrupar los datos por CONCEPTO y COD_RAMO unidad21_VIDAINDIVI
unidad21_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()
unidad21_VIDAINDIVI_a = bd.groupby(['CUENTA', 'COD_RAMO']).agg({'TOTAL': 'sum'}).reset_index()

# Primas cedidas contratos automáticos interior
ai146_005 = abs(unidad21_VIDAINDIVI[(unidad21_VIDAINDIVI['CONCEPTO'] == "Primas cedidas contratos automáticos interior") &
                                       (unidad21_VIDAINDIVI['COD_RAMO'] == 33)]['TOTAL']).sum()

if ai146_005 == 0:
    ai146_005 = 0

ws.cell(row=146, column=35, value=float(ai146_005))

# Primas cedidas contratos facultativos interior
ai147_010 = abs(unidad21_VIDAINDIVI[(unidad21_VIDAINDIVI['CONCEPTO'] == "Primas cedidas contratos facultativos interior") &
                                       (unidad21_VIDAINDIVI['COD_RAMO'] == 33)]['TOTAL']).sum()

if ai147_010 == 0:
    ai147_010 = 0

ws.cell(row=147, column=35, value=float(ai147_010))

# Parte reaseguro al exterior (manteniendo abs para obtener el valor absoluto)
ai148_015 = abs(unidad21_VIDAINDIVI[(unidad21_VIDAINDIVI['CONCEPTO'] == "Parte reaseguro al exterior (resta)") &
                                       (unidad21_VIDAINDIVI['COD_RAMO'] == 33)]['TOTAL']).sum()

ai148_015 = ai148_015 if ag148_015 != 0 else 0
ws.cell(row=148, column=35, value=float(ai148_015))

# Parte reaseguro al interior con cuentas específicas (sin abs para respetar el signo)
cuentas = ["6232101172", "6232101372", "6232101432"]
ai149_020 = unidad21_VIDAINDIVI_a[(unidad21_VIDAINDIVI_a['CUENTA'].isin(cuentas)) &
                                     (unidad21_VIDAINDIVI_a['COD_RAMO'] == 33)]['TOTAL'].sum()

ai149_020 = ai149_020 if ai149_020 != 0 else 0
ws.cell(row=149, column=35, value=float(ai149_020))

# Realizar la suma de los valores individuales extraídos, y aplicar abs() para evitar resultados negativos
ai150_999 = abs(float(ai146_005) + float(ai147_010) - float(ai148_015) - float(ai149_020))

# Escribir el resultado en la celda
ws.cell(row=150, column=35, value=ai150_999)

<Cell 'Hoja1'.AI150>

## 34 PREVISIONAL DE INVALIDEZ Y SOBREVIVENCIA UNIDAD 21

In [299]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_PREVIDEINVAL
unidad21_PREVIDEINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_PREVIDEINVAL['TOTAL'] = unidad21_PREVIDEINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_PREVIDEINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj146_005 = get_value(unidad21_PREVIDEINVAL, "Primas cedidas contratos automáticos interior", 34)
aj147_010 = get_value(unidad21_PREVIDEINVAL, "Primas cedidas contratos facultativos interior", 34)
aj148_015 = get_value(unidad21_PREVIDEINVAL, "Primas cedidas contratos automáticos exterior", 34)
aj149_020 = get_value(unidad21_PREVIDEINVAL, "Primas cedidas contratos facultativos exterior", 34)

# Calcular c150_999 usando la fórmula especificada
aj150_999 = (aj146_005 + aj147_010 - aj148_015 - aj149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=36, value=aj150_999)

<Cell 'Hoja1'.AJ150>

## 35  RIESGOS LABORALES UNIDAD 21

In [300]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad19_RIESGOSLAB
unidad21_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_RIESGOSLAB['TOTAL'] = unidad21_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak146_005 = get_value(unidad21_RIESGOSLAB, "Primas cedidas contratos automáticos interior", 35)
ak147_010 = get_value(unidad21_RIESGOSLAB, "Primas cedidas contratos facultativos interior", 35)
ak148_015 = get_value(unidad21_RIESGOSLAB, "Primas cedidas contratos automáticos exterior", 35)
ak149_020 = get_value(unidad21_RIESGOSLAB, "Primas cedidas contratos facultativos exterior", 35)

# Calcular c150_999 usando la fórmula especificada
ak150_999 = (ak146_005 + ak147_010 - ak148_015 - ak149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=37, value=ak150_999)

<Cell 'Hoja1'.AK150>

##  36 PENSIONES LEY 100 UNIDAD 21

In [301]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad19_PENSIONESLEY100
unidad19_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad19_PENSIONESLEY100['TOTAL'] = unidad19_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad19_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al146_005 = get_value(unidad19_PENSIONESLEY100, "Primas cedidas contratos automáticos interior", 36)
al147_010 = get_value(unidad19_PENSIONESLEY100, "Primas cedidas contratos facultativos interior", 36)
al148_015 = get_value(unidad19_PENSIONESLEY100, "Primas cedidas contratos automáticos exterior", 36)
al149_020 = get_value(unidad19_PENSIONESLEY100, "Primas cedidas contratos facultativos exterior", 36)

# Calcular c150_999 usando la fórmula especificada
al150_999 = (al146_005 + al147_010 - al148_015 - al149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=38, value=al150_999)

<Cell 'Hoja1'.AL150>

## 37 PENSIONES VOLUNTARIAS UNIDAD 21

In [302]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad19_PENSIONESLEY100
unidad21_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_PENSIONESVOLUN['TOTAL'] = unidad21_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am146_005 = get_value(unidad21_PENSIONESVOLUN, "Primas cedidas contratos automáticos interior", 37)
am147_010 = get_value(unidad21_PENSIONESVOLUN, "Primas cedidas contratos facultativos interior", 37)
am148_015 = get_value(unidad21_PENSIONESVOLUN, "Primas cedidas contratos automáticos exterior", 37)
am149_020 = get_value(unidad21_PENSIONESVOLUN, "Primas cedidas contratos facultativos exterior", 37)

# Calcular c150_999 usando la fórmula especificada
am150_999 = (am146_005 + am147_010 - am148_015 - am149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=39, value=am150_999)

<Cell 'Hoja1'.AM150>

## 38 PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 21

In [303]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_PENSIONESCONMU
unidad21_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_PENSIONESCONMU['TOTAL'] = unidad21_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an146_005 = get_value(unidad21_PENSIONESCONMU, "Primas cedidas contratos automáticos interior", 38)
an147_010 = get_value(unidad21_PENSIONESCONMU, "Primas cedidas contratos facultativos interior", 38)
an148_015 = get_value(unidad21_PENSIONESCONMU, "Primas cedidas contratos automáticos exterior", 38)
an149_020 = get_value(unidad21_PENSIONESCONMU, "Primas cedidas contratos facultativos exterior", 38)

# Calcular c150_999 usando la fórmula especificada
an150_999 = (an146_005 + an147_010 - an148_015 - an149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=40, value=an150_999)

<Cell 'Hoja1'.AN150>

##   39 PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 21

In [304]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_PATRIAUTONOMO
unidad21_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_PATRIAUTONOMO['TOTAL'] = unidad21_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao146_005 = get_value(unidad21_PATRIAUTONOMO, "Primas cedidas contratos automáticos interior", 39)
ao147_010 = get_value(unidad21_PATRIAUTONOMO, "Primas cedidas contratos facultativos interior", 39)
ao148_015 = get_value(unidad21_PATRIAUTONOMO, "Primas cedidas contratos automáticos exterior", 39)
ao149_020 = get_value(unidad21_PATRIAUTONOMO, "Primas cedidas contratos facultativos exterior", 39)

# Calcular c150_999 usando la fórmula especificada
ao150_999 = (ao146_005 + ao147_010 - ao148_015 - ao149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=41, value=ao150_999)

<Cell 'Hoja1'.AO150>

## 40 RENTAS VOLUNTARIAS UNIDAD 21

In [305]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_RENTASVOLUN
unidad21_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_RENTASVOLUN['TOTAL'] = unidad21_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap146_005 = get_value(unidad21_RENTASVOLUN, "Primas cedidas contratos automáticos interior", 40)
ap147_010 = get_value(unidad21_RENTASVOLUN, "Primas cedidas contratos facultativos interior", 40)
ap148_015 = get_value(unidad21_RENTASVOLUN, "Primas cedidas contratos automáticos exterior", 40)
ap149_020 = get_value(unidad21_RENTASVOLUN, "Primas cedidas contratos facultativos exterior", 40)

# Calcular c150_999 usando la fórmula especificada
ap150_999 = (ap146_005 + ap147_010 - ap148_015 - ap149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=42, value=ap150_999)

<Cell 'Hoja1'.AP150>

##  41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 21

In [306]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_BENECONOMICOS
unidad21_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_BENECONOMICOS['TOTAL'] = unidad21_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq146_005 = get_value(unidad21_BENECONOMICOS, "Primas cedidas contratos automáticos interior", 41)
aq147_010 = get_value(unidad21_BENECONOMICOS, "Primas cedidas contratos facultativos interior", 41)
aq148_015 = get_value(unidad21_BENECONOMICOS, "Primas cedidas contratos automáticos exterior", 41)
aq149_020 = get_value(unidad21_BENECONOMICOS, "Primas cedidas contratos facultativos exterior", 41)

# Calcular c150_999 usando la fórmula especificada
aq150_999 = (aq146_005 + aq147_010 - aq148_015 - aq149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=43, value=aq150_999)

<Cell 'Hoja1'.AQ150>

## 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 21

In [307]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' en unidad21_OPERNOASIGNABLES
unidad21_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad21_OPERNOASIGNABLES['TOTAL'] = unidad21_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas cedidas contratos automáticos interior", 146),
    ("Primas cedidas contratos facultativos interior", 147),
    ("Primas cedidas contratos automáticos exterior", 148),
    ("Primas cedidas contratos facultativos exterior", 149)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad21_OPERNOASIGNABLES, concept, 42, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as146_005 = get_value(unidad21_OPERNOASIGNABLES, "Primas cedidas contratos automáticos interior", 42)
as147_010 = get_value(unidad21_OPERNOASIGNABLES, "Primas cedidas contratos facultativos interior", 42)
as148_015 = get_value(unidad21_OPERNOASIGNABLES, "Primas cedidas contratos automáticos exterior", 42)
as149_020 = get_value(unidad21_OPERNOASIGNABLES, "Primas cedidas contratos facultativos exterior", 42)

# Calcular c150_999 usando la fórmula especificada
as150_999 = (as146_005 + as147_010 - as148_015 - as149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=45, value=as150_999)

<Cell 'Hoja1'.AS150>

#  1   TOTAL UNIDAD 21

In [308]:
# Sumar los valores de todas las columnas con datos en cada fila
c146_005 = sum([ws.cell(row=146, column=col).value if ws.cell(row=146, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=146, column=3).value = c146_005  

c147_010 = sum([ws.cell(row=147, column=col).value if ws.cell(row=147, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=147, column=3).value = c147_010 

c148_015 = sum([ws.cell(row=148, column=col).value if ws.cell(row=148, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=148, column=3).value = c148_015 

c149_020 = sum([ws.cell(row=149, column=col).value if ws.cell(row=149, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=149, column=3).value = c149_020  

# Calcular el total general según la fórmula dada
c150_999 = abs(c146_005 + c147_010 - c148_015 - c149_020)

# Escribir el total general en Excel
ws.cell(row=150, column=3).value = c150_999


#  2   SUBTOTAL - RAMOS UNIDAD 21

In [309]:
# Sumar los valores de las columnas E a AS para cada fila específica y almacenar el resultado en la columna D
d146_005 = sum([ws.cell(row=146, column=col).value if ws.cell(row=146, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=146, column=4).value = d146_005 

d147_010 = sum([ws.cell(row=147, column=col).value if ws.cell(row=147, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=147, column=4).value = d147_010 

d148_015 = sum([ws.cell(row=148, column=col).value if ws.cell(row=148, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=148, column=4).value = d148_015 

d149_020 = sum([ws.cell(row=149, column=col).value if ws.cell(row=149, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=149, column=4).value = d149_020 

# Calcular el total general para la fila 150 según la fórmula dada
d150_999 = abs(d146_005 + d147_010 - d148_015 - d149_020)

# Escribir el total general en la fila 150, columna D
ws.cell(row=150, column=4).value = d150_999 


## UNIDAD DE CAPTURA 23

#  26  EXEQUIAS UNIDAD 23

In [310]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_EXEQUIAS['TOTAL'] = unidad23_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab154_005 = get_value(unidad23_EXEQUIAS, "Primas emitidas ramo de terremoto", 26)
ab155_010 = get_value(unidad23_EXEQUIAS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 26)
ab156_015 = get_value(unidad23_EXEQUIAS, "Primas cedidas interior y exterior terremoto", 26)
ab157_020 = get_value(unidad23_EXEQUIAS, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 26)

# Calcular c150_999 usando la fórmula especificada
ab158_999 = (ab154_005 + ab155_010 - ab156_015 - ab157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=28, value=ab158_999)

<Cell 'Hoja1'.AB158>

#  27  ACCIDENTES PERSONALES UNIDAD 23

In [311]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_ACCIDENTESPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_ACCIDENTESPERSONALES['TOTAL'] = unidad23_ACCIDENTESPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_ACCIDENTESPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac154_005 = get_value(unidad23_ACCIDENTESPERSONALES, "Primas emitidas ramo de terremoto", 27)
ac155_010 = get_value(unidad23_ACCIDENTESPERSONALES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 27)
ac156_015 = get_value(unidad23_ACCIDENTESPERSONALES, "Primas cedidas interior y exterior terremoto", 27)
ac157_020 = get_value(unidad23_ACCIDENTESPERSONALES, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 27)

# Calcular c150_999 usando la fórmula especificada
ac158_999 = (ac154_005 + ac155_010 - ac156_015 - ac157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=29, value=ac158_999)

<Cell 'Hoja1'.AC158>

#  28  COLECTIVO VIDA UNIDAD 23

In [312]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_COLECTIVOVIDA = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_COLECTIVOVIDA['TOTAL'] = unidad23_COLECTIVOVIDA['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_COLECTIVOVIDA, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad154_005 = get_value(unidad23_COLECTIVOVIDA, "Primas emitidas ramo de terremoto", 28)
ad155_010 = get_value(unidad23_COLECTIVOVIDA, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 28)
ad156_015 = get_value(unidad23_COLECTIVOVIDA, "Primas cedidas interior y exterior terremoto", 28)
ad157_020 = get_value(unidad23_COLECTIVOVIDA, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 28)

# Calcular c150_999 usando la fórmula especificada
ad158_999 = (ad154_005 + ad155_010 - ad156_015 - ad157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=30, value=ad158_999)

<Cell 'Hoja1'.AD158>

#  29  EDUCATIVO UNIDAD 23

In [313]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_EDUCATIVO['TOTAL'] = unidad23_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae154_005 = get_value(unidad23_EDUCATIVO, "Primas emitidas ramo de terremoto", 29)
ae155_010 = get_value(unidad23_EDUCATIVO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 29)
ae156_015 = get_value(unidad23_EDUCATIVO, "Primas cedidas interior y exterior terremoto", 29)
ae157_020 = get_value(unidad23_EDUCATIVO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 29)

# Calcular c150_999 usando la fórmula especificada
ae158_999 = (ae154_005 + ae155_010 - ae156_015 - ae157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=31, value=ae158_999)

<Cell 'Hoja1'.AE158>

#  30  VIDA GRUPO UNIDAD 23

In [314]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_VIDAGRUPO['TOTAL'] = unidad23_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af154_005 = get_value(unidad23_VIDAGRUPO, "Primas emitidas ramo de terremoto", 30)
af155_010 = get_value(unidad23_VIDAGRUPO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 30)
af156_015 = get_value(unidad23_VIDAGRUPO, "Primas cedidas interior y exterior terremoto", 30)
af157_020 = get_value(unidad23_VIDAGRUPO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 30)

# Calcular c150_999 usando la fórmula especificada
af158_999 = (af154_005 + af155_010 - af156_015 - af157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=32, value=af158_999)

<Cell 'Hoja1'.AF158>

#  31  SALUD UNIDAD 23

In [315]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad23_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_SALUD['TOTAL'] = unidad23_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag154_005 = get_value(unidad23_SALUD, "Primas emitidas ramo de terremoto", 31)
ag155_010 = get_value(unidad23_SALUD, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 31)
ag156_015 = get_value(unidad23_SALUD, "Primas cedidas interior y exterior terremoto", 31)
ag157_020 = get_value(unidad23_SALUD, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 31)

# Calcular c150_999 usando la fórmula especificada
ag158_999 = (ag154_005 + ag155_010 - ag156_015 - ag157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=33, value=ag158_999)

<Cell 'Hoja1'.AG158>

#  32  ENFERMEDADES DE ALTO COSTO UNIDAD 23

In [316]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_ENFALTOCOSTO['TOTAL'] = unidad23_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah154_005 = get_value(unidad23_ENFALTOCOSTO, "Primas emitidas ramo de terremoto", 32)
ah155_010 = get_value(unidad23_ENFALTOCOSTO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 32)
ah156_015 = get_value(unidad23_ENFALTOCOSTO, "Primas cedidas interior y exterior terremoto", 32)
ah157_020 = get_value(unidad23_ENFALTOCOSTO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 32)

# Calcular c150_999 usando la fórmula especificada
ah158_999 = (ah154_005 + ah155_010 - ah156_015 - ah157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=34, value=ah158_999)

<Cell 'Hoja1'.AH158>

#  33  VIDA INDIVIDUAL UNIDAD 23

In [317]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_VIDAINDIVI['TOTAL'] = unidad23_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai154_005 = get_value(unidad23_VIDAINDIVI, "Primas emitidas ramo de terremoto", 33)
ai155_010 = get_value(unidad23_VIDAINDIVI, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 33)
ai156_015 = get_value(unidad23_VIDAINDIVI, "Primas cedidas interior y exterior terremoto", 33)
ai157_020 = get_value(unidad23_VIDAINDIVI, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 33)

# Calcular c150_999 usando la fórmula especificada
ai158_999 = (ai154_005 + ai155_010 - ai156_015 - ai157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=35, value=ai158_999)

<Cell 'Hoja1'.AI158>

#  34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 23

In [318]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_PREVIDEINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_PREVIDEINVAL['TOTAL'] = unidad23_PREVIDEINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_PREVIDEINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj154_005 = get_value(unidad23_PREVIDEINVAL, "Primas emitidas ramo de terremoto", 34)
aj155_010 = get_value(unidad23_PREVIDEINVAL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 34)
aj156_015 = get_value(unidad23_PREVIDEINVAL, "Primas cedidas interior y exterior terremoto", 34)
aj157_020 = get_value(unidad23_PREVIDEINVAL, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 34)

# Calcular c150_999 usando la fórmula especificada
aj158_999 = (aj154_005 + aj155_010 - aj156_015 - aj157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=36, value=aj158_999)

<Cell 'Hoja1'.AJ158>

#  35  RIESGOS LABORALES UNIDAD 23

In [319]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_RIESGOSLAB['TOTAL'] = unidad23_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak154_005 = get_value(unidad23_RIESGOSLAB, "Primas emitidas ramo de terremoto", 35)
ak155_010 = get_value(unidad23_RIESGOSLAB, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 35)
ak156_015 = get_value(unidad23_RIESGOSLAB, "Primas cedidas interior y exterior terremoto", 35)
ak157_020 = get_value(unidad23_RIESGOSLAB, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 35)

# Calcular c150_999 usando la fórmula especificada
ak158_999 = (ak154_005 + ak155_010 - ak156_015 - ak157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=37, value=ak158_999)

<Cell 'Hoja1'.AK158>

#  36  PENSIONES LEY 100 UNIDAD 23

In [320]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_PENSIONESLEY100['TOTAL'] = unidad23_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al154_005 = get_value(unidad23_PENSIONESLEY100, "Primas emitidas ramo de terremoto", 36)
al155_010 = get_value(unidad23_PENSIONESLEY100, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 36)
al156_015 = get_value(unidad23_PENSIONESLEY100, "Primas cedidas interior y exterior terremoto", 36)
al157_020 = get_value(unidad23_PENSIONESLEY100, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 36)

# Calcular c150_999 usando la fórmula especificada
al158_999 = (al154_005 + al155_010 - al156_015 - al157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=38, value=al158_999)

<Cell 'Hoja1'.AL158>

#  37  PENSIONES VOLUNTARIAS UNIDAD 23

In [321]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_PENSIONESVOLUN['TOTAL'] = unidad23_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am154_005 = get_value(unidad23_PENSIONESVOLUN, "Primas emitidas ramo de terremoto", 37)
am155_010 = get_value(unidad23_PENSIONESVOLUN, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 37)
am156_015 = get_value(unidad23_PENSIONESVOLUN, "Primas cedidas interior y exterior terremoto", 37)
am157_020 = get_value(unidad23_PENSIONESVOLUN, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 37)

# Calcular c150_999 usando la fórmula especificada
am158_999 = (am154_005 + am155_010 - am156_015 - am157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=39, value=am158_999)

<Cell 'Hoja1'.AM158>

#  38  PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 23

In [322]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_PENSIONESCONMU['TOTAL'] = unidad23_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an154_005 = get_value(unidad23_PENSIONESCONMU, "Primas emitidas ramo de terremoto", 38)
an155_010 = get_value(unidad23_PENSIONESCONMU, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 38)
an156_015 = get_value(unidad23_PENSIONESCONMU, "Primas cedidas interior y exterior terremoto", 38)
an157_020 = get_value(unidad23_PENSIONESCONMU, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 38)

# Calcular c150_999 usando la fórmula especificada
an158_999 = (an154_005 + an155_010 - an156_015 - an157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=40, value=an158_999)

<Cell 'Hoja1'.AN158>

#  39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 23

In [323]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_PATRIAUTONOMO['TOTAL'] = unidad23_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao154_005 = get_value(unidad23_PATRIAUTONOMO, "Primas emitidas ramo de terremoto", 39)
ao155_010 = get_value(unidad23_PATRIAUTONOMO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 39)
ao156_015 = get_value(unidad23_PATRIAUTONOMO, "Primas cedidas interior y exterior terremoto", 39)
ao157_020 = get_value(unidad23_PATRIAUTONOMO, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 39)

# Calcular c150_999 usando la fórmula especificada
ao158_999 = (ao154_005 + ao155_010 - ao156_015 - ao157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=41, value=ao158_999)

<Cell 'Hoja1'.AO158>

# 40 RENTAS VOLUNTARIAS UNIDAD 23

In [324]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_RENTASVOLUN['TOTAL'] = unidad23_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap154_005 = get_value(unidad23_RENTASVOLUN, "Primas emitidas ramo de terremoto", 40)
ap155_010 = get_value(unidad23_RENTASVOLUN, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 40)
ap156_015 = get_value(unidad23_RENTASVOLUN, "Primas cedidas interior y exterior terremoto", 40)
ap157_020 = get_value(unidad23_RENTASVOLUN, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 40)

# Calcular c150_999 usando la fórmula especificada
ap158_999 = (ap154_005 + ap155_010 - ap156_015 - ap157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=42, value=ap158_999)

<Cell 'Hoja1'.AP158>

# 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 23

In [325]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_BENECONOMICOS['TOTAL'] = unidad23_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq154_005 = get_value(unidad23_BENECONOMICOS, "Primas emitidas ramo de terremoto", 41)
aq155_010 = get_value(unidad23_BENECONOMICOS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 41)
aq156_015 = get_value(unidad23_BENECONOMICOS, "Primas cedidas interior y exterior terremoto", 41)
aq157_020 = get_value(unidad23_BENECONOMICOS, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 41)

# Calcular c150_999 usando la fórmula especificada
aq158_999 = (aq154_005 + aq155_010 - aq156_015 - aq157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=43, value=aq158_999)

<Cell 'Hoja1'.AQ158>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 23

In [326]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO' 
unidad23_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad23_OPERNOASIGNABLES['TOTAL'] = unidad23_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 154),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 155),
    ("Primas cedidas interior y exterior terremoto", 156),
    ("Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 157)
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad23_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as154_005 = get_value(unidad23_OPERNOASIGNABLES, "Primas emitidas ramo de terremoto", 43)
as155_010 = get_value(unidad23_OPERNOASIGNABLES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 43)
as156_015 = get_value(unidad23_OPERNOASIGNABLES, "Primas cedidas interior y exterior terremoto", 43)
as157_020 = get_value(unidad23_OPERNOASIGNABLES, "Primas cedidas interior y exterior (coberturas) de terremoto incluidas en otros ramos", 43)

# Calcular c150_999 usando la fórmula especificada
as158_999 = (as154_005 + as155_010 - as156_015 - as157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=45, value=as158_999)

<Cell 'Hoja1'.AS158>

#  1   TOTAL UNIDAD 23

In [327]:
c154_005 = sum([ws.cell(row=154, column=col).value if ws.cell(row=154, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=154, column=3).value = c154_005  

c155_010 = sum([ws.cell(row=155, column=col).value if ws.cell(row=155, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=155, column=3).value = c155_010 

c156_015 = sum([ws.cell(row=156, column=col).value if ws.cell(row=156, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=156, column=3).value = c156_015 

c157_020 = sum([ws.cell(row=157, column=col).value if ws.cell(row=157, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=157, column=3).value = c157_020  

# Calcular el total general según la fórmula dada

c158_999 = abs(c154_005 + c155_010 - c156_015 - c157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=3).value = c158_999

#  2   SUBTOTAL - RAMOS UNIDAD 23

In [328]:
d154_005 = sum([ws.cell(row=154, column=col).value if ws.cell(row=154, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=154, column=4).value = d154_005  

d155_010 = sum([ws.cell(row=155, column=col).value if ws.cell(row=155, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=155, column=4).value = d155_010 

d156_015 = sum([ws.cell(row=156, column=col).value if ws.cell(row=156, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=156, column=4).value = d156_015 

d157_020 = sum([ws.cell(row=157, column=col).value if ws.cell(row=157, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=157, column=4).value = d157_020  

# Calcular el total general según la fórmula dada

d158_999 = abs(d154_005 + d155_010 - d156_015 - d157_020)

# Escribir el total general en Excel
ws.cell(row=158, column=4).value = d158_999

## UNIDAD DE CAPTURA 24

#  26  EXEQUIAS UNIDAD 24

In [329]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_EXEQUIAS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_EXEQUIAS['TOTAL'] = unidad24_EXEQUIAS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_EXEQUIAS, concept, 26, 28, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ab160_005 = get_value(unidad24_EXEQUIAS, "Primas emitidas ramo de terremoto", 26)
ab161_010 = get_value(unidad24_EXEQUIAS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 26)


# Calcular c150_999 usando la fórmula especificada
ab162_999 = (ab160_005 + ab161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=28, value=ab162_999)

<Cell 'Hoja1'.AB162>

#  27  ACCIDENTES PERSONALES UNIDAD 24

In [330]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_ACCPERSONALES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_ACCPERSONALES['TOTAL'] = unidad24_ACCPERSONALES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_ACCPERSONALES, concept, 27, 29, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ac160_005 = get_value(unidad24_ACCPERSONALES, "Primas emitidas ramo de terremoto", 27)
ac161_010 = get_value(unidad24_ACCPERSONALES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 27)


# Calcular c150_999 usando la fórmula especificada
ac162_999 = (ac160_005 + ac161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=29, value=ac162_999)

<Cell 'Hoja1'.AC162>

#  28  COLECTIVO VIDA UNIDAD 24

In [331]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_COLECVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_COLECVI['TOTAL'] = unidad24_COLECVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_COLECVI, concept, 28, 30, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ad160_005 = get_value(unidad24_COLECVI, "Primas emitidas ramo de terremoto", 28)
ad161_010 = get_value(unidad24_COLECVI, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 28)


# Calcular c150_999 usando la fórmula especificada
ad162_999 = (ad160_005 + ad161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=30, value=ad162_999)

<Cell 'Hoja1'.AD162>

#  29  EDUCATIVO UNIDAD 24

In [332]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_EDUCATIVO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_EDUCATIVO['TOTAL'] = unidad24_EDUCATIVO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_EDUCATIVO, concept, 29, 31, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ae160_005 = get_value(unidad24_COLECVI, "Primas emitidas ramo de terremoto", 29)
ae161_010 = get_value(unidad24_COLECVI, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 29)


# Calcular c150_999 usando la fórmula especificada
ae162_999 = (ae160_005 + ae161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=31, value=ae162_999)

<Cell 'Hoja1'.AE162>

#  30  VIDA GRUPO UNIDAD 24

In [333]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_VIDAGRUPO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_VIDAGRUPO['TOTAL'] = unidad24_VIDAGRUPO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_VIDAGRUPO, concept, 30, 32, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
af160_005 = get_value(unidad24_VIDAGRUPO, "Primas emitidas ramo de terremoto", 30)
af161_010 = get_value(unidad24_VIDAGRUPO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 30)


# Calcular c150_999 usando la fórmula especificada
af162_999 = (af160_005 + af161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=32, value=af162_999)

<Cell 'Hoja1'.AF162>

#  31  SALUD UNIDAD 24

In [334]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_SALUD = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_SALUD['TOTAL'] = unidad24_SALUD['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_SALUD, concept, 31, 33, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ag160_005 = get_value(unidad24_SALUD, "Primas emitidas ramo de terremoto", 31)
ag161_010 = get_value(unidad24_SALUD, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 31)


# Calcular c150_999 usando la fórmula especificada
ag162_999 = (ag160_005 + ag161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=33, value=ag162_999)

<Cell 'Hoja1'.AG162>

#  32  ENFERMEDADES DE ALTO COSTO UNIDAD 24

In [335]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_ENFALTOCOSTO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_ENFALTOCOSTO['TOTAL'] = unidad24_ENFALTOCOSTO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_ENFALTOCOSTO, concept, 32, 34, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ah160_005 = get_value(unidad24_ENFALTOCOSTO, "Primas emitidas ramo de terremoto", 32)
ah161_010 = get_value(unidad24_ENFALTOCOSTO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 32)


# Calcular c150_999 usando la fórmula especificada
ah162_999 = (ah160_005 + ah161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=34, value=ah162_999)

<Cell 'Hoja1'.AH162>

#  33  VIDA INDIVIDUAL UNIDAD 24

In [336]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_VIDAINDIVI = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_VIDAINDIVI['TOTAL'] = unidad24_VIDAINDIVI['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_VIDAINDIVI, concept, 33, 35, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ai160_005 = get_value(unidad24_VIDAINDIVI, "Primas emitidas ramo de terremoto", 33)
ai161_010 = get_value(unidad24_VIDAINDIVI, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 33)


# Calcular c150_999 usando la fórmula especificada
ai162_999 = (ai160_005 + ai161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=35, value=ai162_999)

<Cell 'Hoja1'.AI162>

#  34  PREVISIONAL DE  INVALIDEZ.Y SOBREVIVENCIA UNIDAD 24

In [337]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_PREVIDEINVAL = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_PREVIDEINVAL['TOTAL'] = unidad24_PREVIDEINVAL['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_PREVIDEINVAL, concept, 34, 36, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aj160_005 = get_value(unidad24_PREVIDEINVAL, "Primas emitidas ramo de terremoto", 34)
aj161_010 = get_value(unidad24_PREVIDEINVAL, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 34)


# Calcular c150_999 usando la fórmula especificada
aj162_999 = (aj160_005 + aj161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=36, value=aj162_999)

<Cell 'Hoja1'.AJ162>

#  35  RIESGOS LABORALES UNIDAD 24

In [338]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_RIESGOSLAB = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_RIESGOSLAB['TOTAL'] = unidad24_RIESGOSLAB['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_RIESGOSLAB, concept, 35, 37, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ak160_005 = get_value(unidad24_RIESGOSLAB, "Primas emitidas ramo de terremoto", 35)
ak161_010 = get_value(unidad24_RIESGOSLAB, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 35)


# Calcular c150_999 usando la fórmula especificada
ak162_999 = (ak160_005 + ak161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=37, value=ak162_999)

<Cell 'Hoja1'.AK162>

#  36  PENSIONES LEY 100 UNIDAD 24

In [339]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_PENSIONESLEY100 = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_PENSIONESLEY100['TOTAL'] = unidad24_PENSIONESLEY100['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_PENSIONESLEY100, concept, 36, 38, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
al160_005 = get_value(unidad24_PENSIONESLEY100, "Primas emitidas ramo de terremoto", 36)
al161_010 = get_value(unidad24_PENSIONESLEY100, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 36)


# Calcular c150_999 usando la fórmula especificada
al162_999 = (al160_005 + al161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=38, value=al162_999)

<Cell 'Hoja1'.AL162>

#  37  PENSIONES VOLUNTARIAS UNIDAD 24

In [340]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_PENSIONESVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_PENSIONESVOLUN['TOTAL'] = unidad24_PENSIONESVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_PENSIONESVOLUN, concept, 37, 39, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
am160_005 = get_value(unidad24_PENSIONESVOLUN, "Primas emitidas ramo de terremoto", 37)
am161_010 = get_value(unidad24_PENSIONESVOLUN, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 37)


# Calcular c150_999 usando la fórmula especificada
am162_999 = (am160_005 + am161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=39, value=am162_999)

<Cell 'Hoja1'.AM162>

#  38  PENSIONES CON CONMUTACIÓN PENSIONAL UNIDAD 24

In [341]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_PENSIONESCONMU = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_PENSIONESCONMU['TOTAL'] = unidad24_PENSIONESCONMU['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_PENSIONESCONMU, concept, 38, 40, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
an160_005 = get_value(unidad24_PENSIONESCONMU, "Primas emitidas ramo de terremoto", 38)
an161_010 = get_value(unidad24_PENSIONESCONMU, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 38)


# Calcular c150_999 usando la fórmula especificada
an162_999 = (an160_005 + an161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=40, value=an162_999)

<Cell 'Hoja1'.AN162>

#  39  PATRIM. AUTÓNOMO - FDO. PENS. VOLUNTARIAS UNIDAD 24

In [342]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_PATRIAUTONOMO = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_PATRIAUTONOMO['TOTAL'] = unidad24_PATRIAUTONOMO['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_PATRIAUTONOMO, concept, 39, 41, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ao160_005 = get_value(unidad24_PATRIAUTONOMO, "Primas emitidas ramo de terremoto", 39)
ao161_010 = get_value(unidad24_PATRIAUTONOMO, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 39)


# Calcular c150_999 usando la fórmula especificada
ao162_999 = (ao160_005 + ao161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=41, value=ao162_999)

<Cell 'Hoja1'.AO162>

# 40 RENTAS VOLUNTARIAS UNIDAD 24

In [343]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_RENTASVOLUN = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_RENTASVOLUN['TOTAL'] = unidad24_RENTASVOLUN['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_RENTASVOLUN, concept, 40, 42, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
ap160_005 = get_value(unidad24_RENTASVOLUN, "Primas emitidas ramo de terremoto", 40)
ap161_010 = get_value(unidad24_RENTASVOLUN, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 40)


# Calcular c150_999 usando la fórmula especificada
ap162_999 = (ap160_005 + ap161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=42, value=ap162_999)

<Cell 'Hoja1'.AP162>

# 41 BENEFICIOS ECONÓMICOS PERIÓDICOS UNIDAD 24

In [344]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_BENECONOMICOS = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_BENECONOMICOS['TOTAL'] = unidad24_BENECONOMICOS['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_BENECONOMICOS, concept, 41, 43, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
aq160_005 = get_value(unidad24_BENECONOMICOS, "Primas emitidas ramo de terremoto", 41)
aq161_010 = get_value(unidad24_BENECONOMICOS, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 41)


# Calcular c150_999 usando la fórmula especificada
aq162_999 = (aq160_005 + aq161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=43, value=aq162_999)

<Cell 'Hoja1'.AQ162>

# 43 OPERACIONES NO ASIGANBLES A RAMOS UNIDAD 24

In [345]:
# Agrupar los datos por 'CONCEPTO' y 'COD_RAMO'
unidad24_OPERNOASIGNABLES = bd.groupby(['CONCEPTO', 'COD_RAMO'])['TOTAL'].sum().reset_index()
unidad24_OPERNOASIGNABLES['TOTAL'] = unidad24_OPERNOASIGNABLES['TOTAL'].abs()  # Aplicar valor absoluto a la columna 'TOTAL'

# Función para escribir en Excel
def write_to_excel(data_frame, concept, cod_ramo, start_col, start_row):
    row = data_frame[(data_frame['CONCEPTO'] == concept) & (data_frame['COD_RAMO'] == cod_ramo)]
    total = row['TOTAL'].iloc[0] if not row.empty else 0
    ws.cell(row=start_row, column=start_col, value=total)

# Conceptos y correspondientes filas en Excel para escritura
concepts_info = [
    ("Primas emitidas ramo de terremoto", 160),
    ("Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 161),
]

# Escribir cada valor en la hoja de Excel
for concept, row_num in concepts_info:
    write_to_excel(unidad24_OPERNOASIGNABLES, concept, 43, 45, row_num)

# Definir la función para obtener el valor de un concepto específico
def get_value(df, concept, cod_ramo):
    row = df[(df['CONCEPTO'] == concept) & (df['COD_RAMO'] == cod_ramo)]
    return abs(row['TOTAL'].iloc[0]) if not row.empty else 0

# Obtener los valores individuales de cada concepto
as160_005 = get_value(unidad24_OPERNOASIGNABLES, "Primas emitidas ramo de terremoto", 43)
as161_010 = get_value(unidad24_OPERNOASIGNABLES, "Primas emitidas (coberturas) de terremoto incluidas en otros ramos", 43)


# Calcular c150_999 usando la fórmula especificada
as162_999 = (as160_005 + as161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=45, value=as162_999)

<Cell 'Hoja1'.AS162>

#  1   TOTAL UNIDAD 24

In [346]:
c160_005 = sum([ws.cell(row=160, column=col).value if ws.cell(row=160, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=160, column=3).value = c160_005  

c161_010 = sum([ws.cell(row=161, column=col).value if ws.cell(row=161, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=161, column=3).value = c161_010 

# Calcular el total general según la fórmula dada

c162_999 = abs(c160_005 + c161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=3).value = c162_999

#  2   SUBTOTAL - RAMOS UNIDAD 24

In [347]:
d160_005 = sum([ws.cell(row=160, column=col).value if ws.cell(row=160, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=160, column=4).value = d160_005  

d161_010 = sum([ws.cell(row=161, column=col).value if ws.cell(row=161, column=col).value is not None else 0 for col in range(5, 46)])
ws.cell(row=161, column=4).value = d161_010 

# Calcular el total general según la fórmula dada

d162_999 = abs(d160_005 + d161_010)

# Escribir el total general en Excel
ws.cell(row=162, column=4).value = d162_999

# IMPRIMIR FORMATO 290

In [348]:
wb.save('Bases/Formato_290_COVI.xlsx')
wb.close()

# PROFORMA TXT

In [349]:
# Función para formatear una línea con valores dados
def formatear_linea(numero_registro, codigo_cuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)
    tipo_registro = "3"
    codigo_cuenta_str = str(int(codigo_cuenta)).zfill(6)
    valor_str = f"{int(valor):020d}"
    return f"{num_registro_str}{tipo_registro}{codigo_cuenta_str}{signo}{valor_str}"

# Función específica para la parte del Formato 235
def formatear_linea_235(numero_registro, columna_idx, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # 5 dígitos consecutivos
    tipo_registro = "4"  # Siempre '4'
    formato_str = "235"  # Formato 235
    columna_str = str(columna_idx).zfill(2)  # 2 dígitos para columna
    unidad_captura_str = str(unidad_captura).zfill(2)  # 2 dígitos para unidad de captura
    subcuenta_str = str(subcuenta).zfill(3)  # 3 dígitos para subcuenta
    valor_str = f"{int(valor):020d}"  # 20 caracteres con ceros
    return f"{num_registro_str}{tipo_registro}{formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Obtener la fecha (mes vencido)
fecha_actual = datetime.now()
fecha_mes_vencido = fecha_actual.replace(day=1) - timedelta(days=1)
fecha_str = fecha_mes_vencido.strftime("%d%m%Y")

# Encabezados iniciales
registro_inicial = f"00001114000001{fecha_str}01199SVIDCOLSECS0100"
segunda_linea = "000022000000000000"

# Leer la base de datos principal

bd = pd.read_excel(r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COVI.xlsx",
    sheet_name="BALANCE_WP",usecols="A:E",skiprows=5,dtype=str)
bd.columns = ["ALTERNATIVA 6", "NOMBRE", "VALOR", "VALOR2", "VALOR MONEDA EXTRANJERA"]
bd = bd.dropna(subset=['ALTERNATIVA 6'])

# Convertir las columnas numéricas
bd['VALOR'] = pd.to_numeric(bd['VALOR'], errors='coerce').fillna(0)
bd['VALOR2'] = pd.to_numeric(bd['VALOR2'], errors='coerce').fillna(0)
bd['VALOR MONEDA EXTRANJERA'] = pd.to_numeric(bd['VALOR MONEDA EXTRANJERA'], errors='coerce').fillna(0)

# Lista de datos a escribir
txt_datos = [registro_inicial, segunda_linea]
numero_registro = 3

# Primera y segunda partes
for idx, columna in enumerate(['VALOR', 'VALOR2'], start=1):
    for _, row in bd.iterrows():
        codigo_cuenta = row['ALTERNATIVA 6']
        valor = row[columna]
        if valor == 0:
            continue
        signo = '+' if valor >= 0 else '-'
        linea = formatear_linea(numero_registro, codigo_cuenta, signo, abs(valor))
        txt_datos.append(linea)
        numero_registro += 1
    txt_datos.append(f"{str(numero_registro).zfill(5)}200000{idx}000000")
    numero_registro += 1

# Tercera parte
cuentas_vistas = set()
for _, row in bd.iterrows():
    codigo_cuenta = row['ALTERNATIVA 6']
    valor_moneda_ext = row['VALOR MONEDA EXTRANJERA']
    if valor_moneda_ext == 0 or codigo_cuenta in cuentas_vistas:
        continue
    cuentas_vistas.add(codigo_cuenta)
    signo = '+' if valor_moneda_ext >= 0 else '-'
    linea = formatear_linea(numero_registro, codigo_cuenta, signo, abs(valor_moneda_ext))
    txt_datos.append(linea)
    numero_registro += 1

# Cierre de tercera parte
txt_datos.append(f"{str(numero_registro).zfill(5)}2000001100000")
numero_registro += 1

# Determinar si es un mes de cierre trimestral
mes_actual = fecha_actual.month
es_trimestral = mes_actual in [4, 7, 10, 1]  # Abril, Julio, Octubre, Enero

if es_trimestral:
    print("Es un mes de cierre trimestral. Se procesarán los formatos 235, 237 y 242.")

# --------- Cuarta parte: Procesar FORMATO 235 ---------
formato_235_path = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COVI.xlsx"
df_235 = pd.read_excel(formato_235_path, sheet_name="Formato235_TRIMESTRAL", dtype=str)

# Eliminar espacios de los nombres de las columnas
df_235.columns = df_235.columns.str.strip()

# Iterar sobre las columnas de valores relevantes
columnas_valores = ['CALCULO MATEMÁTICO', 'RESERVA DE PRIMAS CEDIDAS', 'TOTAL RESERVAS']

for idx, columna in enumerate(columnas_valores, start=1):  # idx servirá como número de columna
    for _, row in df_235.iterrows():
        subcuenta = row['SUBCUENTA'].strip()  # Subcuenta sin espacios
        unidad_captura = row['UNIDAD DE CAPTURA'].strip()  # Unidad de captura sin espacios
        valor = row[columna]

        # Convertir a numérico y validar si es NaN o cero
        try:
            valor = float(valor)
        except ValueError:
            continue

        if pd.isna(valor) or valor == 0:  # Verificar si es NaN o cero
            continue

        # Determinar el signo
        signo = '+' if valor >= 0 else '-'
        valor_abs = abs(int(valor))  # Valor absoluto como entero

        # Formatear la línea para el Formato 235
        linea = formatear_linea_235(numero_registro, idx, unidad_captura, subcuenta, signo, valor_abs)
        txt_datos.append(linea)
        numero_registro += 1

# Identificar el consecutivo final
consecutivo_final = str(numero_registro).zfill(3)  # Asegura que sea de 3 dígitos, ej: '806'

# Línea de cierre usando el consecutivo final
txt_datos.append(f"{consecutivo_final}2000001100000")
numero_registro += 1  # Incrementar el consecutivo después del cierre


# Quinta parte: Procesar FORMATO 237

# Ruta al archivo Excel del formato 237
formato_237_path = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COVI.xlsx"

# Cargar el archivo Excel
df_237 = pd.read_excel(formato_237_path, sheet_name="Formato237_TRIMESTRAL", dtype=str)

# Definir las columnas clave
columna_subcuenta_237 = 'SUBCUENTA'  # Columna que identifica la subcuenta
columna_unidad_captura_237 = 'UNIDAD DE CAPTURA'  # Columna que identifica la unidad de captura

# Filtrar filas sin datos válidos en las columnas esenciales
df_237 = df_237.dropna(subset=[columna_subcuenta_237, columna_unidad_captura_237])

# Asegurar que las columnas procesadas comiencen desde la columna 'E' en adelante
columnas_datos_237 = df_237.columns[4:]  # Índice 4 corresponde a la columna 'E'

# Recorrer las columnas de datos desde 'E' en adelante
for idx, columna in enumerate(columnas_datos_237, start=1):  # `idx` es el número de columna (comienza en 1)
    for _, row in df_237.iterrows():
        subcuenta = str(row[columna_subcuenta_237]).strip()  # Convertir a texto y eliminar espacios
        unidad_captura = str(row[columna_unidad_captura_237]).strip()  # Convertir a texto y eliminar espacios
        valor = row[columna]  # Valor de la columna actual

        # Validar que el valor sea numérico y no vacío, NaN o cero
        try:
            if pd.isna(valor) or valor.strip() == '':
                continue  # Saltar valores vacíos o NaN
            valor_float = float(valor)  # Intentar convertir el valor a flotante
            if valor_float == 0:
                continue  # Saltar valores iguales a 0
        except ValueError:
            continue  # Saltar valores no numéricos

        # Determinar el signo del valor
        signo = '+' if valor_float >= 0 else '-'
        valor_abs = abs(valor_float)

        # Formatear la línea específica para el Formato 237
        num_registro_str = str(numero_registro).zfill(5)  # Consecutivo con 5 dígitos
        tipo_registro = "4"  # Siempre "4"
        formato = "237"  # Número del formato
        columna_str = str(idx).zfill(2)  # Número de la columna (2 dígitos)
        unidad_captura_str = str(unidad_captura).zfill(2)  # Unidad de captura (2 dígitos)
        subcuenta_str = str(subcuenta).zfill(3)  # Subcuenta (3 dígitos)
        valor_str = f"{int(valor_abs):020d}"  # Valor absoluto con 20 dígitos

        # Construir la línea completa
        linea = f"{num_registro_str}{tipo_registro}{formato}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"
        txt_datos.append(linea)

        # Incrementar el consecutivo
        numero_registro += 1

# Identificar el consecutivo final
consecutivo_final = str(numero_registro).zfill(3)  # Asegura que sea de 3 dígitos

# Línea de cierre usando el consecutivo final
txt_datos.append(f"{consecutivo_final}2000001100000")
numero_registro += 1  # Incrementar el consecutivo después del cierre


# --------- Sexta parte: Formato 242 ---------
# --------- Sexta parte: Formato 242 ---------

# Ruta al archivo Excel del formato 242
formato_242_path = r"D:\OneDrive - Allianz\6_Procesos TRANSMISIONES\Fuentes de Información COVI.xlsx"

# Cargar el archivo Excel
df_242 = pd.read_excel(formato_242_path, sheet_name="Formato242_TRIMESTRAL", dtype=str)

# Filtrar filas que no tengan datos válidos en las columnas esenciales
df_242 = df_242.dropna(subset=['SUBCUENTA', 'UNIDAD DE CAPTURA'])  # Asegurarse de que SUB CUENTA y UNIDAD DE CAPTURA existan

# Excluir filas con "COLUMNAS" en la columna 'SUB CUENTA'
df_242 = df_242[df_242['SUBCUENTA'].str.upper() != "COLUMNAS"]

# Excluir filas donde UNIDAD DE CAPTURA esté vacía o tenga valores en cero
df_242 = df_242[df_242['UNIDAD DE CAPTURA'].str.strip() != '0']  # Filtrar valores cero
df_242 = df_242[df_242['UNIDAD DE CAPTURA'].str.strip() != '']  # Filtrar valores vacíos

# Seleccionar las columnas de datos a partir de la columna 'I' (índice 8 en Python)
columnas_datos = df_242.columns[8:]  # Seleccionar las columnas de datos desde la columna 8

# Convertir las columnas de datos a números, reemplazando valores vacíos o no numéricos con 0
df_242[columnas_datos] = df_242[columnas_datos].apply(pd.to_numeric, errors='coerce').fillna(0)

# Excluir filas con unidad de captura igual a 7 si todos los valores asociados son ceros
df_242 = df_242[~((df_242['UNIDAD DE CAPTURA'] == '7') & (df_242[columnas_datos].sum(axis=1) == 0))]

# Seleccionar las columnas de datos, excluyendo explícitamente la columna UNIDAD DE CAPTURA
columnas_sexta = [col for col in columnas_datos if col.upper() != "UNIDAD DE CAPTURA"]

# Iterar sobre las columnas seleccionadas
for idx, columna in enumerate(columnas_sexta, start=1):
    for _, row in df_242.iterrows():
        subcuenta = str(row['SUBCUENTA']).strip()  # Eliminar espacios en blanco
        unidad_captura = str(row['UNIDAD DE CAPTURA']).strip().zfill(2)  # Usar solo como referencia válida
        valor = row[columna]

        # Saltar valores no válidos (vacíos, ceros o en porcentaje)
        if valor == 0:
            continue

        # Determinar el signo del valor
        try:
            signo = '+' if valor >= 0 else '-'
            valor_abs = abs(valor)

            # Formatear la línea específica para la sexta parte
            num_registro_str = str(numero_registro).zfill(5)  # Número de registro con 5 dígitos
            tipo_registro = "4"  # Tipo de registro fijo
            codigo_formato = "242"  # Código del formato fijo
            columna_str = str(idx).zfill(2)  # Índice de columna con 2 dígitos
            subcuenta_str = str(subcuenta).zfill(3)  # Subcuenta con 3 dígitos
            valor_str = f"{int(valor_abs):020d}"  # Valor absoluto formateado a 20 caracteres

            # Crear la línea
            linea = f"{num_registro_str}{tipo_registro}{codigo_formato}{columna_str}{unidad_captura}{subcuenta_str}{signo}{valor_str}"
            txt_datos.append(linea)

            numero_registro += 1  # Incrementar el número de registro

        except ValueError:
            print(f"Valor no válido en la fila {row.name}, columna {columna}: {valor}")

# Cierre de la sexta parte
txt_datos.append(f"{str(numero_registro).zfill(5)}2000001100000")
numero_registro += 1  # Incrementar el consecutivo después del cierre


# --------- Septima parte: Procesar FORMATO 290 ---------

# Ruta al archivo Excel
ruta_formato_290 = r"D:/OneDrive - Allianz/6_Procesos TRANSMISIONES/FORMATO_290/Bases/Formato_290_COVI.xlsx"
                    
# Cargar el archivo Excel
df_290 = pd.read_excel(ruta_formato_290, dtype=str)

# Definir las columnas clave
columna_subcuenta = 'SUB CUENTA'
columna_unidad_captura = 'UNIDAD DE CAPTURA'

# Filtrar las filas que no tengan datos válidos en la columna 'SUB CUENTA'
df_290 = df_290.dropna(subset=[columna_subcuenta])

# Tomamos las columnas de datos desde la tercera hasta la columna 51 (inclusive)
columnas_datos = df_290.columns[2:45]  # El índice 44 no se incluye, por lo tanto, este abarca hasta la columna 51

# Función para la Sexta parte
def formatear_linea_cuarta_parte(numero_registro, codigo_formato, columna_idx, unidad_captura, subcuenta, signo, valor):
    num_registro_str = str(numero_registro).zfill(5)  # Aseguramos 5 dígitos para el número de registro
    tipo_registro = "4"  # Tipo de registro para la cuarta parte
    codigo_formato_str = str(codigo_formato).zfill(3)  # 3 dígitos para el código del formato
    columna_str = str(columna_idx).zfill(2)  # 2 dígitos para el número de la columna
    unidad_captura_str = str(unidad_captura).zfill(2)  # Aseguramos 2 dígitos para la unidad de captura
    subcuenta_str = str(subcuenta).zfill(3)  # 3 dígitos para la subcuenta
    valor_str = f"{int(valor):020d}"  # Valor sin decimales, formateado a 20 caracteres con ceros a la izquierda
    return f"{num_registro_str}{tipo_registro}{codigo_formato_str}{columna_str}{unidad_captura_str}{subcuenta_str}{signo}{valor_str}"

# Recorrer las columnas de datos (por ejemplo, columnas numéricas con valores)
for idx, columna in enumerate(columnas_datos, start=1):
    for _, row in df_290.iterrows():
        subcuenta = str(row[columna_subcuenta]).strip()  # Convertir a texto y eliminar espacios en blanco
        unidad_captura = str(row[columna_unidad_captura]).strip()  # Convertir a texto y eliminar espacios en blanco
        valor = row[columna]

        # Saltar si el valor es cero o NaN
        if pd.isna(valor) or valor == '0':
            continue

        # Determinar el signo del valor
        try:
            valor_float = float(valor)
            signo = '+' if valor_float >= 0 else '-'
            valor_abs = abs(valor_float)

            # Formatear la línea específica para el Formato 290 (Septima parte)
            linea = formatear_linea_cuarta_parte(numero_registro, 290, idx, unidad_captura, subcuenta, signo, valor_abs)
            txt_datos.append(linea)

            numero_registro += 1

        except ValueError:
            print(f"Valor no válido en la fila {row.name}, columna {columna}: {valor}")

# Agregar la última línea con el conteo de registros y un "5" al final
total_registros = len(txt_datos) + 1  # Contar todas las líneas generadas en las cuatro partes
linea_final = f"{str(total_registros).zfill(5)}5"  # Formatear el total con un "5" al final
txt_datos.append(linea_final)

# Guardar el archivo TXT final con todas las partes
with open("PlanoF290_COVI.txt", "w") as f:
    for linea in txt_datos:
        f.write(f"{linea}\n")